In [1]:
## run-1
## download data sku dari tatanama
## common problems: slow server pyany & make sure path to file

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import time
import pandas as pd
import numpy as np
import math

import smtplib
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from smtplib import SMTP
import smtplib
import sys
import requests
import os


print('Reading Data SKU')
data_SKU = pd.read_excel('SKU_File\data_SKU.xlsx')
print('Scrapping Tatanama')
s = requests.Session()
s.get("https://tatanama.pythonanywhere.com")
s.post("https://tatanama.pythonanywhere.com", data = {'username' : 'ecommerce', 'password' : 'ecommerce'})
r = s.get("https://tatanama.pythonanywhere.com/download")

with open('SKU_File\Master tatanama.xlsx', 'wb') as output:
    output.write(r.content)
    
print('ok')

if os.path.isfile('SKU_File\Master tatanama.xlsx') :    
    SKU_append = pd.read_excel('SKU_File\Master tatanama.xlsx')
    SKU_append.columns = [x.replace('_', ' ') for x in SKU_append.columns]
    data_SKU = data_SKU[~data_SKU['SKU'].astype(str).isin(SKU_append['SKU'].astype(str))]
    data_SKU = data_SKU.append(SKU_append, ignore_index = True, sort = False)

to_excel = data_SKU.to_excel('SKU_File\data_SKU.xlsx', index = False)
print('data SKU done')
data_SKU['Price List NFI'] = data_SKU['Price List NFI'].astype(float).astype('int64')

download = True

Reading Data SKU
Scrapping Tatanama
ok
data SKU done


In [2]:
## run-2
## belum scrap dari forstok
download = False

In [6]:
## run-3
## scrap data forstok & blibli lmen

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from datetime import datetime, timedelta
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import requests
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
import pandas as pd
import numpy as np
from webdriver_manager.chrome import ChromeDriverManager

diff_date = 10
data_now = str(datetime.today().day) + ' ' + str(datetime.today().strftime("%B"))

print('Input Forstok Date')
start_date = datetime.today() - timedelta(days=diff_date)
end_date = datetime.today()
date = start_date.strftime('%Y-%m-%d') + '-' + end_date.strftime('%Y-%m-%d')
print(date)

if not download:
    print('Opening Chrome')
    driver = webdriver.Chrome()
    driver.maximize_window()
    actions = ActionChains(driver)
    driver.get("https://www.forstok.com/dashboard/users/login")
    print('Login Forstok')
    username = driver.find_element_by_id("dashboard_user_email")
    username.clear()
    username.send_keys("timotius.giovandi@nutrifood.co.id")

    password = driver.find_element_by_id("dashboard_user_password")
    password.send_keys("timo123")
    
    driver.find_element_by_name("commit").click()
    
    time.sleep(20)
    datefield = WebDriverWait(driver, 100).until(EC.element_to_be_clickable((By.XPATH,'//*[@type="button" and contains(text(),"Last 30 days")]')))
    datef = WebDriverWait(driver, 200).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="root"]/section/section[2]/article/div/div/aside/div/div[1]/section[5]/section[1]/h2')))
    driver.execute_script("return arguments[0].scrollIntoView(true);", datef)
    time.sleep(30)
    driver.execute_script("return arguments[0].scrollIntoView(true);", datefield)
    time.sleep(10)
    datefield.click()
    pilih=0
    while pilih < 3:    
            n_elem=0
            for i in driver.find_elements_by_xpath('(//*[@class="DateRangePicker__MonthHeaderLabel DateRangePicker__MonthHeaderLabel--month"])'):
                val=(driver.find_elements_by_xpath('(//*[@class="DateRangePicker__MonthHeaderLabel DateRangePicker__MonthHeaderLabel--month"])')[n_elem].text.split()[0])
                if val==start_date.strftime('%B'):
                    # print(n_elem)
                    fin_n_elem=n_elem
                n_elem=n_elem+1
            if fin_n_elem==0:
                text = '(//*[@class="DateRangePicker__Month"])[1]//*[@class="DateRangePicker__DateLabel" and (text()="' + str(start_date.day) + '")]'
                if len(driver.find_elements_by_xpath(text))==2:
                    text = '('+text+')[2]'
            else:
                text = '(//*[@class="DateRangePicker__Month"])[2]//*[@class="DateRangePicker__DateLabel" and (text()="' + str(start_date.day) + '")]'
                if len(driver.find_elements_by_xpath(text))==2:
                    text = '('+text+')[1]'
            driver.find_element_by_xpath(text).click()
            time.sleep(10)

            n_elem=0
            for i in driver.find_elements_by_xpath('(//*[@class="DateRangePicker__MonthHeaderLabel DateRangePicker__MonthHeaderLabel--month"])'):
                val=(driver.find_elements_by_xpath('(//*[@class="DateRangePicker__MonthHeaderLabel DateRangePicker__MonthHeaderLabel--month"])')[n_elem].text.split()[0])
                if val==end_date.strftime('%B'):
                    fin_n_elem=n_elem
                n_elem=n_elem+1
            if fin_n_elem==0:
                text = '(//*[@class="DateRangePicker__Month"])[1]//*[@class="DateRangePicker__DateLabel" and (text()="' + str(end_date.day) + '")]'
                if len(driver.find_elements_by_xpath(text))==2:
                    text = '('+text+')[2]'
            else:
                text = '(//*[@class="DateRangePicker__Month"])[2]//*[@class="DateRangePicker__DateLabel" and (text()="' + str(end_date.day) + '")]'
                if end_date.strftime('%B')==start_date.strftime('%B'):
                    text = '('+text+')'#[2]
                else :
                    text = '('+text+')[1]'
            driver.find_element_by_xpath(text).click()
            time.sleep(10)
            pilih=pilih+1
    driver.find_element_by_xpath('//*[@type="button" and contains(text(),"Apply")]').click()
    time.sleep(100)
    WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, "(//*[@type='button' and contains(text(),'Export')])[1]"))).click()
    driver.find_element_by_xpath('//span[text()="Sales Order"]').click()
    driver.find_element_by_xpath('//span[text()="Sales Order"]').click()
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "(//*[@type='button' and contains(text(),'Export')])[2]"))).click()
    time.sleep(2)

    driver.quit()
    
    print('Forstok Downloaded')
    download = True

print('Download Data SKU')
# Import library

import time
import pandas as pd
import numpy as np
import math

import smtplib 
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from smtplib import SMTP
import smtplib
import sys
import requests
import os

print('Waiting to Download Forstok')
s = requests.Session()
r = s.get("https://www.forstok.com/dashboard/users/login")
data = {'dashboard_user_email' : 'andra.miftah@nutrifood.co.id', "dashboard_user_password" : 'nutrimart1234'}
r = s.post("https://www.forstok.com/dashboard/users/login", data = data)

text = 'https://forstok-staging-storage.s3.ap-southeast-1.amazonaws.com/items_import/nutrifood--forstok-sales_orders-version_1-' + date + '.xls'
print(text)
while True:
    r = s.get(text)
    print('Waiting to Download Forstok')
    
    if r.status_code == requests.codes.ok:
        print(r.status_code)
        with open('Input Data/Forstok_new_input.xls', 'wb') as output:
            output.write(r.content)
        break
    else :
        time.sleep(60)
           
with open('Input Data/nutrifood--forstok-sales_orders-' + date + '.xls', 'wb') as output:
    output.write(r.content)
    
# Import data
start_time = time.time()
print("Import Data ====== 1/10")

data_forstok = pd.read_excel(r'Input Data/Forstok_new_input.xls')
data_forstok_pure = data_forstok.copy()

data_forstok = data_forstok.dropna(how = 'all')

pd.options.mode.chained_assignment = None
# Forstok formatting
print("--- %s seconds ---" % (time.time() - start_time))
print("Formatting Data ====== 2/10")

if 'Unnamed: 35' in data_forstok:
    data_forstok = data_forstok.drop(['Unnamed: 35'], axis = 'columns')
if 'Unnamed: 36' in data_forstok:
    data_forstok = data_forstok.rename(columns={'Unnamed: 36' : 'Comment'})
if 'Unnamed: 37' in data_forstok:
    data_forstok = data_forstok.drop(['Unnamed: 37'], axis = 'columns')
if 'Unnamed: 38' in data_forstok:
    data_forstok = data_forstok.drop(['Unnamed: 38'], axis = 'columns')
if 'Unnamed: 39' in data_forstok:
    data_forstok = data_forstok.drop(['Unnamed: 39'], axis = 'columns')
if 'Unnamed: 40' in data_forstok:
    data_forstok = data_forstok.drop(['Unnamed: 40'], axis = 'columns')
if 'Unnamed: 41' in data_forstok:
    data_forstok = data_forstok.drop(['Unnamed: 41'], axis = 'columns')
    
data_forstok["Order Date"] = pd.to_datetime(data_forstok["Order Date"], errors = 'coerce')
data_forstok["Paid Date"] = pd.to_datetime(data_forstok["Paid Date"], errors = 'coerce')
data_forstok["Cancelled Date"] = pd.to_datetime(data_forstok["Cancelled Date"], errors = 'coerce')
data_forstok['Customer Name'] = data_forstok['Customer Name'].fillna(data_forstok['Shipping Name'])
data_forstok = data_forstok.rename(columns = {'Seller Voucher' : 'Seller Discount'})
data_forstok = data_forstok[~((data_forstok['Store']=='Shopee')&(data_forstok['Order Date']<'21-12-2021'))] ###TMO###

# Phone formatting
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('=','', regex = False)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('"','', regex = False)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('(','', regex = False)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace(')','', regex = False)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('-','', regex = False)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('+','', regex = False)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('^620','0', regex = True)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('^62','0', regex = True)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('^620','0', regex = True)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('^8','08', regex = True)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('^21','021', regex = True)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('^008','08', regex = True)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('(021)','021', regex = False)
data_forstok['Shipping Phone'] = data_forstok['Shipping Phone'].astype(str).str.replace('+62','0', regex = False)

# Forstok SKU
print("--- %s seconds ---" % (time.time() - start_time))
print("Fulfilling SKU ====== 3/10")
indeks = data_forstok[data_forstok['Item Name'].astype(str) == 'Buy 1 Get 1 FREE Tropicana Slim Goldenmil Vanilla Manuka Honey (6 Sch)'][data_forstok[data_forstok['Item Name'].astype(str) == 'Buy 1 Get 1 FREE Tropicana Slim Goldenmil Vanilla Manuka Honey (6 Sch)']['SKU'] == 'PE8B27'].index.to_list()
data_forstok['SKU'][indeks] = '2101384106P2'
data_forstok.loc[data_forstok['SKU']=="(E)2101492P04",'SKU']="(E)2101492P4"
data_forstok.loc[data_forstok['SKU']=="P2101151180P2",'SKU']="PT7(2)B65"
data_forstok.loc[data_forstok['SKU']==' PT28(2)B65','SKU']='PT28(2)B65'
data_forstok.loc[data_forstok['SKU']==' PT6(2)B65','SKU']='PT6(2)B65'
data_forstok.loc[data_forstok['SKU']==' PT1(2)B65','SKU']='PT1(2)B65'
data_forstok.loc[(data_forstok['SKU']=='2101865450P3')&(data_forstok['Item Name'].astype(str)=='Tropicana Slim White Coffee 4 Sachet x 3 pcs - Kopi Susu Nikmat Tanpa Gula Pasir'),'SKU']='2104170164P3'
data_forstok.loc[(data_forstok['SKU']=='2104170104P3')&(data_forstok['Item Name'].astype(str)=='Triple Pack: Tropicana Slim White Coffee - Kopi Bebas Gula'),'SKU']='2104170164P3'
data_forstok.loc[(data_forstok['SKU']=='COLLETTEP24')&(data_forstok['Item Name'].astype(str)=='HiLo School Cotton Candy 200ml - Ready to Drink (24 Pcs)'),'SKU']='2HF1403250P24'
data_forstok.loc[(data_forstok['SKU']=='COLLETTEP4')&(data_forstok['Item Name'].astype(str)=='HiLo School Cotton Candy 200ml - Ready to Drink (4 Pcs) Tinggi Kalsium'),'SKU']='2HF1403250P4'

skushopee = data_forstok[data_forstok['SKU'].astype(str).str.contains('(S)',regex = False)]
data_forstok = data_forstok[~data_forstok['SKU'].astype(str).isin(skushopee['SKU'].astype(str))]

skushopee = skushopee.reset_index(drop = True)
data_forstok = data_forstok.reset_index(drop = True)

forstok_all_sku = pd.read_excel(r'SKU_File\forstok_all_sku.xlsx')
indeks = data_forstok[data_forstok['SKU'].isnull()].index.to_list()

for i in indeks:
    if str(data_forstok['Item Name'][i]).lower() in data_SKU['Nama Produk'].astype(str).str.lower().values:
        data_forstok['SKU'][i] = data_SKU['SKU'].loc[str(data_forstok['Item Name'][i]).lower() == data_SKU['Nama Produk'].astype(str).str.lower()].values[0]

indeks = data_forstok[data_forstok['SKU'].isnull()].index.to_list()
for i in indeks:
    if str(data_forstok['Item Name'][i]).lower() in forstok_all_sku['Item Name'].astype(str).str.lower().values:
        data_forstok['SKU'][i] = forstok_all_sku['SKU'].loc[str(data_forstok['Item Name'][i]).lower() == forstok_all_sku['Item Name'].astype(str).str.lower()].values[0]

indeks = data_forstok[data_forstok['Item Name'].astype(str).str.contains(' - ')].index.to_list()

# Formatting double name
for i in indeks:
    if data_forstok['Item Name'][i].count(' - ') == 1 :
        if (data_forstok['Item Name'][i].split(' - ')[0] == data_forstok['Item Name'][i].split(' - ')[1]):
            data_forstok['Item Name'][i] = data_forstok['Item Name'][i].split(' - ')[0]
    elif data_forstok['Item Name'][i].count(' - ') > 1:
        temp = math.ceil(data_forstok['Item Name'][i].count(' - ')/2)
        itemname = ''
        duplicate = ''
        for j in range(temp):
            if j == 0:
                itemname = itemname + data_forstok['Item Name'][i].split(' - ')[j]
                duplicate = duplicate + data_forstok['Item Name'][i].split(' - ')[temp+j]
            else :
                itemname = itemname + ' ' +data_forstok['Item Name'][i].split(' - ')[j]
                duplicate = duplicate + ' ' +data_forstok['Item Name'][i].split(' - ')[temp+j]
        if itemname == duplicate:
            data_forstok['Item Name'][i] = itemname
        
data_forstok['Item Name'] = data_forstok['Item Name'].str.replace(r' - $', '')
data_forstok["Item Name"] = data_forstok["Item Name"].str.replace('Special Promo by Nutrimart Serba 12 RIBU Varian:', '', regex=False)
data_forstok["Item Name"] = data_forstok["Item Name"].str.replace('Khusus Jabodetabek', '- Khusus Jabodetabek', regex=False)
data_forstok['Item Name'] = data_forstok['Item Name'].str.replace('Buy 1 Get F', 'Buy 1 Get 1 F')
data_forstok['Item Name'] = data_forstok['Item Name'].str.replace('Buy 1 Get H', 'Buy 1 Get 1 H')
data_forstok['Item Name'] = data_forstok['Item Name'].str.replace('Buy 12 FREE', 'Buy 12 FREE 12')

# Formatting SKU based on name
indeks = data_forstok[data_forstok['SKU'].isnull()].index.to_list()
for i in indeks:
    if str(data_forstok['Item Name'][i]).lower().strip() in data_SKU['Nama Produk'].astype(str).str.lower().str.strip().values:
        data_forstok['SKU'][i] = data_SKU['SKU'].loc[str(data_forstok['Item Name'][i]).lower().strip() == data_SKU['Nama Produk'].astype(str).str.lower().str.strip()].values[0]

indeks = data_forstok[data_forstok['SKU'].isnull()].index.to_list()
for i in indeks:
    if str(data_forstok['Item Name'][i]).lower() in forstok_all_sku['Item Name'].astype(str).str.lower().values:
        data_forstok['SKU'][i] = forstok_all_sku['SKU'].loc[str(data_forstok['Item Name'][i]).lower() == forstok_all_sku['Item Name'].astype(str).str.lower()].values[0]

indeks = data_forstok[data_forstok['SKU'].isnull()].index.tolist()

data_adasku = data_forstok[['Item Name', 'SKU']]
data_adasku = data_adasku[data_adasku['SKU'].notnull()]

data_nosku = data_forstok[['Item Name', 'SKU']]
data_nosku = data_nosku[data_nosku['SKU'].isnull()]

for i in indeks:
    if data_adasku['SKU'].loc[data_nosku['Item Name'][i] == data_adasku['Item Name']].size != 0:
        data_forstok['SKU'][i] = data_adasku['SKU'].loc[data_nosku['Item Name'][i] == data_adasku['Item Name']].values[0]

for i in indeks:
    if data_forstok['Item Name'][i] == 'Lokalate Kopi Durian 10s':
        data_forstok['SKU'][i] = '1101675318'
    elif data_forstok['Item Name'][i] == 'Nutrisari Madu Kurma Isi 16 Renceng X 10 Sachet Karton' or data_forstok['Item Name'][i] =='Nutrisari Madu Kurma Isi 16 Renceng X 10 SachetKarton': 
        data_forstok['SKU'][i] = 'PN30(16)'
    elif data_forstok['Item Name'][i] == 'L-Men Protein Bar Crunchy Chocolate Isi X12 (Exp Date:10-Apr-2019)':
        data_forstok['SKU'][i] = '2306592173'
    elif data_forstok['Item Name'][i] == 'FS Hilo Active Chocolate Minuman Kesehatan [750 gr]' or data_forstok['Item Name'][i] == 'FSHilo Active Chocolate Minuman Kesehatan [750 gr]':
        data_forstok['SKU'][i] = '2101452190'
    elif data_forstok['Item Name'][i] == 'FS L-Men Platinum Suplemen Kesehatan + Free Spider Bottle [800 g] Hitam' or data_forstok['Item Name'][i] == 'FSL-Men Platinum Suplemen Kesehatan + Free Spider Bottle [800 g] Hitam':
        data_forstok['SKU'][i] = '2305551288P1G26'
    elif data_forstok['Item Name'][i] == 'NutriSari Premium ala Jus Mangga':
        data_forstok['SKU'][i] = '1100534104'
    elif data_forstok['Item Name'][i] == 'Buy 1 Get 1 FREE Tropicana Slim Sweetener Honey (50 Sch) - FS':
        data_forstok['SKU'][i] = '2102501125P1G53'

indeks = data_forstok[~data_forstok['SKU'].astype(str).isin(data_SKU['SKU'].astype(str))].index.to_list()
for i in indeks:
    if str(data_forstok['Item Name'][i]).lower() in forstok_all_sku['Item Name'].astype(str).str.lower().values:
        data_forstok['SKU'][i] = forstok_all_sku['SKU'].loc[str(data_forstok['Item Name'][i]).lower() == forstok_all_sku['Item Name'].astype(str).str.lower()].values[0]

list_alias = []
list_alias_name = []
for colname in data_SKU.columns:
    if 'Alias SKU' in colname:
        list_alias.append(colname)
    if 'Alias Nama' in colname:
        list_alias_name.append(colname)


for i in indeks:
    for j in list_alias:
        if str(data_forstok['SKU'][i]) in data_SKU[j].astype(str).values:
            idx = data_SKU[str(data_forstok['SKU'][i]) == data_SKU[j].astype(str)].index.to_list()
            for k in idx:
                if str(data_forstok['Item Name'][i]) == data_SKU[j.replace('SKU', 'Nama')][k]:
                    data_forstok['SKU'][i] = data_SKU['SKU'][k]
                        
indeks = data_forstok[~data_forstok['SKU'].astype(str).isin(data_SKU['SKU'].astype(str))].index.to_list()

for i in indeks:
    for j in list_alias_name:
        if str(data_forstok['Item Name'][i]).lower() in data_SKU[j].astype(str).str.lower().values:
            data_forstok['SKU'][i] = data_SKU['SKU'].loc[str(data_forstok['Item Name'][i]).lower() == data_SKU[j].astype(str).str.lower()].values[0]

indeks = data_forstok[~data_forstok['SKU'].astype(str).isin(data_SKU['SKU'].astype(str))].index.to_list()

for i in indeks:
    if str(data_forstok['Item Name'][i]).lower() in data_SKU['Nama Produk'].astype(str).str.lower().values:
        data_forstok['SKU'][i] = data_SKU['SKU'].loc[str(data_forstok['Item Name'][i]).lower() == data_SKU['Nama Produk'].astype(str).str.lower()].values[0]

indeks = skushopee[~skushopee['SKU'].astype(str).str.replace('(S)','', regex = False).isin(data_SKU['SKU'].astype(str))].index.to_list()
for i in indeks:
    if str(skushopee['Item Name'][i]).lower() in data_SKU['Nama Produk'].astype(str).str.lower().values:
        skushopee['SKU'][i] = data_SKU['SKU'].loc[str(skushopee['Item Name'][i]).lower() == data_SKU['Nama Produk'].astype(str).str.lower()].values[0]
        skushopee['SKU'][i] = '(S)' + str(skushopee['SKU'][i])

indeks = skushopee[~skushopee['SKU'].astype(str).str.replace('(S)','', regex = False).isin(data_SKU['SKU'].astype(str))].index.to_list()
for i in indeks:
    for j in list_alias:
        if str(skushopee['SKU'][i]).replace('(S)','') in data_SKU[j].astype(str).values:
            idx = data_SKU[str(skushopee['SKU'][i]).replace('(S)','') == data_SKU[j].astype(str)].index.to_list()
            for k in idx:
                if str(skushopee['Item Name'][i]) == data_SKU[j.replace('SKU', 'Nama')][k]:
                    skushopee['SKU'][i] = data_SKU['SKU'][k]

indeks = data_forstok[data_forstok['SKU'].astype(str) == 'Gift Sosro'].index.to_list()
data_forstok = data_forstok.drop(indeks, axis = 0).reset_index(drop = True)

indeks = data_forstok[data_forstok['Item Name'].astype(str).str.contains('Fricella')].index.to_list()
data_forstok = data_forstok.drop(indeks, axis = 0).reset_index(drop = True)

indeks = data_forstok[data_forstok['Item Name'].astype(str).str.contains('Zaskia Mecca')].index.to_list()
data_forstok = data_forstok.drop(indeks, axis = 0).reset_index(drop = True)

print("--- %s seconds ---" % (time.time() - start_time))
print("Listing SKU Missing ====== 4/10")

data_forstok = data_forstok.append(skushopee, ignore_index = True, sort = False)
data_forstok = data_forstok.reset_index(drop = True)

indeks = data_forstok[data_forstok['SKU'].astype(str) == '71210111'].index.to_list()
data_forstok['SKU'][indeks] = 'PN28N29N34(2)N53N54N55'

indeks = data_forstok[data_forstok['SKU'].astype(str) == '71210112'].index.to_list()
data_forstok['SKU'][indeks] = 'PN19N22N30N34(2)N53N55'

indeks = data_forstok[data_forstok['SKU'].astype(str) == '(B) 2101656'].index.to_list()
data_forstok['SKU'][indeks] = '(B)2101656'

indeks = data_forstok[data_forstok['Item Name'].astype(str) == 'PRE ORDER NutriSari NUTRI-C1000 40sch Suplemen Kesehatan Vit C 1000mg '].index.to_list()
data_forstok['SKU'][indeks] = '1102110453'

data_forstok.loc[data_forstok['Item Name']=="HiLo Active Caramel Latte 500gr - Susu Tinggi Kalsium",'SKU']="2101478180"
data_forstok.loc[data_forstok['Item Name']=="NutriSari Es Rujak 40 Sachet - Minuman Buah Vitamin C - Expired 4 Bulan",'SKU']="(E)1101984453"
data_forstok.loc[data_forstok['SKU']=='PH39(12)U8(12) ','SKU']='PH39(12)U8(12)'


shopee_con = pd.read_excel(r'data_supp\Shopee Converter.xlsx')
indeks = data_forstok[data_forstok['SKU'].astype(str).str.replace('(S)','', regex = False).isin(shopee_con['SKU'].astype(str))][data_forstok[data_forstok['SKU'].astype(str).str.replace('(S)','', regex = False).isin(shopee_con['SKU'].astype(str))]['Item Name'].astype(str).str.lower().isin(shopee_con['Product Name'].astype(str).str.lower())].index.to_list()
product = data_forstok[data_forstok['SKU'].astype(str).str.replace('(S)','', regex = False).isin(shopee_con['SKU'].astype(str))][data_forstok[data_forstok['SKU'].astype(str).str.replace('(S)','', regex = False).isin(shopee_con['SKU'].astype(str))]['Item Name'].astype(str).str.lower().isin(shopee_con['Product Name'].astype(str).str.lower())]['Item Name'].unique()

for i in product:
    indeks_shopee = data_forstok.iloc[indeks][data_forstok.iloc[indeks]['Item Name'].astype(str).str.lower() == str(i).lower()].index.to_list()
    data_forstok['SKU'][indeks_shopee] = shopee_con[shopee_con['Product Name'].astype(str).str.lower() == str(i).lower()]['Real SKU'].values[0]


to_excel = data_forstok.to_excel(r'forstok_new.xls', index = False)

skushopee = data_forstok[data_forstok['SKU'].astype(str).str.contains('(S)',regex = False)]
data_forstok = data_forstok[~data_forstok['SKU'].astype(str).isin(skushopee['SKU'].astype(str))]

skushopee = skushopee.reset_index(drop = True)
data_forstok = data_forstok.reset_index(drop = True)

sku_exclude = ['5337754001-1623228299159-0',"TESSS"]
data_forstok = data_forstok[~data_forstok['SKU'].isin(sku_exclude)]

data_forstok.loc[data_forstok['Item Name']=="HiLo Teen Strawberry Milkshake 500 gram – Susu Tinggi Kalsium - Milkshake",'SKU']="2101683180"
data_forstok.loc[data_forstok['Item Name']=="Paket Minyak Hemat – Tropicana Slim Canola Oil & Tropicana Slim Sunflower Oil - Package",'SKU']="PT3T46"
data_forstok.loc[data_forstok['Item Name']=="Buy 2 Get 1 - L-Men Protein Crunch BBQ Beef (20gr) - Triplepack",'SKU']="2309005300P3"
data_forstok.loc[data_forstok['Item Name']=="Paket Kecap Sehat - Topicana Slim Kecap Manis & Tropicana Slim Kecap Asin - Package","SKU"]="PT26T44"
data_forstok.loc[(data_forstok['Item Name']=="(Free Gift) 3pcs L-Men Protein Crunch BBQ Beef 20g")&
                 (data_forstok['Store']=="Lazada"),"SKU"]="(B)2309005305P3"
data_forstok.loc[data_forstok['Item Name']=="WHS - Paket NutriSari Jeruk Peras, HiLo Es Ketan Hitam, dan HiLo Klepon Latte Powder Drink Refill 500 gram",'SKU']="PH81H82N60G187"

idx = []
idx = idx + data_forstok[data_forstok['SKU'].isnull()].drop_duplicates().index.to_list()
idx = idx + data_forstok[~data_forstok['SKU'].astype(str).isin(data_SKU['SKU'].astype(str))].index.to_list()
idx = list(dict.fromkeys(idx))
idx_s = skushopee[~skushopee['SKU'].astype(str).str.replace('(S)','', regex = False).isin(data_SKU['SKU'].astype(str))].index.to_list()
idx_s = list(dict.fromkeys(idx_s))


for i in product:
    indeks_shopee = data_forstok.iloc[indeks][data_forstok.iloc[indeks]['Item Name'].astype(str).str.lower() == str(i).lower()].index.to_list()
    data_forstok['SKU'][indeks_shopee] = shopee_con[shopee_con['Product Name'].astype(str).str.lower() == str(i).lower()]['Real SKU'].values[0]

print("--- %s seconds ---" % (time.time() - start_time))
if len(idx) != 0 or len(idx_s) != 0:
    alert = data_forstok.iloc[idx, ][['SKU', 'Item Name', 'Channel']].drop_duplicates()
    alert = alert.append(skushopee.iloc[idx_s][['SKU', 'Item Name', 'Channel', 'Selling Price']].drop_duplicates(), ignore_index = True, sort = False)
    alert['SKU'] = alert['SKU'].astype(str)
    alert['SKU Valid'] = np.nan
    to_excel = alert.to_excel('ALERT_FORSTOK_SKU_MISSING.xlsx')
    print("Some SKU Missing Please Complete It ====== 5/10")
    print("--- %s seconds ---" % (time.time() - start_time))
    
    # creates SMTP session 
    s = smtplib.SMTP('smtp.gmail.com', 587) 

    # start TLS for security 
    s.starttls() 

    # Authentication 
    s.login("automationnfi@gmail.com", "nutrifood2020") 

    msg = MIMEMultipart()
    msg['Subject'] = "ALERT SKU FORSTOK MISSING"


    html = """\
    <html>
      <head></head>
      <body>
        {0}
      </body>
    </html>
    """.format(alert.to_html())

    part1 = MIMEText(html, 'html')
    msg.attach(part1)

    # sending the mail 
    s.sendmail("automationnfi@gmail.com", "andra.miftah@nutrifood.co.id", msg.as_string()) 

    # terminating the session 
    s.quit() 
else :
    print("Filling Brand ====== 5/10")
    data_forstok['SKU'] = data_forstok['SKU'].astype(str)
    data_forstok['Item Name'] = data_forstok['Item Name'].astype(str)
    data_SKU['Real SKU'] = data_SKU['SKU'].astype(str).str.replace('(S)', '', regex = False)
    data_SKU['Real Nama Produk'] = data_SKU['Nama Produk'].astype(str)
    
    data_forstok = data_forstok.merge(data_SKU[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')

    temp = data_forstok[data_forstok['Real SKU'].isnull()].copy()
    temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
    temp = temp.merge(data_SKU[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
    temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
    temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
    temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
    temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

    indeks = data_forstok[data_forstok['Real SKU'].isnull()].index.to_list()
    data_forstok['Real SKU'][indeks] = temp['Real SKU'][indeks]
    data_forstok['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

    temp = data_forstok[data_forstok['Real SKU'].isnull()].copy()
    temp['SKU'] = temp['SKU'].astype(str).str.replace('hd','', regex = False)
    temp['SKU'] = temp['SKU'].astype(str).str.replace('HD','', regex = False)
    temp = temp.merge(data_SKU[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
    temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
    temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
    temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
    temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

    indeks = data_forstok[data_forstok['Real SKU'].isnull()].index.to_list()
    data_forstok['Real SKU'][indeks] = temp['Real SKU'][indeks]
    data_forstok['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

    data_forstok['Real SKU'] = data_forstok['Real SKU'].astype(str)
    data_forstok = data_forstok.merge(data_SKU[['SKU', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
    data_forstok = data_forstok.drop(['SKU_y'], axis = 1)
    data_forstok = data_forstok.rename(columns = {'SKU_x':'SKU'})

    print("--- %s seconds ---" % (time.time() - start_time))
    print("Unbundling ====== 6/10")        
    # Forstok Unbundling    
    list_col = ['SKU'] + data_SKU.columns[data_SKU.columns.get_loc('Produk 1'):data_SKU.columns.get_loc('Harga Organik 7')+1].to_list()
    data_forstok = data_forstok.merge(data_SKU[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
    list_pcs = [x for x in data_forstok.columns if 'PCS' in x]
    for i in list_pcs:
        data_forstok[i] = data_forstok[i] * data_forstok['Quantity']
    data_forstok = data_forstok.drop(['SKU_y'], axis = 1)
    data_forstok = data_forstok.rename(columns = {'SKU_x':'SKU'})

    indeks = data_forstok[data_forstok['Brand'] == 'Bundle'].index.to_list()
    data_forstok['Bundle Flag'] = np.nan
    data_forstok['Bundle Flag'][indeks] = 'Bundle'

    indeks = data_forstok[data_forstok['Brand'] == 'Bundle'][data_forstok[data_forstok['Brand'] == 'Bundle']['SKU'].astype(str).str.contains('(S)', regex = False)].index.to_list()
    data_forstok['SKU Produk 1'][indeks] = '(S)' + data_forstok['SKU Produk 1'][indeks].astype(str)
    data_forstok['SKU Produk 2'][indeks] = '(S)' + data_forstok['SKU Produk 2'][indeks].astype(str)
    data_forstok['SKU Produk 3'][indeks] = '(S)' + data_forstok['SKU Produk 3'][indeks].astype(str)
    data_forstok['SKU Produk 4'][indeks] = '(S)' + data_forstok['SKU Produk 4'][indeks].astype(str)
    data_forstok['SKU Produk 5'][indeks] = '(S)' + data_forstok['SKU Produk 5'][indeks].astype(str)
    data_forstok['SKU Produk 6'][indeks] = '(S)' + data_forstok['SKU Produk 6'][indeks].astype(str)
    data_forstok['SKU Produk 7'][indeks] = '(S)' + data_forstok['SKU Produk 7'][indeks].astype(str)

    print("--- %s seconds ---" % (time.time() - start_time))
    print("Filling Date ====== 7/10")
    data_forstok['Date'] = np.nan
    data_forstok['Month'] = np.nan
    data_forstok['Year'] = np.nan

    for i in range(data_forstok.shape[0]):
        if int(data_forstok['Order Date'][i].strftime('%d')) <= 12:
            data_forstok['Date'][i] = pd.to_datetime(data_forstok['Order Date'][i].strftime('%Y-%d-%m %H:%M')).day
            data_forstok['Month'][i] = pd.to_datetime(data_forstok['Order Date'][i].strftime('%Y-%d-%m %H:%M')).month_name()
            data_forstok['Year'][i] = pd.to_datetime(data_forstok['Order Date'][i].strftime('%Y-%d-%m %H:%M')).year
        else :
            data_forstok['Date'][i] = pd.to_datetime(data_forstok['Order Date'][i]).day
            data_forstok['Month'][i] = pd.to_datetime(data_forstok['Order Date'][i]).month_name()
            data_forstok['Year'][i] = pd.to_datetime(data_forstok['Order Date'][i]).year

    quarter = pd.DataFrame([['January', 1], ['February', 1], ['March', 1], ['April', 2], ['May', 2], ['June', 2], 
            ['July', 3], ['August', 3], ['September', 3],['October', 4], ['November', 4], ['December', 4]], columns = ['Bulan', 'Quarter'])
    data_forstok = data_forstok.merge(quarter, how = 'left', left_on = 'Month', right_on = 'Bulan')
    data_forstok = data_forstok.drop(['Bulan'], axis = 1)
    data_bulan = pd.DataFrame([{'Bulan' : 'December', 'Number' : 12} ,
            {'Bulan' : 'January' , 'Number': 1},
            {'Bulan' : 'February' , 'Number': 2},
            {'Bulan' : 'March' , 'Number': 3},
            {'Bulan' : 'April' , 'Number': 4},
            {'Bulan' : 'May' , 'Number': 5},
            {'Bulan' : 'June', 'Number': 6},
            {'Bulan' : 'July' , 'Number': 7},
            {'Bulan' : 'August', 'Number' : 8},
            {'Bulan' : 'September', 'Number' : 9},
            {'Bulan' : 'October' , 'Number': 10},
            {'Bulan' : 'November' , 'Number': 11}])
    temp = data_forstok.copy()
    temp['Day'] = temp['Date']
    temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
    temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
    data_forstok['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.week
    temp['Hour'] = pd.to_datetime(data_forstok['Order Date']).dt.hour
    temp['Minute'] = pd.to_datetime(data_forstok['Order Date']).dt.minute
    temp['Second'] = pd.to_datetime(data_forstok['Order Date']).dt.second
    data_forstok['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']])
    
    
    forstok_all = data_forstok
    forstok_all['Total'] = forstok_all['Sub Total']
    forstok_all['Price List NFI'] = np.nan
    forstok_all['Total Net'] = np.nan

    forstok_all = forstok_all.rename(columns={'Channel Order ID' : 'Order #',
                                            'Status' : 'Order Status',
                                            'Order Date' : 'Order date',
                                            'Item Name' :'Product Name',
                                            'Bundle Name' : 'Bundle',
                                            'Shipping Country' : 'Country',
                                            'Shipping Province' : 'Region',
                                            'Shipping City' : 'City',
                                            'Shipping Zip' : 'Zip Code',
                                            'Shipping Address1' : 'Address',
                                            'Shipping Phone' : 'Phone',
                                            'Quantity' : 'Qty. Invoiced',
                                            'Item Price' : 'Regular Price',
                                            'Sub Total' : 'Subtotal'})
    forstok_all['Kecamatan'] = np.nan
    forstok_all['Kelurahan'] = np.nan
    forstok_all['Store'] = forstok_all['Channel']

    print("Filling Location ===== 7/10")
    indeks = forstok_all[forstok_all['City'].astype(str).str.contains('/')]['City'].index.to_list()
    if len(indeks)>0:
        forstok_all['Kecamatan'][indeks] = forstok_all['City'][indeks].str.split('/', n = 1,expand = True)[1]
        forstok_all['City'][indeks] = forstok_all['City'][indeks].str.split('/', n = 1,expand = True)[0]

    indeks = forstok_all[forstok_all['Kecamatan'].astype(str).str.contains('-')]['Kecamatan'].index.to_list()
    if len(indeks)>0:
        forstok_all['Kelurahan'][indeks] = forstok_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[1]
        forstok_all['Kecamatan'][indeks] = forstok_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[0]

    indeks = forstok_all[forstok_all['City'].astype(str).str.contains(',')]['City'].index.to_list()
    if len(indeks)>0:
        forstok_all['Kecamatan'][indeks] = forstok_all['City'][indeks].str.split(',', n = 1,expand = True)[1]
        forstok_all['City'][indeks] = forstok_all['City'][indeks].str.split(',', n = 1,expand = True)[0]

    indeks = forstok_all[forstok_all['Kecamatan'].astype(str).str.contains(',')]['Kecamatan'].index.to_list()
    if len(indeks)>0:
        forstok_all['Kelurahan'][indeks] = forstok_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[1]
        forstok_all['Kecamatan'][indeks] = forstok_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[0]

    forstok_all['City'] = forstok_all['City'].astype(str).str.replace('Kab\.', 'Kabupaten' ,case = False)

    master_map = pd.read_csv(r'All Data/Province.csv', names = ['Kode Prov', 'Province'], header= 0)
    master_map2 = pd.read_csv(r'All Data/City.csv', names = ['Kode City', 'Kode Prov', 'City'], header = 0)
    master_map = master_map.merge(master_map2, how = 'right', on = 'Kode Prov')
    master_map['Kode Prov'][515] = 14
    master_map['Province'][515] = 'Riau'
    master_map['Kode Prov'] = master_map['Kode Prov'].astype(int)
    master_map['Province'] = master_map['Province'].str.title()
    master_map['City'] = master_map['City'].str.title()

    city = pd.read_excel(r'All Data/list_city.xlsx')
    temp = forstok_all.copy()
    temp['City'] = temp['City'].astype(str).str.lower()
    temp['City'] = temp['City'].astype(str).str.replace('kab. ', 'kabupaten ', regex = False, case = False)
    city['All City'] = city['All City'].astype(str).str.lower()
    temp = temp.merge(city.drop_duplicates('All City'), how = 'left', left_on = 'City', right_on = 'All City').set_index(temp.index)
    indeks = temp[temp['Real City'].notnull()].index.to_list()
    forstok_all['City'][indeks] = temp['Real City'][indeks]

    province = pd.read_excel(r'All Data/list_province.xlsx')
    temp = forstok_all.copy()
    temp['Region'] = temp['Region'].astype(str).str.lower()
    province['All Province'] = province['All Province'].astype(str).str.lower()
    temp = temp.merge(province.drop_duplicates('All Province'), how = 'left', left_on = 'Region', right_on = 'All Province').set_index(temp.index)
    indeks = temp[temp['Real Province'].notnull()].index.to_list()
    forstok_all['Region'][indeks] = temp['Real Province'][indeks]

    temp = forstok_all.copy()
    temp = temp[temp['Region'].isnull()]
    temp['Region'] = temp.merge(master_map, how = 'left', on = 'City').set_index(temp.index)['Province']
    forstok_all['Region'][temp.index] = temp['Region']  
    
    district = pd.read_excel(r'All Data/list_district.xlsx')
    temp = forstok_all.copy()
    temp['Kecamatan'] = temp['Kecamatan'].astype(str).str.lower()
    district['All District'] = district['All District'].astype(str).str.lower()
    temp = temp.merge(district.drop_duplicates('All District'), how = 'left', left_on = 'Kecamatan', right_on = 'All District').set_index(temp.index)
    indeks = temp[temp['Real District'].notnull()].index.to_list()
    forstok_all['Kecamatan'][indeks] = temp['Real District'][indeks]

    temp = forstok_all.copy()
    temp2 = temp[['Region', 'City', 'Kecamatan']].merge(master_map, how = 'left', on = 'City')
    indeks = temp2[temp2['Region'] != temp2['Province']][temp2[temp2['Region'] != temp2['Province']]['City'].notnull()].index.to_list()
    forstok_all['City'][indeks] = np.nan

    data_SKU['Real SKU'] = data_SKU['SKU'].astype(str)
    data_SKU['Real Nama Produk'] = data_SKU['Nama Produk'].astype(str)

    print("Unbundling ===== 9/10")
    data_bundle1 = forstok_all[~forstok_all['Produk 1'].isnull()]
    data_bundle1['Bundle Name'] = data_bundle1['Product Name']
    data_bundle1['Product Name'] = data_bundle1['Produk 1']
    data_bundle1['SKU'] = data_bundle1['SKU Produk 1']
    data_bundle1['Qty. Invoiced'] = data_bundle1['PCS Produk 1']
    data_bundle1['Price List NFI'] = data_bundle1['Price List NFI 1']
    data_bundle1['Total Net'] = data_bundle1['Price List NFI 1']
    data_bundle1['Bundle Flag'] = np.nan

    data_bundle2 = forstok_all[~forstok_all['Produk 2'].isnull()]
    data_bundle2['Bundle Name'] = data_bundle2['Product Name']
    data_bundle2['Product Name'] = data_bundle2['Produk 2']
    data_bundle2['SKU'] = data_bundle2['SKU Produk 2']
    data_bundle2['Qty. Invoiced'] = data_bundle2['PCS Produk 2']
    data_bundle2['Price List NFI'] = data_bundle2['Price List NFI 2']
    data_bundle2['Total Net'] = data_bundle2['Price List NFI 2'] 
    data_bundle2['Bundle Flag'] = np.nan

    data_bundle3 = forstok_all[~forstok_all['Produk 3'].isnull()]
    data_bundle3['Bundle Name'] = data_bundle3['Product Name']
    data_bundle3['Product Name'] = data_bundle3['Produk 3']
    data_bundle3['SKU'] = data_bundle3['SKU Produk 3']
    data_bundle3['Qty. Invoiced'] = data_bundle3['PCS Produk 3']
    data_bundle3['Price List NFI'] = data_bundle3['Price List NFI 3']
    data_bundle3['Total Net'] = data_bundle3['Price List NFI 3'] 
    data_bundle3['Bundle Flag'] = np.nan

    data_bundle4 = forstok_all[~forstok_all['Produk 4'].isnull()]
    data_bundle4['Bundle Name'] = data_bundle4['Product Name']
    data_bundle4['Product Name'] = data_bundle4['Produk 4']
    data_bundle4['SKU'] = data_bundle4['SKU Produk 4']
    data_bundle4['Qty. Invoiced'] = data_bundle4['PCS Produk 4']
    data_bundle4['Price List NFI'] = data_bundle4['Price List NFI 4']
    data_bundle4['Total Net'] = data_bundle4['Price List NFI 4'] 
    data_bundle4['Bundle Flag'] = np.nan

    data_bundle5 = forstok_all[~forstok_all['Produk 5'].isnull()]
    data_bundle5['Bundle Name'] = data_bundle5['Product Name']
    data_bundle5['Product Name'] = data_bundle5['Produk 5']
    data_bundle5['SKU'] = data_bundle5['SKU Produk 5']
    data_bundle5['Qty. Invoiced'] = data_bundle5['PCS Produk 5']
    data_bundle5['Price List NFI'] = data_bundle5['Price List NFI 5']
    data_bundle5['Total Net'] = data_bundle5['Price List NFI 5']
    data_bundle5['Bundle Flag'] = np.nan

    data_bundle6 = forstok_all[~forstok_all['Produk 6'].isnull()]
    data_bundle6['Bundle Name'] = data_bundle6['Product Name']
    data_bundle6['Product Name'] = data_bundle6['Produk 6']
    data_bundle6['SKU'] = data_bundle6['SKU Produk 6']
    data_bundle6['Qty. Invoiced'] = data_bundle6['PCS Produk 6']
    data_bundle6['Price List NFI'] = data_bundle6['Price List NFI 6']
    data_bundle6['Total Net'] = data_bundle6['Price List NFI 6']
    data_bundle6['Bundle Flag'] = np.nan

    data_bundle7 = forstok_all[~forstok_all['Produk 7'].isnull()]
    data_bundle7['Bundle Name'] = data_bundle7['Product Name']
    data_bundle7['Product Name'] = data_bundle7['Produk 7']
    data_bundle7['SKU'] = data_bundle7['SKU Produk 7']
    data_bundle7['Qty. Invoiced'] = data_bundle7['PCS Produk 7']
    data_bundle7['Price List NFI'] = data_bundle7['Price List NFI 7']
    data_bundle7['Total Net'] = data_bundle7['Price List NFI 7']
    data_bundle7['Bundle Flag'] = np.nan

    data_bundle = data_bundle1.append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)
    data_bundle['SKU'] = data_bundle['SKU'].astype(str)
    data_bundle['SKU'] = data_bundle['SKU'].str.replace('\.0$', '', regex = True)
    data_bundle[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']] = data_bundle.merge(data_SKU[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item_y', 'Parent SKU_y']]

    temp = data_bundle[data_bundle['Real SKU'].isnull()].copy()
    temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
    temp = temp.merge(data_SKU[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)

    indeks = data_bundle[data_bundle['Real SKU'].isnull()].index.to_list()
    data_bundle['Real SKU'][indeks] = temp['Real SKU_y'][indeks]
    data_bundle['Real Nama Produk'][indeks] = temp['Nama Produk'][indeks]
    data_bundle['Brand'][indeks] = temp['Brand_y'][indeks]
    data_bundle['Sub Brand'][indeks] = temp['Sub Brand_y'][indeks]
    data_bundle['Parent Item'][indeks] = temp['Parent Item_y'][indeks]
    data_bundle['Parent SKU'][indeks] = temp['Parent SKU_y'][indeks]

    print("Pricing ===== 10/10")
    forstok_all = forstok_all.append(data_bundle, ignore_index = True, sort = False)
    
    forstok_all = forstok_all.merge(data_SKU[['SKU', 'Price List NFI', 'Harga Cost']].drop_duplicates('SKU'), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(forstok_all.index)
    forstok_all['Price List NFI_x'] = forstok_all['Price List NFI_x'].fillna(forstok_all['Price List NFI_y'])
    forstok_all =  forstok_all.drop(['Price List NFI_y', 'SKU_y'], axis = 1)
    forstok_all = forstok_all.rename(columns = {'SKU_x' : 'SKU', 'Price List NFI_x' : 'Price List NFI'})

    forstok_all['Price List NFI'] = pd.to_numeric(forstok_all['Price List NFI']).astype(int)
    forstok_all['Harga Cost'] = pd.to_numeric(forstok_all['Harga Cost']).astype(int)
    forstok_all['Qty. Invoiced'] = pd.to_numeric(forstok_all['Qty. Invoiced']).astype(int)
    
    forstok_all['Total Net'] = forstok_all['Price List NFI'] * forstok_all['Qty. Invoiced']
    forstok_all['Total Harga Cost'] = forstok_all['Harga Cost'] * forstok_all['Qty. Invoiced']
    
    forstok_all = forstok_all.reset_index(drop = True)
    forstok_all['Order #'] = forstok_all['Order #'].astype(str).str.replace('.0', '', regex = False)
    before = os.listdir(os.getcwd() + '/Input Data')

    options = Options()
    options.add_experimental_option("prefs", {
            "download.default_directory": os.path.abspath("Input Data"),
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            "safebrowsing.enabled": False
    })

    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    driver.get("https://seller.blibli.com/sign-in")
    time.sleep(40)

    username = driver.find_element_by_id("email")
    username.clear()
    username.send_keys("customer1@nutrimart.co.id")

    password = driver.find_element_by_id("password")
    password.send_keys("Nutrimart12345")

    driver.find_element_by_id("sign-in").click()
    time.sleep(30)

    driver.get("https://seller.blibli.com/MTA/order/summary")
    time.sleep(50)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'btn-primary-mta'))).click()
    time.sleep(10)
    driver.find_element_by_id("semuaFilter").click()

    driver.find_element_by_class_name("order-filter-input").click()
    start_date = datetime.today() - timedelta(days=14)
    end_date = datetime.today()
    date = start_date.strftime('%Y-%m-%d') + '-' + end_date.strftime('%Y-%m-%d')

    if start_date.month == end_date.month:
        text = "//div[@class='calendar left']//td[@class = 'available' or @class='weekend available'][text() = " + str(start_date.day) + "]"
        driver.find_element_by_xpath(text).click()
        text = "//div[@class='calendar left']//td[contains(@class, 'today')][text() = " + str(end_date.day) + "]"
        driver.find_element_by_xpath(text).click()
    else :
        driver.find_element_by_css_selector("th.prev.available").click()
        text = "//div[@class='calendar left']//td[@class = 'available' or @class='weekend available'][text() = " + str(start_date.day) + "]"
        driver.find_element_by_xpath(text).click()
        text = "//div[@class='calendar right']//td[contains(@class, 'today')][text() = " + str(end_date.day) + "]"
        driver.find_element_by_xpath(text).click()
    driver.find_element_by_css_selector('button.btn.col-sm-3.btn-default.order-option-buttons.dropdown-toggle').click()
    driver.find_element_by_xpath("/html/body/div[3]/div[2]/div/div/div[1]/div[3]/div[2]/ul").click()
    try:
        time.sleep(2)
        driver.find_element_by_class_name("btn-primary-mta").click()
        time.sleep(2)
    except NoSuchElementException:
        pass

    cond = False
    while not cond:
        time.sleep(40)
        driver.find_element_by_class_name('fa-bell-o').click()
        print('click 1')
        time.sleep(10)
        driver.find_element_by_class_name('fa-bell-o').click()
        print('click 2')
        #time.sleep(10)
        #driver.find_element_by_class_name('fa-bell-o').click()
        #print('click 3')

        time_now = str(datetime.now().hour) + ':' + str(datetime.now().minute)
        time.sleep(10)
        time_notif = driver.find_element_by_xpath('//*[@id="viewNotification"]/div[2]/div[2]/ul/li[1]/span[2]').text
        print(time_notif)
        time_notif1 = driver.find_element_by_css_selector('li.ng-scope.notification-unread').text[-5:]
        end = datetime.strptime(time_notif, '%H:%M').time()
        print(end)
        duration = int((datetime.today() - timedelta(hours=end.hour, minutes=end.minute)).strftime('%M'))
        print(duration)
        if duration < 3:
            time.sleep(1)
            driver.find_element_by_xpath('//ul[@class = "notification-item-container"]//li[@class = "ng-scope notification-unread"]').click()
            cond = True
        else :
            driver.find_element_by_class_name('fa-bell-o').click()
            time.sleep(30)
            
    print('Blibli LMS Downloaded')     

   
    time.sleep(15)
    after = os.listdir(os.getcwd() + '/Input Data')
    change = set(after) - set(before)
    if len(change) == 1:
        file_name = change.pop()
    else:
        print("More than one file or no file downloaded")
    print(file_name)
    new_name = 'Blibli ' + str(date) + '.csv'
    if os.path.isfile('Input Data/' + str(new_name)) :
        os.remove('Input Data/' + str(new_name))
    os.rename('Input Data/' + str(file_name), 'Input Data/' + str(new_name))
    driver.quit()
    print("Import Data ====== 1/10")
    
    lmen = pd.read_csv(r'Input Data/' + str(new_name) , dayfirst = True)
    lmen = lmen.dropna(how = 'all')
    lmen_pure = lmen.copy()

    pd.options.mode.chained_assignment = None
    print("--- %s seconds ---" % (time.time() - start_time))
    print("Formatting Data ====== 2/10")
    lmen['No. Order'] = lmen['No. Order'].astype(str).str.replace('^="','')
    lmen['No. Order'] = lmen['No. Order'].astype(str).str.replace('"$','')
    lmen['No. Order Item'] = lmen['No. Order Item'].astype(str).str.replace('^="','')
    lmen['No. Order Item'] = lmen['No. Order Item'].astype(str).str.replace('"$','')
    lmen['No. Awb'] = lmen['No. Awb'].astype(str).str.replace('^="','')
    lmen['No. Awb'] = lmen['No. Awb'].astype(str).str.replace('"$','')
    lmen['Merchant SKU'] = lmen['Merchant SKU'].astype(str).str.replace('^="','')
    lmen['Merchant SKU'] = lmen['Merchant SKU'].astype(str).str.replace('"$','')
    
    #INPUT SKU BLIBLI LMS ERROR
    lmen.loc[lmen['Nama Produk']=='Daily Protein Package - L-Men Daily Popcorn Caramel 250g & L-Men Protein Bar Crunchy Chocolate 12 Sachet','Merchant SKU']='PL19L23'
    lmen.loc[lmen['Nama Produk']=='[Tebus Murah] L-Men Lose Weight Mango Sticky Rice 300g - Suplemen Penurun Berat Badan Tinggi Whey Protein ','Merchant SKU']='2304576112'
    for i in lmen[lmen['Merchant SKU']=='nan']['Blibli SKU'].unique():
        print(i)
        skunfi=lmen[(lmen['Blibli SKU']==i)&(lmen['Merchant SKU']!='nan')].reset_index()['Merchant SKU'][0]
        lmen.loc[(lmen['Blibli SKU']==i)&(lmen['Merchant SKU']=='nan'),'Merchant SKU']=skunfi

    print("--- %s seconds ---" % (time.time() - start_time))
    print("Listing SKU ====== 3/10")
    lmen_sku = pd.read_excel(r'SKU_File/Converter Paket Blibli.xlsx')

    lmen['Merchant SKU'] = lmen['Merchant SKU'].astype(str).str.replace('\.0$','', regex = True)

    null_sku = lmen[lmen['Merchant SKU'] == 'nan'].copy()
    lmen = lmen[lmen['Merchant SKU'] != 'nan']

    null_sku['Nama Produk'] = null_sku['Nama Produk'].astype(str).str.strip()
    lmen_sku['Nama Produk'] = lmen_sku['Nama Produk'].astype(str).str.strip()
    null_sku['Merchant SKU'] = null_sku.merge(lmen_sku[['SKU Merchant', 'Nama Produk']].drop_duplicates(['Nama Produk']), how = 'left', on = 'Nama Produk').set_index(null_sku.index)['SKU Merchant']

    lmen = lmen.append(null_sku, ignore_index = True, sort = False)

    gaada_sku = lmen[~lmen['Merchant SKU'].astype(str).isin(data_SKU['SKU'].astype(str))].copy()
    lmen = lmen[lmen['Merchant SKU'].astype(str).isin(data_SKU['SKU'].astype(str))]

    gaada_sku['Nama Produk'] = gaada_sku['Nama Produk'].astype(str).str.strip()
    lmen_sku['Nama Produk'] = lmen_sku['Nama Produk'].astype(str).str.strip()
    gaada_sku['Merchant SKU'] = gaada_sku.merge(lmen_sku[['SKU Merchant', 'Nama Produk']].drop_duplicates(['Nama Produk']), how = 'left', on = 'Nama Produk').set_index(gaada_sku.index)['SKU Merchant']
    lmen = lmen.append(gaada_sku, ignore_index = True, sort = False)

    indeks = []
    indeks = indeks + lmen[lmen['Merchant SKU'].isnull()].index.to_list()
    indeks = indeks + lmen[~lmen['Merchant SKU'].astype(str).str.replace('\.0$','', regex = True).isin(data_SKU['SKU'].astype(str))].index.to_list()

    if len(indeks) != 0 :
        alert = lmen.iloc[indeks][['Blibli SKU','Merchant SKU', 'Nama Produk']].drop_duplicates()
        alert['SKU Valid'] = np.nan
        to_excel = alert.to_excel('ALERT_LMEN-STORE_SKU_MISSING.xlsx')
        print("Some SKU Missing Please Complete It ====== 4/10")
        print("--- %s seconds ---" % (time.time() - start_time))
    else :
        data_SKU['Real Nama Produk'] = data_SKU['Nama Produk'].astype(str)
        data_SKU['Real SKU'] = data_SKU['SKU'].astype(str)
        lmen['Merchant SKU'] = lmen['Merchant SKU'].astype(str)
        data_SKU['Price List NFI'] = data_SKU['Price List NFI'].astype(float).astype('int64')
        lmen = lmen.merge(data_SKU[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU', 'Price List NFI']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'Merchant SKU', right_on = 'Real SKU')
        lmen['Total Net'] = lmen['Price List NFI'] * lmen['Total Barang']

        indeks = lmen[lmen['Brand'] == 'Bundle'].index.to_list()
        lmen['Bundle Flag'] = np.nan
        lmen['Bundle Flag'][indeks] = 'Bundle'

        lmen['Date'] = pd.to_datetime(lmen['Tanggal Order'], format = '%d/%m/%Y %H:%M').dt.day
        lmen['Month'] = pd.to_datetime(lmen['Tanggal Order'], format = '%d/%m/%Y %H:%M').dt.month_name()
        lmen['Year'] = pd.to_datetime(lmen['Tanggal Order'], format = '%d/%m/%Y %H:%M').dt.year

        quarter = pd.DataFrame([['January', 1], ['February', 1], ['March', 1], ['April', 2], ['May', 2], ['June', 2], 
                    ['July', 3], ['August', 3], ['September', 3],['October', 4], ['November', 4], ['December', 4]], columns = ['Bulan', 'Quarter'])
        lmen = lmen.merge(quarter, how = 'left', left_on = 'Month', right_on = 'Bulan')
        lmen = lmen.drop(['Bulan'], axis = 1)
        data_bulan = pd.DataFrame([{'Bulan' : 'December', 'Number' : 12} ,
                {'Bulan' : 'January' , 'Number': 1},
                {'Bulan' : 'February' , 'Number': 2},
                {'Bulan' : 'March' , 'Number': 3},
                {'Bulan' : 'April' , 'Number': 4},
                {'Bulan' : 'May' , 'Number': 5},
                {'Bulan' : 'June', 'Number': 6},
                {'Bulan' : 'July' , 'Number': 7},
                {'Bulan' : 'August', 'Number' : 8},
                {'Bulan' : 'September', 'Number' : 9},
                {'Bulan' : 'October' , 'Number': 10},
                {'Bulan' : 'November' , 'Number': 11}])
        temp = lmen.copy()
        temp['Day'] = temp['Date']
        temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
        temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
        lmen['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.week
        lmen['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day']])

        lmen = lmen.rename(columns = {'No. Order' : 'Order #', 'No. Order Item' : 'No. Order Item L-Men Blibli',
                                    'Tanggal Order' : 'Order date', 'Nama Pemesan' : 'Customer Name',
                                    'Merchant SKU' : 'SKU', 'Nama Produk' : 'Product Name', 'Total Barang':'Qty. Invoiced',
                                    'Harga Produk' : 'Selling Price', 'Servis Logistik' : 'Shipping Courier', 'Nama Store' : 'Channel',
                                    'Toko/Gudang' : 'Warehouse Name', 'No. Awb' : 'AWB'
                                    })

        lmen['Channel'] = 'L-Men Store Blibli'
        lmen['Store'] = 'Blibli'

        data_SKU['Real SKU'] = data_SKU['SKU'].astype(str)
        data_SKU['Real Nama Produk'] = data_SKU['Nama Produk'].astype(str)

        lmen['SKU'] = lmen['SKU'].astype(str)
        list_col = ['SKU'] + data_SKU.columns[data_SKU.columns.get_loc('Produk 1'):data_SKU.columns.get_loc('Harga Cost 7')+1].to_list()
        lmen = lmen.merge(data_SKU[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        lmen = lmen.drop(['SKU_y'], axis = 1)
        lmen = lmen.rename(columns = {'SKU_x':'SKU'})
        data_bundle1 = lmen[~lmen['Produk 1'].isnull()]

        data_bundle1['Bundle Name'] = data_bundle1['Product Name']
        data_bundle1['Product Name'] = data_bundle1['Produk 1']
        data_bundle1['SKU'] = data_bundle1['SKU Produk 1']
        data_bundle1['Qty. Invoiced'] = data_bundle1['PCS Produk 1'] * data_bundle1['Qty. Invoiced']
        data_bundle1['Price List NFI'] = data_bundle1['Price List NFI 1']
        data_bundle1['Total Net'] = data_bundle1['Price List NFI 1'] * data_bundle1['Qty. Invoiced']
        data_bundle1['Bundle Flag'] = np.nan

        data_bundle2 = lmen[~lmen['Produk 2'].isnull()]
        data_bundle2['Bundle Name'] = data_bundle2['Product Name']
        data_bundle2['Product Name'] = data_bundle2['Produk 2']
        data_bundle2['SKU'] = data_bundle2['SKU Produk 2']
        data_bundle2['Qty. Invoiced'] = data_bundle2['PCS Produk 2'] * data_bundle2['Qty. Invoiced']
        data_bundle2['Price List NFI'] = data_bundle2['Price List NFI 2']
        data_bundle2['Total Net'] = data_bundle2['Price List NFI 2'] * data_bundle2['Qty. Invoiced']
        data_bundle2['Bundle Flag'] = np.nan

        data_bundle3 = lmen[~lmen['Produk 3'].isnull()]
        data_bundle3['Bundle Name'] = data_bundle3['Product Name']
        data_bundle3['Product Name'] = data_bundle3['Produk 3']
        data_bundle3['SKU'] = data_bundle3['SKU Produk 3']
        data_bundle3['Qty. Invoiced'] = data_bundle3['PCS Produk 3']* data_bundle3['Qty. Invoiced']
        data_bundle3['Price List NFI'] = data_bundle3['Price List NFI 3']
        data_bundle3['Total Net'] = data_bundle3['Price List NFI 3'] * data_bundle3['Qty. Invoiced']
        data_bundle3['Bundle Flag'] = np.nan

        data_bundle4 = lmen[~lmen['Produk 4'].isnull()]
        data_bundle4['Bundle Name'] = data_bundle4['Product Name']
        data_bundle4['Product Name'] = data_bundle4['Produk 4']
        data_bundle4['SKU'] = data_bundle4['SKU Produk 4']
        data_bundle4['Qty. Invoiced'] = data_bundle4['PCS Produk 4']* data_bundle4['Qty. Invoiced']
        data_bundle4['Price List NFI'] = data_bundle4['Price List NFI 4']
        data_bundle4['Total Net'] = data_bundle4['Price List NFI 4'] * data_bundle4['Qty. Invoiced']
        data_bundle4['Bundle Flag'] = np.nan

        data_bundle5 = lmen[~lmen['Produk 5'].isnull()]
        data_bundle5['Bundle Name'] = data_bundle5['Product Name']
        data_bundle5['Product Name'] = data_bundle5['Produk 5']
        data_bundle5['SKU'] = data_bundle5['SKU Produk 5']
        data_bundle5['Qty. Invoiced'] = data_bundle5['PCS Produk 5']* data_bundle5['Qty. Invoiced']
        data_bundle5['Price List NFI'] = data_bundle5['Price List NFI 5']
        data_bundle5['Total Net'] = data_bundle5['Price List NFI 5'] * data_bundle5['Qty. Invoiced']
        data_bundle5['Bundle Flag'] = np.nan

        data_bundle6 = lmen[~lmen['Produk 6'].isnull()]
        data_bundle6['Bundle Name'] = data_bundle6['Product Name']
        data_bundle6['Product Name'] = data_bundle6['Produk 6']
        data_bundle6['SKU'] = data_bundle6['SKU Produk 6']
        data_bundle6['Qty. Invoiced'] = data_bundle6['PCS Produk 6']* data_bundle6['Qty. Invoiced']
        data_bundle6['Price List NFI'] = data_bundle6['Price List NFI 6']
        data_bundle6['Total Net'] = data_bundle6['Price List NFI 6'] * data_bundle6['Qty. Invoiced']
        data_bundle6['Bundle Flag'] = np.nan

        data_bundle7 = lmen[~lmen['Produk 7'].isnull()]
        data_bundle7['Bundle Name'] = data_bundle7['Product Name']
        data_bundle7['Product Name'] = data_bundle7['Produk 7']
        data_bundle7['SKU'] = data_bundle7['SKU Produk 7']
        data_bundle7['Qty. Invoiced'] = data_bundle7['PCS Produk 7']* data_bundle7['Qty. Invoiced']
        data_bundle7['Price List NFI'] = data_bundle7['Price List NFI 7']
        data_bundle7['Total Net'] = data_bundle7['Price List NFI 7'] * data_bundle7['Qty. Invoiced']
        data_bundle7['Bundle Flag'] = np.nan

        data_bundle = data_bundle1.append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)
        data_bundle['SKU'] = data_bundle['SKU'].astype(str)
        data_bundle['SKU'] = data_bundle['SKU'].str.replace('\.0$', '', regex = True)
        data_bundle[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']] = data_bundle.merge(data_SKU[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item_y', 'Parent SKU_y']]

        lmen = lmen.append(data_bundle, ignore_index = True, sort = False)
        lmen = lmen.merge(data_SKU[['Real SKU', 'Harga Cost']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')
        lmen['Total Harga Cost'] = pd.to_numeric(lmen['Harga Cost'], errors = 'coerce') * lmen['Qty. Invoiced']
        print(lmen[lmen['Harga Cost'] == 0][['SKU','Product Name']].drop_duplicates())
        lmen['Total'] = lmen['Selling Price'] * lmen['Qty. Invoiced']
        lmen['Subtotal'] = lmen['Selling Price'] * lmen['Qty. Invoiced']
        lmen = lmen.rename(columns = {'Real SKU_x' : 'Real SKU'})

        forstok_all = forstok_all.append(lmen, ignore_index = True, sort = False)

        list_bundle = forstok_all[forstok_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Subtotal', 'Total']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = forstok_all[forstok_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        forstok_all['Total'][list_nobundle.index] = list_nobundle['Total_y']
        forstok_all['Subtotal'][list_nobundle.index] = list_nobundle['Subtotal_y']

        temp = forstok_all[forstok_all['Bundle Name'].notnull()]
        temp['Subtotal'] = temp['Subtotal'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        temp['Selling Price'] = temp['Subtotal']/temp['Qty. Invoiced']
        temp['Total'] = temp['Total'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        forstok_all['Total'][temp.index] = temp['Total']
        forstok_all['Subtotal'][temp.index] = temp['Subtotal']
        forstok_all['Selling Price'][temp.index] = temp['Selling Price']
        

        forstok_all['Order #'] = forstok_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        list_bundle = forstok_all[forstok_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Seller Discount']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = forstok_all[forstok_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        forstok_all['Seller Discount'][list_nobundle.index] = list_nobundle['Seller Discount_y']
        temp = forstok_all[forstok_all['Bundle Name'].notnull()]
        temp['Seller Discount'] = temp['Seller Discount'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        forstok_all['Seller Discount'][temp.index] = temp['Seller Discount']


        temp = forstok_all[forstok_all['Bundle Name'].isnull()]
        temp_group = temp[['Order #','Shipping']].groupby(['Order #']).sum().reset_index()

        temp = forstok_all.merge(temp_group, how = 'left', on = 'Order #').set_index(forstok_all.index)
        temp['Shipping_x'] = temp['Shipping_y'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        forstok_all['Shipping'][temp.index] = temp['Shipping_y']
        list_bundle = forstok_all[forstok_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Shipping']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = forstok_all[forstok_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        forstok_all['Shipping'][list_nobundle.index] = list_nobundle['Shipping_y']
        temp = forstok_all[forstok_all['Bundle Name'].notnull()]
        temp['Shipping'] = temp['Shipping'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        forstok_all['Shipping'][temp.index] = temp['Shipping']
        forstok_all['True datetime'] = pd.to_datetime(forstok_all['True datetime'])
        forstok_all['Promo'] = np.nan
        forstok_all['Discount MC'] = np.nan


print('DONE')

Input Forstok Date
2022-11-19-2022-11-29
Download Data SKU
Waiting to Download Forstok
https://forstok-staging-storage.s3.ap-southeast-1.amazonaws.com/items_import/nutrifood--forstok-sales_orders-version_1-2022-11-19-2022-11-29.xls
Waiting to Download Forstok
200
Import Data ====== 1/10
--- 9.328515529632568 seconds ---
Formatting Data ====== 2/10


C:\Users\steven.nathanael\Anaconda3\lib\site-packages\pandas\core\arraylike.py:48: UserWarning: Parsing '21-12-2021' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return self._cmp_method(other, operator.lt)


--- 14.009727954864502 seconds ---
Fulfilling SKU ====== 3/10
--- 59.205955505371094 seconds ---
Listing SKU Missing ====== 4/10
--- 114.94974255561829 seconds ---
Filling Brand ====== 5/10
--- 115.28125739097595 seconds ---
Unbundling ====== 6/10
--- 115.77750515937805 seconds ---
Filling Date ====== 7/10
Filling Location ===== 7/10
Unbundling ===== 9/10
Pricing ===== 10/10
click 1
click 2
10:44
10:44:00
1
Blibli LMS Downloaded
bulk-order-download-template.csv
Import Data ====== 1/10
--- 383.24010491371155 seconds ---
Formatting Data ====== 2/10
LMS-60022-00238-00001
--- 383.49089908599854 seconds ---
Listing SKU ====== 3/10
               SKU                                 Product Name
471    (B)71210147                          (FREE) Masker L-Men
686  (B)2309005305  (PBS) L-Men Protein Crunch BBQ Beef 20BX20G
DONE


In [13]:
print('Done')

Done


In [14]:
idx

[]

SyntaxError: invalid syntax (3948359156.py, line 1)

In [7]:
skushopee[['Item Name', 'SKU']].drop_duplicates().values
### ini harus null juga, jgn ada item masuk pake sku shopee (S)

array([], shape=(0, 2), dtype=object)

In [8]:
### opsional , if errror
### cek sku missing (belum ada di tatanama)
### to do: regist single/bundle/alias 

data_forstok[data_forstok['SKU'].isnull()]['Item Name'].unique()
data_forstok[~data_forstok['SKU'].astype(str).isin(data_SKU['SKU'].astype(str))][['Store','Item Name','SKU']].drop_duplicates().values#.unique()

array([], shape=(0, 3), dtype=object)

In [10]:
## Run-4
## scrap magento detail order blibli update order status
## kadang auto logout di blibli -> solusi: run partial / add code


magento_scrape = False
magentoUser_scrape = False

from datetime import datetime

if not magento_scrape: 
    print("Starting Magento")

    # Download the file using Selenium here

    options = Options()
    options.add_experimental_option("prefs", {
            "download.default_directory": os.path.abspath("Input Data"),
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            "safebrowsing.enabled": False
    })

    print('Opening Chrome')
    driver = webdriver.Chrome("chromedriver.exe",options=options)
    driver.get("https://nutrimart.co.id/backoffice")
    print('Login Magento')
    username = driver.find_element_by_id("username")
    username.clear()
    username.send_keys("andra")
    password = driver.find_element_by_id("login")
    password.send_keys("andra123")

    print('Go To Sales Form')
    time.sleep(5)
    before = os.listdir(os.getcwd() + '/Input Data')
    driver.find_element_by_xpath("//button[@class = 'action-login action-primary']").click()
    time.sleep(5)
    WebDriverWait(driver, 300).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="menu-magento-sales-sales"]'))).click()
    time.sleep(5)
    driver.find_element_by_xpath('//*[@id="menu-magento-sales-sales"]/div/ul/li[2]/ul/li[2]/div/ul/li[1]/a').click()
    driver.find_element_by_xpath('//*[@id="profile_id"]').click()
    driver.find_element_by_xpath('//*[@id="profile_id"]/optgroup[2]/option[2]').click()
    

    start_date = datetime.today() - timedelta(days=12)
    print(start_date)
    end_date = datetime.today()
    print(end_date)
    date = start_date.strftime('%d %b %Y') + '-' + end_date.strftime('%d %b %Y')

    print('Input Date')
    date_from = driver.find_element_by_name('daterange_from')
    date_from.clear()
    date_from.send_keys(start_date.strftime('%m/%d/%Y'))

    date_to = driver.find_element_by_name('daterange_to')
    date_to.clear()
    date_to.send_keys(end_date.strftime('%m/%d/%Y'))
    print('Download Magento')
    driver.find_element_by_xpath('//*[@id="export_button"]').click()

    time.sleep(120)
    after = os.listdir(os.getcwd() + '/Input Data')
    change = set(after) - set(before)
    if len(change) == 1:
        magento_name = change.pop()
    else:
        print("More than one file or no file downloaded")
    
    data_magento = pd.read_csv('Input Data/' + magento_name, sep = ',', index_col = False)
    magento_scrape = True
else :
    data_magento = pd.read_csv('Input Data/' + magento_name, sep = ',', index_col = False)
    
if not magentoUser_scrape:
    before = os.listdir(os.getcwd() + '/Input Data')

    driver.find_element_by_xpath('//*[@id="profile_id"]').click()
    driver.find_element_by_xpath('//*[@id="profile_id"]/optgroup[2]/option[1]').click()

    start_date = datetime.today() - timedelta(days=15)
    end_date = datetime.today()
    date = start_date.strftime('%d %b %Y') + '-' + end_date.strftime('%d %b %Y')

    date_from = driver.find_element_by_name('daterange_from')
    date_from.clear()
    date_from.send_keys(start_date.strftime('%m/%d/%Y'))
    date_to = driver.find_element_by_name('daterange_to')
    date_to.clear()
    date_to.send_keys(end_date.strftime('%m/%d/%Y'))

    driver.find_element_by_xpath('//*[@id="export_button"]').click()
    time.sleep(60)
    after = os.listdir(os.getcwd() + '/Input Data')
    change = set(after) - set(before)
    if len(change) == 1:
        magentouser_name = change.pop()
        print(file_name)
    else:
        print("More than one file or no file downloaded")
        print(change)
    driver.quit()

    data_magentoUser2 = pd.read_csv('Input Data/' + magentouser_name, sep = ',', index_col = False)
    for i in range(data_magentoUser2.shape[1]):
        data_magentoUser2 = data_magentoUser2.rename(columns={ data_magentoUser2.columns[i] : data_magentoUser2.columns[i].replace("    "," ")})
    magentoUser_scrape = True
else :
    data_magentoUser2 = pd.read_csv('Input Data/' + magentouser_name, sep = ',', index_col = False)
    for i in range(data_magentoUser2.shape[1]):
        data_magentoUser2 = data_magentoUser2.rename(columns={ data_magentoUser2.columns[i] : data_magentoUser2.columns[i].replace("    "," ")})

data_SKU = pd.read_excel(r"SKU_File\data_SKU.xlsx")

s = requests.Session()
s.get("https://tatanama.pythonanywhere.com")
s.post("https://tatanama.pythonanywhere.com", data = {'username' : 'ecommerce', 'password' : 'ecommerce'})
r = s.get("https://tatanama.pythonanywhere.com/download")

with open(r"SKU_File\Master tatanama.xlsx", 'wb') as output:
    output.write(r.content)

if os.path.isfile(r"SKU_File\Master tatanama.xlsx") :    
    SKU_append = pd.read_excel(r"SKU_File\Master tatanama.xlsx")
    SKU_append.columns = [x.replace('_', ' ') for x in SKU_append.columns]
    data_SKU = data_SKU[~data_SKU['SKU'].astype(str).isin(SKU_append['SKU'].astype(str))]
    data_SKU = data_SKU.append(SKU_append, ignore_index = True, sort = False)

to_excel = data_SKU.to_excel(r"SKU_File\data_SKU.xlsx", index = False)


# Import library
import time
import pandas as pd
import numpy as np
import datetime
import math
import os

# Import data
start_time = time.time()
print("Import Data ====== 1/10")

data_magentoUser = pd.read_excel('All Data\data_magentoUser_2020.xlsx', index_col = False)
data_magentoUser = data_magentoUser[~data_magentoUser['Sales Order Id'].astype(str).isin(data_magentoUser2['Sales Order Id'].astype(str))]
data_magentoUser = data_magentoUser.append(data_magentoUser2, ignore_index = True, sort = False)
magento_pure = data_magento.copy()

list_skumiss = []

# Formatting Magento
print("--- %s seconds ---" % (time.time() - start_time))
print("Formatting Data ====== 2/10")

pd.options.mode.chained_assignment = None
if set(['Qty. Ordered"', 'Qty. Shipped"']).issubset(data_magento.columns):
    data_magento = data_magento.rename(columns={'Qty. Ordered"' : 'Qty. Ordered', 
                                            'Qty. Shipped"' : 'Qty. Shipped'})
if "Manufacturer" in data_magento.columns:
    data_magento= data_magento.drop(["Manufacturer"], axis=1)
if "Item Cost" in data_magento.columns:
    data_magento= data_magento.drop(["Item Cost"], axis=1)
if "Package Name" in data_magento.columns:
    data_magento= data_magento.drop(["Package Name"], axis=1)
if "Tax Refunded" in data_magento.columns:
    data_magento= data_magento.drop(["Tax Refunded"], axis=1)
if "Catalog Name Rule" in data_magento.columns:
    data_magento= data_magento.drop(["Catalog Name Rule"], axis=1)
if "Order date" in data_magento.columns:
    data_magento["Order date"] = pd.to_datetime(data_magento["Order date"], errors = 'coerce')
if "Customer Email" in data_magento.columns:
    data_magento["Customer Email"] = data_magento["Customer Email"].replace(regex = r'"', value="")
if "Qty. Ordered" in data_magento.columns:
    data_magento['Qty. Ordered'] = data_magento['Qty. Ordered'].fillna(0)
    if "Qty. Invoiced" in data_magento.columns:
        data_magento['Qty. Invoiced'] = data_magento['Qty. Invoiced'].fillna(0)
    if "Qty. Shipped" in data_magento.columns:
        data_magento['Qty. Shipped'] = data_magento['Qty. Shipped'].fillna(0)
    if "Qty. Refunded" in data_magento.columns:
        data_magento['Qty. Refunded'] = data_magento['Qty. Refunded'].fillna(0)
    if "Item Price" in data_magento.columns:
        data_magento['Item Price'] = data_magento['Item Price'].fillna(0)
    if "Subtotal" in data_magento.columns:
        data_magento['Subtotal'] = data_magento['Subtotal'].fillna(0)
    if "Discounts" in data_magento.columns:
        data_magento['Discounts'] = data_magento['Discounts'].fillna(0)
    if "Invoiced" in data_magento.columns:
        data_magento['Invoiced'] = data_magento['Invoiced'].fillna(0)
    if "Tax Invoiced" in data_magento.columns:
        data_magento['Tax Invoiced'] = data_magento['Tax Invoiced'].fillna(0)
    if "Voucher Amount" in data_magento.columns:
        data_magento[data_magento['Voucher Amount'].astype(str).str.isdigit()]['Voucher Amount'] = data_magento[data_magento['Voucher Amount'].fillna(0).astype(str).str.isdigit()]['Voucher Amount'].astype(float).div(10000)
    if "Discount Poin Reward" in data_magento.columns:
        data_magento['Discount Poin Reward'] = data_magento['Discount Poin Reward'].fillna(0)
    if "Discount Product" in data_magento.columns:
        data_magento['Discount Product'] = data_magento['Discount Product'].fillna(0)
if "Tax" in data_magento.columns:
    data_magento['Tax'] = data_magento['Tax'].astype(np.float32)
if "Total" in data_magento.columns:
    data_magento['Total'] = data_magento['Total'].astype(np.float32)
if "Total incl. Tax" in data_magento.columns:
    data_magento['Total incl. Tax'] = pd.to_numeric(data_magento['Total incl. Tax'], errors = 'coerce').astype(np.float32)
if "Invoiced incl. Tax" in data_magento.columns:
    data_magento['Invoiced incl. Tax'] = data_magento['Invoiced incl. Tax'].astype(np.float32)
if "Refunded" in data_magento.columns:
    data_magento['Refunded'] = data_magento['Refunded'].astype(np.float32)
if "Refunded incl. Tax" in data_magento.columns:
    data_magento['Refunded incl. Tax'] = data_magento['Refunded incl. Tax'].astype(np.float32)
if "Ship Date" in data_magento.columns:
    data_magento['Ship Date'] = data_magento['Ship Date'].astype(str).str.replace('false', '')

# Customer Information Filling  
for i in range(data_magentoUser.shape[1]):
    data_magentoUser = data_magentoUser.rename(columns={ data_magentoUser.columns[i] : data_magentoUser.columns[i].replace("    "," ")})

for i in range(data_magento.shape[0]):
    if str(data_magento["Customer Group"][i]) == 'nan':
        if data_magento["Order #"][i] in data_magentoUser["Sales Order Id"].values:
            data_magento["Customer Name"][i] = data_magentoUser["Shipping Name"].loc[data_magento["Order #"][i]==data_magentoUser["Sales Order Id"]].values[0]
            data_magento["Customer Group"][i] = "Not Logged In"
            data_magento["Country"][i] = data_magentoUser["Shipping Country"].loc[data_magento["Order #"][i]==data_magentoUser["Sales Order Id"]].values[0]
            data_magento["Region"][i] = data_magentoUser["Shipping Province"].loc[data_magento["Order #"][i]==data_magentoUser["Sales Order Id"]].values[0]
            data_magento["City"][i] = data_magentoUser["Shipping City"].loc[data_magento["Order #"][i]==data_magentoUser["Sales Order Id"]].values[0]
            data_magento["Zip Code"][i] = data_magentoUser["Shipping Zip"].loc[data_magento["Order #"][i]==data_magentoUser["Sales Order Id"]].values[0]
            data_magento["Address"][i] = data_magentoUser["Shipping Address1"].loc[data_magento["Order #"][i]==data_magentoUser["Sales Order Id"]].values[0]
            data_magento["Phone"][i] = data_magentoUser["Shipping Phone"].loc[data_magento["Order #"][i]==data_magentoUser["Sales Order Id"]].values[0]

indeks = data_magento[data_magento['Phone'].isnull()].index.to_list()
data_magento['Phone Condition'] = np.nan
data_magento['Phone Condition'][indeks] = 'X'

data_magentoUser['Sales Order Id'] = data_magentoUser['Sales Order Id'].astype(str)
data_magento['Order #'] = data_magento['Order #'].astype(str)

temp2 = data_magento.merge(data_magentoUser[['Sales Order Id', 'AWB', 'Shipping Cost', 'Shipping Address1', 'Shipping City', 'Shipping Province', 'Shipping Phone', 'Shipping Courier']].drop_duplicates('Sales Order Id'), how = 'left', left_on = 'Order #', right_on = 'Sales Order Id').set_index(data_magento.index)

data_magento['Shipping'] = np.nan
data_magento['AWB'] = np.nan
data_magento['Shipping Courier'] = np.nan

data_magento['Shipping'][temp2.index] = temp2['Shipping Cost']
data_magento['AWB'][temp2.index] = temp2['AWB']
data_magento['Shipping Courier'][temp2.index] = temp2['Shipping Courier']


# Phone formatting
data_magento["Phone"] = data_magento["Phone"].fillna(0.0)
data_magento["Phone"] = data_magento["Phone"].astype(str).str.replace(" ","")
data_magento["Phone"] = data_magento["Phone"].astype(str).str.replace("-","")
temp = data_magento["Phone"].astype(str).str.split(",", expand = True)
data_magento["Phone"] = temp[0]
temp = data_magento["Phone"].astype(str).str.split("/", expand = True)
data_magento["Phone"] = temp[0]
data_magento["Phone"] = data_magento["Phone"].astype(str).str.replace("^\+620", "0", regex = True)
data_magento["Phone"] = data_magento["Phone"].astype(str).str.replace("^\+62", "0")
data_magento["Phone"] = data_magento["Phone"].astype(str).str.replace("^620", "0", regex = True)
data_magento["Phone"] = data_magento["Phone"].astype(str).str.replace("^62", "0", regex = True)
data_magento["Phone"] = data_magento["Phone"].astype(str).str.replace("^8", "08", regex = True)
data_magento["Phone"] = data_magento["Phone"].astype(str).str.replace("^62", "0", regex = True)
data_magento["Phone"] = data_magento["Phone"].astype(str).str.replace("(021)", "021")
data_magento["Phone"] = data_magento["Phone"].astype(str).str.replace("^21", "021", regex = True)


data_magento['Shipping Address1'] = np.nan
data_magento['Shipping City'] = np.nan
data_magento['Shipping Province'] = np.nan
data_magento['Shipping Phone'] = np.nan


data_magento["Shipping Phone"] = data_magento["Shipping Phone"].fillna(0.0)
data_magento["Shipping Phone"] = data_magento["Shipping Phone"].astype(str).str.replace(" ","")
data_magento["Shipping Phone"] = data_magento["Shipping Phone"].astype(str).str.replace("-","")
temp = data_magento["Shipping Phone"].astype(str).str.split(",", expand = True)
data_magento["Shipping Phone"] = temp[0]
temp = data_magento["Shipping Phone"].astype(str).str.split("/", expand = True)
data_magento["Shipping Phone"] = temp[0]
data_magento["Shipping Phone"] = data_magento["Shipping Phone"].astype(str).str.replace("^\+620", "0", regex = True)
data_magento["Shipping Phone"] = data_magento["Shipping Phone"].astype(str).str.replace("^\+62", "0")
data_magento["Shipping Phone"] = data_magento["Shipping Phone"].astype(str).str.replace("^620", "0", regex = True)
data_magento["Shipping Phone"] = data_magento["Shipping Phone"].astype(str).str.replace("^62", "0", regex = True)
data_magento["Shipping Phone"] = data_magento["Shipping Phone"].astype(str).str.replace("^8", "08", regex = True)
data_magento["Shipping Phone"] = data_magento["Shipping Phone"].astype(str).str.replace("^62", "0", regex = True)
data_magento["Shipping Phone"] = data_magento["Shipping Phone"].astype(str).str.replace("(021)", "021")
data_magento["Shipping Phone"] = data_magento["Shipping Phone"].astype(str).str.replace("^21", "021", regex = True)

# Master tatanama
print("--- %s seconds ---" % (time.time() - start_time))
print("Fulfilling SKU ====== 3/10")

data_magento['SKU'] = data_magento['SKU'].astype(str).str.split('-').str[0]
indeks = data_magento[~data_magento['SKU'].astype(str).isin(data_SKU['SKU'].astype(str))]['SKU'].index.to_list()

skuhd = data_magento[data_magento['SKU'].astype(str).str.contains('hd', case = False)]
skushopee = data_magento[data_magento['SKU'].astype(str).str.contains('(S)',regex = False)]
data_magento = data_magento[~data_magento['SKU'].astype(str).isin(skushopee['SKU'].astype(str))]
data_magento = data_magento[~data_magento['SKU'].astype(str).isin(skuhd['SKU'].astype(str))]

skuhd = skuhd.reset_index(drop = True)
skushopee = skushopee.reset_index(drop = True)
data_magento = data_magento.reset_index(drop = True)

indeks = data_magento[~data_magento['SKU'].astype(str).isin(data_SKU['SKU'].astype(str))]['SKU'].index.to_list()
for i in indeks:
    if data_magento['Product Name'][i].lower() in data_SKU['Nama Produk'].str.lower().values:
        data_magento['SKU'][i] = data_SKU['SKU'].loc[data_SKU['Nama Produk'].str.lower() == str(data_magento['Product Name'][i]).lower()].values[0]

list_alias = []
list_alias_name = []
for colname in data_SKU.columns:
    if 'Alias SKU' in colname:
        list_alias.append(colname)
    if 'Alias Nama' in colname:
        list_alias_name.append(colname)

for i in indeks:
    for j in list_alias:
        if str(data_magento['SKU'][i]) in data_SKU[j].astype(str).values:
            idx = data_SKU[str(data_magento['SKU'][i]) == data_SKU[j].astype(str)].index.to_list()
            for k in idx:
                if str(data_magento['SKU'][i]) == data_SKU[j.replace('SKU', 'Nama')][k]:
                    data_magento['SKU'][i] = data_SKU['SKU'][k]

indeks = data_magento[~data_magento['SKU'].astype(str).isin(data_SKU['SKU'].astype(str))]['SKU'].index.to_list()

for i in indeks:
    for j in list_alias_name:
        if str(data_magento['Product Name'][i]).lower() in data_SKU[j].astype(str).str.lower().values:
            data_magento['SKU'][i] = data_SKU['SKU'].loc[str(data_magento['Product Name'][i]).lower() == data_SKU[j].astype(str).str.lower()].values[0]

data_magento = data_magento[data_magento['Product Name'].astype(str) != 'WRP Everyday Low Fat Milk Coklat 4 Pillow Bag']
data_magento = data_magento[data_magento['Product Name'].astype(str) != 'Heavenly Kitchen Vietnamese Coffee Latte (12 Sch)']
data_magento = data_magento[data_magento['Product Name'].astype(str) != 'Heavenly Blush Yo Raspberry Pumpkin (24 pcs)']
data_magento = data_magento[data_magento['Product Name'].astype(str) != 'Heavenly Blush Yo Raspberry Pumpkin (1 pc)']
data_magento = data_magento[data_magento['Product Name'].astype(str) != 'Heavenly Kitchen Thai Tea Latte (12 Sch)']
data_magento = data_magento[data_magento['Product Name'].astype(str) != 'Heavenly Blush Greek Yogurt Blueberry (24 Pcs)']
data_magento = data_magento[data_magento['Product Name'].astype(str) != 'Heavenly Blush Greek Yogurt Blueberry']
data_magento = data_magento[data_magento['Product Name'].astype(str) != 'Heavenly Blush Yo Lychee Spinach (24 Pcs)']
data_magento = data_magento[data_magento['Product Name'].astype(str) != 'Heavenly Blush Yo Lychee Spinach (1 pc)']



data_magento['SKU'] = data_magento['SKU'].astype(str).str.replace('7300281P24', '7300281')

data_magento = data_magento[~data_magento['Product Name'].astype(str).str.contains('JANGAN DIORDER INI TESTING')]
data_magento = data_magento[~data_magento['SKU'].astype(str).str.contains('7300851')]
data_magento = data_magento[~data_magento['SKU'].astype(str).str.contains('F0301002006')]
data_magento = data_magento[~data_magento['SKU'].astype(str).str.contains('F0302004002')]
data_magento = data_magento[~data_magento['SKU'].astype(str).str.contains('F0301002005P24')]
data_magento = data_magento[~data_magento['SKU'].astype(str).str.contains('F0301002005')]


print("--- %s seconds ---" % (time.time() - start_time))
print("Listing SKU Missing ====== 4/10")   
idx = data_magento[['SKU']][data_magento['SKU'].isnull()].drop_duplicates().index.to_list()
idx = idx + data_magento[['SKU', 'Product Name']][~data_magento['SKU'].astype(str).isin(data_SKU['SKU'].astype(str))].index.to_list()
idx = list(dict.fromkeys(idx))
idx_s = skushopee[~skushopee['SKU'].astype(str).str.replace('(S)','', regex = False).isin(data_SKU['SKU'].astype(str))].index.to_list()
idx_s = list(dict.fromkeys(idx_s))
idx_hd = skuhd[~skuhd['SKU'].astype(str).str.replace('hd','',case = False).isin(data_SKU['SKU'].astype(str))].index.to_list()
idx_hd = list(dict.fromkeys(idx_hd))

to_excel = data_magento.to_excel(r'magento_new.xls', index = False)
print("--- %s seconds ---" % (time.time() - start_time))
if len(idx) != 0 or len(idx_s) != 0 or len(idx_hd) != 0:
    print('SKU is missing')
    alert = data_magento.iloc[idx, ][['SKU', 'Product Name']].drop_duplicates()
    alert = alert.append(skushopee.iloc[idx_s][['SKU', 'Product Name']].drop_duplicates(), ignore_index = True, sort = False)
    alert['SKU Valid'] = np.nan
    to_excel = alert.to_excel('ALERT_MAGENTO_SKU_MISSING.xlsx')
    print("Some SKU Missing Please Complete It ====== 5/10")
    print("--- %s seconds ---" % (time.time() - start_time))
    s = smtplib.SMTP('smtp.gmail.com', 587) 

    # start TLS for security 
    s.starttls() 

    # Authentication 
    s.login("automationnfi@gmail.com", "nutrifood2020") 

    msg = MIMEMultipart()
    msg['Subject'] = "ALERT SKU MAGENTO MISSING"


    html = """\
    <html>
    <head></head>
    <body>
        {0}
    </body>
    </html>
    """.format(alert.to_html())

    part1 = MIMEText(html, 'html')
    msg.attach(part1)

    # sending the mail 
    s.sendmail("automationnfi@gmail.com", "andra.miftah@nutrifood.co.id", msg.as_string()) 

    # terminating the session 
    s.quit()
else :
    print("Preparing Appending Magento to Masterdata")
    print("Filling Brand ====== 5/10")  
    data_magento = data_magento.append(skushopee, ignore_index = True, sort = False)
    data_magento = data_magento.reset_index(drop = True)

    data_magento['SKU'] = data_magento['SKU'].astype(str)
    data_magento['Product Name'] = data_magento['Product Name'].astype(str)
    data_SKU['Real SKU'] = data_SKU['SKU'].astype(str)
    data_SKU['Real Nama Produk'] = data_SKU['Nama Produk'].astype(str)

    data_magento = data_magento.merge(data_SKU[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')

    temp = data_magento[data_magento['Real SKU'].isnull()].copy()
    temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
    temp = temp.merge(data_SKU[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
    temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
    temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
    temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
    temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

    indeks = data_magento[data_magento['Real SKU'].isnull()].index.to_list()
    data_magento['Real SKU'][indeks] = temp['Real SKU'][indeks]
    data_magento['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

    temp = data_magento[data_magento['Real SKU'].isnull()].copy()
    temp['SKU'] = temp['SKU'].astype(str).str.replace('hd','', regex = False)
    temp = temp.merge(data_SKU[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
    temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
    temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
    temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
    temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

    indeks = data_magento[data_magento['Real SKU'].isnull()].index.to_list()
    data_magento['Real SKU'][indeks] = temp['Real SKU'][indeks]
    data_magento['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

    data_magento['Real SKU'] = data_magento['Real SKU'].astype(str)
    data_magento = data_magento.merge(data_SKU[['SKU', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
    data_magento = data_magento.drop(['SKU_y'], axis = 1)
    data_magento = data_magento.rename(columns = {'SKU_x':'SKU'})

    print("--- %s seconds ---" % (time.time() - start_time))
    print("Unbundling ====== 6/10")
    list_col = ['SKU'] + data_SKU.columns[data_SKU.columns.get_loc('Produk 1'):data_SKU.columns.get_loc('Harga Organik 7')+1].to_list()
    data_magento = data_magento.merge(data_SKU[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
    data_magento = data_magento.drop(['SKU_y'], axis = 1)
    data_magento = data_magento.rename(columns = {'SKU_x':'SKU'})

    indeks = data_magento[data_magento['Brand'] == 'Bundle'].index.to_list()
    data_magento['Bundle Flag'] = np.nan
    data_magento['Bundle Flag'][indeks] = 'Bundle'

    print("--- %s seconds ---" % (time.time() - start_time))
    print("Filling Date ====== 7/10")
    data_magento['Date'] = np.nan
    data_magento['Month'] = np.nan
    data_magento['Year'] = np.nan

    for i in range(data_magento.shape[0]):
        if int(data_magento['Order date'][i].strftime('%d')) <= 12:
            data_magento['Date'][i] = pd.to_datetime(data_magento['Order date'][i].strftime('%Y-%d-%m %H:%M')).day
            data_magento['Month'][i] = pd.to_datetime(data_magento['Order date'][i].strftime('%Y-%d-%m %H:%M')).month_name()
            data_magento['Year'][i] = pd.to_datetime(data_magento['Order date'][i].strftime('%Y-%d-%m %H:%M')).year
        else :
            data_magento['Date'][i] = pd.to_datetime(data_magento['Order date'][i]).day
            data_magento['Month'][i] = pd.to_datetime(data_magento['Order date'][i]).month_name()
            data_magento['Year'][i] = pd.to_datetime(data_magento['Order date'][i]).year


    quarter = pd.DataFrame([['January', 1], ['February', 1], ['March', 1], ['April', 2], ['May', 2], ['June', 2], 
            ['July', 3], ['August', 3], ['September', 3],['October', 4], ['November', 4], ['December', 4]], columns = ['Bulan', 'Quarter'])
    data_magento = data_magento.merge(quarter, how = 'left', left_on = 'Month', right_on = 'Bulan')
    data_magento = data_magento.drop(['Bulan'], axis = 1)
    data_bulan = pd.DataFrame([{'Bulan' : 'December', 'Number' : 12} ,
            {'Bulan' : 'January' , 'Number': 1},
            {'Bulan' : 'February' , 'Number': 2},
            {'Bulan' : 'March' , 'Number': 3},
            {'Bulan' : 'April' , 'Number': 4},
            {'Bulan' : 'May' , 'Number': 5},
            {'Bulan' : 'June', 'Number': 6},
            {'Bulan' : 'July' , 'Number': 7},
            {'Bulan' : 'August', 'Number' : 8},
            {'Bulan' : 'September', 'Number' : 9},
            {'Bulan' : 'October' , 'Number': 10},
            {'Bulan' : 'November' , 'Number': 11}])
    temp = data_magento.copy()
    temp['Day'] = temp['Date']
    temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
    temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
    data_magento['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.week
    data_magento['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day']])
    data_magento['Channel'] = 'Nutrimart'
    data_magento['Store'] = 'Nutrimart'
    data_magento['Selling Price'] = data_magento['Item Price']
    print("--- %s seconds ---" % (time.time() - start_time))
    print("Pricing")
    data_magento = data_magento.merge(data_SKU[['SKU', 'Price List NFI', 'Harga Cost']].drop_duplicates('SKU'), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(data_magento.index)
    data_magento = data_magento.rename(columns = {'SKU_x' : 'SKU', 'Price List NFI_x' : 'Price List NFI'})
    data_magento['Price List NFI'] = pd.to_numeric(data_magento['Price List NFI']).astype(int)
    data_magento['Harga Cost'] = pd.to_numeric(data_magento['Harga Cost'], errors = 'coerce').fillna(0).astype(int)
    data_magento['Qty. Invoiced'] = pd.to_numeric(data_magento['Qty. Invoiced']).astype(int)
    data_magento['Total Net'] = data_magento['Price List NFI'] * data_magento['Qty. Invoiced']
    data_magento['Total Harga Cost'] = data_magento['Harga Cost'] * data_magento['Qty. Invoiced']

    data_magento = data_magento.drop('SKU_y', axis = 1)
    data_magento = data_magento.reset_index(drop = True)
    data_magento['Order #'] = data_magento['Order #'].astype(str).str.replace('.0', '', regex = False)

    list_bundle = data_magento[data_magento['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Subtotal', 'Total']].groupby(['Order #', 'Product Name']).sum().reset_index()
    list_nobundle = data_magento[data_magento['Bundle Name'].notnull()]
    list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
    list_nobundle

    data_magento['Total'][list_nobundle.index] = list_nobundle['Total_y']
    data_magento['Subtotal'][list_nobundle.index] = list_nobundle['Subtotal_y']

    temp = data_magento[data_magento['Bundle Name'].notnull()]
    temp['Subtotal'] = temp['Subtotal'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
    temp['Total'] = temp['Total'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
    temp['Selling Price'] = temp['Subtotal'] / temp['Qty. Invoiced']

    data_magento['Total'][temp.index] = temp['Total'].fillna(0).astype(int)
    data_magento['Subtotal'][temp.index] = temp['Subtotal'].fillna(0).astype(int)
    data_magento['Selling Price'][temp.index] = temp['Selling Price'].fillna(0).astype(int)

    print("Filling Location")
    data_magento['Kecamatan'] = np.nan
    data_magento['Kelurahan'] = np.nan

    indeks = data_magento[data_magento['City'].astype(str).str.contains('/')]['City'].index.to_list()
    if len(indeks)>0:
        data_magento['Kecamatan'][indeks] = data_magento['City'][indeks].str.split('/', n = 1,expand = True)[1]
        data_magento['City'][indeks] = data_magento['City'][indeks].str.split('/', n = 1,expand = True)[0]

    indeks = data_magento[data_magento['Kecamatan'].astype(str).str.contains('-')]['Kecamatan'].index.to_list()
    if len(indeks)>0:
        data_magento['Kelurahan'][indeks] = data_magento['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[1]
        data_magento['Kecamatan'][indeks] = data_magento['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[0]

    indeks = data_magento[data_magento['City'].astype(str).str.contains(',')]['City'].index.to_list()
    if len(indeks)>0:
        data_magento['Kecamatan'][indeks] = data_magento['City'][indeks].str.split(',', n = 1,expand = True)[1]
        data_magento['City'][indeks] = data_magento['City'][indeks].str.split(',', n = 1,expand = True)[0]

    indeks = data_magento[data_magento['Kecamatan'].astype(str).str.contains(',')]['Kecamatan'].index.to_list()
    if len(indeks)>0:
        data_magento['Kelurahan'][indeks] = data_magento['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[1]
        data_magento['Kecamatan'][indeks] = data_magento['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[0]

    city = pd.read_excel(r'All Data/list_city.xlsx')
    temp = data_magento.copy()
    temp['City'] = temp['City'].astype(str).str.lower()
    city['All City'] = city['All City'].astype(str).str.lower()
    temp = temp.merge(city.drop_duplicates('All City'), how = 'left', left_on = 'City', right_on = 'All City').set_index(temp.index)
    indeks = temp[temp['Real City'].notnull()].index.to_list()
    data_magento['City'][indeks] = temp['Real City'][indeks]

    province = pd.read_excel(r'All Data/list_province.xlsx')
    temp = data_magento.copy()
    temp['Region'] = temp['Region'].astype(str).str.lower()
    province['All Province'] = province['All Province'].astype(str).str.lower()
    temp = temp.merge(province.drop_duplicates('All Province'), how = 'left', left_on = 'Region', right_on = 'All Province').set_index(temp.index)
    indeks = temp[temp['Real Province'].notnull()].index.to_list()
    data_magento['Region'][indeks] = temp['Real Province'][indeks]

    temp = data_magento.copy()
    temp = temp[temp['Region'].isnull()]
    temp['Region'] = temp.merge(master_map, how = 'left', on = 'City').set_index(temp.index)['Province']
    data_magento['Region'][temp.index] = temp['Region']

    district = pd.read_excel(r'All Data/list_district.xlsx')
    temp = data_magento.copy()
    temp['Kecamatan'] = temp['Kecamatan'].astype(str).str.lower()
    district['All District'] = district['All District'].astype(str).str.lower()
    temp = temp.merge(district.drop_duplicates('All District'), how = 'left', left_on = 'Kecamatan', right_on = 'All District').set_index(temp.index)
    indeks = temp[temp['Real District'].notnull()].index.to_list()
    data_magento['Kecamatan'][indeks] = temp['Real District'][indeks]

    temp = data_magento.copy()
    temp2 = temp[['Region', 'City', 'Kecamatan']].merge(master_map, how = 'left', on = 'City')
    indeks = temp2[temp2['Region'] != temp2['Province']][temp2[temp2['Region'] != temp2['Province']]['City'].notnull()].index.to_list()
    data_magento['City'][indeks] = np.nan

    indeks = data_magento[data_magento['Shipping City'].astype(str).str.contains('/')]['Shipping City'].index.to_list()
    if len(indeks)>0:
        data_magento['Shipping City'][indeks] = data_magento['Shipping City'][indeks].str.split('/', n = 1,expand = True)[0]

    indeks = data_magento[data_magento['Shipping City'].astype(str).str.contains(',')]['Shipping City'].index.to_list()
    if len(indeks)>0:
        data_magento['Shipping City'][indeks] = data_magento['Shipping City'][indeks].str.split(',', n = 1,expand = True)[0]

    city = pd.read_excel(r'All Data/list_city.xlsx')
    temp = data_magento.copy()
    temp['Shipping City'] = temp['Shipping City'].astype(str).str.lower()
    city['All City'] = city['All City'].astype(str).str.lower()
    temp = temp.merge(city.drop_duplicates('All City'), how = 'left', left_on = 'Shipping City', right_on = 'All City').set_index(temp.index)
    indeks = temp[temp['Real City'].notnull()].index.to_list()
    data_magento['Shipping City'][indeks] = temp['Real City'][indeks]

    province = pd.read_excel(r'All Data/list_province.xlsx')
    temp = data_magento.copy()
    temp['Shipping Province'] = temp['Shipping Province'].astype(str).str.lower()
    province['All Province'] = province['All Province'].astype(str).str.lower()
    temp = temp.merge(province.drop_duplicates('All Province'), how = 'left', left_on = 'Shipping Province', right_on = 'All Province').set_index(temp.index)
    indeks = temp[temp['Real Province'].notnull()].index.to_list()
    data_magento['Shipping Province'][indeks] = temp['Real Province'][indeks]

    temp = data_magento.copy()
    temp = temp[temp['Shipping Province'].isnull()]
    temp['Shipping Province'] = temp.merge(master_map, how = 'left', left_on = 'Shipping City', right_on = 'City').set_index(temp.index)['Shipping Province']
    data_magento['Shipping Province'][temp.index] = temp['Shipping Province']

    temp = data_magento.copy()
    temp2 = temp[['Shipping Province', 'Shipping City']].merge(master_map, how = 'left', left_on = 'Shipping City', right_on = 'City')
    indeks = temp2[temp2['Shipping Province'] != temp2['Province']][temp2[temp2['Shipping Province'] != temp2['Province']]['Shipping City'].notnull()].index.to_list()
    data_magento['Shipping City'][indeks] = np.nan
    data_magento['Warehouse Name'] = 'Primary Warehouse'

    import os
    import glob

    list_of_files = glob.glob('D:\Masterdata\Clean Data\*.csv')
    latest_file = max(list_of_files, key=os.path.getctime)

    data_all = pd.read_csv(latest_file, sep = ';',converters = {'Phone' : str, 'Order #' : str, 'AWB' : str})
    data_all = data_all[data_all['Store Type'] != 'Retail Online']

    data_all['Phone'] = data_all['Phone'].astype(str).str.replace('="', '', regex = False)
    data_all['Phone'] = data_all['Phone'].astype(str).str.replace('"', '', regex = False)

    lmen_store = forstok_all[forstok_all['Channel'] == 'L-Men Store Blibli']
    forstok_all = forstok_all[forstok_all['Channel'] != 'L-Men Store Blibli']

    lmen_store = lmen_store[~lmen_store['Order #'].astype(str).isin(data_all['Order #'].astype(str))]
    lmen_store['Sales Order ID'] = lmen_store['Sales Order ID'].fillna(lmen_store['No. Order Item L-Men Blibli'])
    ornum = lmen_store[['Order #', 'Sales Order ID']].drop_duplicates('Order #')

    options = Options()
    options.add_experimental_option("prefs", {
            "download.default_directory": str(os.getcwd())  + '/Input Data',
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            "safebrowsing.enabled": False
    })

    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    driver.get("https://seller.blibli.com/sign-in")
    time.sleep(30)


    username = driver.find_element_by_id("email")
    username.clear()
    username.send_keys("customer1@nutrimart.co.id")

    password = driver.find_element_by_id("password")
    password.send_keys("Nutrimart12345")

    driver.find_element_by_id("sign-in").click()
    time.sleep(25)

    driver.get("https://seller.blibli.com/MTA/order/summary")
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'btn-primary-mta'))).click()
    driver.find_element_by_id("semuaFilter").click()

    ornum['Phone'] = np.nan
    ornum['Email'] = np.nan
    ornum['Address'] = np.nan

    for i in ornum.index:
        orderNo = ornum['Order #'][i]
        orderItemNo = int(pd.to_numeric(ornum['Sales Order ID'][i]))
        link = "https://merchant.blibli.com/MTA/neo/order/order-detail/" + "?orderNo=" + str(orderNo) + "&orderItemNo=" + str(orderItemNo)
        driver.get(link)
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//*[@id="customer-phone"]/div/span')))
        hp = driver.find_elements_by_xpath('//*[@id="customer-phone"]/div/span')[0].text
        if len(driver.find_elements_by_xpath('//*[@id="customer-email"]/span')) != 0:
            email = driver.find_elements_by_xpath('//*[@id="customer-email"]/span')[0].text
            ornum['Email'][i] = email
        address = driver.find_elements_by_xpath('//*[@id="alamat-pengiriman"]//div//p')[0].text
        ornum['Phone'][i] = hp
        ornum['Address'][i] = address

    driver.quit()
    if len(ornum) != 0:
        tes = ornum.copy()
        tes['Real Address'] = tes['Address'].str.split('\n', expand = True)[1].str.strip()
        tes['Kelurahan'] = tes['Address'].str.split('\n', expand = True)[3].str.split('-', expand = True)[0].str.strip()
        tes['Kecamatan'] = tes['Address'].str.split('\n', expand = True)[3].str.split('-', expand = True)[1].str.strip()
        tes['City'] = tes['Address'].str.split('\n', expand = True)[4].str.split(',', expand = True)[0].str.strip()
        tes['Region'] = tes['Address'].str.split('\n', expand = True)[4].str.split(',', expand = True)[1].str.strip()
        tes['Zip Code'] = tes['Address'].str.split('\n', expand = True)[5].str.strip()

        lmen_store['Order #'] = lmen_store['Order #'].astype(str)
        tes['Order #'] = tes['Order #'].astype(str)
        lmen_store = lmen_store.merge(tes[['Order #', 'Phone', 'Email', 'Real Address', 'Kelurahan', 'Kecamatan', 'City', 'Region', 'Zip Code']].drop_duplicates('Order #'), how = 'left', on = 'Order #')

        indeks = lmen_store[lmen_store['Phone_y'].notnull()].index.to_list()
        lmen_store['Phone_x'][indeks] = lmen_store['Phone_y'][indeks]
        lmen_store['Customer Email'][indeks] = lmen_store['Email'][indeks]
        lmen_store['Phone_x'][indeks] = lmen_store['Phone_y'][indeks]
        lmen_store['Region_x'][indeks] = lmen_store['Region_y'][indeks]
        lmen_store['City_x'][indeks] = lmen_store['City_y'][indeks]
        lmen_store['Kecamatan_x'][indeks] = lmen_store['Kecamatan_y'][indeks]
        lmen_store['Kelurahan_x'][indeks] = lmen_store['Kelurahan_y'][indeks]
        lmen_store['Zip Code_x'][indeks] = lmen_store['Zip Code_y'][indeks]
        lmen_store['Address'][indeks] = lmen_store['Real Address'][indeks]

        lmen_store = lmen_store.drop(['Phone_y', 'Email', 'Region_y', 'City_y', 'Kecamatan_y', 'Kelurahan_y', 'Real Address', 'Zip Code_y'], axis = 1)
        lmen_store = lmen_store.rename(columns = {'Phone_x' : 'Phone', 'Region_x' : 'Region', 'City_x' : 'City', 'Kecamatan_x' : 'Kecamatan', 'Kelurahan_x' : 'Kelurahan', 'Zip Code_x' : 'Zip Code'})

    forstok_all = forstok_all.append(lmen_store, ignore_index = True, sort = False)

    indeks = data_all[data_all['Channel'] == 'L-Men Store Blibli'].index.to_list()
    data_all['Store'][indeks] = 'Blibli'

    print('Blibli Done')
    
    data_all = data_all[~data_all['Order #'].astype(str).isin(forstok_all['Order #'].astype(str))]
    data_all = data_all[~data_all['Order #'].astype(str).isin(data_magento['Order #'].astype(str))]
    data_all = data_all.append(forstok_all, ignore_index = True, sort = False)
    data_all = data_all.append(data_magento, ignore_index = True, sort = False)
    data_all['Brand'] = data_all['Brand'].astype(str).str.replace('Tropicana Slim', 'TS')

    indeks = data_all[data_all['Qty. Invoiced'] == 0].index.to_list()
    data_all['Qty. Invoiced'][indeks] = data_all['Qty. Shipped'][indeks]
    data_all['Total Net'][indeks] = data_all['Price List NFI'][indeks] * data_all['Qty. Invoiced'][indeks]
    indeks = data_all[data_all['Qty. Invoiced'] == 0].index.to_list()
    data_all['Qty. Invoiced'][indeks] = data_all['Qty. Ordered'][indeks]
    data_all['Total Net'][indeks] = data_all['Price List NFI'][indeks] * data_all['Qty. Invoiced'][indeks]

#   Cleaning Phone Number
    import re 
    from re import sub            
    data_all['Phone'] = data_all['Phone'].astype(str).str.split(';', expand = True)[0]
    data_all['Phone'] = data_all['Phone'].astype(str).str.replace('.0','', regex = False)
    data_all['Phone'] = data_all['Phone'].astype(str).str.replace('+','', regex = False)
    non_phone = data_all[(data_all['Phone'].astype(str).str.startswith('62'))]['Phone'].unique()
    non_phone1 = data_all[(data_all['Phone'].astype(str).str.startswith('62'))][['Store','Phone']].drop_duplicates()
    for i in non_phone:
        index = data_all[data_all['Phone'].astype(str) == str(i)].index.to_list()
        phone = str(i).replace('62','0', 1)
        print(i, phone, sep='-')
        data_all['Phone'][index] = phone
            
    print('done')
    print("--- %s seconds ---" % (time.time() - start_time))
    data_all['Phone'] = data_all['Phone'].apply('="{}"'.format)
    data_all['Shipping Phone'] = data_all['Shipping Phone'].apply('="{}"'.format)
    indeks = data_all[data_all['City'] == 'nan'].index.to_list()
    data_all['City'][indeks] = np.nan

    indeks = data_all[data_all['Region'] == 'nan'].index.to_list()
    data_all['Region'][indeks] = np.nan

    indeks = data_all[data_all['Channel'] == 'FBL'].index.to_list()
    data_all['Warehouse Name'][indeks] = 'Lazada Warehouse'

    indeks = data_all[data_all['Warehouse Name'] == 'Lazada Warehouse'].index.to_list()
    data_all['Channel'][indeks] = 'FBL'

    print("Read Data Settlement")
    
    print('Opening Chrome')
    driver = webdriver.Chrome()
    driver.maximize_window()
    actions = ActionChains(driver)
    driver.get("https://www.forstok.com/dashboard/users/login")
    print('Login Forstok')
    username = driver.find_element_by_id("dashboard_user_email")
    username.clear()
    username.send_keys("timotius.giovandi@nutrifood.co.id")

    password = driver.find_element_by_id("dashboard_user_password")
    password.send_keys("timo123")
    
    driver.find_element_by_name("commit").click()

    time.sleep(20)
    datefield = WebDriverWait(driver, 100).until(EC.element_to_be_clickable((By.XPATH,'//*[@type="button" and contains(text(),"Last 30 days")]')))
    datef = WebDriverWait(driver, 200).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="root"]/section/section[2]/article/div/div/aside/div/div[1]/section[5]/section[1]/h2')))
    driver.execute_script("return arguments[0].scrollIntoView(true);", datef)
    time.sleep(30)
    driver.execute_script("return arguments[0].scrollIntoView(true);", datefield)
    time.sleep(10)
    datefield.click()
    pilih=0
    while pilih < 3:    
            n_elem=0
            for i in driver.find_elements_by_xpath('(//*[@class="DateRangePicker__MonthHeaderLabel DateRangePicker__MonthHeaderLabel--month"])'):
                val=(driver.find_elements_by_xpath('(//*[@class="DateRangePicker__MonthHeaderLabel DateRangePicker__MonthHeaderLabel--month"])')[n_elem].text.split()[0])
                if val==start_date.strftime('%B'):
                    # print(n_elem)
                    fin_n_elem=n_elem
                n_elem=n_elem+1
            if fin_n_elem==0:
                text = '(//*[@class="DateRangePicker__Month"])[1]//*[@class="DateRangePicker__DateLabel" and (text()="' + str(start_date.day) + '")]'
                if len(driver.find_elements_by_xpath(text))==2:
                    text = '('+text+')[2]'
            else:
                text = '(//*[@class="DateRangePicker__Month"])[2]//*[@class="DateRangePicker__DateLabel" and (text()="' + str(start_date.day) + '")]'
                if len(driver.find_elements_by_xpath(text))==2:
                    text = '('+text+')[1]'
            driver.find_element_by_xpath(text).click()
            time.sleep(10)

            n_elem=0
            for i in driver.find_elements_by_xpath('(//*[@class="DateRangePicker__MonthHeaderLabel DateRangePicker__MonthHeaderLabel--month"])'):
                val=(driver.find_elements_by_xpath('(//*[@class="DateRangePicker__MonthHeaderLabel DateRangePicker__MonthHeaderLabel--month"])')[n_elem].text.split()[0])
                if val==end_date.strftime('%B'):
                    fin_n_elem=n_elem
                n_elem=n_elem+1
            if fin_n_elem==0:
                text = '(//*[@class="DateRangePicker__Month"])[1]//*[@class="DateRangePicker__DateLabel" and (text()="' + str(end_date.day) + '")]'
                if len(driver.find_elements_by_xpath(text))==2:
                    text = '('+text+')[2]'
            else:
                text = '(//*[@class="DateRangePicker__Month"])[2]//*[@class="DateRangePicker__DateLabel" and (text()="' + str(end_date.day) + '")]'
                if end_date.strftime('%B')==start_date.strftime('%B'):
                    text = '('+text+')'#[2]'
                else :
                    text = '('+text+')[1]'
            driver.find_element_by_xpath(text).click()
            time.sleep(10)
            pilih=pilih+1
    driver.find_element_by_xpath('//*[@type="button" and contains(text(),"Apply")]').click()
    time.sleep(100)
    WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, "(//*[@type='button' and contains(text(),'Export')])[1]"))).click()
    driver.find_element_by_xpath('//span[text()="Sales Order"]').click()
    driver.find_element_by_xpath('//span[text()="Sales Order"]').click()
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "(//*[@type='button' and contains(text(),'Export')])[2]"))).click()
    time.sleep(2)

    driver.quit()

    print('Forstok Downloaded')

    print('Download Data SKU')
    # Import library

    import time
    import pandas as pd
    import numpy as np
    import math

    import smtplib 
    from email.mime.text import MIMEText
    from email.mime.application import MIMEApplication
    from email.mime.multipart import MIMEMultipart
    from smtplib import SMTP
    import smtplib
    import sys
    import requests
    import os

    date1 = start_date.strftime('%Y-%m-%d') + '-' + end_date.strftime('%Y-%m-%d')
    print(date1)
    print('Waiting to Download Forstok 1')
    s = requests.Session()
    r = s.get("https://www.forstok.com/dashboard/users/login")
    data = {'dashboard_user_email' : 'timotius.giovandi@nutrifood.co.id', "dashboard_user_password" : 'timo123'}
    r = s.post("https://www.forstok.com/dashboard/users/login", data = data)

    text = 'https://forstok-staging-storage.s3.ap-southeast-1.amazonaws.com/items_import/nutrifood--forstok-sales_orders-version_1-' + date1 + '.xls'
    print(text)
    while True:
        r = s.get(text)
        print('Waiting to Download Forstok 2')

        if r.status_code == requests.codes.ok:
            print(r.status_code)
            with open('Input Data/Forstok_new_input.xls', 'wb') as output:
                output.write(r.content)
            print('output out')
            break
        else :
            time.sleep(60)

    with open('Input Data/nutrifood--forstok-sales_orders-' + date + '.xls', 'wb') as output:
        output.write(r.content)

    orstat_forstok = pd.read_excel(r'Input Data/Forstok_new_input.xls')
    orstat_forstok['Date'] = np.nan
    orstat_forstok['Month'] = np.nan
    orstat_forstok['Year'] = np.nan

    orstat_forstok['Order Date'] = pd.to_datetime(orstat_forstok['Order Date'])
    for i in range(orstat_forstok.shape[0]):
        if int(orstat_forstok['Order Date'][i].strftime('%d')) <= 12:
            orstat_forstok['Date'][i] = pd.to_datetime(orstat_forstok['Order Date'][i].strftime('%Y-%d-%m %H:%M')).day
            orstat_forstok['Month'][i] = pd.to_datetime(orstat_forstok['Order Date'][i].strftime('%Y-%d-%m %H:%M')).month_name()
            orstat_forstok['Year'][i] = pd.to_datetime(orstat_forstok['Order Date'][i].strftime('%Y-%d-%m %H:%M')).year
        else :
            orstat_forstok['Date'][i] = pd.to_datetime(orstat_forstok['Order Date'][i]).day
            orstat_forstok['Month'][i] = pd.to_datetime(orstat_forstok['Order Date'][i]).month_name()
            orstat_forstok['Year'][i] = pd.to_datetime(orstat_forstok['Order Date'][i]).year

    temp = orstat_forstok.copy()
    temp['Day'] = temp['Date']
    temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
    temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
    orstat_forstok['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.week
    temp['Hour'] = pd.to_datetime(orstat_forstok['Order Date']).dt.hour
    temp['Minute'] = pd.to_datetime(orstat_forstok['Order Date']).dt.minute
    temp['Second'] = pd.to_datetime(orstat_forstok['Order Date']).dt.second
    orstat_forstok['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']])

    print("Update Order Status")
    date_min  = orstat_forstok['True datetime'].min()
    orstat_forstok = orstat_forstok[['Sales Order ID','Status']].drop_duplicates('Sales Order ID')
    shopee_bp = data_all[data_all['Customer Name'] == 'Shopee Brand Portal']
    data_all = data_all[data_all['Customer Name'] != 'Shopee Brand Portal']
    data_all['True datetime1'] = pd.to_datetime(data_all['True datetime'])
    temp_all = data_all[data_all['True datetime1'] >= pd.to_datetime(date_min)][~data_all[data_all['True datetime1'] >= pd.to_datetime(date_min)]['Channel'].isin(['Nutrimart', 'L-Men Store Blibli', 'Order Online', 'Order Online Jakarta', 'Order Online Surabaya'])]
    orstat_forstok['Sales Order ID'] = orstat_forstok['Sales Order ID'].astype(str)
    temp_all['Sales Order ID'] = temp_all['Sales Order ID'].astype(str)
    temp_all = temp_all.merge(orstat_forstok, how = 'left', on = 'Sales Order ID')
    temp_all['Status'] = temp_all['Status'].fillna('Canceled')
    temp_all['Order Status'] = temp_all['Status']
    temp_all = temp_all.drop('Status', axis = 1)
    data_all['Order #'] = data_all['Order #'].astype(str)
    temp_all['Order #'] = temp_all['Order #'].astype(str)
    data_all = data_all[~data_all['Order #'].isin(temp_all['Order #'])]
    data_all = data_all.append(temp_all, ignore_index = True, sort = False)
    data_all = data_all.append(shopee_bp, ignore_index = True, sort = False)
    print("Export Master Data")


Starting Magento
Opening Chrome
Login Magento
Go To Sales Form
2022-11-17 11:41:39.945142
2022-11-29 11:41:39.945142
Input Date
Download Magento
bulk-order-download-template.csv
Import Data ====== 1/10
--- 4.552905082702637 seconds ---
Formatting Data ====== 2/10
--- 4.80031418800354 seconds ---
Fulfilling SKU ====== 3/10
--- 4.838092565536499 seconds ---
Listing SKU Missing ====== 4/10
--- 5.339282035827637 seconds ---
Preparing Appending Magento to Masterdata
Filling Brand ====== 5/10
--- 5.370523691177368 seconds ---
Unbundling ====== 6/10
--- 5.3861777782440186 seconds ---
Filling Date ====== 7/10
--- 5.809260368347168 seconds ---
Pricing
Filling Location


C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_10148\1561600918.py:627: DtypeWarning: Columns (3,12,15,20,23,24,25,27,29,30,31,32,33,34,35,36,37,39,53,54,57,59,60,66,67,73,74,80,81,87,88,94,95,101,102,110,122,123,134,135,136,153,159,161,167,168,169,196,197) have mixed types. Specify dtype option on import or set low_memory=False.
  data_all = pd.read_csv(latest_file, sep = ';',converters = {'Phone' : str, 'Order #' : str, 'AWB' : str})


Blibli Done
6285649261415-085649261415
6287860054375-087860054375
6285257216829-085257216829
6285736946413-085736946413
6285754635475-085754635475
6289685343071-089685343071
6287766595378-087766595378
6281515580809-081515580809
6281938777090-081938777090
6283831111384-083831111384
628130232121-08130232121
6282142044477-082142044477
6282232000676-082232000676
6281775229185-081775229185
6281232554553-081232554553
6285606555040-085606555040
6285747231589-085747231589
6285736522843-085736522843
6285704004677-085704004677
6287758890030-087758890030
6285778681114-085778681114
62895710281000-0895710281000
6281234842380-081234842380
6285707340134-085707340134
6285601067466-085601067466
6289693292000-089693292000
628564926145-08564926145
6281249722401-081249722401
6281217498009-081217498009
6285855619383-085855619383
62895364823770-0895364823770
6282139432399-082139432399
6288805436598-088805436598
6282113120664-082113120664
62838311113840-0838311113840
6285859768839-085859768839
6289688256600-

6282144325592-082144325592
6287855083527-087855083527
6282146957614-082146957614
6281238654142-081238654142
6285768289214-085768289214
6281934381250-081934381250
6282313722322-082313722322
628814728614-08814728614
6281916303952-081916303952
6281337982377-081337982377
6287853683048-087853683048
6281353230582-081353230582
6282116921227-082116921227
6281242766439-081242766439
628884285164-08884285164
6285341600924-085341600924
6285823843882-085823843882
6282189526262-082189526262
6285395665859-085395665859
6281355987308-081355987308
6282292302858-082292302858
6281310345681-081310345681
6282293931145-082293931145
6282347720909-082347720909
6285240462641-085240462641
6281524439105-081524439105
6282290470697-082290470697
6282349296438-082349296438
6281455987308-081455987308
6285256710849-085256710849
6281928693964-081928693964
6282296776995-082296776995
6285741888137-085741888137
6281241920815-081241920815
6285398019992-085398019992
6285242411990-085242411990
6281222409496-081222409496
62812

In [18]:
print('done')

done


In [11]:
### if sku missing di magento
### to do : listing di tatanama rerun kode atas Run -4 
data_magento[~data_magento['SKU'].isin(data_SKU['SKU'])]['Product Name'].unique()
data_magento[~data_magento['SKU'].isin(data_SKU['SKU'])]['SKU'].unique()

array([], dtype=object)

In [12]:
## run-5
indeks = data_all[data_all['Qty. Invoiced'].isnull()].index.to_list()
data_all['Qty. Invoiced'][indeks] = 0
data_all['Total Net'][indeks] = data_all['Price List NFI'][indeks] * data_all['Qty. Invoiced'][indeks]

In [13]:
##run-6
order_online_cond = False
shopee_scrap = False
shopee_done = False

In [14]:
##run-7
order_online_jatim = False
order_online_jkt = False
order_online_jateng = False
order_online_bali = False
order_online_makasar = False
order_online_samarinda = False
order_online_medan = False
order_online_lampung = False
order_online_pekanbaru = False
order_online_banjarmasin = False
order_online_jabar = False

In [21]:
## run 8
## until all region true
## error : add missing prod to code line jatim jkt jateng, other to "D:\Masterdata\Order Online\SKU buat andra updated maret 2021.xlsx"

start_time = time.time()
if not order_online_jatim:

    import pandas as pd
    import numpy as np
    import requests
    import os
    
    print('order_online_jatim')
    ##komen sementara

    before = os.listdir(os.getcwd() + '/Input Data')

    options = Options()
    options.add_experimental_option("prefs", {
            "download.default_directory": os.path.abspath("Input Data"),
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            "safebrowsing.enabled": False
    })

    driver = webdriver.Chrome(options=options)
    driver.fullscreen_window()
    driver.get("https://app.orderonline.id")
    
    time.sleep(30)

    username = driver.find_element_by_name("email")
    username.clear()
    username.send_keys("customer@nutrimart.co.id")

    password = driver.find_element_by_name("password")
    password.send_keys("jatimhomdel")
    password.click()

    driver.find_element_by_class_name("btn-submit").click()

    WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="main-nav-dropdown"]/ul/li[3]/a'))).click()
    time.sleep(5)
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[1]/div/div[2]/div/div/div').click()
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="app"]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[1]/div[1]/ul/li[6]'))).click()
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[2]/button[2]').click()
    time.sleep(10)
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[3]/div/button').click()
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[3]/div/div/button[1]').click()

    time.sleep(60)

    after = os.listdir(os.getcwd() + '/Input Data')
    change = set(after) - set(before)
    if len(change) == 1:
        file_name = change.pop()
    elif len(change) == 0: 
        print("No file downloaded")
    else :
        print("More than one file downloaded")

    data_order = pd.read_csv(r'Input Data/' + str(file_name))
    
    #komen sementara
    # data_order=pd.read_csv(r"D:\Masterdata\Input Data\orderonline_orders_all_products_08-07-2022_MOoUrPq6Pzq0pu2y.csv")
    driver.close()
    product_order = pd.read_csv(r'data_supp/orderonline_products_Surabaya.csv')
    
    data_order['order_id'] = data_order['order_id'].fillna(method='ffill')
    data_order = data_order.drop('quantity', axis = 1)
    temp = data_order.drop_duplicates('order_id').drop('product', axis = 1)
    data_order = data_order[['order_id', 'product']]
    data_order = data_order.merge(temp, how = 'left', on = 'order_id')
    
    data_order['order_id'] = data_order['order_id'].astype(int)
    data_order['product'] = data_order['product'].astype(str).str.replace('L-Men Hi Protein 2 Go Chocolate (24 pcs) - 12gr Protein / Serving', 'L-Men Hi Protein 2 Go Chocolate (24 pcs)', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Shirataki Noodles 71gr (x2) ', 'Twin Pack: Tropicana Slim Shirataki Noodles 71gr ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Saus Tiram 200ml (x2) ', 'Twin Pack: Tropicana Slim Saus Tiram 200ml ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Sweetener Rose Vanilla 50 sch (x2) ', 'Twin Pack: Tropicana Slim Sweetener Rose Vanilla 50 sch ', regex = False)

    
    product = data_order['product'].str.split("\(x",1, expand = True)
    data_order['Quantity'] = product[1].astype(str).str.replace(')', '', regex = False).astype(int)
    data_order['product'] = product[0].str.strip().str.replace('  ', ' ').str.replace('6 SCH', '6sch')
                        
    product_order['title'] = product_order['title'].str.strip().str.replace('  ', ' ')
    product_order['title'] = product_order['title'].str.strip().str.replace('L-Men Gain Mass Chocolate 500 gr', 'L Men Gain Mass Chocolate 500 gr')


    product_order = product_order.append(pd.DataFrame([['Nutrisari Mango Smoothie 200ml (6pcs)', 36300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Hi Protein 2 Go Chocolate (6pcs)', 48000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Bar Crunchy Chocolate (12sch)', 5500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Sweetener Honey (50sch)', 39500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Sirup Orange 750ml', 28600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Gain Mass Chocolate 225gr', 57500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Gainmass Taro 225gr', 69600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Gain Mass Chocolate 500 gr', 139200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Hokkaido Cheese 100gr', 19800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Nutrisari Mangga Gandaria (40 sch)', 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo School Chocolate 750gr + Free Kertas Gambar', 85800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Paket Nutrisari Jeruk Peras (40sch x 2) + Nutrisari Mangga Gandaria (40sch x 1)', 157500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Paket Nutrisari Blewah (40sch x 2) + Nutrisari Jeruk Maroko (40sch x 1)', 157500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Paket Nutrisari American Sweet Orange (40sch x 2) + Nutrisari Milky Orange (40sch x 1)', 157500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo School Chocolate 500gr + Free Kertas Gambar', 117600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Gold Chocolate 750gr', 127100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Buy 1 Get 1 FREE: Lokalate Kopi Berondong (10 Sch)', 15000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Es Cincau 40 sch', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Sunflower Oil 946 ml', 52800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Lokalate Kopi Berondong 10's", 15000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["L-Men Lose Weight Chocolate Cereal 300gr", 118800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["L-Men Platinum Choco Latte Dus 6 sch", 80000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["L-Men Hi Protein 2 Go Chocolate (24 pcs)", 264000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["L-Men Protein Crunch BBQ Beef (20gr)", 10500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["L-Men Gain Mass Banana 225gr", 61000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["L-Men Protein Bar Chocolate 12 sch", 105600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Hilo Chocolate Taro (10 sch)", 14700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["BUY 1 GET 1 - W'Dank Empon Empon 10 Sachet Renceng", 15000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Nanas 40 sch", 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Semangka 40 sch", 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["W'Dank Empon-Empon 10 sch", 15000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(FREE) W'Dank Empon-Empon 10 sch", 0]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo Teen Chocolate 1000 gr", 136400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["L-MEN Gain Mass Taro 225 gr", 52800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["L-Men Whey Daily Dark Chocolate 250gr", 49500]], columns = ['title', 'price']), ignore_index = True, sort = False)                                        
    product_order = product_order.append(pd.DataFrame([["HiLo Es Teler 10 sch", 13200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Hilo Es Ketan Hitam 10 sch", 13200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo School Honey 500gr", 71500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo Teen Vanilla 750gr", 102300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Twin Pack: Hilo Es Ketan Hitam 10 sch x 2", 26400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Twin Pack: HiLo Es Teler 10 sch x 2", 26400]], columns = ['title', 'price']), ignore_index = True, sort = False)

    product_order = product_order.append(pd.DataFrame([['Triple Pack: Tropicana Slim Korean Garlic Butter Cookies (5 Sch)', 52000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Tropicana Slim Sweetener Jahe 50 sch", 38500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Tropicana Slim Korean Garlic Butter Cookies 5 Sch", 17500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Sambal Terasi 200 gr', 29700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Tropicana Slim Avocado Coffee 4 Sch", 12100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Protein Bar Chocolate (12 Sch) + L-Men Protein Crunch BBQ Beef x 2', 107800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Buy 5 Get 5 L-Men Protein Crunch BBQ Beef (20gr)', 67500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Protein Bar Chocolate 12 sch', 105600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Protein Crunch BBQ Beef (20gr)', 9900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: Tropicana Slim Avocado Coffee 4 Sch x 2', 24200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Avocado Coffee 4 Sch', 12100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Teen Taro 500gr', 71500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Sweetener Lemongrass Pandan 50 sch', 28900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Active Ketan Hitam 175gr', 20700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Cocopandan 40 sch', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: Tropicana Slim Shirataki Noodles 71gr', 28100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: Tropicana Slim Saus Tiram 200ml', 39600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Shirataki Noodles 71gr', 14200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Saus Tiram 200ml', 19800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim White Coffee (4sch)', 17600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hampers Idul Fitri Tropicana Slim', 99000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Paket Takjil HiLo Dessert', 67500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Paket Ramadhan NutriSari 2021', 60000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Paket Kehangatan WDank - Ramadhan Edition', 60000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Platinum Kacang Hijau 800g', 275000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Topping Kental Manis 150ml - SUGAR FREE', 28600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Whey Advanced Cappuccino 250gr', 79200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Minyak Kanola 946ml - 100% Pure Canola Oil - Free Korean Garlic Butter Cookies 1 sch', 59400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Minyak Kanola 946ml', 59400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['(EACH)Tropicana Slim Korean Garlic Butter Cookies - Sampling', 0]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Platinum Ketan Hitam 800g (25g protein / serving) - Suplemen Tinggi Whey Protein Rendah Lemak', 302500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Whey Advanced Choco Vanilla 500gr', 148500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['FREE Cookies - Tropicana Slim Susu Skim Chocolate 500gr - Bantu Turunkan Kolesterol', 106700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['FREE Cookies - Tropicana Slim Cafe Latte 10 Sachet 14gr - Kopi Susu Nikmat Tanpa Gula Pasir', 27500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Platinum Choco Latte 800gr (25gr protein)', 302500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['FREE Cookies - Tropicana Slim Susu Skim Plain 1000gr - Bantu Turunkan Kolesterol', 180400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['FREE Cookies - Tropicana Slim Susu Skim Coffee 500gr - Bantu Turunkan Kolesterol', 106700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack - HiLo Active Es Teler 175gr', 38000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Active Es Teler 175gr', 19000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Apel Jeruk 40 Sachet', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: HiLo Teen Popcorn Caramel 500 gram x 2', 99000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Avocado Coffee 4 Sch', 12100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: Tropicana Slim Sweetener Rose Vanilla 50 sch', 57800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Sweetener Rose Vanilla 50 sch', 28900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Milk Skim Chocolate 500gr', 106700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Milk Skim Coffee 500gr', 106700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Extra Virgin Olive Oil 500 ml - [Free] Tropicana Slim Shirataki Noodles 71gr', 83500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School Chocolate 750g Susu Tinggi Kalsium Lebih Rendah Lemak - [FREE] HiLo Active Ketan Hitam 175g', 111000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim DIABTX (100 sch) - [FREE] Tropicana Slim Avocado Coffee 4 Sch', 76000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Milk Skim Original 1kg', 187700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Oat Drink 190 ml (RTD) x 4 pcs', 22600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Es Rujak Jeruk Bali 40 Sachet', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Oat Drink 190 ml (RTD)', 5650]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: Tropicana Slim Mayonnaise 200 g x 2', 34800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Mayonnaise 200 g - Bantu Dukung Hidup Sehat', 17400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Active Chocolate 1000 gr', 115500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: L-Men Daily Popcorn Caramel 250gr x 2', 78200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Extra Virgin Olive Oil 500 ml', 83500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Daily Popcorn Caramel 250gr', 39100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Buy 1 Get 1 - HiLo Gold Sweet Potato 500 gram', 73200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NuTrilogi Seri Jeje Si Jeli', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Kecap Asin 200 ml', 24600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Gold Sweet Potato 500 gram', 73200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['(FREE) HiLo Gold Sweet Potato 500 gram', 0]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Buy 1 Get 1 FREE HiLo Joint Plus 140gr', 38300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Joint Plus Orange 140 gram', 38300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['(FREE) HiLo Joint Plus Orange 140 gram', 0]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: Tropicana Slim Korean Goguma Cookies (5 Sch) x 2', 32200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Korean Goguma Cookies (5 Sch)', 16100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NuTrilogi Seri Mba Nana Si Penuh Pesona', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Teen Popcorn Caramel 500gr', 68600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School Chocolate 1000 gr', 141900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['BUY 1 GET 1 - Lokalate Kopi Andaliman 10 Sachet', 13700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['BUY 1 GET 1 - Lokalate Kopi Tape Ketan 10 Sachet', 13700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Special Bundle - Tropicana Slim Sweetener Rose Vanilla & Tropicana Slim Korean Goguma Cookies', 55000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Lokalate Kopi Andaliman 10 Sachet', 6850]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Lokalate Kopi Tape Ketan 10 Sachet', 6850]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Sweetener Rose Vanilla 50 sch', 40000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['(FREE) Lokalate Kopi Andaliman 10 Sachet', 0]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['(FREE) Lokalate Kopi Tape Ketan 10 Sachet', 0]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Korean Goguma Cookies (5 Sch)', 16100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Gold Plain 1000 gr', 128200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NuTrilogi Seri Mas Kaka yang Banyak Akal', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: HiLo School Strawberry Cheesecake 500 gram x 2', 110600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School Strawberry Cheesecake 500 gram', 55300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: HiLo Joint Plus Orange 140 gram x 2', 55300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School Vanilla Vegiberi 750gr', 76600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Madu Kurma Pack (4R)', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Nutrisari Jeruk Jeju [40 Sachet]', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Es Rujak (4R)', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Nutrisari Jeruk Nipis (40 sch)', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Madu Lemon (40 Sch)', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Thai Tea (10 sch)', 13700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Chocolate Taro RTD 200ml (4pcs)', 26100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Swiss Chocolate (10 sch) Renceng', 17200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo Teen Coffee Tiramisu Ready To Drink Susu UHT [4 pcs]', 26100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo Teen Chocolate Ready To Drink Susu [4 pcs]', 26100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo School Vanilla Vegiberi Ready To Drink Susu [200ml/4 pcs]', 26100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Lychee Tea 40 sch', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Yuzu Orange 40 sch', 46200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Jeruk Manis 500gr', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: HiLo Gold Sweet Potato 500 gram x 2', 105600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin pack - Lokalate Kopi Andaliman 10 Sachet', 19800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Gold Sweet Potato 500 gram', 52800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Lokalate Kopi Andaliman 10 Sachet', 13700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin pack - Lokalate Kopi Tape Ketan 10 Sachet', 19800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Lokalate Kopi Tape Ketan 10 Sachet', 13700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim DIABTX (100 sch)', 76100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Nutty Chocolate Cookies 200gr', 40000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Markisa (40 Sch)', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Nutrisari Madu Jeruk (40sch)', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari NUTRI C1000 Jeruk 40 Sachet', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Active Chocolate 500gr', 61800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["W'Dank Bajigur (10 sch)", 17200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Sweetener I Sweet', 21700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Active Kacang Hijau 200gr', 40000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack - Lokalate Kopi Tape Ketan 10 Sachet', 19800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Lokalate Kopi Tape Ketan 10 Sachet', 13700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Classic Refill 500gr', 98400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Milk Skim Fiber Pro Plain 500gr', 111000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Teen Vanilla Caramel 500gr', 76600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim High Fiber High Calcium Milk Chocolate 500gr', 112100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack - Lokalate Kopi Andaliman 10 Sachet', 35000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Lokalate Kopi Andaliman 10 Sachet', 13700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: HiLo Almond Milk Coconut 200gr x 2', 70000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Almond Milk Coconut 200gr', 35000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([['HiLo Yoghurt Smoothie Bowl Strawberry (8 sch)', 62900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["W'Dank Bandrek 10 sachet Renceng", 17200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo School Chocolate Candy (10 sch)", 18300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["2102500320 (CI160)", 72160]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["2102500336 (CS)", 45100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([["NutriSari Jeruk Maroko (40 Sch) FREE Lokalate Andaliman (10 Sch)", 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Jeruk Peras Refill 500 gr", 34300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Nutrisari Jeruk Maroko (40 sch)", 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(FREE) Lokalate Kopi Andaliman 10 Sachet", 13700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Twin Pack:HiLo Teen Strawberry Milkshake 500g x 2", 100000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo Teen Strawberry Milkshake 500g", 50000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo School Chocolate Ready To Drink (4 tetrapack)", 26100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo School RTD Coklat 200ml", 6525]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Tropicana Slim Sirup Leci 750ml", 30900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([["NutriSari Mango Smoothie 200ml (4 Pcs)", 27500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari RTD Jeruk Madu 200 ml (4 tetrapack)", 22900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari RTD Mango Smoothie 200 ml", 6900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari RTD Jeruk Madu 200 ml", 5750]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo Active Chocolate 750gr", 89200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["L-Men Hi Protein 2 Go Chocolate x 4 pcs", 38400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["L-Men Hi Protein 2 Go Chocolate", 9600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Sweetener Honey 50 sachet', 40000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Triple Pack - Tropicana Slim Klepon Cookies (5 Sch) - Snack Bebas Gula, 100 Kalori', 50000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Klepon Cookies (5 Sch)', 16700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Platinum Swiss Chocolate 420gr', 91500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School Chocolate 250g (Health & Green Science Competition)', 38300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['BUY 1 GET 1 - Tropicana Slim Shirataki Rice 72 g', 50000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Shirataki Rice 72 g', 25000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Jeruk Nipis Jahe 40 Sachet', 35000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Active Vanilla 500gr', 65200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Teen Yoghurt Banana 250gr', 38400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['TS SHIRATAKI RICE RASA NASI UDUK 24PX72G', 29700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School Cotton Candy 500g', 62500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Es Kuwud Nipis', 37500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Lychee Tea Refill 500 gram', 34300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Thai Tea Ready to Drink 200ml x 4pcs', 23600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo RTD Thai Tea 200ml', 5900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Buy 1 Get 1 Free - HiLo School Bubble Gum 500g', 92400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['(Near ED) HiLo School RTD Coklat 200ML', 6270]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['BUY 1 GET 1 - Tropicana Slim Brownies Instant Powder Cake Mix 230 g - Bantu Dukung Hidup Sehat', 59400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Klepon Latte Refill 500g - Powder Drink Tinggi Kalsium', 49500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim BROWNIES  Instant Powder Cake Mix 230 g', 29700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Active Caramel Latte 500g', 50500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Avocado Chocolate (10 Sch)', 127160]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Bundle - HiLo Teen Taro 500gr & HiLo Teen Strawberry Milkshake 500g', 11400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Teen Taro 500gr', 63600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Teen Strawberry Milkshake 500g', 63600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['(Near ED)Bundle L-Men Platinum Ketan Hitam 800g & L-Men Platinum Kacang Hijau 800g', 314600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['(Near ED)L-Men Platinum Ketan Hitam 800g ', 157300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['(Near ED)L-Men Platinum Kacang Hijau 800g', 157300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['(Near ED)Twinpack: L-Men Platinum Kacang Hijau 800g', 314600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['(Near ED)L-Men Platinum Kacang Hijau 800g', 157300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Es Ketan Hitam 500g - Powder Drink Tinggi Kalsium', 44000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Teen Biscuit Caramel 500gr', 65000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Gold Plain (Original) 500gr', 77800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Lemon Tea 500 gram', 34300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['BUY 1 GET 1 - Lokalate Kopi Pisang Bakar 10 sch', 17500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Lokalate Kopi Pisang Bakar Epe 10 sch', 8750]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School Bubble Gum 500g', 80100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack/B1G1 HiLo Multigrain Original 500g', 176000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Multigrain Original 500g', 88000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Lemon-C (25 sch)', 25200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Belgian Chocolate (10 sch)', 49200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([["HiLo Chocolate Polos 10's", 12000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo White Chocolate (10 Sch)', 12000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Cookies Paket Hampers Idul Fitri Parcel Lebaran', 131600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Chocolate Banana 10 sch', 12000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Hokkaido Cheese Cookies', 22300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Nutty Chocolate Cookies 200gr', 40000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Korean Garlic Butter Cookies 5 Sch', 22300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Korean Goguma Cookies (5 Sch)', 22300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Klepon Cookies (5 Sch)', 21500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Gift - Paper Bag Ramadhan Tropicana Slim', 0]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([['HiLo Es Pisang Ijo 10 Sch x 2 pcs', 17500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Gain Mass Mangga 500gr', 132700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Buy 1 Get 1 - Tropicana Slim Mint Cocoa - Minuman Cokelat Mint Nikmat Tanpa Gula Pasir', 20000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Buy 1 Get 1 FREE L-Men Lose Weight Mango Sticky Rice 300gr', 145200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Buy 1 Get 1 FREE Tropicana Slim Susu Low Fat Macchiato Coffee 500gr', 143000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Diabetamil Milk Vanilla 150 gram', 29700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Susu Low Fat Macchiato Coffee 500 gram', 71500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Brownies Instant Powder Cake Mix 230 g - Bantu Dukung Hidup Sehat', 30900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-MEN PLANTPROTEIN OGURA 6Dx216G', 103900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Active Ketan Hitam 175gr - Near ED', 16150]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Choco Series Paket Hampers Idul Fitri Parcel Lebaran', 139600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Gold Chocolate 1000g - Susu Tinggi Kalsium Lebih Rendah Lemak', 104800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School Vanilla 1000g - Susu Tinggi Kalsium Lebih Rendah Lemak', 115500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Active Vanilla 1000g - Susu Tinggi Kalsium Lebih Rendah Lemak', 86650]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Es Pisang Ijo 10 Sch', 14300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['BUY 1 GET 1 - Tropicana Slim Soy Latte - Kopi Plant Based Rendah Gula', 36900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Mint Cocoa 4 Sachet', 20000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Gain Mass Taro 500g x 2 pcs', 133900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Buy 1 Get 1 Free - HiLo Gold Biscuit Cereal 500g', 80800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Lokalate Kopi Alpukat 500 gram - Tinggi Vitamin A Lebih Rendah Gula', 44400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Thai Tea Refill 500g', 48000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Soy Latte - Kopi Susu Kacang Bebas Gula', 36900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Lokalate Kopi Berondong 500 gram - Tinggi Vitamin A Lebih Rendah Gula', 44400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['2102900319 (D)', 90400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Gold Biscuit Cereal 500g', 80800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Anggur Hijau 40 Sachet', 43300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Gula Asem 40 Sachet', 43300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack - Tropicana Slim Low Fat Milk Korean Strawberry 500g', 159900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Low Fat Milk Korean Strawberry 500g', 79900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Hokkaido Cheese Cookies 100gr', 22500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Wdank Madu Temulawak 10 Sachet', 16600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Es Ketan Hitam Refill 500g - Powder Drink Tinggi Kalsium', 59400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([["(Near ED) Lokalate Kopi Tape Ketan 10's", 7500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['(Near ED) HiLo Joint Plus Orange 140 gram', 19150]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['(Near ED) BUY 1 GET 1 L-Men Gain Mass Mangga 500gr', 132700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['(Near ED) HiLo Teen Taro 500gr', 43300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Bundle - Lokalate Kopi Alpukat Refill 500 gram & Kopi Berondong Refill 500 gram', 92400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Thai Tea RTD 200ml x 4pcs', 24000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["BUY 1 GET 1 - W'Dank Temulawak Madu", 33300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([['TRIPLE HEMAT - HiLo Thai Tea Refill 500g', 100900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['TRIPLE HEMAT - NutriSari Lemon Tea 500 gram', 72700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["W'dank Sarabba Extra 10 Sachet", 18870]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["W'dank Madu Temulawak 10 Sachet", 16650]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([['Nutrisari Blewah 40 sch', 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Nutrisari Mangga Gandaria 40 sch', 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Nutrisari Jeruk Nipis 40 sch", 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Nutrisari American Sweet Orange 40 sch", 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Nutrisari Jeruk Peras 40 sch', 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Nutrisari Milky Orange 40 sch', 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo Chocolate Polos 10 sch", 12100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Madu Lemon 40 sch", 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([["Nutrisari Florida Orange 40 sch", 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Anggur Hijau 40 sch", 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari NUTRI C1000 Jeruk 40 sch', 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Markisa 40 sch', 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Es Kuwud Nipis 40 sch", 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["TRIPLE HEMAT - NutriSari Lychee Tea Refill 500 gram", 72700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([["Nutrisari Jeruk Maroko 40 sch", 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Apel Jeruk 40 sch", 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo Chocolate Taro 10 sch', 12100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Avocado Chocolate 10 sch', 12100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["BUY 1 GET 1 - HiLo Platinum Original 360 gram", 111000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["TRIPLE DEALS - NutriSari Lychee Tea & Lemon Tea 500 gram Refill", 72700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Gula Asem 40 sch", 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Nutrisari Madu Jeruk 40 sch", 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Lemon Tea Refill 500 gram", 34600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["TRIPLE HEMAT - HiLo Klepon Latte Refill 500g", 109100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Twinpack Tropicana Slim Bumbu Saus Telur Asin 3 Sachet - Lebih Rendah Lemak dan Garam", 29000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Madu Kurma Pack 40 sch", 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo White Chocolate 10 sch", 12100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo Taro Latte Refill 500g", 32700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Es Rujak 40 sch", 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Es Rujak Jeruk Bali 40 sch", 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["TRIPLE DEALS - HiLo Thai Tea , Klepon Latte , Taro Latte Refill 500g", 103900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([["Buy 3 Get 3 - HiLo Milky Vanilla Cookies Ready to Drink 200ml", 60000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Twinpack Lokalate Kopi Alpukat Refill 500 gram", 92400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([["Twinpack Lokalate Kopi Berondong Refill 500 gram", 92400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Nutrisari Jeruk Jeju 40 sch", 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([["NutriSari Jeruk Nipis Jahe 40 sch", 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["2102500320 (CI160) TS SWT 160S IND", 76590]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([["HiLo Platinum Original 360g (12 Sachet)", 127500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    price = product_order[product_order['title'] == 'Tropicana Slim Goldenmil Vanilla (6sch)']['price'].values[0]
    product_order = product_order.append(pd.DataFrame([['BUY 1 GET 1 Tropicana Slim Goldenmil Vanilla (6sch)', price]], columns = ['title', 'price']), ignore_index = True, sort = False)

    price = product_order[product_order['title'] == 'Lokalate Kopi Kawista (10sch)']['price'].values[0]
    product_order = product_order.append(pd.DataFrame([['BUY 1 GET 1 Lokalate Kopi Kawista (10sch)', price]], columns = ['title', 'price']), ignore_index = True, sort = False)

    price = product_order[product_order['title'] == 'Tropicana Slim Kecap Manis 200ml']['price'].values[0]
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Kecap Manis 200m', price]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    
    
    
    data_SKU = pd.read_excel(r'Order Online\SKU buat andra.xlsx')
    data_SKU = data_SKU.rename(columns = {'Unnamed: 0' : 'SKU', 'ref.1' : 'product', 'Pricelist Nutrimart' : 'Price List NFI', 'Harga jua Homdel' : 'Harga Display'})
    data_SKU['SKU'] = data_SKU['SKU'].astype(str).str.replace('.0', '', regex = False)
    data_SKU = data_SKU[data_SKU['SKU'] != 'nan'][data_SKU[data_SKU['SKU'] != 'nan']['SKU'] != '0']
    data_SKU['product'] = data_SKU['product'].str.strip()

    data_order['product'] = data_order['product'].astype(str)
    data_SKU['product'] = data_SKU['product'].astype(str)
    product_order['title'] = product_order['title'].astype(str)

    data_order = data_order.merge(data_SKU[['SKU', 'product']].drop_duplicates('product'), how = 'left', on = 'product')
    data_order = data_order.merge(product_order[['title', 'price']].drop_duplicates('title'), how = 'left', left_on = 'product', right_on = 'title').drop('title', axis = 1)

    data_order = data_order.reset_index(drop = True)

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()
    for i in indeks:
        col = [x for x in data_SKU.columns if 'Alias Nama' in x]
        for j in col:
            if data_order['product'][i] in data_SKU[j].astype(str).values:
                SKU = data_SKU[data_SKU[j].astype(str) == data_order['product'][i]]['SKU'].values[0]
                data_order['SKU'][i] = SKU

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    data_SKU2 = pd.read_excel(r'SKU_File\data_SKU.xlsx')
    data_SKU2['Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

    s = requests.Session()
    s.get("http://tatanama.pythonanywhere.com")
    s.post("http://tatanama.pythonanywhere.com", data = {'username' : 'ecommerce', 'password' : 'ecommerce'})
    r = s.get("http://tatanama.pythonanywhere.com/download")

    with open(r'SKU_File\Master tatanama.xlsx', 'wb') as output:
        output.write(r.content)

    if os.path.isfile(r'SKU_File\Master tatanama.xlsx') :    
        SKU_append = pd.read_excel(r'SKU_File\Master tatanama.xlsx')
        SKU_append.columns = [x.replace('_', ' ') for x in SKU_append.columns]
        data_SKU2 = data_SKU2[~data_SKU2['SKU'].astype(str).isin(SKU_append['SKU'].astype(str))]
        data_SKU2 = data_SKU2.append(SKU_append, ignore_index = True, sort = False)

    to_excel = data_SKU2.to_excel(r'SKU_File\data_SKU.xlsx', index = False)

    for i in indeks:
        if str(data_order['product'][i]).lower() in data_SKU2['Nama Produk'].astype(str).str.lower().values:
            data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2['Nama Produk'].astype(str).str.lower()].values[0]

    list_alias_name = [x for x in data_SKU2.columns if 'Alias Nama' in x]

    for i in indeks:
        for j in list_alias_name:
            if str(data_order['product'][i]).lower() in data_SKU2[j].astype(str).str.lower().values:
                data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2[j].astype(str).str.lower()].values[0]

    
    
    
    indeks = data_order[data_order['SKU'].isnull()].index.to_list()
    

    if len(indeks) != 0:
        print('Alert SKU Missing')
        data_order['product'][indeks].drop_duplicates().to_excel('Alert SKU Missing.xlsx', index = False)
    else :
        data_order['phone'] = data_order['phone'].astype(str).str.replace('+628', '08', regex = False)

        data_order['zip'] = data_order['zip'].replace('.0', '', regex = False)

        data_order = data_order.rename(columns = {'order_id' : 'Sales Order ID', 'name' : 'Customer Name', 'product' : 'Item Name', 'price' : 'Price', 'shipping_cost' : 'Shipping Cost', 'address' : 'Shipping Address1', 'city' : 'Shipping City', 'zip' : 'Shipping Zip', 'province' : 'Shipping Province', 'phone' : 'Shipping Phone', 'courier' : 'Shipping Courier'})
        data_order['Channel Order ID'] = data_order['Sales Order ID']
        data_order['Invoice Number'] = data_order['Sales Order ID']
        data_order['Shipping Name'] = data_order['Customer Name']
        data_order['Shipping Address2'] = 0
        data_order['Shipping Country'] = 'Indonesia'
        data_order['AWB'] = 0
        data_order['Channel'] = 'Order Online Surabaya'

        data_order['Order Date'] = pd.to_datetime(data_order['created_at'])

        indeks = data_order[data_order['Item Name'].astype(str).str.contains('pcs')].index.to_list()
        temp = data_order.iloc[indeks].copy()
        product = temp['Item Name'].str.split("\(",1, expand = True)

        data_order['SKU'] = data_order['SKU'].astype(str)
        data_order['Item Name'] = data_order['Item Name'].astype(str)
        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str).str.replace('(S)', '', regex = False)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        data_order = data_order.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('hd','', regex = False)
        temp['SKU'] = temp['SKU'].astype(str).str.replace('HD','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        data_order['Real SKU'] = data_order['Real SKU'].astype(str)
        data_order = data_order.merge(data_SKU2[['SKU', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Unbundling ====== 6/10")        
        # Forstok Unbundling    
        list_col = ['SKU'] + data_SKU2.columns[data_SKU2.columns.get_loc('Produk 1'):data_SKU2.columns.get_loc('Harga Organik 7')+1].to_list()
        data_order = data_order.merge(data_SKU2[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        list_pcs = [x for x in data_order.columns if 'PCS' in x]
        for i in list_pcs:
            data_order[i] = data_order[i] * data_order['Quantity']
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        indeks = data_order[data_order['Brand'] == 'Bundle'].index.to_list()
        data_order['Bundle Flag'] = np.nan
        data_order['Bundle Flag'][indeks] = 'Bundle'

        indeks = data_order[data_order['Brand'] == 'Bundle'][data_order[data_order['Brand'] == 'Bundle']['SKU'].astype(str).str.contains('(S)', regex = False)].index.to_list()
        data_order['SKU Produk 1'][indeks] = '(S)' + data_order['SKU Produk 1'][indeks].astype(str)
        data_order['SKU Produk 2'][indeks] = '(S)' + data_order['SKU Produk 2'][indeks].astype(str)
        data_order['SKU Produk 3'][indeks] = '(S)' + data_order['SKU Produk 3'][indeks].astype(str)
        data_order['SKU Produk 4'][indeks] = '(S)' + data_order['SKU Produk 4'][indeks].astype(str)
        data_order['SKU Produk 5'][indeks] = '(S)' + data_order['SKU Produk 5'][indeks].astype(str)
        data_order['SKU Produk 6'][indeks] = '(S)' + data_order['SKU Produk 6'][indeks].astype(str)
        data_order['SKU Produk 7'][indeks] = '(S)' + data_order['SKU Produk 7'][indeks].astype(str)

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Filling Date ====== 7/10")
        data_order['Date'] = np.nan
        data_order['Month'] = np.nan
        data_order['Year'] = np.nan

        for i in range(data_order.shape[0]):
            if int(data_order['Order Date'][i].strftime('%d')) <= 12:
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).year
            else :
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i]).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i]).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i]).year

        quarter = pd.DataFrame([['January', 1], ['February', 1], ['March', 1], ['April', 2], ['May', 2], ['June', 2], 
                ['July', 3], ['August', 3], ['September', 3],['October', 4], ['November', 4], ['December', 4]], columns = ['Bulan', 'Quarter'])
        data_order = data_order.merge(quarter, how = 'left', left_on = 'Month', right_on = 'Bulan')
        data_order = data_order.drop(['Bulan'], axis = 1)
        data_bulan = pd.DataFrame([{'Bulan' : 'December', 'Number' : 12} ,
                {'Bulan' : 'January' , 'Number': 1},
                {'Bulan' : 'February' , 'Number': 2},
                {'Bulan' : 'March' , 'Number': 3},
                {'Bulan' : 'April' , 'Number': 4},
                {'Bulan' : 'May' , 'Number': 5},
                {'Bulan' : 'June', 'Number': 6},
                {'Bulan' : 'July' , 'Number': 7},
                {'Bulan' : 'August', 'Number' : 8},
                {'Bulan' : 'September', 'Number' : 9},
                {'Bulan' : 'October' , 'Number': 10},
                {'Bulan' : 'November' , 'Number': 11}])
        temp = data_order.copy()
        temp['Day'] = temp['Date']
        temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
        temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
        data_order['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.week
        temp['Hour'] = pd.to_datetime(data_order['Order Date']).dt.hour
        temp['Minute'] = pd.to_datetime(data_order['Order Date']).dt.minute
        temp['Second'] = pd.to_datetime(data_order['Order Date']).dt.second
        data_order['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']])



        order_all = data_order.copy()
        index = order_all[order_all['SKU'].astype(str) == '2306551174'].index.to_list()
        order_all['SKU'][index] = '2306592173'
        order_all['Real SKU'][index] = '2306592173'
        order_all['Total'] = order_all['net_revenue']
        order_all['Price List NFI'] = np.nan
        order_all['Total Net'] = np.nan

        order_all = order_all.rename(columns={'Channel Order ID' : 'Order #',
                                                'Status' : 'Order Status',
                                                'Order Date' : 'Order date',
                                                'Item Name' :'Product Name',
                                                'Bundle Name' : 'Bundle',
                                                'Shipping Country' : 'Country',
                                                'Shipping Province' : 'Region',
                                                'Shipping City' : 'City',
                                                'Shipping Zip' : 'Zip Code',
                                                'Shipping Address1' : 'Address',
                                                'Shipping Phone' : 'Phone',
                                                'Quantity' : 'Qty. Invoiced',
                                                'Price' : 'Regular Price',
                                                'net_revenue' : 'Subtotal'})
        
        
        order_all['Kecamatan'] = np.nan
        order_all['Kelurahan'] = np.nan

        print("Filling Location")
        indeks = order_all[order_all['City'].astype(str).str.contains('/')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains('-')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[0]

        indeks = order_all[order_all['City'].astype(str).str.contains(',')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains(',')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[0]

        order_all['City'] = order_all['City'].astype(str).str.replace('Kab\.', 'Kabupaten' ,case = False)

        master_map = pd.read_csv(r'All Data/Province.csv', names = ['Kode Prov', 'Province'], header= 0)
        master_map2 = pd.read_csv(r'All Data/City.csv', names = ['Kode City', 'Kode Prov', 'City'], header = 0)
        master_map = master_map.merge(master_map2, how = 'right', on = 'Kode Prov')
        master_map['Kode Prov'][515] = 14
        master_map['Province'][515] = 'Riau'
        master_map['Kode Prov'] = master_map['Kode Prov'].astype(int)
        master_map['Province'] = master_map['Province'].str.title()
        master_map['City'] = master_map['City'].str.title()

        city = pd.read_excel(r'All Data/list_city.xlsx')
        temp = order_all.copy()
        temp['City'] = temp['City'].astype(str).str.lower()
        temp['City'] = temp['City'].astype(str).str.replace('kab. ', 'kabupaten ', regex = False, case = False)
        city['All City'] = city['All City'].astype(str).str.lower()
        temp = temp.merge(city.drop_duplicates('All City'), how = 'left', left_on = 'City', right_on = 'All City').set_index(temp.index)
        indeks = temp[temp['Real City'].notnull()].index.to_list()
        order_all['City'][indeks] = temp['Real City'][indeks]

        province = pd.read_excel(r'All Data/list_province.xlsx')
        temp = order_all.copy()
        temp['Region'] = temp['Region'].astype(str).str.lower()
        province['All Province'] = province['All Province'].astype(str).str.lower()
        temp = temp.merge(province.drop_duplicates('All Province'), how = 'left', left_on = 'Region', right_on = 'All Province').set_index(temp.index)
        indeks = temp[temp['Real Province'].notnull()].index.to_list()
        order_all['Region'][indeks] = temp['Real Province'][indeks]

        temp = order_all.copy()
        temp = temp[temp['Region'].isnull()]
        temp['Region'] = temp.merge(master_map, how = 'left', on = 'City').set_index(temp.index)['Province']
        order_all['Region'][temp.index] = temp['Region']  

        district = pd.read_excel(r'All Data/list_district.xlsx')
        temp = order_all.copy()
        temp['Kecamatan'] = temp['Kecamatan'].astype(str).str.lower()
        district['All District'] = district['All District'].astype(str).str.lower()
        temp = temp.merge(district.drop_duplicates('All District'), how = 'left', left_on = 'Kecamatan', right_on = 'All District').set_index(temp.index)
        indeks = temp[temp['Real District'].notnull()].index.to_list()
        order_all['Kecamatan'][indeks] = temp['Real District'][indeks]

        temp = order_all.copy()
        temp2 = temp[['Region', 'City', 'Kecamatan']].merge(master_map, how = 'left', on = 'City')
        indeks = temp2[temp2['Region'] != temp2['Province']][temp2[temp2['Region'] != temp2['Province']]['City'].notnull()].index.to_list()
        order_all['City'][indeks] = np.nan

        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        print("Unbundling")
        data_bundle1 = order_all[~order_all['Produk 1'].isnull()]
        data_bundle1['Bundle Name'] = data_bundle1['Product Name']
        data_bundle1['Product Name'] = data_bundle1['Produk 1']
        data_bundle1['SKU'] = data_bundle1['SKU Produk 1']
        data_bundle1['Qty. Invoiced'] = data_bundle1['PCS Produk 1']
        data_bundle1['Price List NFI'] = data_bundle1['Price List NFI 1']
        data_bundle1['Total Net'] = data_bundle1['Price List NFI 1']
        data_bundle1['Bundle Flag'] = np.nan

        data_bundle2 = order_all[~order_all['Produk 2'].isnull()]
        data_bundle2['Bundle Name'] = data_bundle2['Product Name']
        data_bundle2['Product Name'] = data_bundle2['Produk 2']
        data_bundle2['SKU'] = data_bundle2['SKU Produk 2']
        data_bundle2['Qty. Invoiced'] = data_bundle2['PCS Produk 2']
        data_bundle2['Price List NFI'] = data_bundle2['Price List NFI 2']
        data_bundle2['Total Net'] = data_bundle2['Price List NFI 2'] 
        data_bundle2['Bundle Flag'] = np.nan

        data_bundle3 = order_all[~order_all['Produk 3'].isnull()]
        data_bundle3['Bundle Name'] = data_bundle3['Product Name']
        data_bundle3['Product Name'] = data_bundle3['Produk 3']
        data_bundle3['SKU'] = data_bundle3['SKU Produk 3']
        data_bundle3['Qty. Invoiced'] = data_bundle3['PCS Produk 3']
        data_bundle3['Price List NFI'] = data_bundle3['Price List NFI 3']
        data_bundle3['Total Net'] = data_bundle3['Price List NFI 3'] 
        data_bundle3['Bundle Flag'] = np.nan

        data_bundle4 = order_all[~order_all['Produk 4'].isnull()]
        data_bundle4['Bundle Name'] = data_bundle4['Product Name']
        data_bundle4['Product Name'] = data_bundle4['Produk 4']
        data_bundle4['SKU'] = data_bundle4['SKU Produk 4']
        data_bundle4['Qty. Invoiced'] = data_bundle4['PCS Produk 4']
        data_bundle4['Price List NFI'] = data_bundle4['Price List NFI 4']
        data_bundle4['Total Net'] = data_bundle4['Price List NFI 4'] 
        data_bundle4['Bundle Flag'] = np.nan

        data_bundle5 = order_all[~order_all['Produk 5'].isnull()]
        data_bundle5['Bundle Name'] = data_bundle5['Product Name']
        data_bundle5['Product Name'] = data_bundle5['Produk 5']
        data_bundle5['SKU'] = data_bundle5['SKU Produk 5']
        data_bundle5['Qty. Invoiced'] = data_bundle5['PCS Produk 5']
        data_bundle5['Price List NFI'] = data_bundle5['Price List NFI 5']
        data_bundle5['Total Net'] = data_bundle5['Price List NFI 5']
        data_bundle5['Bundle Flag'] = np.nan

        data_bundle6 = order_all[~order_all['Produk 6'].isnull()]
        data_bundle6['Bundle Name'] = data_bundle6['Product Name']
        data_bundle6['Product Name'] = data_bundle6['Produk 6']
        data_bundle6['SKU'] = data_bundle6['SKU Produk 6']
        data_bundle6['Qty. Invoiced'] = data_bundle6['PCS Produk 6']
        data_bundle6['Price List NFI'] = data_bundle6['Price List NFI 6']
        data_bundle6['Total Net'] = data_bundle6['Price List NFI 6']
        data_bundle6['Bundle Flag'] = np.nan

        data_bundle7 = order_all[~order_all['Produk 7'].isnull()]
        data_bundle7['Bundle Name'] = data_bundle7['Product Name']
        data_bundle7['Product Name'] = data_bundle7['Produk 7']
        data_bundle7['SKU'] = data_bundle7['SKU Produk 7']
        data_bundle7['Qty. Invoiced'] = data_bundle7['PCS Produk 7']
        data_bundle7['Price List NFI'] = data_bundle7['Price List NFI 7']
        data_bundle7['Total Net'] = data_bundle7['Price List NFI 7']
        data_bundle7['Bundle Flag'] = np.nan

        data_bundle = data_bundle1.append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)
        data_bundle['SKU'] = data_bundle['SKU'].astype(str)
        data_bundle['SKU'] = data_bundle['SKU'].str.replace('\.0$', '', regex = True)
        data_bundle[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']] = data_bundle.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item_y', 'Parent SKU_y']]

        temp = data_bundle[data_bundle['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)

        indeks = data_bundle[data_bundle['Real SKU'].isnull()].index.to_list()
        data_bundle['Real SKU'][indeks] = temp['Real SKU_y'][indeks]
        data_bundle['Real Nama Produk'][indeks] = temp['Nama Produk'][indeks]
        data_bundle['Brand'][indeks] = temp['Brand_y'][indeks]
        data_bundle['Sub Brand'][indeks] = temp['Sub Brand_y'][indeks]
        data_bundle['Parent Item'][indeks] = temp['Parent Item_y'][indeks]
        data_bundle['Parent SKU'][indeks] = temp['Parent SKU_y'][indeks]

        print("Pricing")
        order_all = order_all.append(data_bundle, ignore_index = True, sort = False)

        colname = temp.columns[temp.columns.get_loc('Produk 1') : temp.columns.get_loc('Harga Cost 7') + 1]
        colname_str = [x for x in colname if 'Subtotal' not in x and 'Harga' not in x]
        colname_int = [x for x in colname if x not in colname_str]

        for i in colname_str:
            temp[i] = np.nan

        for i in colname_int:
            temp[i] = 0

        data_order = data_order.append(temp , ignore_index = True, sort = False)


        order_all = order_all.merge(data_SKU2[['SKU', 'Price List NFI', 'Harga Cost']].drop_duplicates('SKU'), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(order_all.index)
        order_all['Price List NFI_x'] = order_all['Price List NFI_x'].fillna(order_all['Price List NFI_y'])
        order_all =  order_all.drop(['Price List NFI_y', 'SKU_y'], axis = 1)
        order_all = order_all.rename(columns = {'SKU_x' : 'SKU', 'Price List NFI_x' : 'Price List NFI'})

        order_all['Price List NFI'] = pd.to_numeric(order_all['Price List NFI']).astype(int)
        order_all['Harga Cost'] = pd.to_numeric(order_all['Harga Cost']).astype(int)
        order_all['Qty. Invoiced'] = pd.to_numeric(order_all['Qty. Invoiced']).astype(int)

        order_all['Total Net'] = order_all['Price List NFI'] * order_all['Qty. Invoiced']
        order_all['Total Harga Cost'] = order_all['Harga Cost'] * order_all['Qty. Invoiced']
        
        temp = order_all[order_all['Brand'] != 'Bundle']
        temp['discount'] = temp['discount'] * temp['Total Harga Cost']/temp.groupby(['Order #'])['Total Harga Cost'].transform('sum')
        order_all['discount'][temp.index] = temp['discount']
        
        order_all['Selling Price'] = order_all['Regular Price'].astype(int)
        
        data_SKU3 = pd.read_excel(r'Order Online\SKU buat andra updated maret 2021.xlsx')
        data_SKU3 = data_SKU3.rename(columns = {'Product' : 'product', 'PL NFI' : 'Price List NFI'})
        data_SKU3['SKU'] = data_SKU3['SKU'].astype(str).str.replace('.0', '', regex = False)
        data_SKU3 = data_SKU3[data_SKU3['SKU'] != 'nan'][data_SKU3[data_SKU3['SKU'] != 'nan']['SKU'] != '0'][data_SKU3[data_SKU3['SKU'] != 'nan'][data_SKU3[data_SKU3['SKU'] != 'nan']['SKU'] != '0']['Harga Coret'].notnull()]
        data_SKU3['product'] = data_SKU3['product'].str.strip()
        data_SKU3 = data_SKU3.reset_index(drop = True)
        
        for i in range(data_SKU3.shape[0]):
            if data_SKU3['SKU'][i] in order_all['SKU'].values:
                indeks = order_all[order_all['SKU'].astype(str) == i].index.to_list()
                order_all['Regular Price'][indeks] = data_SKU3['Harga Display'][i]
                order_all['Selling Price'][indeks] = data_SKU3['Harga Coret'][i]
        
        
        order_all['Subtotal'] = order_all['Selling Price'] * order_all['Qty. Invoiced']
        order_all['Total'] = order_all['Selling Price'] * order_all['Qty. Invoiced']

        order_all = order_all.reset_index(drop = True)
        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        order_all['Seller Discount'] = order_all['discount']
        order_all['Shipping'] = order_all['Shipping Cost']

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Subtotal', 'Total']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Total'][list_nobundle.index] = list_nobundle['Total_y']
        order_all['Subtotal'][list_nobundle.index] = list_nobundle['Subtotal_y']

        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Subtotal'] = temp['Subtotal'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        temp['Selling Price'] = temp['Subtotal']/temp['Qty. Invoiced']
        temp['Total'] = temp['Total'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Total'][temp.index] = temp['Total']
        order_all['Subtotal'][temp.index] = temp['Subtotal']
        order_all['Selling Price'][temp.index] = temp['Selling Price']


        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Seller Discount']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Seller Discount'][list_nobundle.index] = list_nobundle['Seller Discount_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Seller Discount'] = temp['Seller Discount'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Seller Discount'][temp.index] = temp['Seller Discount']


        temp = order_all[order_all['Bundle Name'].isnull()]
        temp_group = temp[['Order #','Shipping']].groupby(['Order #']).sum().reset_index()

        temp = order_all.merge(temp_group, how = 'left', on = 'Order #').set_index(order_all.index)
        temp['Shipping_x'] = temp['Shipping_y'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Shipping'][temp.index] = temp['Shipping_y']
        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Shipping']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Shipping'][list_nobundle.index] = list_nobundle['Shipping_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Shipping'] = temp['Shipping'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Shipping'][temp.index] = temp['Shipping']
        order_all['True datetime'] = pd.to_datetime(order_all['True datetime'])
        order_all['Promo'] = np.nan
        order_all['Discount MC'] = np.nan

        order_all['Warehouse Name'] = 'Order Online Surabaya Warehouse'
        order_all['Store'] = 'Order Online'

        order_all['Customer Email'] = order_all['email']
        order_all['Order Status'] = order_all['status']
        order_all['Payment Channel'] = order_all['payment_method']
        order_all['Coupon Code'] = order_all['coupon']
        order_all.columns.to_list()

        order_all_append = order_all[['Sales Order ID', 'Store',
         'Product Name',
         'Customer Name',
         'Phone',
         'Address',
         'Region',
         'City',
         'Zip Code',
         'payment_status',
         'Regular Price',
         'Shipping Courier',
         'Shipping Cost',
         'Subtotal',
         'Qty. Invoiced',
         'SKU',
         'Order #',
         'Invoice Number',
         'Shipping Name',
         'Shipping Address2',
         'Country',
         'AWB',
         'Channel',
         'Order date',
         'Real SKU',
         'Real Nama Produk',
         'Brand',
         'Sub Brand',
         'Parent Item',
         'Parent SKU',
         'Produk 1',
         'SKU Produk 1',
         'PCS Produk 1',
         'Price List NFI 1',
         'Subtotal Produk 1',
         'Harga Display 1',
         'Harga Cost 1',
         'Harga Organik 1',
         'Produk 2',
         'SKU Produk 2',
         'PCS Produk 2',
         'Price List NFI 2',
         'Subtotal Produk 2',
         'Harga Display 2',
         'Harga Cost 2',
         'Harga Organik 2',
         'Produk 3',
         'SKU Produk 3',
         'PCS Produk 3',
         'Price List NFI 3',
         'Subtotal Produk 3',
         'Harga Display 3',
         'Harga Cost 3',
         'Harga Organik 3',
         'Produk 4',
         'SKU Produk 4',
         'PCS Produk 4',
         'Price List NFI 4',
         'Subtotal Produk 4',
         'Harga Display 4',
         'Harga Cost 4',
         'Harga Organik 4',
         'Produk 5',
         'SKU Produk 5',
         'PCS Produk 5',
         'Price List NFI 5',
         'Subtotal Produk 5',
         'Harga Display 5',
         'Harga Cost 5',
         'Harga Organik 5',
         'Produk 6',
         'SKU Produk 6',
         'PCS Produk 6',
         'Price List NFI 6',
         'Subtotal Produk 6',
         'Harga Display 6',
         'Harga Cost 6',
         'Harga Organik 6',
         'Produk 7',
         'SKU Produk 7',
         'PCS Produk 7',
         'Price List NFI 7',
         'Subtotal Produk 7',
         'Harga Display 7',
         'Harga Cost 7',
         'Harga Organik 7',
         'Bundle Flag',
         'Date',
         'Month',
         'Year',
         'Quarter',
         'Week',
         'True datetime',
         'Total',
         'Price List NFI',
         'Total Net',
         'Selling Price',
         'Kecamatan',
         'Kelurahan',
         'Bundle Name',
         'Harga Cost',
         'Total Harga Cost',
         'Seller Discount',
         'Shipping',
         'Promo',
         'Discount MC',
         'Warehouse Name',
         'Customer Email',
         'Order Status',
         'Payment Channel',
         'Coupon Code']]

        data_all = data_all[~data_all['Order #'].astype(str).isin(order_all_append['Order #'].astype(str))]
        data_all = data_all.append(order_all_append, ignore_index = True, sort = False)

        order_online_jatim = True
        del file_name

if not order_online_jkt:
    import pandas as pd 
    import numpy as np
    import requests
    import os
    
    print('order_online_jkt')
    #komen sementara
    before = os.listdir(os.getcwd() + '/Input Data')

    options = Options()
    options.add_experimental_option("prefs", {
            "download.default_directory": os.path.abspath("Input Data"),
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            "safebrowsing.enabled": False
    })

    driver = webdriver.Chrome(options=options)
    driver.fullscreen_window()
    driver.get("https://app.orderonline.id")

    username = driver.find_element_by_name("email")
    username.clear()
    username.send_keys("nutrimart.nfi@gmail.com")

    password = driver.find_element_by_name("password")
    password.send_keys("sumselhomdel")
    password.click()

    driver.find_element_by_class_name("btn-submit").click()

    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="main-nav-dropdown"]/ul/li[3]/a'))).click()
    time.sleep(5)
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[1]/div/div[2]/div/div/div').click()
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="app"]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[1]/div[1]/ul/li[6]'))).click()
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[2]/button[2]').click()
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[3]/div/button').click()
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[3]/div/div/button[1]').click()

    time.sleep(20)

    after = os.listdir(os.getcwd() + '/Input Data')
    change = set(after) - set(before)
    if len(change) == 1:
        file_name = change.pop()
    elif len(change) == 0: 
        print("No file downloaded")
    else :
        print("More than one file downloaded")

    data_order = pd.read_csv(r'Input Data/' + str(file_name))
    driver.close()

    product_order = pd.read_csv(r'data_supp/orderonline_products_Jakarta.csv')

    data_order['order_id'] = data_order['order_id'].fillna(method='ffill')
    data_order = data_order.drop('quantity', axis = 1)
    temp = data_order.drop_duplicates('order_id').drop('product', axis = 1)
    data_order = data_order[['order_id', 'product']]
    data_order = data_order.merge(temp, how = 'left', on = 'order_id')
    data_order['order_id'] = data_order['order_id'].astype(int)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Shirataki Noodles 71gr (x2) ', 'Twin Pack: Tropicana Slim Shirataki Noodles 71gr ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Saus Tiram 200ml (x2) ', 'Twin Pack: Tropicana Slim Saus Tiram 200ml ', regex = False)

    product = data_order['product'].str.split("\(x",1, expand = True)
    data_order['Quantity'] = product[1].astype(str).str.replace(')', '', regex = False).astype(int)
    data_order['product'] = product[0].str.strip().str.replace('  ', ' ').str.replace('6 SCH', '6sch')

    data_SKU = pd.read_excel(r'Order Online\SKU buat andra.xlsx')
    data_SKU = data_SKU.rename(columns = {'Unnamed: 0' : 'SKU', 'ref.1' : 'product', 'Pricelist Nutrimart' : 'Price List NFI', 'Harga jua Homdel' : 'Harga Display'})
    data_SKU['SKU'] = data_SKU['SKU'].astype(str).str.replace('.0', '', regex = False)
    data_SKU = data_SKU[data_SKU['SKU'] != 'nan'][data_SKU[data_SKU['SKU'] != 'nan']['SKU'] != '0']
    data_SKU['product'] = data_SKU['product'].str.strip()
    product_order['title'] = product_order['title'].str.strip().str.replace('  ', ' ')
    product_order['title'] = product_order['title'].str.strip().str.replace('L-Men Gain Mass Chocolate 500 gr', 'L Men Gain Mass Chocolate 500 gr')

    product_order = product_order.append(pd.DataFrame([['Nutrisari Mango Smoothie 200ml (6pcs)', 36300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Hi Protein 2 Go Chocolate (6pcs)', 48000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Hi Protein 2 Go Chocolate (24 TETRAPAK)', 264000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Bar Crunchy Chocolate (12sch)', 5500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Sweetener Honey (50sch)', 39500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Sirup Orange 750ml', 28600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Gain Mass Chocolate 225gr', 57500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Gainmass Taro 225gr', 69600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo Milk Brown Sugar RTD 200ml (6pcs)', 33000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Lose Weight Chocolate Cereal (12sch)', 99000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Gain Mass Chocolate 500 gr', 139200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Strawberry Jam 375gr', 72600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Hokkaido Cheese 100gr', 19800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Nutrisari Mangga Gandaria (40 sch)', 45000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Paket Cegah Diabetes (+Kaos)', 116100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo School Chocolate 250gr + Free Kertas Gambar', 40500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo School Chocolate 750gr + Free Kertas Gambar', 85800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Paket Nutrisari Jeruk Peras (40sch x 2) + Nutrisari Mangga Gandaria (40sch x 1)', 157500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Paket Nutrisari Blewah (40sch x 2) + Nutrisari Jeruk Maroko (40sch x 1)', 157500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Paket Nutrisari American Sweet Orange (40sch x 2) + Nutrisari Milky Orange (40sch x 1)', 157500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo School Chocolate 500gr + Free Kertas Gambar', 117600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Paket Ngopi Lokalate', 136000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Gold Chocolate 750gr', 127100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Paket Nutrisari Florida Orange (40sch x 2) + Nutrisari Markisa (40sch x 1)', 157500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Extra Virgin Olive Oil 500 ml', 82000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Paket Bundle HiLo (HiLo Active Chocolate 500gr + HiLo Teen Yoghurt Banana 250gr)', 94000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Buy 1 Get 1 FREE: Lokalate Kopi Berondong (10 Sch)', 15000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Lokalate Kopi Berondong 10's", 15000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["L-Men Platinum Choco Latte Dus 6 sch", 80000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo Teen Chocolate 750gr", 104000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Tropicana Slim DIABTX (100 sch)", 75000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Semangka 40 sch", 42500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Es Cincau 40 sch", 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["NutriSari Nanas 40 sch", 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Tropicana Slim Milk Skim Fiber Pro Plain 500gr", 180400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo Es Teler 10 sch", 13200]], columns = ['title', 'price']), ignore_index = True, sort = False)

    product_order = product_order.append(pd.DataFrame([["Twin Pack: Hilo Es Ketan Hitam 10 sch x 2", 26400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Twin Pack: HiLo Es Teler 10 sch x 2", 26400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Hilo Es Ketan Hitam 10 sch", 13200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Nutrisari Jeruk Nipis (40 sch)', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Madu Lemon (40 Sch)', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Sweet Mango (40 sch)', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Protein Bar Chocolate (12 Sch) + L-Men Protein Crunch BBQ Beef x 2', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Protein Bar Chocolate 12 sch', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Protein Crunch BBQ Beef (20gr)', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo Chocolate Taro (10 sch)', 11000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Markisa (40 Sch)', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Buy 5 Get 5 L-Men Protein Crunch BBQ Beef (20gr)', 67500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: Tropicana Slim Avocado Coffee 4 Sch x 2', 18200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Avocado Coffee 4 Sch', 12100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Cocopandan 40 sch', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Sambal Terasi 200 gr', 22300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Lemon Tea 40 sch', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)

    product_order = product_order.append(pd.DataFrame([["W'Dank Empon-Empon 10 sch", 13200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Teen Vanilla 750gr', 102300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo RTD Kacang Hijau 200ml (6pcs)', 29700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Active Chocolate 250gr', 29150]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Nutrisari Milky Orange (40sch)', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Sweetener Lemongrass Pandan 50 sch', 28900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Triple Pack: Tropicana Slim Korean Garlic Butter Cookies (5 Sch)', 52000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Gold Vanilla 500gr', 66000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Sweetener Jahe 50 sch', 38500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Active Ketan Hitam 175gr', 13200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([["HiLo Gold Plain (Original) 750gr", 92400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo School Vanilla Vegiberi 500gr', 71500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Lose Weight Chocolate Cereal 300gr', 104500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School Honey 500gr', 71500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Korean Garlic Butter Cookies 5 Sch', 20400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Teen Taro 500gr', 71500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Hi Protein 2 Go Chocolate (24 pcs) - 12gr Protein / Serving', 211200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-MEN Gain Mass Taro 225 gr', 52800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School RTD Chocolate 200ml (6pcs)', 36300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Sunflower Oil 946 ml', 57200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([["L-Men Whey Daily Dark Chocolate 250gr", 49500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Active Vanilla 200gr', 59400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Teen Chocolate 1000 gr', 136400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["BUY 1 GET 1 - W'Dank Empon Empon 10 Sachet Renceng", 26400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Gain Mass Banana 225gr', 60500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Nutrisari Mangga Gandaria', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo RTD Kacang Hijau 200 ml', 4950]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Hi Protein 2 Go Chocolate', 8800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(FREE) W'Dank Empon-Empon 10 sch", 0]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Khusus Homdel - HiLo Chocolate Banana 10 sch', 11000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Lychee Tea 40 sch', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Strawberry 40 Sch', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)

    product_order = product_order.append(pd.DataFrame([['HiLo School RTD Vanila Vegiberi 200ml (6pcs)', 36300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: Tropicana Slim Saus Tiram 200ml', 39600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: Tropicana Slim Shirataki Noodles 71gr', 28100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School RTD Vanila Vegiberi 200ml', 6100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Saus Tiram 200ml', 19800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Shirataki Noodles 71gr', 14100]], columns = ['title', 'price']), ignore_index = True, sort = False)

    product_order = product_order.append(pd.DataFrame([['Nutrisari American Sweet Orange (40 Sch)', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Madu Kurma Pack (4R)', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Milky Orange (40 sch)', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo Chocolate (10 sch)', 11000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Swiss Chocolate (10 sch) Renceng', 16500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Leci 40 sch', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Sweet Guava (40 sch)', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([['Paket Hilo - Lebaran Edition', 56100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Paket Ramadhan NutriSari 7 Rasa (7 x 10 sch)', 294000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Nutrisari Jeruk Peras (40 Sch)', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Paket Lebaran Tropicana Slim', 96300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Paket Lokalate untuk Sobatku - Idul Fitri Edition', 54120]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Topping Kental Manis 150ml - SUGAR FREE', 28600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Nutrisari Florida Orange (40 Sch)', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo RTD Milky Brown Sugar 200 ml', 4950]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Belgian Chocolate (10 sch)', 41800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Mangga Gandaria (40 sch)', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Kecap Asin 200 ml', 23650]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Apel Jeruk 40 Sachet', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Minyak Jagung 1lt', 69300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Teen Coffee Tiramisu 200ml (6pcs) - Ready to Drink', 36300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Nutrisari Jeruk Madu Jeruk 40'sachet (4 Renceng)", 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Teen RTD Coffee Tiramisu 200ml', 6050]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Milk Skim Chocolate 500gr', 111000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: Tropicana Slim Mayonnaise 200 g x 2', 34800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Oat Drink 190 ml (RTD) x 4 pcs', 22600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Mayonnaise 200 g - Bantu Dukung Hidup Sehat', 17400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Oat Drink 190 ml (RTD)', 5650]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Baru! NutriSari NUTRI C1000 Jeruk 40 Sachet', 35000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: L-Men Daily Popcorn Caramel 250gr x 2', 78300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Daily Popcorn Caramel 250gr', 39150]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Buy 1 Get 1 FREE HiLo Joint Plus 140gr', 38300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Es Rujak Jeruk Bali 40 Sachet', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['BUY 1 GET 1 - Lokalate Kopi Tape Ketan 10 Sachet', 13700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Lokalate Kopi Tape Ketan 10 Sachet', 13700]], columns = ['title', 'price']), ignore_index = True, sort = False) 
    product_order = product_order.append(pd.DataFrame([['(FREE) Lokalate Kopi Tape Ketan 10 Sachet', 0]], columns = ['title', 'price']), ignore_index = True, sort = False) 
    product_order = product_order.append(pd.DataFrame([['Twin Pack: Tropicana Slim Korean Goguma Cookies (5 Sch) x 2', 32200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Korean Goguma Cookies (5 Sch)', 16100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['BUY 1 GET 1 - Lokalate Kopi Andaliman 10 Sachet', 13700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Lokalate Kopi Andaliman 10 Sachet', 6850]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['(FREE) Lokalate Kopi Andaliman 10 Sachet', 0]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Korean Goguma Cookies (5 Sch)', 16100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Gold Plain 1000 gr', 128200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NuTrilogi Seri Mas Kaka yang Banyak Akal', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School Vanilla Vegiberi 500gr', 76600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Teen Vanilla Caramel 500gr', 76600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari NUTRI C1000 Jeruk 40 Sachet', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: HiLo Joint Plus Orange 140 gram x 2', 55300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Lokalate Kopi Tape Ketan 10 Sachet', 13700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Teen Popcorn Caramel 500 gram - Susu Tinggi Kalsium Lebih Rendah Lemak', 68600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin Pack: HiLo School Strawberry Cheesecake 500 gram x 2', 110600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Yuzu Orange 40 sch', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Nutrisari Jeruk Jeju [40 Sachet]', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School Strawberry Cheesecake 500 gram', 55300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin pack - Lokalate Kopi Andaliman 10 Sachet', 19800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Paket Masak Sehat', 125000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Extra Virgin Olive Oil 500 ml', 85800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Kecap Manis 200ml', 27500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Santan 5 sachet', 15400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Twin pack - Lokalate Kopi Tape Ketan 10 Sachet', 19800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Lokalate Kopi Tape Ketan 10 Sachet', 13700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Chocolate Taro RTD 200ml (4pcs)', 22900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Chocolate Taro RTD 200ml', 5750]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([['Paket Ngemil Sehat', 125000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Korean Goguma Cookies (5 Sch)', 21200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Oat Drink 190 ml (RTD)', 7500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Topping Kental Manis 150ml - SUGAR FREE', 29700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Hokkaido Cheese Cookies', 21200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Avocado Coffee 4 Sch', 12600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['2102900319 (D)', 82500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['2102500336 (classic stick)', 46900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([['NutriSari Jeruk Maroko (40 Sch) FREE Lokalate Andaliman (10 Sch)', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Nutty Chocolate Cookies 200gr', 40000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Es Rujak (4R)', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Nutrisari Jeruk Maroko (40 sch)', 42600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['(FREE) Lokalate Kopi Andaliman 10 Sachet', 13600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School Chocolate 750gr', 111000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Yoghurt Smoothie Bowl Strawberry (8 sch)', 62900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School Chocolate Ready To Drink (4 tetrapack)', 26100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo Teen Chocolate Ready To Drink Susu [4 pcs]', 26100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School RTD Coklat 200ml', 6525]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Teen RTD Coklat 200ml', 6525]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([['Twin Pack: HiLo Almond Milk Coconut 200gr x 2', 70000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Hi Protein 2 Go Chocolate x 4 pcs', 38400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Almond Milk Coconut 200gr', 35000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['L-Men Hi Protein 2 Go Chocolate', 9600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Thai Tea Ready to Drink 200ml x 4pcs', 22900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo RTD Thai Tea 200ml', 5750]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["W'Dank Bajigur (10 sch)", 17200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["W'Dank Bandrek 10 sachet Renceng", 17200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo Milky Brown Sugar Ready To Drink 200ml (4 Tetrapack)', 22900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Hilo School Vanilla Vegiberi Ready To Drink Susu [200ml/4 pcs]', 26100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Hilo RTD Milky Brown Sugar 200 ml", 5725]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo School RTD Vanila Vegiberi 200ml", 6525]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["HiLo School Chocolate Candy (10 sch)", 18300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["2102500320 (CI160)", 72160]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School Chocolate 1000 gr', 141900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Triple Pack: Tropicana Slim Klepon Cookies (5 Sch)', 50000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Klepon Cookies (5 Sch)', 16700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Jeruk Nipis Jahe 40 Sachet', 35000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['BUY 1 GET 1 - Tropicana Slim Shirataki Rice 72 g', 50000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Shirataki Rice 72 g', 25000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Sirup Leci 750ml', 30900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Mango Smoothie 200ml (4 Pcs)', 27500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari RTD Mango Smoothie 200 ml', 6900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Active Es Teler 175gr', 26300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Platinum Swiss Chocolate 420gr', 91500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Es Kuwud Nipis', 37500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Classic Refill 500gr', 104100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([['BUY 1 GET 1 - Tropicana Slim Brownies Instant Powder Cake Mix 230 g - Bantu Dukung Hidup Sehat', 59400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Bundle - HiLo Teen Taro 500gr & HiLo Teen Strawberry Milkshake 500g', 127200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Buy 1 Get 1 Free - HiLo School Bubble Gum 500g', 92400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HILO ALMOND MILK COCONUT 12DX200G', 51500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Teen Taro 500gr', 80100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['(Near ED) HiLo School RTD Coklat 200ML', 6900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Teen Strawberry Milkshake 500g', 75500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Sweetener I Sweet', 25200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['BUY 1 GET 1 - Lokalate Kopi Pisang Bakar 10 sch', 17500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Lokalate Kopi Pisang Bakar Epe 10 sch', 8750]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo School Vanilla Vegiberi 750gr', 116700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['HiLo Coffee Milk Pillow Bag (2 sch)', 10300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Lychee Tea Refill 500 gram', 34300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Cookies Paket Hampers Idul Fitri Parcel Lebaran', 131600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Lemon-C (25 sch)', 25200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim High Fiber High Calcium Milk Chocolate 500gr', 116700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Lemon Tea 500 gram', 42000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['NutriSari Jeruk Peras Refill 500 gr', 37800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['LOKALATE KOPI PISANG BAKAR EPE', 15000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Buy 1 Get 1 - Tropicana Slim Mint Cocoa - Minuman Cokelat Mint Nikmat Tanpa Gula Pasir', 20000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'HiLo Es Pisang Ijo 10 Sch x 2 pcs', 17500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ '(Near ED)L-Men Platinum Kacang Hijau 800g', 158750]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'L-Men Platinum Choco Latte 800 gr - Near ED', 164500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'NutriSari RTD Jeruk Madu 200 ml', 6000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'NutriSari Jeruk Manis 500gr', 38100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'L-Men Gain Mass Mangga 500gr', 133900]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'HiLo Avocado Chocolate (10 Sch)', 12100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'HiLo White Chocolate (10 Sch)', 12100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'Diabetamil Milk Vanilla 150 gram', 35000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'Buy 1 Get 1 FREE Tropicana Slim Susu Low Fat Macchiato Coffee 500gr', 143000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'HiLo Active Chocolate 1000 gr', 121200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'HiLo Joint Plus Orange 140 gram', 38700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'Tropicana Slim Susu Low Fat Macchiato Coffee 500 gram', 71500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'HiLo Teen Biscuit Caramel 500gr', 65000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'HiLo Active Caramel Latte 500g', 65800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'Buy 1 Get 1 Free - HiLo Gold Biscuit Cereal 500g', 80800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'HILO TEEN STRAWBERRY MILKSHAKE\xa0 12DX500G', 76200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'HiLo School Bubble Gum 500g', 80800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ '2102500336 (CS)', 49200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'HiLo Thai Tea Refill 500g', 48000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'Hilo Teen Coffee Tiramisu Ready To Drink Susu UHT [4 pcs]', 27600]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ '(Near ED)L-Men Platinum Ketan Hitam 800g', 157300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'HiLo Klepon Latte Refill 500g - Powder Drink Tinggi Kalsium', 46800]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'NutriSari Gula Asem 40 Sachet', 43300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'NutriSari Anggur Hijau 40 Sachet', 43300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'HiLo Es Pisang Ijo 10 Sch', 14300]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'Buy 1 Get 1 FREE L-Men Lose Weight Mango Sticky Rice 300gr', 145200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([[ 'TRIPLE DEALS - NutriSari Lychee Tea & Lemon Tea 500 gram Refill', 72700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'TRIPLE HEMAT - NutriSari Lemon Tea 500 gram', 72700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'TRIPLE HEMAT - HiLo Thai Tea Refill 500g', 72700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'TRIPLE HEMAT - HiLo Klepon Latte Refill 500g', 109100]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'Bundle - Lokalate Kopi Alpukat Refill 500 gram & Kopi Berondong Refill 500 gram', 46200]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'TRIPLE HEMAT - NutriSari Lychee Tea Refill 500 gram', 72700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ '(Near ED)L-Men Gain Mass Mangga 500gr', 66350]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'Lokalate Kopi Berondong 500 gram - Tinggi Vitamin A Lebih Rendah Gula', 44400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([[ 'Lokalate Kopi Alpukat 500 gram - Tinggi Vitamin A Lebih Rendah Gula', 44400]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    product_order = product_order.append(pd.DataFrame([["Buy 1 Get 1 - Tropicana Slim Bumbu Kaldu Ayam dan Jamur 100 gram", 48500]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["BUY 1 GET 1 - HiLo Platinum Original 360 gram", 255000]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) HiLo School Chocolate 1000 gr", 148700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["Near ED - HiLo Teen Chocolate 1000 gr", 148700]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([["(Near ED) HiLo Joint Plus Orange 140 gram", 19150]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    price = product_order[product_order['title'] == 'Tropicana Slim Goldenmil Vanilla (6sch)']['price'].values[0]
    product_order = product_order.append(pd.DataFrame([['BUY 1 GET 1 Tropicana Slim Goldenmil Vanilla (6sch)', price]], columns = ['title', 'price']), ignore_index = True, sort = False)

    price = product_order[product_order['title'] == 'Lokalate Kopi Kawista (10sch)']['price'].values[0]
    product_order = product_order.append(pd.DataFrame([['BUY 1 GET 1 Lokalate Kopi Kawista (10sch)', price]], columns = ['title', 'price']), ignore_index = True, sort = False)
    product_order = product_order.append(pd.DataFrame([['Lokalate Kopi Kawista', price]], columns = ['title', 'price']), ignore_index = True, sort = False)

    price = product_order[product_order['title'] == 'Tropicana Slim Kecap Manis 200ml']['price'].values[0]
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Kecap Manis 200m', price]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    price = product_order[product_order['title'] == 'Tropicana Slim Diabtx 50 sch']['price'].values[0]
    product_order = product_order.append(pd.DataFrame([['Tropicana Slim Diabtx (50 sch)', price]], columns = ['title', 'price']), ignore_index = True, sort = False)
    
    data_order['product'] = data_order['product'].astype(str)
    data_SKU['product'] = data_SKU['product'].astype(str)
    product_order['title'] = product_order['title'].astype(str)

    data_order = data_order.merge(data_SKU[['SKU', 'product']].drop_duplicates('product'), how = 'left', on = 'product')
    data_order = data_order.merge(product_order[['title', 'price']].drop_duplicates('title'), how = 'left', left_on = 'product', right_on = 'title').drop('title', axis = 1)
    data_order = data_order.reset_index(drop = True)

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()
    for i in indeks:
        col = [x for x in data_SKU.columns if 'Alias Nama' in x]
        for j in col:
            if data_order['product'][i] in data_SKU[j].astype(str).values:
                SKU = data_SKU[data_SKU[j].astype(str) == data_order['product'][i]]['SKU'].values[0]
                data_order['SKU'][i] = SKU

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    data_SKU2 = pd.read_excel(r'SKU_File\data_SKU.xlsx')
    data_SKU2['Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

    s = requests.Session()
    s.get("http://tatanama.pythonanywhere.com")
    s.post("http://tatanama.pythonanywhere.com", data = {'username' : 'ecommerce', 'password' : 'ecommerce'})
    r = s.get("http://tatanama.pythonanywhere.com/download")

    with open(r'SKU_File\Master tatanama.xlsx', 'wb') as output:
        output.write(r.content)

    if os.path.isfile(r'SKU_File\Master tatanama.xlsx') :    
        SKU_append = pd.read_excel(r'SKU_File\Master tatanama.xlsx')
        SKU_append.columns = [x.replace('_', ' ') for x in SKU_append.columns]
        data_SKU2 = data_SKU2[~data_SKU2['SKU'].astype(str).isin(SKU_append['SKU'].astype(str))]
        data_SKU2 = data_SKU2.append(SKU_append, ignore_index = True, sort = False)

    to_excel = data_SKU2.to_excel(r'SKU_File\data_SKU.xlsx', index = False)

    for i in indeks:
        if str(data_order['product'][i]).lower() in data_SKU2['Nama Produk'].astype(str).str.lower().values:
            data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2['Nama Produk'].astype(str).str.lower()].values[0]

    list_alias_name = [x for x in data_SKU2.columns if 'Alias Nama' in x]

    for i in indeks:
        for j in list_alias_name:
            if str(data_order['product'][i]).lower() in data_SKU2[j].astype(str).str.lower().values:
                data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2[j].astype(str).str.lower()].values[0]

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()
    indeks
    
    if len(indeks) != 0:
        print('Alert SKU Missing')
        data_order['product'][indeks].drop_duplicates().to_excel('Alert SKU Missing.xlsx', index = False)
    else :
        data_order['phone'] = data_order['phone'].astype(str).str.replace('+628', '08', regex = False)

        data_order['zip'] = data_order['zip'].replace('.0', '', regex = False)

        data_order = data_order.rename(columns = {'order_id' : 'Sales Order ID', 'name' : 'Customer Name', 'product' : 'Item Name', 'price' : 'Price', 'shipping_cost' : 'Shipping Cost', 'address' : 'Shipping Address1', 'city' : 'Shipping City', 'zip' : 'Shipping Zip', 'province' : 'Shipping Province', 'phone' : 'Shipping Phone', 'courier' : 'Shipping Courier'})
        data_order['Channel Order ID'] = data_order['Sales Order ID']
        data_order['Invoice Number'] = data_order['Sales Order ID']
        data_order['Shipping Name'] = data_order['Customer Name']
        data_order['Shipping Address2'] = 0
        data_order['Shipping Country'] = 'Indonesia'
        data_order['AWB'] = 0
        data_order['Channel'] = 'Order Online Jakarta'

        data_order['Order Date'] = pd.to_datetime(data_order['created_at'])

        indeks = data_order[data_order['Item Name'].astype(str).str.contains('pcs')].index.to_list()
        temp = data_order.iloc[indeks].copy()
        product = temp['Item Name'].str.split("\(",1, expand = True)

        data_order['SKU'] = data_order['SKU'].astype(str)
        data_order['Item Name'] = data_order['Item Name'].astype(str)
        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str).str.replace('(S)', '', regex = False)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        data_order = data_order.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('hd','', regex = False)
        temp['SKU'] = temp['SKU'].astype(str).str.replace('HD','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        data_order['Real SKU'] = data_order['Real SKU'].astype(str)
        data_order = data_order.merge(data_SKU2[['SKU', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Unbundling ====== 6/10")        
        # Forstok Unbundling    
        list_col = ['SKU'] + data_SKU2.columns[data_SKU2.columns.get_loc('Produk 1'):data_SKU2.columns.get_loc('Harga Organik 7')+1].to_list()
        data_order = data_order.merge(data_SKU2[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        list_pcs = [x for x in data_order.columns if 'PCS' in x]
        for i in list_pcs:
            data_order[i] = data_order[i] * data_order['Quantity']
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        indeks = data_order[data_order['Brand'] == 'Bundle'].index.to_list()
        data_order['Bundle Flag'] = np.nan
        data_order['Bundle Flag'][indeks] = 'Bundle'

        indeks = data_order[data_order['Brand'] == 'Bundle'][data_order[data_order['Brand'] == 'Bundle']['SKU'].astype(str).str.contains('(S)', regex = False)].index.to_list()
        data_order['SKU Produk 1'][indeks] = '(S)' + data_order['SKU Produk 1'][indeks].astype(str)
        data_order['SKU Produk 2'][indeks] = '(S)' + data_order['SKU Produk 2'][indeks].astype(str)
        data_order['SKU Produk 3'][indeks] = '(S)' + data_order['SKU Produk 3'][indeks].astype(str)
        data_order['SKU Produk 4'][indeks] = '(S)' + data_order['SKU Produk 4'][indeks].astype(str)
        data_order['SKU Produk 5'][indeks] = '(S)' + data_order['SKU Produk 5'][indeks].astype(str)
        data_order['SKU Produk 6'][indeks] = '(S)' + data_order['SKU Produk 6'][indeks].astype(str)
        data_order['SKU Produk 7'][indeks] = '(S)' + data_order['SKU Produk 7'][indeks].astype(str)

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Filling Date ====== 7/10")
        data_order['Date'] = np.nan
        data_order['Month'] = np.nan
        data_order['Year'] = np.nan

        for i in range(data_order.shape[0]):
            if int(data_order['Order Date'][i].strftime('%d')) <= 12:
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).year
            else :
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i]).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i]).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i]).year

        quarter = pd.DataFrame([['January', 1], ['February', 1], ['March', 1], ['April', 2], ['May', 2], ['June', 2], 
                ['July', 3], ['August', 3], ['September', 3],['October', 4], ['November', 4], ['December', 4]], columns = ['Bulan', 'Quarter'])
        data_order = data_order.merge(quarter, how = 'left', left_on = 'Month', right_on = 'Bulan')
        data_order = data_order.drop(['Bulan'], axis = 1)
        data_bulan = pd.DataFrame([{'Bulan' : 'December', 'Number' : 12} ,
                {'Bulan' : 'January' , 'Number': 1},
                {'Bulan' : 'February' , 'Number': 2},
                {'Bulan' : 'March' , 'Number': 3},
                {'Bulan' : 'April' , 'Number': 4},
                {'Bulan' : 'May' , 'Number': 5},
                {'Bulan' : 'June', 'Number': 6},
                {'Bulan' : 'July' , 'Number': 7},
                {'Bulan' : 'August', 'Number' : 8},
                {'Bulan' : 'September', 'Number' : 9},
                {'Bulan' : 'October' , 'Number': 10},
                {'Bulan' : 'November' , 'Number': 11}])
        temp = data_order.copy()
        temp['Day'] = temp['Date']
        temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
        temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
        data_order['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.week
        temp['Hour'] = pd.to_datetime(data_order['Order Date']).dt.hour
        temp['Minute'] = pd.to_datetime(data_order['Order Date']).dt.minute
        temp['Second'] = pd.to_datetime(data_order['Order Date']).dt.second
        data_order['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']])



        order_all = data_order.copy()
        index = order_all[order_all['SKU'].astype(str) == '2306551174'].index.to_list()
        order_all['SKU'][index] = '2306592173'
        order_all['Real SKU'][index] = '2306592173'
        order_all['Total'] = order_all['net_revenue']
        order_all['Price List NFI'] = np.nan
        order_all['Total Net'] = np.nan

        order_all = order_all.rename(columns={'Channel Order ID' : 'Order #',
                                                'Status' : 'Order Status',
                                                'Order Date' : 'Order date',
                                                'Item Name' :'Product Name',
                                                'Bundle Name' : 'Bundle',
                                                'Shipping Country' : 'Country',
                                                'Shipping Province' : 'Region',
                                                'Shipping City' : 'City',
                                                'Shipping Zip' : 'Zip Code',
                                                'Shipping Address1' : 'Address',
                                                'Shipping Phone' : 'Phone',
                                                'Quantity' : 'Qty. Invoiced',
                                                'Price' : 'Regular Price',
                                                'net_revenue' : 'Subtotal'})
        order_all['Kecamatan'] = np.nan
        order_all['Kelurahan'] = np.nan

        print("Filling Location")
        indeks = order_all[order_all['City'].astype(str).str.contains('/')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains('-')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[0]

        indeks = order_all[order_all['City'].astype(str).str.contains(',')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains(',')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[0]

        order_all['City'] = order_all['City'].astype(str).str.replace('Kab\.', 'Kabupaten' ,case = False)

        master_map = pd.read_csv(r'All Data/Province.csv', names = ['Kode Prov', 'Province'], header= 0)
        master_map2 = pd.read_csv(r'All Data/City.csv', names = ['Kode City', 'Kode Prov', 'City'], header = 0)
        master_map = master_map.merge(master_map2, how = 'right', on = 'Kode Prov')
        master_map['Kode Prov'][515] = 14
        master_map['Province'][515] = 'Riau'
        master_map['Kode Prov'] = master_map['Kode Prov'].astype(int)
        master_map['Province'] = master_map['Province'].str.title()
        master_map['City'] = master_map['City'].str.title()

        city = pd.read_excel(r'All Data/list_city.xlsx')
        temp = order_all.copy()
        temp['City'] = temp['City'].astype(str).str.lower()
        temp['City'] = temp['City'].astype(str).str.replace('kab. ', 'kabupaten ', regex = False, case = False)
        city['All City'] = city['All City'].astype(str).str.lower()
        temp = temp.merge(city.drop_duplicates('All City'), how = 'left', left_on = 'City', right_on = 'All City').set_index(temp.index)
        indeks = temp[temp['Real City'].notnull()].index.to_list()
        order_all['City'][indeks] = temp['Real City'][indeks]

        province = pd.read_excel(r'All Data/list_province.xlsx')
        temp = order_all.copy()
        temp['Region'] = temp['Region'].astype(str).str.lower()
        province['All Province'] = province['All Province'].astype(str).str.lower()
        temp = temp.merge(province.drop_duplicates('All Province'), how = 'left', left_on = 'Region', right_on = 'All Province').set_index(temp.index)
        indeks = temp[temp['Real Province'].notnull()].index.to_list()
        order_all['Region'][indeks] = temp['Real Province'][indeks]

        temp = order_all.copy()
        temp = temp[temp['Region'].isnull()]
        temp['Region'] = temp.merge(master_map, how = 'left', on = 'City').set_index(temp.index)['Province']
        order_all['Region'][temp.index] = temp['Region']  

        district = pd.read_excel(r'All Data/list_district.xlsx')
        temp = order_all.copy()
        temp['Kecamatan'] = temp['Kecamatan'].astype(str).str.lower()
        district['All District'] = district['All District'].astype(str).str.lower()
        temp = temp.merge(district.drop_duplicates('All District'), how = 'left', left_on = 'Kecamatan', right_on = 'All District').set_index(temp.index)
        indeks = temp[temp['Real District'].notnull()].index.to_list()
        order_all['Kecamatan'][indeks] = temp['Real District'][indeks]

        temp = order_all.copy()
        temp2 = temp[['Region', 'City', 'Kecamatan']].merge(master_map, how = 'left', on = 'City')
        indeks = temp2[temp2['Region'] != temp2['Province']][temp2[temp2['Region'] != temp2['Province']]['City'].notnull()].index.to_list()
        order_all['City'][indeks] = np.nan

        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        print("Unbundling")
        data_bundle1 = order_all[~order_all['Produk 1'].isnull()]
        data_bundle1['Bundle Name'] = data_bundle1['Product Name']
        data_bundle1['Product Name'] = data_bundle1['Produk 1']
        data_bundle1['SKU'] = data_bundle1['SKU Produk 1']
        data_bundle1['Qty. Invoiced'] = data_bundle1['PCS Produk 1']
        data_bundle1['Price List NFI'] = data_bundle1['Price List NFI 1']
        data_bundle1['Total Net'] = data_bundle1['Price List NFI 1']
        data_bundle1['Bundle Flag'] = np.nan

        data_bundle2 = order_all[~order_all['Produk 2'].isnull()]
        data_bundle2['Bundle Name'] = data_bundle2['Product Name']
        data_bundle2['Product Name'] = data_bundle2['Produk 2']
        data_bundle2['SKU'] = data_bundle2['SKU Produk 2']
        data_bundle2['Qty. Invoiced'] = data_bundle2['PCS Produk 2']
        data_bundle2['Price List NFI'] = data_bundle2['Price List NFI 2']
        data_bundle2['Total Net'] = data_bundle2['Price List NFI 2'] 
        data_bundle2['Bundle Flag'] = np.nan

        data_bundle3 = order_all[~order_all['Produk 3'].isnull()]
        data_bundle3['Bundle Name'] = data_bundle3['Product Name']
        data_bundle3['Product Name'] = data_bundle3['Produk 3']
        data_bundle3['SKU'] = data_bundle3['SKU Produk 3']
        data_bundle3['Qty. Invoiced'] = data_bundle3['PCS Produk 3']
        data_bundle3['Price List NFI'] = data_bundle3['Price List NFI 3']
        data_bundle3['Total Net'] = data_bundle3['Price List NFI 3'] 
        data_bundle3['Bundle Flag'] = np.nan

        data_bundle4 = order_all[~order_all['Produk 4'].isnull()]
        data_bundle4['Bundle Name'] = data_bundle4['Product Name']
        data_bundle4['Product Name'] = data_bundle4['Produk 4']
        data_bundle4['SKU'] = data_bundle4['SKU Produk 4']
        data_bundle4['Qty. Invoiced'] = data_bundle4['PCS Produk 4']
        data_bundle4['Price List NFI'] = data_bundle4['Price List NFI 4']
        data_bundle4['Total Net'] = data_bundle4['Price List NFI 4'] 
        data_bundle4['Bundle Flag'] = np.nan

        data_bundle5 = order_all[~order_all['Produk 5'].isnull()]
        data_bundle5['Bundle Name'] = data_bundle5['Product Name']
        data_bundle5['Product Name'] = data_bundle5['Produk 5']
        data_bundle5['SKU'] = data_bundle5['SKU Produk 5']
        data_bundle5['Qty. Invoiced'] = data_bundle5['PCS Produk 5']
        data_bundle5['Price List NFI'] = data_bundle5['Price List NFI 5']
        data_bundle5['Total Net'] = data_bundle5['Price List NFI 5']
        data_bundle5['Bundle Flag'] = np.nan

        data_bundle6 = order_all[~order_all['Produk 6'].isnull()]
        data_bundle6['Bundle Name'] = data_bundle6['Product Name']
        data_bundle6['Product Name'] = data_bundle6['Produk 6']
        data_bundle6['SKU'] = data_bundle6['SKU Produk 6']
        data_bundle6['Qty. Invoiced'] = data_bundle6['PCS Produk 6']
        data_bundle6['Price List NFI'] = data_bundle6['Price List NFI 6']
        data_bundle6['Total Net'] = data_bundle6['Price List NFI 6']
        data_bundle6['Bundle Flag'] = np.nan

        data_bundle7 = order_all[~order_all['Produk 7'].isnull()]
        data_bundle7['Bundle Name'] = data_bundle7['Product Name']
        data_bundle7['Product Name'] = data_bundle7['Produk 7']
        data_bundle7['SKU'] = data_bundle7['SKU Produk 7']
        data_bundle7['Qty. Invoiced'] = data_bundle7['PCS Produk 7']
        data_bundle7['Price List NFI'] = data_bundle7['Price List NFI 7']
        data_bundle7['Total Net'] = data_bundle7['Price List NFI 7']
        data_bundle7['Bundle Flag'] = np.nan

        data_bundle = data_bundle1.append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)
        data_bundle['SKU'] = data_bundle['SKU'].astype(str)
        data_bundle['SKU'] = data_bundle['SKU'].str.replace('\.0$', '', regex = True)
        data_bundle[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']] = data_bundle.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item_y', 'Parent SKU_y']]

        temp = data_bundle[data_bundle['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)

        indeks = data_bundle[data_bundle['Real SKU'].isnull()].index.to_list()
        data_bundle['Real SKU'][indeks] = temp['Real SKU_y'][indeks]
        data_bundle['Real Nama Produk'][indeks] = temp['Nama Produk'][indeks]
        data_bundle['Brand'][indeks] = temp['Brand_y'][indeks]
        data_bundle['Sub Brand'][indeks] = temp['Sub Brand_y'][indeks]
        data_bundle['Parent Item'][indeks] = temp['Parent Item_y'][indeks]
        data_bundle['Parent SKU'][indeks] = temp['Parent SKU_y'][indeks]

        print("Pricing")
        order_all = order_all.append(data_bundle, ignore_index = True, sort = False)

        colname = temp.columns[temp.columns.get_loc('Produk 1') : temp.columns.get_loc('Harga Cost 7') + 1]
        colname_str = [x for x in colname if 'Subtotal' not in x and 'Harga' not in x]
        colname_int = [x for x in colname if x not in colname_str]

        for i in colname_str:
            temp[i] = np.nan

        for i in colname_int:
            temp[i] = 0

        data_order = data_order.append(temp , ignore_index = True, sort = False)
        
        order_all = order_all.reset_index()

        order_all = order_all.merge(data_SKU2[['SKU', 'Price List NFI', 'Harga Cost']].drop_duplicates('SKU'), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(order_all.index)
        order_all['Price List NFI_x'] = order_all['Price List NFI_x'].fillna(order_all['Price List NFI_y'])
        order_all =  order_all.drop(['Price List NFI_y', 'SKU_y'], axis = 1)
        order_all = order_all.rename(columns = {'SKU_x' : 'SKU', 'Price List NFI_x' : 'Price List NFI'})

        order_all['Price List NFI'] = pd.to_numeric(order_all['Price List NFI']).astype(int)
        order_all['Harga Cost'] = pd.to_numeric(order_all['Harga Cost'], errors = 'coerce').fillna(0).astype(int)
        order_all['Qty. Invoiced'] = pd.to_numeric(order_all['Qty. Invoiced']).astype(int)

        order_all['Total Net'] = order_all['Price List NFI'] * order_all['Qty. Invoiced']
        order_all['Total Harga Cost'] = order_all['Harga Cost'] * order_all['Qty. Invoiced']
        
        temp = order_all[order_all['Brand'] != 'Bundle']
        temp['discount'] = temp['discount'] * temp['Total Harga Cost']/temp.groupby(['Order #'])['Total Harga Cost'].transform('sum')
        order_all['discount'][temp.index] = temp['discount']

        order_all['Selling Price'] = order_all['Regular Price'].astype(int)

        data_SKU3 = pd.read_excel(r'Order Online\SKU buat andra updated maret 2021.xlsx')
        data_SKU3 = data_SKU3.rename(columns = {'Product' : 'product', 'PL NFI' : 'Price List NFI'})
        data_SKU3['SKU'] = data_SKU3['SKU'].astype(str).str.replace('.0', '', regex = False)
        data_SKU3 = data_SKU3[data_SKU3['SKU'] != 'nan'][data_SKU3[data_SKU3['SKU'] != 'nan']['SKU'] != '0'][data_SKU3[data_SKU3['SKU'] != 'nan'][data_SKU3[data_SKU3['SKU'] != 'nan']['SKU'] != '0']['Harga Coret'].notnull()]
        data_SKU3['product'] = data_SKU3['product'].str.strip()
        data_SKU3 = data_SKU3.reset_index(drop = True)

        for i in range(data_SKU3.shape[0]):
            if data_SKU3['SKU'][i] in order_all['SKU'].values:
                indeks = order_all[order_all['SKU'].astype(str) == i].index.to_list()
                order_all['Regular Price'][indeks] = data_SKU3['Harga Display'][i]
                order_all['Selling Price'][indeks] = data_SKU3['Harga Coret'][i]

        order_all['Subtotal'] = order_all['Selling Price'] * order_all['Qty. Invoiced']
        order_all['Total'] = order_all['Selling Price'] * order_all['Qty. Invoiced']

        order_all = order_all.reset_index(drop = True)
        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        order_all['Seller Discount'] = order_all['discount']
        order_all['Shipping'] = order_all['Shipping Cost']

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Subtotal', 'Total']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Total'][list_nobundle.index] = list_nobundle['Total_y']
        order_all['Subtotal'][list_nobundle.index] = list_nobundle['Subtotal_y']

        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Subtotal'] = temp['Subtotal'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        temp['Selling Price'] = temp['Subtotal']/temp['Qty. Invoiced']
        temp['Total'] = temp['Total'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Total'][temp.index] = temp['Total']
        order_all['Subtotal'][temp.index] = temp['Subtotal']
        order_all['Selling Price'][temp.index] = temp['Selling Price']


        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Seller Discount']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Seller Discount'][list_nobundle.index] = list_nobundle['Seller Discount_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Seller Discount'] = temp['Seller Discount'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Seller Discount'][temp.index] = temp['Seller Discount']


        temp = order_all[order_all['Bundle Name'].isnull()]
        temp_group = temp[['Order #','Shipping']].groupby(['Order #']).sum().reset_index()

        temp = order_all.merge(temp_group, how = 'left', on = 'Order #').set_index(order_all.index)
        temp['Shipping_x'] = temp['Shipping_y'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Shipping'][temp.index] = temp['Shipping_y']
        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Shipping']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Shipping'][list_nobundle.index] = list_nobundle['Shipping_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Shipping'] = temp['Shipping'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Shipping'][temp.index] = temp['Shipping']
        order_all['True datetime'] = pd.to_datetime(order_all['True datetime'])
        order_all['Promo'] = np.nan
        order_all['Discount MC'] = np.nan

        order_all['Warehouse Name'] = 'Primary Warehouse'
        order_all['Store'] = 'Order Online'

        order_all['Customer Email'] = order_all['email']
        order_all['Order Status'] = order_all['status']
        order_all['Payment Channel'] = order_all['payment_method']
        order_all['Coupon Code'] = order_all['coupon']
        order_all.columns.to_list()

        order_all_append = order_all[['Sales Order ID', 'Store',
            'Product Name',
            'Customer Name',
            'Phone',
            'Address',
            'Region',
            'City',
            'Zip Code',
            'payment_status',
            'Regular Price',
            'Shipping Courier',
            'Shipping Cost',
            'Subtotal',
            'Qty. Invoiced',
            'SKU',
            'Order #',
            'Invoice Number',
            'Shipping Name',
            'Shipping Address2',
            'Country',
            'AWB',
            'Channel',
            'Order date',
            'Real SKU',
            'Real Nama Produk',
            'Brand',
            'Sub Brand',
            'Parent Item',
            'Parent SKU',
            'Produk 1',
            'SKU Produk 1',
            'PCS Produk 1',
            'Price List NFI 1',
            'Subtotal Produk 1',
            'Harga Display 1',
            'Harga Cost 1',
            'Harga Organik 1',
            'Produk 2',
            'SKU Produk 2',
            'PCS Produk 2',
            'Price List NFI 2',
            'Subtotal Produk 2',
            'Harga Display 2',
            'Harga Cost 2',
            'Harga Organik 2',
            'Produk 3',
            'SKU Produk 3',
            'PCS Produk 3',
            'Price List NFI 3',
            'Subtotal Produk 3',
            'Harga Display 3',
            'Harga Cost 3',
            'Harga Organik 3',
            'Produk 4',
            'SKU Produk 4',
            'PCS Produk 4',
            'Price List NFI 4',
            'Subtotal Produk 4',
            'Harga Display 4',
            'Harga Cost 4',
            'Harga Organik 4',
            'Produk 5',
            'SKU Produk 5',
            'PCS Produk 5',
            'Price List NFI 5',
            'Subtotal Produk 5',
            'Harga Display 5',
            'Harga Cost 5',
            'Harga Organik 5',
            'Produk 6',
            'SKU Produk 6',
            'PCS Produk 6',
            'Price List NFI 6',
            'Subtotal Produk 6',
            'Harga Display 6',
            'Harga Cost 6',
            'Harga Organik 6',
            'Produk 7',
            'SKU Produk 7',
            'PCS Produk 7',
            'Price List NFI 7',
            'Subtotal Produk 7',
            'Harga Display 7',
            'Harga Cost 7',
            'Harga Organik 7',
            'Bundle Flag',
            'Date',
            'Month',
            'Year',
            'Quarter',
            'Week',
            'True datetime',
            'Total',
            'Price List NFI',
            'Total Net',
            'Selling Price',
            'Kecamatan',
            'Kelurahan',
            'Bundle Name',
            'Harga Cost',
            'Total Harga Cost',
            'Seller Discount',
            'Shipping',
            'Promo',
            'Discount MC',
            'Warehouse Name',
            'Customer Email',
            'Order Status',
            'Payment Channel',
            'Coupon Code']]
        
        order_all_append['True datetime'] = pd.to_datetime(order_all_append['True datetime'])

        indeks = order_all_append[
            (order_all_append['True datetime'] > pd.to_datetime('2021-02-28')) &
            (order_all_append['Channel'] == 'Order Online Jakarta')
        ].index.to_list()

        order_all_append['Channel'][indeks] = 'Order Online Sumsel'

        data_all = data_all[~data_all['Order #'].astype(str).isin(order_all_append['Order #'].astype(str))]
        data_all = data_all.append(order_all_append, ignore_index = True, sort = False)

        order_online_jkt = True
        del file_name

if not order_online_jateng:
            
    import pandas as pd
    import numpy as np
    import requests
    import os
    
    print('order_online_jateng')

    before = os.listdir(os.getcwd() + '/Input Data')

    options = Options()
    options.add_experimental_option("prefs", {
            "download.default_directory": os.path.abspath("Input Data"),
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            "safebrowsing.enabled": False
    })

    driver = webdriver.Chrome(options=options)
    driver.fullscreen_window()
    driver.get("https://app.orderonline.id")

    username = driver.find_element_by_name("email")
    username.clear()
    username.send_keys("customer1@nutrimart.co.id")

    password = driver.find_element_by_name("password")
    password.send_keys("jatenghomdel")
    password.click()

    driver.find_element_by_class_name("btn-submit").click()

    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="main-nav-dropdown"]/ul/li[3]/a'))).click()
    time.sleep(5)
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[1]/div/div[2]/div/div/div').click()
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="app"]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[1]/div[1]/ul/li[6]'))).click()
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[2]/button[2]').click()
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[3]/div/button').click()
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[3]/div/div/button[1]').click()

    time.sleep(15)

    after = os.listdir(os.getcwd() + '/Input Data')
    change = set(after) - set(before)
    if len(change) == 1:
        file_name = change.pop()
    elif len(change) == 0: 
        print("No file downloaded")
    else :
        print("More than one file downloaded")

    data_order = pd.read_csv(r'Input Data/' + str(file_name), error_bad_lines=False, engine="python")
    
    driver.close()

    data_order['order_id'] = data_order['order_id'].fillna(method='ffill')

    temp = data_order.drop_duplicates('order_id').drop('product', axis = 1)
    data_order = data_order[['order_id', 'product']]
    data_order = data_order.merge(temp, how = 'left', on = 'order_id')
    data_order['order_id'] = data_order['order_id'].astype(int)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Shirataki Noodles 71gr (x2) ', 'Twin Pack: Tropicana Slim Shirataki Noodles 71gr ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Saus Tiram 200ml (x2) ', 'Twin Pack: Tropicana Slim Saus Tiram 200ml ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Sweetener Rose Vanilla 50 sch (x2) ', 'Twin Pack: Tropicana Slim Sweetener Rose Vanilla 50 sch ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Tropicana Slim Oat Drink 190 ml \(RTD\) x 4 pcs$', 'Tropicana Slim Oat Drink 190 ml (RTD) x 4 pcs (x1)')
    data_order=data_order[data_order['product']!="2102500336 (CS)"]
    data_order=data_order.reset_index(drop=True)
    
    product = data_order['product'].str.split("\(x",1, expand = True)
    product = product[product[0] != 'nan']
    data_order['Quantity'] = product[1].astype(str).str.replace(')', '', regex = False).astype(int)
    data_order['product'] = product[0].str.strip().str.replace('  ', ' ').str.replace('6 SCH', '6sch').str.replace("W'Dank", "W'dank").str.replace('L-men', 'L-Men').str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("Rtd", "RTD").str.replace('Turmeric 180gr', 'Turmeric (6 sch)').str.replace('L-Men Gainmass Taro 225gr', 'L-Men Gain Mass Taro 225 gr').str.replace('L-Men Lose Weight Chocolate Cereal (12sch)', 'L-Men Lose Weight Chocolate Cereal 300gr').str.replace(' (+kaos)', ' (+Kaos)', regex = False).str.replace('40 sch', '40sch', regex = False).str.replace('Coklat', 'Chocolate', regex = False)
    data_order['product'] = data_order['product'].str.replace("HiLo Thai Tea \(10sch\)$", 'HiLo Thai Tea (10 sch)', regex = True)

    data_SKU = pd.read_excel(r'Order Online\SKU buat andra updated maret 2021.xlsx')
    data_SKU = data_SKU.rename(columns = {'Product' : 'product', 'PL NFI' : 'Price List NFI'})
    data_SKU['SKU'] = data_SKU['SKU'].astype(str).str.replace('.0', '', regex = False)
    data_SKU = data_SKU[data_SKU['SKU'] != 'nan'][data_SKU[data_SKU['SKU'] != 'nan']['SKU'] != '0']
    data_SKU['product'] = data_SKU['product'].str.strip().str.replace('L-men', 'L-Men').str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("Rtd", "RTD").str.replace('Turmeric 180gr', 'Turmeric (6 sch)').str.replace('L-Men Gainmass Taro 225gr', 'L-Men Gain Mass Taro 225 gr').str.replace('L-Men Lose Weight Chocolate Cereal (12sch)', 'L-Men Lose Weight Chocolate Cereal 300gr').str.replace(' (+kaos)', ' (+Kaos)', regex = False).str.replace('40 sch', '40sch', regex = False).str.replace('Coklat', 'Chocolate', regex = False)


    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Kecap Manis 200ml'].copy()
    price['product'] = 'Tropicana Slim Kecap Manis 200m'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Diabtx (50 sch)'].copy()
    price['product'] = 'Tropicana Slim Diabtx 50 sch'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Hokkaido Cheese 100gr'].copy()
    price['product'] = 'Tropicana Slim Hokaido Cheese 100gr'


    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['L-Men Bar Crunchy Chocolate 12sch', 5500, 5500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Mangga Gandaria', 45000, 45000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Teen Vanilla Caramel 500gr', 71500, 71500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Paket Bundle HiLo Renceng (Hilo Chocolate Banana (10 sch) + Hilo Chocolate Taro (10 sch) + HiLo Thai Tea (10sch))', 35200, 35200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Berondong 10's", 15000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Kecap Asin 200 ml", 28500, 26000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim DIABTX (100 sch)", 87700, 75000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Teen Chocolate 750gr", 117800, 104000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Paket Ngopi Lokalate", 136000, 109200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Hi Protein 2 Go Chocolate (24 TETRAPAK)", 240000, 238000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 Tropicana Slim Goldenmil Vanilla (6sch)", 39160, 31000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Kawista", 17500, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["Paket Bundle HiLo (HiLo Active Chocolate 500gr + HiLo Teen Yoghurt Banana 250gr)", 122900, 101850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Strawberry Jam 375gr", 72600, 58500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Crunch BBQ Beef (20gr)", 13500, 10500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Cocopandan 40 sch", 52500, 52500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 - W'dank Empon Empon 10 Sachet Renceng", 35000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Semangka 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Nanas 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["W'Dank Empon-Empon 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Milk Skim Fiber Pro Plain 500gr", 135000, 106700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Es Teler 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: HiLo Es Teler 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Hilo Es Ketan Hitam 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Triple Pack: Tropicana Slim Korean Garlic Butter Cookies (5 Sch)", 73500, 52000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Avocado Coffee 4 Sch", 21200, 12100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Sambal Terasi 200 gr", 35600, 29700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Tropicana Slim Avocado Coffee 4 sch x 2", 42400, 24200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Bar Chocolate (12 Sch) + L-Men Protein Crunch BBQ Beef x 2", 159000, 107800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Buy 5 Get 5 L-Men Protein Crunch BBQ Beef (20gr)", 130000, 67500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Active Ketan Hitam 175gr', 48500, 20700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Es Ketan Hitam 10 sch', 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Sweetener Lemongrass Pandan 50 sch', 44200, 28900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 1 Get 1 FREE: Lokalate Kopi Berondong (10 sch)', 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Platinum Swiss Chocolate 420gr', 91500, 91500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim White Coffee (4 sch)', 21200, 18400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['L-MEN Gain Mass Taro 225 gr', 58300, 58300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Extra Virgin Olive Oil 500ml', 85800, 85800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Jeruk Peras Refill 500gr', 34300, 34300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['L-Men Gain Mass Chocolate 500gr', 125800, 125800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Jeruk Manis Refill 500gr', 37800, 37800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Roasted Sesame Mayonnaise 200g - Bantu Dukung Hidup Sehat', 22900, 22900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['BUY 1 GET 1 - Tropicana Slim Shirataki Rice 72 g', 50000, 50000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Jeruk Nipis Jahe 40 Sachet', 62500, 35000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo school Chocolate 250g (Health & green Science Competition)', 38300, 38300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Kacang Hijau Ready to Drink 200ml (4 tetrapack)', 22900, 22900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Shirataki Noodles 71gr', 9700, 9700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo RTD Kacang Hijau 200 ml', 2850, 2850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo RTD Milky Brown Sugar 200 ml', 2550, 2550]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HILO TEEN STRAWBERRY MILKSHAKEÂ\xa0 12Dx500G', 66800, 66800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) NutriSari Cocopandan 40sch', 21300, 21300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Lemon Tea (40sch)', 42600, 42600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Mayonnaise 200 g', 11450, 11450]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Active Caramel Latte 500g', 95000, 50500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo school Cotton Candy 500g', 88400, 62500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Oat Drink 190ml (RTD)', 9999, 7500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 1 Get 1 Free - HiLo school Bubble Gum 500g', 154000, 92400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['BUY 1 GET 1 - Tropicana Slim Brownies Instant Powder Cake Mix 230 g - Bantu Dukung Hidup Sehat', 59400, 59400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['L-Men Hi Protein 2 Go Chocolate (24 pcs) - 12gr Protein / Serving', 237600, 199000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Klepon Latte Refill 500g - Powder Drink Tinggi Kalsium', 59400, 49500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['BUY 1 GET 1 - Tropicana Slim Brownies Instant Powder Cake Mix 230g - Bantu Dukung Hidup Sehat', 59400, 59400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HILO ALMOND MILK COCONUT 12Dx200G', 51500, 51500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo school Bubble Gum 500g', 80100, 80100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Lemon Tea 500 gram', 42000, 34300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Teen Biscuit Caramel 500gr', 92400, 65000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 1 Get 1 FREE L-Men Lose Weight Mango Sticky Rice 300gr', 290400, 145200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 1 Get 1 - Tropicana Slim Mint Cocoa - Minuman Cokelat Mint Nikmat Tanpa Gula Pasir', 40000, 20000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Es Pisang Ijo 10 sch x 2 pcs', 35000, 17500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NuTrilogi Seri Mas Kaka yang Banyak Akal', 70000, 42600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 1 Get 1 FREE Tropicana Slim Susu Low Fat Macchiato Coffee 500gr', 257400, 143000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Lokalate Kopi Pisang Bakar 10 sch', 15000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Choco Series Paket Hampers Idul Fitri Parcel Lebaran', 185300, 139600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Brownies Instant Powder Cake Mix 230g - Bantu Dukung Hidup Sehat', 31200, 31200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    
    data_SKU = data_SKU.append(pd.DataFrame([['BUY 1 GET 1 - Tropicana Slim Soy Latte - Kopi Plant Based Rendah Gula', 81800, 36900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Susu Low Fat Macchiato Coffee 500 gram', 75000, 75000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Teen Taro 500gr - Khusus Homdel', 80800, 40400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari RTD Jeruk Madu 200 ml (4 tetrapack)', 24000, 24000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["(Near ED) W'dank Bandrek 10 sachet Renceng - Khusus Homdel", 17300, 8650]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["(Near ED) W'dank Empon-Empon 10 sch - Khusus Homdel", 15000, 7500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Buy 1 Get 1 Free - HiLo Gold Biscuit Cereal 500g", 161600, 80800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Berondong 500 gram - Tinggi Vitamin A Lebih Rendah Gula", 59400, 44400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["(Near ED) HiLo Joint Plus Orange 140 gram", 38300, 19150]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["(Near ED) HiLo Active Es Teler 175gr", 29700, 14850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["(Near ED) Lokalate Kopi Tape Ketan 10's", 14900, 7450]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Sweetener Jahe (50 sch)", 45000, 45000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Anggur Hijau 40 Sachet", 62000, 43290]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Gula Asem 40 Sachet", 62000, 43290]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack - Tropicana Slim Low Fat Milk Korean Strawberry 500g", 257400, 159900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 - W'dank Temulawak Madu", 42000, 33300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack - Tropicana Slim Low Fat Milk Korean Strawberry 500g", 257400, 159900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 - W'dank Temulawak Madu", 42000, 33300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([["TRIPLE HEMAT - HiLo Taro Latte Refill 500g", 145500, 101800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["TRIPLE HEMAT - HiLo Thai Tea Refill 500g", 144000, 100900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["TRIPLE DEALS - NutriSari Lychee Tea & Lemon Tea 500 gram Refill", 103900, 72700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twinpack Tropicana Slim Bumbu Saus Telur Asin 3 Sachet - Lebih Rendah Lemak dan Garam", 58000, 29000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Near ED - Lokalate Kopi Andaliman 10 Sachet", 15000, 7500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([["HiLo RTD Chocolate Avocado 200ml (4pcs)", 25400, 25400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Buy 1 Get 1 - L-Men Gain Mass Klepon Latte", 277100, 138500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["HILO TEEN STRAWBERRY MILKSHAKE\xa0 12Dx500G", 86600, 86600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_order['product'] = data_order['product'].astype(str)
    data_SKU['product'] = data_SKU['product'].astype(str)

    data_order['product'] = data_order['product'].str.replace('L Men Gain Mass Chocolate 500 gr', 'L-Men Gain Mass Chocolate 500 gr')



    data_order = data_order.merge(data_SKU[['SKU', 'product', 'Harga Display', 'Harga Coret']].drop_duplicates('product'), how = 'left', on = 'product')
  
    data_order = data_order.reset_index(drop = True)

    indeks = data_order[data_order['product'] == "Lokalate Kopi Berondong 10's"].index.to_list()
    data_order['Harga Display'][indeks] = 15000
    data_order['Harga Coret'][indeks] = 15000
    indeks = data_order[data_order['SKU'].isnull()].index.to_list()
    for i in indeks:
        col = [x for x in data_SKU.columns if 'Alias Nama' in x]
        for j in col:
            if data_order['product'][i] in data_SKU[j].astype(str).values:
                SKU = data_SKU[data_SKU[j].astype(str) == data_order['product'][i]]['SKU'].values[0]
                data_order['SKU'][i] = SKU

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    data_SKU2 = pd.read_excel(r'SKU_File\data_SKU.xlsx')
    data_SKU2['Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

    s = requests.Session()
    s.get("http://tatanama.pythonanywhere.com")
    s.post("http://tatanama.pythonanywhere.com", data = {'username' : 'ecommerce', 'password' : 'ecommerce'})
    r = s.get("http://tatanama.pythonanywhere.com/download")

    with open(r'SKU_File\Master tatanama.xlsx', 'wb') as output:
        output.write(r.content)

    if os.path.isfile(r'SKU_File\Master tatanama.xlsx') :    
        SKU_append = pd.read_excel(r'SKU_File\Master tatanama.xlsx')
        SKU_append.columns = [x.replace('_', ' ') for x in SKU_append.columns]
        data_SKU2 = data_SKU2[~data_SKU2['SKU'].astype(str).isin(SKU_append['SKU'].astype(str))]
        data_SKU2 = data_SKU2.append(SKU_append, ignore_index = True, sort = False)

    to_excel = data_SKU2.to_excel(r'SKU_File\data_SKU.xlsx', index = False)

    for i in indeks:
        if str(data_order['product'][i]).lower() in data_SKU2['Nama Produk'].astype(str).str.lower().values:
            data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2['Nama Produk'].astype(str).str.lower()].values[0]

    list_alias_name = [x for x in data_SKU2.columns if 'Alias Nama' in x]

    for i in indeks:
        for j in list_alias_name:
            if str(data_order['product'][i]).lower() in data_SKU2[j].astype(str).str.lower().values:
                data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2[j].astype(str).str.lower()].values[0]

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    if len(indeks) != 0:
        print('Alert SKU Missing')
        data_order['product'][indeks].drop_duplicates().to_excel('Alert SKU Missing.xlsx', index = False)
    else :
        data_order['phone'] = data_order['phone'].astype(str).str.replace('+628', '08', regex = False)

        data_order['zip'] = data_order['zip'].replace('.0', '', regex = False)

        data_order = data_order.rename(columns = {'order_id' : 'Sales Order ID', 'name' : 'Customer Name', 'product' : 'Item Name', 'price' : 'Price', 'shipping_cost' : 'Shipping Cost', 'address' : 'Shipping Address1', 'city' : 'Shipping City', 'zip' : 'Shipping Zip', 'province' : 'Shipping Province', 'phone' : 'Shipping Phone', 'courier' : 'Shipping Courier'})
        data_order['Channel Order ID'] = data_order['Sales Order ID']
        data_order['Invoice Number'] = data_order['Sales Order ID']
        data_order['Shipping Name'] = data_order['Customer Name']
        data_order['Shipping Address2'] = 0
        data_order['Shipping Country'] = 'Indonesia'
        data_order['AWB'] = 0
        data_order['Channel'] = 'Order Online Jateng'


        data_order['Order Date'] = pd.to_datetime(data_order['created_at'])

        data_order['SKU'] = data_order['SKU'].astype(str)
        data_order['Item Name'] = data_order['Item Name'].astype(str)
        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str).str.replace('(S)', '', regex = False)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        index = data_order[data_order['SKU'].astype(str) == '2306551174'].index.to_list()
        data_order['SKU'][index] = '2306592173'

        data_order = data_order.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('hd','', regex = False)
        temp['SKU'] = temp['SKU'].astype(str).str.replace('HD','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        data_order['Real SKU'] = data_order['Real SKU'].astype(str)
        data_order = data_order.merge(data_SKU2[['SKU', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Unbundling ====== 6/10")        
        # Forstok Unbundling    
        list_col = ['SKU'] + data_SKU2.columns[data_SKU2.columns.get_loc('Produk 1'):data_SKU2.columns.get_loc('Harga Organik 7')+1].to_list()
        data_order = data_order.merge(data_SKU2[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        list_pcs = [x for x in data_order.columns if 'PCS' in x]
        for i in list_pcs:
            data_order[i] = data_order[i] * data_order['Quantity']
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        indeks = data_order[data_order['Brand'] == 'Bundle'].index.to_list()
        data_order['Bundle Flag'] = np.nan
        data_order['Bundle Flag'][indeks] = 'Bundle'

        indeks = data_order[data_order['Brand'] == 'Bundle'][data_order[data_order['Brand'] == 'Bundle']['SKU'].astype(str).str.contains('(S)', regex = False)].index.to_list()
        data_order['SKU Produk 1'][indeks] = '(S)' + data_order['SKU Produk 1'][indeks].astype(str)
        data_order['SKU Produk 2'][indeks] = '(S)' + data_order['SKU Produk 2'][indeks].astype(str)
        data_order['SKU Produk 3'][indeks] = '(S)' + data_order['SKU Produk 3'][indeks].astype(str)
        data_order['SKU Produk 4'][indeks] = '(S)' + data_order['SKU Produk 4'][indeks].astype(str)
        data_order['SKU Produk 5'][indeks] = '(S)' + data_order['SKU Produk 5'][indeks].astype(str)
        data_order['SKU Produk 6'][indeks] = '(S)' + data_order['SKU Produk 6'][indeks].astype(str)
        data_order['SKU Produk 7'][indeks] = '(S)' + data_order['SKU Produk 7'][indeks].astype(str)

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Filling Date ====== 7/10")
        data_order['Date'] = np.nan
        data_order['Month'] = np.nan
        data_order['Year'] = np.nan

        for i in range(data_order.shape[0]):
            if int(data_order['Order Date'][i].strftime('%d')) <= 12:
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).year
            else :
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i]).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i]).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i]).year

        quarter = pd.DataFrame([['January', 1], ['February', 1], ['March', 1], ['April', 2], ['May', 2], ['June', 2], 
                ['July', 3], ['August', 3], ['September', 3],['October', 4], ['November', 4], ['December', 4]], columns = ['Bulan', 'Quarter'])
        data_order = data_order.merge(quarter, how = 'left', left_on = 'Month', right_on = 'Bulan')
        data_order = data_order.drop(['Bulan'], axis = 1)
        data_bulan = pd.DataFrame([{'Bulan' : 'December', 'Number' : 12} ,
                {'Bulan' : 'January' , 'Number': 1},
                {'Bulan' : 'February' , 'Number': 2},
                {'Bulan' : 'March' , 'Number': 3},
                {'Bulan' : 'April' , 'Number': 4},
                {'Bulan' : 'May' , 'Number': 5},
                {'Bulan' : 'June', 'Number': 6},
                {'Bulan' : 'July' , 'Number': 7},
                {'Bulan' : 'August', 'Number' : 8},
                {'Bulan' : 'September', 'Number' : 9},
                {'Bulan' : 'October' , 'Number': 10},
                {'Bulan' : 'November' , 'Number': 11}])
        temp = data_order.copy()
        temp['Day'] = temp['Date']
        temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
        temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
        data_order['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.week
        temp['Hour'] = pd.to_datetime(data_order['Order Date']).dt.hour
        temp['Minute'] = pd.to_datetime(data_order['Order Date']).dt.minute
        temp['Second'] = pd.to_datetime(data_order['Order Date']).dt.second
        data_order['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']])



        order_all = data_order.copy()
        order_all['Total'] = order_all['net_revenue']
        order_all['Price List NFI'] = np.nan
        order_all['Total Net'] = np.nan



        order_all = order_all.rename(columns={'Channel Order ID' : 'Order #',
                                                'Status' : 'Order Status',
                                                'Order Date' : 'Order date',
                                                'Item Name' :'Product Name',
                                                'Bundle Name' : 'Bundle',
                                                'Shipping Country' : 'Country',
                                                'Shipping Province' : 'Region',
                                                'Shipping City' : 'City',
                                                'Shipping Zip' : 'Zip Code',
                                                'Shipping Address1' : 'Address',
                                                'Shipping Phone' : 'Phone',
                                                'Quantity' : 'Qty. Invoiced',
                                                'Harga Display' : 'Regular Price',
                                                'net_revenue' : 'Subtotal'})

        
        order_all['Selling Price'] = order_all['Harga Coret'].astype(int)
        order_all['Kecamatan'] = np.nan
        order_all['Kelurahan'] = np.nan

        print("Filling Location")
        indeks = order_all[order_all['City'].astype(str).str.contains('/')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains('-')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[0]

        indeks = order_all[order_all['City'].astype(str).str.contains(',')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains(',')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[0]

        order_all['City'] = order_all['City'].astype(str).str.replace('Kab\.', 'Kabupaten' ,case = False)

        master_map = pd.read_csv(r'All Data/Province.csv', names = ['Kode Prov', 'Province'], header= 0)
        master_map2 = pd.read_csv(r'All Data/City.csv', names = ['Kode City', 'Kode Prov', 'City'], header = 0)
        master_map = master_map.merge(master_map2, how = 'right', on = 'Kode Prov')
        master_map['Kode Prov'][515] = 14
        master_map['Province'][515] = 'Riau'
        master_map['Kode Prov'] = master_map['Kode Prov'].astype(int)
        master_map['Province'] = master_map['Province'].str.title()
        master_map['City'] = master_map['City'].str.title()

        city = pd.read_excel(r'All Data/list_city.xlsx')
        temp = order_all.copy()
        temp['City'] = temp['City'].astype(str).str.lower()
        temp['City'] = temp['City'].astype(str).str.replace('kab. ', 'kabupaten ', regex = False, case = False)
        city['All City'] = city['All City'].astype(str).str.lower()
        temp = temp.merge(city.drop_duplicates('All City'), how = 'left', left_on = 'City', right_on = 'All City').set_index(temp.index)
        indeks = temp[temp['Real City'].notnull()].index.to_list()
        order_all['City'][indeks] = temp['Real City'][indeks]

        province = pd.read_excel(r'All Data/list_province.xlsx')
        temp = order_all.copy()
        temp['Region'] = temp['Region'].astype(str).str.lower()
        province['All Province'] = province['All Province'].astype(str).str.lower()
        temp = temp.merge(province.drop_duplicates('All Province'), how = 'left', left_on = 'Region', right_on = 'All Province').set_index(temp.index)
        indeks = temp[temp['Real Province'].notnull()].index.to_list()
        order_all['Region'][indeks] = temp['Real Province'][indeks]

        temp = order_all.copy()
        temp = temp[temp['Region'].isnull()]
        temp['Region'] = temp.merge(master_map, how = 'left', on = 'City').set_index(temp.index)['Province']
        order_all['Region'][temp.index] = temp['Region']  

        district = pd.read_excel(r'All Data/list_district.xlsx')
        temp = order_all.copy()
        temp['Kecamatan'] = temp['Kecamatan'].astype(str).str.lower()
        district['All District'] = district['All District'].astype(str).str.lower()
        temp = temp.merge(district.drop_duplicates('All District'), how = 'left', left_on = 'Kecamatan', right_on = 'All District').set_index(temp.index)
        indeks = temp[temp['Real District'].notnull()].index.to_list()
        order_all['Kecamatan'][indeks] = temp['Real District'][indeks]

        temp = order_all.copy()
        temp2 = temp[['Region', 'City', 'Kecamatan']].merge(master_map, how = 'left', on = 'City')
        indeks = temp2[temp2['Region'] != temp2['Province']][temp2[temp2['Region'] != temp2['Province']]['City'].notnull()].index.to_list()
        order_all['City'][indeks] = np.nan

        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        print("Unbundling")
        data_bundle1 = order_all[~order_all['Produk 1'].isnull()]
        data_bundle1['Bundle Name'] = data_bundle1['Product Name']
        data_bundle1['Product Name'] = data_bundle1['Produk 1']
        data_bundle1['SKU'] = data_bundle1['SKU Produk 1']
        data_bundle1['Qty. Invoiced'] = data_bundle1['PCS Produk 1']
        data_bundle1['Price List NFI'] = data_bundle1['Price List NFI 1']
        data_bundle1['Total Net'] = data_bundle1['Price List NFI 1']
        data_bundle1['Bundle Flag'] = np.nan

        data_bundle2 = order_all[~order_all['Produk 2'].isnull()]
        data_bundle2['Bundle Name'] = data_bundle2['Product Name']
        data_bundle2['Product Name'] = data_bundle2['Produk 2']
        data_bundle2['SKU'] = data_bundle2['SKU Produk 2']
        data_bundle2['Qty. Invoiced'] = data_bundle2['PCS Produk 2']
        data_bundle2['Price List NFI'] = data_bundle2['Price List NFI 2']
        data_bundle2['Total Net'] = data_bundle2['Price List NFI 2'] 
        data_bundle2['Bundle Flag'] = np.nan

        data_bundle3 = order_all[~order_all['Produk 3'].isnull()]
        data_bundle3['Bundle Name'] = data_bundle3['Product Name']
        data_bundle3['Product Name'] = data_bundle3['Produk 3']
        data_bundle3['SKU'] = data_bundle3['SKU Produk 3']
        data_bundle3['Qty. Invoiced'] = data_bundle3['PCS Produk 3']
        data_bundle3['Price List NFI'] = data_bundle3['Price List NFI 3']
        data_bundle3['Total Net'] = data_bundle3['Price List NFI 3'] 
        data_bundle3['Bundle Flag'] = np.nan

        data_bundle4 = order_all[~order_all['Produk 4'].isnull()]
        data_bundle4['Bundle Name'] = data_bundle4['Product Name']
        data_bundle4['Product Name'] = data_bundle4['Produk 4']
        data_bundle4['SKU'] = data_bundle4['SKU Produk 4']
        data_bundle4['Qty. Invoiced'] = data_bundle4['PCS Produk 4']
        data_bundle4['Price List NFI'] = data_bundle4['Price List NFI 4']
        data_bundle4['Total Net'] = data_bundle4['Price List NFI 4'] 
        data_bundle4['Bundle Flag'] = np.nan

        data_bundle5 = order_all[~order_all['Produk 5'].isnull()]
        data_bundle5['Bundle Name'] = data_bundle5['Product Name']
        data_bundle5['Product Name'] = data_bundle5['Produk 5']
        data_bundle5['SKU'] = data_bundle5['SKU Produk 5']
        data_bundle5['Qty. Invoiced'] = data_bundle5['PCS Produk 5']
        data_bundle5['Price List NFI'] = data_bundle5['Price List NFI 5']
        data_bundle5['Total Net'] = data_bundle5['Price List NFI 5']
        data_bundle5['Bundle Flag'] = np.nan

        data_bundle6 = order_all[~order_all['Produk 6'].isnull()]
        data_bundle6['Bundle Name'] = data_bundle6['Product Name']
        data_bundle6['Product Name'] = data_bundle6['Produk 6']
        data_bundle6['SKU'] = data_bundle6['SKU Produk 6']
        data_bundle6['Qty. Invoiced'] = data_bundle6['PCS Produk 6']
        data_bundle6['Price List NFI'] = data_bundle6['Price List NFI 6']
        data_bundle6['Total Net'] = data_bundle6['Price List NFI 6']
        data_bundle6['Bundle Flag'] = np.nan

        data_bundle7 = order_all[~order_all['Produk 7'].isnull()]
        data_bundle7['Bundle Name'] = data_bundle7['Product Name']
        data_bundle7['Product Name'] = data_bundle7['Produk 7']
        data_bundle7['SKU'] = data_bundle7['SKU Produk 7']
        data_bundle7['Qty. Invoiced'] = data_bundle7['PCS Produk 7']
        data_bundle7['Price List NFI'] = data_bundle7['Price List NFI 7']
        data_bundle7['Total Net'] = data_bundle7['Price List NFI 7']
        data_bundle7['Bundle Flag'] = np.nan

        data_bundle = data_bundle1.append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)
        data_bundle['SKU'] = data_bundle['SKU'].astype(str)
        data_bundle['SKU'] = data_bundle['SKU'].str.replace('\.0$', '', regex = True)
        data_bundle[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']] = data_bundle.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item_y', 'Parent SKU_y']]

        temp = data_bundle[data_bundle['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)

        indeks = data_bundle[data_bundle['Real SKU'].isnull()].index.to_list()
        data_bundle['Real SKU'][indeks] = temp['Real SKU_y'][indeks]
        data_bundle['Real Nama Produk'][indeks] = temp['Nama Produk'][indeks]
        data_bundle['Brand'][indeks] = temp['Brand_y'][indeks]
        data_bundle['Sub Brand'][indeks] = temp['Sub Brand_y'][indeks]
        data_bundle['Parent Item'][indeks] = temp['Parent Item_y'][indeks]
        data_bundle['Parent SKU'][indeks] = temp['Parent SKU_y'][indeks]

        print("Pricing")
        order_all = order_all.append(data_bundle, ignore_index = True, sort = False)

        colname = temp.columns[temp.columns.get_loc('Produk 1') : temp.columns.get_loc('Harga Cost 7') + 1]
        colname_str = [x for x in colname if 'Subtotal' not in x and 'Harga' not in x]
        colname_int = [x for x in colname if x not in colname_str]

        for i in colname_str:
            temp[i] = np.nan

        for i in colname_int:
            temp[i] = 0

        data_order = data_order.append(temp , ignore_index = True, sort = False)


        order_all = order_all.merge(data_SKU2[['SKU', 'Price List NFI', 'Harga Cost']].drop_duplicates('SKU'), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(order_all.index)
        order_all['Price List NFI_x'] = order_all['Price List NFI_x'].fillna(order_all['Price List NFI_y'])
        order_all =  order_all.drop(['Price List NFI_y', 'SKU_y'], axis = 1)
        order_all = order_all.rename(columns = {'SKU_x' : 'SKU', 'Price List NFI_x' : 'Price List NFI'})
        
        indeks = order_all[order_all['Product Name'] == 'HiLo Teen Chocolate 250gr'].index.to_list()
        order_all['Real Nama Produk'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Parent Item'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Brand'][indeks] = 'HiLo'
        order_all['Sub Brand'][indeks] = 'HILO TEEN'
        order_all['Price List NFI'][indeks] = 36850
        order_all['Harga Cost'][indeks] = 36850
        order_all['Real SKU'][indeks] = '2101651155'
        order_all['Parent SKU'][indeks] = '2101651155'
        

        order_all['Price List NFI'] = pd.to_numeric(order_all['Price List NFI']).astype(int)

        order_all['Harga Cost'] = pd.to_numeric(order_all['Harga Cost'], errors = 'coerce').fillna(0).astype(int)
        order_all['Qty. Invoiced'] = pd.to_numeric(order_all['Qty. Invoiced']).astype(int)

        order_all['Total Net'] = order_all['Price List NFI'] * order_all['Qty. Invoiced']
        order_all['Total Harga Cost'] = order_all['Harga Cost'] * order_all['Qty. Invoiced']
        order_all['Subtotal'] = order_all['Selling Price'] * order_all['Qty. Invoiced']
        order_all['Total'] = order_all['Selling Price'] * order_all['Qty. Invoiced']

        order_all = order_all.reset_index(drop = True)
        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        order_all['Seller Discount'] = order_all['discount']
        order_all['Shipping'] = order_all['Shipping Cost']
        
        temp = order_all[order_all['Brand'] != 'Bundle']
        temp['discount'] = temp['discount'] * temp['Total Harga Cost']/temp.groupby(['Order #'])['Total Harga Cost'].transform('sum')
        order_all['discount'][temp.index] = temp['discount']

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Subtotal', 'Total']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Total'][list_nobundle.index] = list_nobundle['Total_y']
        order_all['Subtotal'][list_nobundle.index] = list_nobundle['Subtotal_y']

        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Subtotal'] = temp['Subtotal'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        temp['Selling Price'] = temp['Subtotal']/temp['Qty. Invoiced']
        temp['Total'] = temp['Total'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Total'][temp.index] = temp['Total']
        order_all['Subtotal'][temp.index] = temp['Subtotal']
        order_all['Selling Price'][temp.index] = temp['Selling Price']


        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Seller Discount']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Seller Discount'][list_nobundle.index] = list_nobundle['Seller Discount_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Seller Discount'] = temp['Seller Discount'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Seller Discount'][temp.index] = temp['Seller Discount']


        temp = order_all[order_all['Bundle Name'].isnull()]
        temp_group = temp[['Order #','Shipping']].groupby(['Order #']).sum().reset_index()

        temp = order_all.merge(temp_group, how = 'left', on = 'Order #').set_index(order_all.index)
        temp['Shipping_x'] = temp['Shipping_y'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Shipping'][temp.index] = temp['Shipping_y']
        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Shipping']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Shipping'][list_nobundle.index] = list_nobundle['Shipping_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Shipping'] = temp['Shipping'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Shipping'][temp.index] = temp['Shipping']
        order_all['True datetime'] = pd.to_datetime(order_all['True datetime'])
        order_all['Promo'] = np.nan
        order_all['Discount MC'] = np.nan
        

        order_all['Warehouse Name'] = 'Primary Warehouse'
        order_all['Store'] = 'Order Online'

        order_all['Customer Email'] = order_all['email']
        order_all['Order Status'] = order_all['status']
        order_all['Payment Channel'] = order_all['payment_method']
        order_all['Coupon Code'] = order_all['coupon']
        order_all.columns.to_list()

        order_all_append = order_all[['Sales Order ID', 'Store',
            'Product Name',
            'Customer Name',
            'Phone',
            'Address',
            'Region',
            'City',
            'Zip Code',
            'payment_status',
            'Regular Price',
            'Shipping Courier',
            'Shipping Cost',
            'Subtotal',
            'Qty. Invoiced',
            'SKU',
            'Order #',
            'Invoice Number',
            'Shipping Name',
            'Shipping Address2',
            'Country',
            'AWB',
            'Channel',
            'Order date',
            'Real SKU',
            'Real Nama Produk',
            'Brand',
            'Sub Brand',
            'Parent Item',
            'Parent SKU',
            'Produk 1',
            'SKU Produk 1',
            'PCS Produk 1',
            'Price List NFI 1',
            'Subtotal Produk 1',
            'Harga Display 1',
            'Harga Cost 1',
            'Harga Organik 1',
            'Produk 2',
            'SKU Produk 2',
            'PCS Produk 2',
            'Price List NFI 2',
            'Subtotal Produk 2',
            'Harga Display 2',
            'Harga Cost 2',
            'Harga Organik 2',
            'Produk 3',
            'SKU Produk 3',
            'PCS Produk 3',
            'Price List NFI 3',
            'Subtotal Produk 3',
            'Harga Display 3',
            'Harga Cost 3',
            'Harga Organik 3',
            'Produk 4',
            'SKU Produk 4',
            'PCS Produk 4',
            'Price List NFI 4',
            'Subtotal Produk 4',
            'Harga Display 4',
            'Harga Cost 4',
            'Harga Organik 4',
            'Produk 5',
            'SKU Produk 5',
            'PCS Produk 5',
            'Price List NFI 5',
            'Subtotal Produk 5',
            'Harga Display 5',
            'Harga Cost 5',
            'Harga Organik 5',
            'Produk 6',
            'SKU Produk 6',
            'PCS Produk 6',
            'Price List NFI 6',
            'Subtotal Produk 6',
            'Harga Display 6',
            'Harga Cost 6',
            'Harga Organik 6',
            'Produk 7',
            'SKU Produk 7',
            'PCS Produk 7',
            'Price List NFI 7',
            'Subtotal Produk 7',
            'Harga Display 7',
            'Harga Cost 7',
            'Harga Organik 7',
            'Bundle Flag',
            'Date',
            'Month',
            'Year',
            'Quarter',
            'Week',
            'True datetime',
            'Total',
            'Price List NFI',
            'Total Net',
            'Selling Price',
            'Kecamatan',
            'Kelurahan',
            'Bundle Name',
            'Harga Cost',
            'Total Harga Cost',
            'Seller Discount',
            'Shipping',
            'Promo',
            'Discount MC',
            'Warehouse Name',
            'Customer Email',
            'Order Status',
            'Payment Channel',
            'Coupon Code']]

        data_all = data_all[~data_all['Order #'].astype(str).isin(order_all_append['Order #'].astype(str))]
        data_all = data_all.append(order_all_append, ignore_index = True, sort = False)

        order_online_jateng = True
        del file_name

if not order_online_bali:

    import pandas as pd
    import numpy as np
    import requests
    import os
    
    print('order_online_bali')

    before = os.listdir(os.getcwd() + '/Input Data')

    options = Options()
    options.add_experimental_option("prefs", {
            "download.default_directory": os.path.abspath("Input Data"),
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            "safebrowsing.enabled": False
    })

    driver = webdriver.Chrome(options=options)
    driver.fullscreen_window()
    driver.get("https://app.orderonline.id")

    username = driver.find_element_by_name("email")
    username.clear()
    username.send_keys("customerbali@nutrimart.co.id")

    password = driver.find_element_by_name("password")
    password.send_keys("balihomdel")
    password.click()

    driver.find_element_by_class_name("btn-submit").click()
    time.sleep(15)
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="main-nav-dropdown"]/ul/li[3]/a'))).click()
    time.sleep(5)
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[1]/div/div[2]/div/div/div').click()
    time.sleep(5)
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="app"]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[1]/div[1]/ul/li[6]'))).click()
    time.sleep(5)
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[2]/button[2]').click()
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[3]/div/button').click()
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[3]/div/div/button[1]').click()

    time.sleep(15)

    after = os.listdir(os.getcwd() + '/Input Data')
    change = set(after) - set(before)
    if len(change) == 1:
        file_name = change.pop()
    elif len(change) == 0: 
        print("No file downloaded")
    else :
        print("More than one file downloaded")

    data_order = pd.read_csv(r'Input Data/' + str(file_name))
    driver.close()
#         data_order = pd.read_csv(r'Input Data/orderonline_orders_all_products_Jakarta.csv')
#             product_order = pd.read_csv(r'Input Data/orderonline_products_Jakarta.csv')

    data_order['order_id'] = data_order['order_id'].fillna(method='ffill')
    data_order = data_order.drop('quantity', axis = 1)
    temp = data_order.drop_duplicates('order_id').drop('product', axis = 1)
    data_order = data_order[['order_id', 'product']]
    data_order = data_order.merge(temp, how = 'left', on = 'order_id')
    data_order['order_id'] = data_order['order_id'].astype(int)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Shirataki Noodles 71gr (x2) ', 'Twin Pack: Tropicana Slim Shirataki Noodles 71gr ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Saus Tiram 200ml (x2) ', 'Twin Pack: Tropicana Slim Saus Tiram 200ml ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Sweetener Rose Vanilla 50 sch (x2) ', 'Twin Pack: Tropicana Slim Sweetener Rose Vanilla 50 sch ', regex = False)

    product = data_order['product'].str.split("\(x",1, expand = True)
    data_order['Quantity'] = product[1].astype(str).str.replace(')', '', regex = False).astype(int)
    data_order['product'] = product[0].str.strip().str.replace('  ', ' ').str.replace('6 SCH', '6sch').str.replace("W'Dank", "W'dank").str.replace('L-men', 'L-Men').str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("school", "School").str.replace("Rtd", "RTD")
    data_order['product'] = data_order['product'].str.replace("HiLo Thai Tea \(10sch\)$", 'HiLo Thai Tea (10 sch)', regex = True)

    data_SKU = pd.read_excel(r'Order Online\SKU buat andra updated maret 2021.xlsx')
    data_SKU = data_SKU.rename(columns = {'Product' : 'product', 'PL NFI' : 'Price List NFI'})
    data_SKU['SKU'] = data_SKU['SKU'].astype(str).str.replace('.0', '', regex = False)
    data_SKU = data_SKU[data_SKU['SKU'] != 'nan'][data_SKU[data_SKU['SKU'] != 'nan']['SKU'] != '0']
    data_SKU['product'] = data_SKU['product'].str.strip().str.replace('L-men', 'L-Men').str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("school", "School").str.replace("Rtd", "RTD")


    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Kecap Manis 200ml'].copy()
    price['product'] = 'Tropicana Slim Kecap Manis 200m'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Diabtx (50 sch)'].copy()
    price['product'] = 'Tropicana Slim Diabtx 50 sch'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Hokkaido Cheese 100gr'].copy()
    price['product'] = 'Tropicana Slim Hokaido Cheese 100gr'


    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['L-Men Bar Crunchy Chocolate 12sch', 5500, 5500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
  
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Mangga Gandaria', 45000, 45000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
  
    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Teen Vanilla Caramel 500gr', 71500, 71500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Paket Bundle HiLo Renceng (Hilo Chocolate Banana (10 sch) + Hilo Chocolate Taro (10 sch) + HiLo Thai Tea (10sch))', 35200, 35200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Berondong 10's", 15000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Kecap Asin 200 ml", 28500, 26000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim DIABTX (100 sch)", 87700, 75000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Teen Chocolate 750gr", 117800, 104000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Paket Ngopi Lokalate", 136000, 109200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Hi Protein 2 Go Chocolate (24 TETRAPAK)", 240000, 238000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 Tropicana Slim Goldenmil Vanilla (6sch)", 39160, 31000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Kawista", 17500, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["Paket Bundle HiLo (HiLo Active Chocolate 500gr + HiLo Teen Yoghurt Banana 250gr)", 122900, 101850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Strawberry Jam 375gr", 72600, 58500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Crunch BBQ Beef (20gr)", 13500, 10500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Cocopandan 40 sch", 52500, 52500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 - W'dank Empon Empon 10 Sachet Renceng", 35000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Semangka 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Nanas 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["W'Dank Empon-Empon 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Milk Skim Fiber Pro Plain 500gr", 135000, 106700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Es Teler 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: HiLo Es Teler 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Hilo Es Ketan Hitam 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Triple Pack: Tropicana Slim Korean Garlic Butter Cookies (5 Sch)", 73500, 52000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Avocado Coffee 4 Sch", 21200, 12100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Sambal Terasi 200 gr", 35600, 29700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Tropicana Slim Avocado Coffee 4 sch x 2", 42400, 24200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Bar Chocolate (12 Sch) + L-Men Protein Crunch BBQ Beef x 2", 159000, 107800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Buy 5 Get 5 L-Men Protein Crunch BBQ Beef (20gr)", 130000, 67500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Active Ketan Hitam 175gr', 48500, 20700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Es Ketan Hitam 10 sch', 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Sweetener Lemongrass Pandan 50 sch', 44200, 28900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 1 Get 1 FREE: Lokalate Kopi Berondong (10 sch)', 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)HiLo School Chocolate 500gr', 38300, 38300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)Tropicana Slim Extra Virgin Olive Oil 500 ml', 42900, 42900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)HiLo Active Vanilla 500gr', 32600, 32600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)HiLo Gold Plain (Original) 500gr', 36600, 36600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)HiLo Teen Chocolate 750gr', 55500, 55500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)HiLo School Chocolate 750gr', 55500, 55500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)L-Men Gain Mass Chocolate 500gr', 62900, 62900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['L-Men Platinum Choco 800 gr - Near ED', 157300, 157300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)HiLo Teen Chocolate 500gr', 38300, 38300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo RTD Milky Brown Sugar 200 ml', 2574, 2574]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo RTD Thai Tea 200ml - Near ED', 2860, 2860]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 1 Get 1 - L-Men Gain Mass Klepon Latte', 277100, 138500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 12 Get 12 - HiLo Chocolate Avocado Ready to Drink 200ml - Minuman Siap Minum Tinggi Kalsium', 152400, 84000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Korean Garlic Butter Cookies 5 sch - Khusus Homdel', 11150, 11150]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Active Chocolate 1000 gr', 60050, 60050]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) L-Men PlantProtein Ogura 216g', 57700, 57700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Active Caramel Latte 500g', 32600, 32600]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_order['product'] = data_order['product'].astype(str)
    data_SKU['product'] = data_SKU['product'].astype(str)

    data_order['product'] = data_order['product'].str.replace('L Men Gain Mass Chocolate 500 gr', 'L-Men Gain Mass Chocolate 500 gr')



    data_order = data_order.merge(data_SKU[['SKU', 'product', 'Harga Display', 'Harga Coret']].drop_duplicates('product'), how = 'left', on = 'product')

    data_order = data_order.reset_index(drop = True)

    indeks = data_order[data_order['product'] == "Lokalate Kopi Berondong 10's"].index.to_list()
    data_order['Harga Display'][indeks] = 15000
    data_order['Harga Coret'][indeks] = 15000
    indeks = data_order[data_order['SKU'].isnull()].index.to_list()
    for i in indeks:
        col = [x for x in data_SKU.columns if 'Alias Nama' in x]
        for j in col:
            if data_order['product'][i] in data_SKU[j].astype(str).values:
                SKU = data_SKU[data_SKU[j].astype(str) == data_order['product'][i]]['SKU'].values[0]
                data_order['SKU'][i] = SKU

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    data_SKU2 = pd.read_excel(r'SKU_File\data_SKU.xlsx')
    data_SKU2['Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

    for i in indeks:
        if str(data_order['product'][i]).lower() in data_SKU2['Nama Produk'].astype(str).str.lower().values:
            data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2['Nama Produk'].astype(str).str.lower()].values[0]

    list_alias_name = [x for x in data_SKU2.columns if 'Alias Nama' in x]

    for i in indeks:
        for j in list_alias_name:
            if str(data_order['product'][i]).lower() in data_SKU2[j].astype(str).str.lower().values:
                data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2[j].astype(str).str.lower()].values[0]

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    if len(indeks) != 0:
        print('Alert SKU Missing')
        data_order['product'][indeks].drop_duplicates().to_excel('Alert SKU Missing.xlsx', index = False)
    else :
        data_order['phone'] = data_order['phone'].astype(str).str.replace('+628', '08', regex = False)

        data_order['zip'] = data_order['zip'].replace('.0', '', regex = False)

        data_order = data_order.rename(columns = {'order_id' : 'Sales Order ID', 'name' : 'Customer Name', 'product' : 'Item Name', 'price' : 'Price', 'shipping_cost' : 'Shipping Cost', 'address' : 'Shipping Address1', 'city' : 'Shipping City', 'zip' : 'Shipping Zip', 'province' : 'Shipping Province', 'phone' : 'Shipping Phone', 'courier' : 'Shipping Courier'})
        data_order['Channel Order ID'] = data_order['Sales Order ID']
        data_order['Invoice Number'] = data_order['Sales Order ID']
        data_order['Shipping Name'] = data_order['Customer Name']
        data_order['Shipping Address2'] = 0
        data_order['Shipping Country'] = 'Indonesia'
        data_order['AWB'] = 0
        data_order['Channel'] = 'Order Online Bali'


        data_order['Order Date'] = pd.to_datetime(data_order['created_at'])

        data_order['SKU'] = data_order['SKU'].astype(str)
        data_order['Item Name'] = data_order['Item Name'].astype(str)
        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str).str.replace('(S)', '', regex = False)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        index = data_order[data_order['SKU'].astype(str) == '2306551174'].index.to_list()
        data_order['SKU'][index] = '2306592173'

        data_order = data_order.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('hd','', regex = False)
        temp['SKU'] = temp['SKU'].astype(str).str.replace('HD','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        data_order['Real SKU'] = data_order['Real SKU'].astype(str)
        data_order = data_order.merge(data_SKU2[['SKU', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Unbundling ====== 6/10")        
        # Forstok Unbundling    
        list_col = ['SKU'] + data_SKU2.columns[data_SKU2.columns.get_loc('Produk 1'):data_SKU2.columns.get_loc('Harga Organik 7')+1].to_list()
        data_order = data_order.merge(data_SKU2[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        list_pcs = [x for x in data_order.columns if 'PCS' in x]
        for i in list_pcs:
            data_order[i] = data_order[i] * data_order['Quantity']
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        indeks = data_order[data_order['Brand'] == 'Bundle'].index.to_list()
        data_order['Bundle Flag'] = np.nan
        data_order['Bundle Flag'][indeks] = 'Bundle'

        indeks = data_order[data_order['Brand'] == 'Bundle'][data_order[data_order['Brand'] == 'Bundle']['SKU'].astype(str).str.contains('(S)', regex = False)].index.to_list()
        data_order['SKU Produk 1'][indeks] = '(S)' + data_order['SKU Produk 1'][indeks].astype(str)
        data_order['SKU Produk 2'][indeks] = '(S)' + data_order['SKU Produk 2'][indeks].astype(str)
        data_order['SKU Produk 3'][indeks] = '(S)' + data_order['SKU Produk 3'][indeks].astype(str)
        data_order['SKU Produk 4'][indeks] = '(S)' + data_order['SKU Produk 4'][indeks].astype(str)
        data_order['SKU Produk 5'][indeks] = '(S)' + data_order['SKU Produk 5'][indeks].astype(str)
        data_order['SKU Produk 6'][indeks] = '(S)' + data_order['SKU Produk 6'][indeks].astype(str)
        data_order['SKU Produk 7'][indeks] = '(S)' + data_order['SKU Produk 7'][indeks].astype(str)

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Filling Date ====== 7/10")
        data_order['Date'] = np.nan
        data_order['Month'] = np.nan
        data_order['Year'] = np.nan

        for i in range(data_order.shape[0]):
            if int(data_order['Order Date'][i].strftime('%d')) <= 12:
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).year
            else :
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i]).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i]).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i]).year

        quarter = pd.DataFrame([['January', 1], ['February', 1], ['March', 1], ['April', 2], ['May', 2], ['June', 2], 
                ['July', 3], ['August', 3], ['September', 3],['October', 4], ['November', 4], ['December', 4]], columns = ['Bulan', 'Quarter'])
        data_order = data_order.merge(quarter, how = 'left', left_on = 'Month', right_on = 'Bulan')
        data_order = data_order.drop(['Bulan'], axis = 1)
        data_bulan = pd.DataFrame([{'Bulan' : 'December', 'Number' : 12} ,
                {'Bulan' : 'January' , 'Number': 1},
                {'Bulan' : 'February' , 'Number': 2},
                {'Bulan' : 'March' , 'Number': 3},
                {'Bulan' : 'April' , 'Number': 4},
                {'Bulan' : 'May' , 'Number': 5},
                {'Bulan' : 'June', 'Number': 6},
                {'Bulan' : 'July' , 'Number': 7},
                {'Bulan' : 'August', 'Number' : 8},
                {'Bulan' : 'September', 'Number' : 9},
                {'Bulan' : 'October' , 'Number': 10},
                {'Bulan' : 'November' , 'Number': 11}])
        temp = data_order.copy()
        temp['Day'] = temp['Date']
        temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
        temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
        data_order['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.week
        temp['Hour'] = pd.to_datetime(data_order['Order Date']).dt.hour
        temp['Minute'] = pd.to_datetime(data_order['Order Date']).dt.minute
        temp['Second'] = pd.to_datetime(data_order['Order Date']).dt.second
        data_order['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']])



        order_all = data_order.copy()
        order_all['Total'] = order_all['net_revenue']
        order_all['Price List NFI'] = np.nan
        order_all['Total Net'] = np.nan



        order_all = order_all.rename(columns={'Channel Order ID' : 'Order #',
                                                'Status' : 'Order Status',
                                                'Order Date' : 'Order date',
                                                'Item Name' :'Product Name',
                                                'Bundle Name' : 'Bundle',
                                                'Shipping Country' : 'Country',
                                                'Shipping Province' : 'Region',
                                                'Shipping City' : 'City',
                                                'Shipping Zip' : 'Zip Code',
                                                'Shipping Address1' : 'Address',
                                                'Shipping Phone' : 'Phone',
                                                'Quantity' : 'Qty. Invoiced',
                                                'Harga Display' : 'Regular Price',
                                                'net_revenue' : 'Subtotal'})


        order_all['Selling Price'] = order_all['Harga Coret'].astype(int)
        order_all['Kecamatan'] = np.nan
        order_all['Kelurahan'] = np.nan

        print("Filling Location")
        indeks = order_all[order_all['City'].astype(str).str.contains('/')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains('-')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[0]

        indeks = order_all[order_all['City'].astype(str).str.contains(',')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains(',')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[0]

        order_all['City'] = order_all['City'].astype(str).str.replace('Kab\.', 'Kabupaten' ,case = False)

        master_map = pd.read_csv(r'All Data/Province.csv', names = ['Kode Prov', 'Province'], header= 0)
        master_map2 = pd.read_csv(r'All Data/City.csv', names = ['Kode City', 'Kode Prov', 'City'], header = 0)
        master_map = master_map.merge(master_map2, how = 'right', on = 'Kode Prov')
        master_map['Kode Prov'][515] = 14
        master_map['Province'][515] = 'Riau'
        master_map['Kode Prov'] = master_map['Kode Prov'].astype(int)
        master_map['Province'] = master_map['Province'].str.title()
        master_map['City'] = master_map['City'].str.title()

        city = pd.read_excel(r'All Data/list_city.xlsx')
        temp = order_all.copy()
        temp['City'] = temp['City'].astype(str).str.lower()
        temp['City'] = temp['City'].astype(str).str.replace('kab. ', 'kabupaten ', regex = False, case = False)
        city['All City'] = city['All City'].astype(str).str.lower()
        temp = temp.merge(city.drop_duplicates('All City'), how = 'left', left_on = 'City', right_on = 'All City').set_index(temp.index)
        indeks = temp[temp['Real City'].notnull()].index.to_list()
        order_all['City'][indeks] = temp['Real City'][indeks]

        province = pd.read_excel(r'All Data/list_province.xlsx')
        temp = order_all.copy()
        temp['Region'] = temp['Region'].astype(str).str.lower()
        province['All Province'] = province['All Province'].astype(str).str.lower()
        temp = temp.merge(province.drop_duplicates('All Province'), how = 'left', left_on = 'Region', right_on = 'All Province').set_index(temp.index)
        indeks = temp[temp['Real Province'].notnull()].index.to_list()
        order_all['Region'][indeks] = temp['Real Province'][indeks]

        temp = order_all.copy()
        temp = temp[temp['Region'].isnull()]
        temp['Region'] = temp.merge(master_map, how = 'left', on = 'City').set_index(temp.index)['Province']
        order_all['Region'][temp.index] = temp['Region']  

        district = pd.read_excel(r'All Data/list_district.xlsx')
        temp = order_all.copy()
        temp['Kecamatan'] = temp['Kecamatan'].astype(str).str.lower()
        district['All District'] = district['All District'].astype(str).str.lower()
        temp = temp.merge(district.drop_duplicates('All District'), how = 'left', left_on = 'Kecamatan', right_on = 'All District').set_index(temp.index)
        indeks = temp[temp['Real District'].notnull()].index.to_list()
        order_all['Kecamatan'][indeks] = temp['Real District'][indeks]

        temp = order_all.copy()
        temp2 = temp[['Region', 'City', 'Kecamatan']].merge(master_map, how = 'left', on = 'City')
        indeks = temp2[temp2['Region'] != temp2['Province']][temp2[temp2['Region'] != temp2['Province']]['City'].notnull()].index.to_list()
        order_all['City'][indeks] = np.nan

        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        print("Unbundling")
        data_bundle1 = order_all[~order_all['Produk 1'].isnull()]
        data_bundle1['Bundle Name'] = data_bundle1['Product Name']
        data_bundle1['Product Name'] = data_bundle1['Produk 1']
        data_bundle1['SKU'] = data_bundle1['SKU Produk 1']
        data_bundle1['Qty. Invoiced'] = data_bundle1['PCS Produk 1']
        data_bundle1['Price List NFI'] = data_bundle1['Price List NFI 1']
        data_bundle1['Total Net'] = data_bundle1['Price List NFI 1']
        data_bundle1['Bundle Flag'] = np.nan

        data_bundle2 = order_all[~order_all['Produk 2'].isnull()]
        data_bundle2['Bundle Name'] = data_bundle2['Product Name']
        data_bundle2['Product Name'] = data_bundle2['Produk 2']
        data_bundle2['SKU'] = data_bundle2['SKU Produk 2']
        data_bundle2['Qty. Invoiced'] = data_bundle2['PCS Produk 2']
        data_bundle2['Price List NFI'] = data_bundle2['Price List NFI 2']
        data_bundle2['Total Net'] = data_bundle2['Price List NFI 2'] 
        data_bundle2['Bundle Flag'] = np.nan

        data_bundle3 = order_all[~order_all['Produk 3'].isnull()]
        data_bundle3['Bundle Name'] = data_bundle3['Product Name']
        data_bundle3['Product Name'] = data_bundle3['Produk 3']
        data_bundle3['SKU'] = data_bundle3['SKU Produk 3']
        data_bundle3['Qty. Invoiced'] = data_bundle3['PCS Produk 3']
        data_bundle3['Price List NFI'] = data_bundle3['Price List NFI 3']
        data_bundle3['Total Net'] = data_bundle3['Price List NFI 3'] 
        data_bundle3['Bundle Flag'] = np.nan

        data_bundle4 = order_all[~order_all['Produk 4'].isnull()]
        data_bundle4['Bundle Name'] = data_bundle4['Product Name']
        data_bundle4['Product Name'] = data_bundle4['Produk 4']
        data_bundle4['SKU'] = data_bundle4['SKU Produk 4']
        data_bundle4['Qty. Invoiced'] = data_bundle4['PCS Produk 4']
        data_bundle4['Price List NFI'] = data_bundle4['Price List NFI 4']
        data_bundle4['Total Net'] = data_bundle4['Price List NFI 4'] 
        data_bundle4['Bundle Flag'] = np.nan

        data_bundle5 = order_all[~order_all['Produk 5'].isnull()]
        data_bundle5['Bundle Name'] = data_bundle5['Product Name']
        data_bundle5['Product Name'] = data_bundle5['Produk 5']
        data_bundle5['SKU'] = data_bundle5['SKU Produk 5']
        data_bundle5['Qty. Invoiced'] = data_bundle5['PCS Produk 5']
        data_bundle5['Price List NFI'] = data_bundle5['Price List NFI 5']
        data_bundle5['Total Net'] = data_bundle5['Price List NFI 5']
        data_bundle5['Bundle Flag'] = np.nan

        data_bundle6 = order_all[~order_all['Produk 6'].isnull()]
        data_bundle6['Bundle Name'] = data_bundle6['Product Name']
        data_bundle6['Product Name'] = data_bundle6['Produk 6']
        data_bundle6['SKU'] = data_bundle6['SKU Produk 6']
        data_bundle6['Qty. Invoiced'] = data_bundle6['PCS Produk 6']
        data_bundle6['Price List NFI'] = data_bundle6['Price List NFI 6']
        data_bundle6['Total Net'] = data_bundle6['Price List NFI 6']
        data_bundle6['Bundle Flag'] = np.nan

        data_bundle7 = order_all[~order_all['Produk 7'].isnull()]
        data_bundle7['Bundle Name'] = data_bundle7['Product Name']
        data_bundle7['Product Name'] = data_bundle7['Produk 7']
        data_bundle7['SKU'] = data_bundle7['SKU Produk 7']
        data_bundle7['Qty. Invoiced'] = data_bundle7['PCS Produk 7']
        data_bundle7['Price List NFI'] = data_bundle7['Price List NFI 7']
        data_bundle7['Total Net'] = data_bundle7['Price List NFI 7']
        data_bundle7['Bundle Flag'] = np.nan

        data_bundle = data_bundle1.append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)
        data_bundle['SKU'] = data_bundle['SKU'].astype(str)
        data_bundle['SKU'] = data_bundle['SKU'].str.replace('\.0$', '', regex = True)
        data_bundle[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']] = data_bundle.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item_y', 'Parent SKU_y']]

        temp = data_bundle[data_bundle['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)

        indeks = data_bundle[data_bundle['Real SKU'].isnull()].index.to_list()
        data_bundle['Real SKU'][indeks] = temp['Real SKU_y'][indeks]
        data_bundle['Real Nama Produk'][indeks] = temp['Nama Produk'][indeks]
        data_bundle['Brand'][indeks] = temp['Brand_y'][indeks]
        data_bundle['Sub Brand'][indeks] = temp['Sub Brand_y'][indeks]
        data_bundle['Parent Item'][indeks] = temp['Parent Item_y'][indeks]
        data_bundle['Parent SKU'][indeks] = temp['Parent SKU_y'][indeks]

        print("Pricing")
        order_all = order_all.append(data_bundle, ignore_index = True, sort = False)

        colname = temp.columns[temp.columns.get_loc('Produk 1') : temp.columns.get_loc('Harga Cost 7') + 1]
        colname_str = [x for x in colname if 'Subtotal' not in x and 'Harga' not in x]
        colname_int = [x for x in colname if x not in colname_str]

        for i in colname_str:
            temp[i] = np.nan

        for i in colname_int:
            temp[i] = 0

        data_order = data_order.append(temp , ignore_index = True, sort = False)


        order_all = order_all.merge(data_SKU2[['SKU', 'Price List NFI', 'Harga Cost']].drop_duplicates('SKU'), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(order_all.index)
        order_all['Price List NFI_x'] = order_all['Price List NFI_x'].fillna(order_all['Price List NFI_y'])
        order_all =  order_all.drop(['Price List NFI_y', 'SKU_y'], axis = 1)
        order_all = order_all.rename(columns = {'SKU_x' : 'SKU', 'Price List NFI_x' : 'Price List NFI'})

        indeks = order_all[order_all['Product Name'] == 'HiLo Teen Chocolate 250gr'].index.to_list()
        order_all['Real Nama Produk'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Parent Item'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Brand'][indeks] = 'HiLo'
        order_all['Sub Brand'][indeks] = 'HILO TEEN'
        order_all['Price List NFI'][indeks] = 36850
        order_all['Harga Cost'][indeks] = 36850
        order_all['Real SKU'][indeks] = '2101651155'
        order_all['Parent SKU'][indeks] = '2101651155'


        order_all['Price List NFI'] = pd.to_numeric(order_all['Price List NFI']).astype(int)
        order_all['Harga Cost'] = pd.to_numeric(order_all['Harga Cost']).astype(int)
        order_all['Qty. Invoiced'] = pd.to_numeric(order_all['Qty. Invoiced']).astype(int)

        order_all['Total Net'] = order_all['Price List NFI'] * order_all['Qty. Invoiced']
        order_all['Total Harga Cost'] = order_all['Harga Cost'] * order_all['Qty. Invoiced']
        order_all['Subtotal'] = order_all['Selling Price'] * order_all['Qty. Invoiced']
        order_all['Total'] = order_all['Selling Price'] * order_all['Qty. Invoiced']

        order_all = order_all.reset_index(drop = True)
        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        order_all['Seller Discount'] = order_all['discount']
        order_all['Shipping'] = order_all['Shipping Cost']

        temp = order_all[order_all['Brand'] != 'Bundle']
        temp['discount'] = temp['discount'] * temp['Total Harga Cost']/temp.groupby(['Order #'])['Total Harga Cost'].transform('sum')
        order_all['discount'][temp.index] = temp['discount']

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Subtotal', 'Total']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Total'][list_nobundle.index] = list_nobundle['Total_y']
        order_all['Subtotal'][list_nobundle.index] = list_nobundle['Subtotal_y']

        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Subtotal'] = temp['Subtotal'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        temp['Selling Price'] = temp['Subtotal']/temp['Qty. Invoiced']
        temp['Total'] = temp['Total'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Total'][temp.index] = temp['Total']
        order_all['Subtotal'][temp.index] = temp['Subtotal']
        order_all['Selling Price'][temp.index] = temp['Selling Price']


        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Seller Discount']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Seller Discount'][list_nobundle.index] = list_nobundle['Seller Discount_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Seller Discount'] = temp['Seller Discount'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Seller Discount'][temp.index] = temp['Seller Discount']


        temp = order_all[order_all['Bundle Name'].isnull()]
        temp_group = temp[['Order #','Shipping']].groupby(['Order #']).sum().reset_index()

        temp = order_all.merge(temp_group, how = 'left', on = 'Order #').set_index(order_all.index)
        temp['Shipping_x'] = temp['Shipping_y'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Shipping'][temp.index] = temp['Shipping_y']
        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Shipping']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Shipping'][list_nobundle.index] = list_nobundle['Shipping_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Shipping'] = temp['Shipping'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Shipping'][temp.index] = temp['Shipping']
        order_all['True datetime'] = pd.to_datetime(order_all['True datetime'])
        order_all['Promo'] = np.nan
        order_all['Discount MC'] = np.nan


        order_all['Warehouse Name'] = 'Primary Warehouse'
        order_all['Store'] = 'Order Online'

        order_all['Customer Email'] = order_all['email']
        order_all['Order Status'] = order_all['status']
        order_all['Payment Channel'] = order_all['payment_method']
        order_all['Coupon Code'] = order_all['coupon']
        order_all.columns.to_list()

        order_all_append = order_all[['Sales Order ID', 'Store',
            'Product Name',
            'Customer Name',
            'Phone',
            'Address',
            'Region',
            'City',
            'Zip Code',
            'payment_status',
            'Regular Price',
            'Shipping Courier',
            'Shipping Cost',
            'Subtotal',
            'Qty. Invoiced',
            'SKU',
            'Order #',
            'Invoice Number',
            'Shipping Name',
            'Shipping Address2',
            'Country',
            'AWB',
            'Channel',
            'Order date',
            'Real SKU',
            'Real Nama Produk',
            'Brand',
            'Sub Brand',
            'Parent Item',
            'Parent SKU',
            'Produk 1',
            'SKU Produk 1',
            'PCS Produk 1',
            'Price List NFI 1',
            'Subtotal Produk 1',
            'Harga Display 1',
            'Harga Cost 1',
            'Harga Organik 1',
            'Produk 2',
            'SKU Produk 2',
            'PCS Produk 2',
            'Price List NFI 2',
            'Subtotal Produk 2',
            'Harga Display 2',
            'Harga Cost 2',
            'Harga Organik 2',
            'Produk 3',
            'SKU Produk 3',
            'PCS Produk 3',
            'Price List NFI 3',
            'Subtotal Produk 3',
            'Harga Display 3',
            'Harga Cost 3',
            'Harga Organik 3',
            'Produk 4',
            'SKU Produk 4',
            'PCS Produk 4',
            'Price List NFI 4',
            'Subtotal Produk 4',
            'Harga Display 4',
            'Harga Cost 4',
            'Harga Organik 4',
            'Produk 5',
            'SKU Produk 5',
            'PCS Produk 5',
            'Price List NFI 5',
            'Subtotal Produk 5',
            'Harga Display 5',
            'Harga Cost 5',
            'Harga Organik 5',
            'Produk 6',
            'SKU Produk 6',
            'PCS Produk 6',
            'Price List NFI 6',
            'Subtotal Produk 6',
            'Harga Display 6',
            'Harga Cost 6',
            'Harga Organik 6',
            'Produk 7',
            'SKU Produk 7',
            'PCS Produk 7',
            'Price List NFI 7',
            'Subtotal Produk 7',
            'Harga Display 7',
            'Harga Cost 7',
            'Harga Organik 7',
            'Bundle Flag',
            'Date',
            'Month',
            'Year',
            'Quarter',
            'Week',
            'True datetime',
            'Total',
            'Price List NFI',
            'Total Net',
            'Selling Price',
            'Kecamatan',
            'Kelurahan',
            'Bundle Name',
            'Harga Cost',
            'Total Harga Cost',
            'Seller Discount',
            'Shipping',
            'Promo',
            'Discount MC',
            'Warehouse Name',
            'Customer Email',
            'Order Status',
            'Payment Channel',
            'Coupon Code']]

        data_all = data_all[~data_all['Order #'].astype(str).isin(order_all_append['Order #'].astype(str))]
        data_all = data_all.append(order_all_append, ignore_index = True, sort = False)

        order_online_bali = True
        del file_name

if not order_online_makasar :
                    
    import pandas as pd
    import numpy as np
    import requests
    import os

    print('order_online_makasar')
    
    before = os.listdir(os.getcwd() + '/Input Data')

    options = Options()
    options.add_experimental_option("prefs", {
            "download.default_directory": os.path.abspath("Input Data"),
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            "safebrowsing.enabled": False
    })

    driver = webdriver.Chrome(options=options)
    driver.fullscreen_window()
    driver.get("https://app.orderonline.id")

    username = driver.find_element_by_name("email")
    username.clear()
    username.send_keys("customermakassar@nutrimart.co.id")

    password = driver.find_element_by_name("password")
    password.send_keys("makassarhomdel")
    password.click()

    driver.find_element_by_class_name("btn-submit").click()

    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="main-nav-dropdown"]/ul/li[3]/a'))).click()
    time.sleep(5)
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[1]/div/div[2]/div/div/div').click()
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="app"]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[1]/div[1]/ul/li[6]'))).click()
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[2]/button[2]').click()
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[3]/div/button').click()
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[3]/div/div/button[1]').click()

    time.sleep(20)

    after = os.listdir(os.getcwd() + '/Input Data')
    change = set(after) - set(before)
    if len(change) == 1:
        file_name = change.pop()
    elif len(change) == 0: 
        print("No file downloaded")
    else :
        print("More than one file downloaded")

    data_order = pd.read_csv(r'Input Data/' + str(file_name))
    driver.close()
#         data_order = pd.read_csv(r'Input Data/orderonline_orders_all_products_Jakarta.csv')
#             product_order = pd.read_csv(r'Input Data/orderonline_products_Jakarta.csv')

    data_order['order_id'] = data_order['order_id'].fillna(method='ffill')
    data_order = data_order.drop('quantity', axis = 1)
    temp = data_order.drop_duplicates('order_id').drop('product', axis = 1)
    data_order = data_order[['order_id', 'product']]
    data_order = data_order.merge(temp, how = 'left', on = 'order_id')
    data_order['order_id'] = data_order['order_id'].astype(int)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Shirataki Noodles 71gr (x2) ', 'Twin Pack: Tropicana Slim Shirataki Noodles 71gr ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Saus Tiram 200ml (x2) ', 'Twin Pack: Tropicana Slim Saus Tiram 200ml ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Sweetener Rose Vanilla 50 sch (x2) ', 'Twin Pack: Tropicana Slim Sweetener Rose Vanilla 50 sch ', regex = False)

    product = data_order['product'].str.split("\(x",1, expand = True)
    data_order['Quantity'] = product[1].astype(str).str.replace(')', '', regex = False).astype(int)
    data_order['product'] = product[0].str.strip().str.replace('  ', ' ').str.replace('6 SCH', '6sch').str.replace("W'Dank", "W'dank").str.replace('L-men', 'L-Men').str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("school", "School").str.replace("Rtd", "RTD").str.replace("tetra", "Tetra").str.replace("[6pcs]", "(6pcs)", regex = False)
    data_order['product'] = data_order['product'].str.replace("HiLo Thai Tea \(10sch\)$", 'HiLo Thai Tea (10 sch)', regex = True)
    data_order['product'] = data_order['product'].str.replace("Tropicana Slim Goldenmil Vanilla Turmeric 180gr", 'Tropicana Slim Goldenmil Vanilla Turmeric (6 sch)')
    data_order['product'] = data_order['product'].str.replace("NutriSari Mango Smoothie Ready to Drink Jus 200ml (6pcs)", 'NutriSari Mango Smoothie 200ml (6pcs)', regex = False)
    
    data_SKU = pd.read_excel(r'Order Online\SKU buat andra updated maret 2021.xlsx')
    data_SKU = data_SKU.rename(columns = {'Product' : 'product', 'PL NFI' : 'Price List NFI'})
    data_SKU['SKU'] = data_SKU['SKU'].astype(str).str.replace('.0', '', regex = False)
    data_SKU = data_SKU[data_SKU['SKU'] != 'nan'][data_SKU[data_SKU['SKU'] != 'nan']['SKU'] != '0']
    data_SKU['product'] = data_SKU['product'].str.strip().str.replace('L-men', 'L-Men').str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("school", "School").str.replace("Rtd", "RTD").str.replace("tetra", "Tetra").str.replace("[6pcs]", "(6pcs)", regex = False)


    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Kecap Manis 200ml'].copy()
    price['product'] = 'Tropicana Slim Kecap Manis 200m'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Diabtx (50 sch)'].copy()
    price['product'] = 'Tropicana Slim Diabtx 50 sch'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Hokkaido Cheese 100gr'].copy()
    price['product'] = 'Tropicana Slim Hokaido Cheese 100gr'


    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
   
    data_SKU = data_SKU.append(pd.DataFrame([['L-Men Bar Crunchy Chocolate 12sch', 5500, 5500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
 
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Mangga Gandaria', 45000, 45000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
 
    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Teen Vanilla Caramel 500gr', 71500, 71500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Paket Bundle HiLo Renceng (Hilo Chocolate Banana (10 sch) + Hilo Chocolate Taro (10 sch) + HiLo Thai Tea (10sch))', 35200, 35200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Berondong 10's", 15000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Kecap Asin 200 ml", 28500, 26000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim DIABTX (100 sch)", 87700, 75000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Teen Chocolate 750gr", 117800, 104000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Paket Ngopi Lokalate", 136000, 109200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Hi Protein 2 Go Chocolate (24 TETRAPAK)", 240000, 238000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 Tropicana Slim Goldenmil Vanilla (6sch)", 39160, 31000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Kawista", 17500, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["Paket Bundle HiLo (HiLo Active Chocolate 500gr + HiLo Teen Yoghurt Banana 250gr)", 122900, 101850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Strawberry Jam 375gr", 72600, 58500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Crunch BBQ Beef (20gr)", 13500, 10500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Cocopandan 40 sch", 52500, 52500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 - W'dank Empon Empon 10 Sachet Renceng", 35000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Semangka 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Nanas 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["W'Dank Empon-Empon 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Milk Skim Fiber Pro Plain 500gr", 135000, 106700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Es Teler 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: HiLo Es Teler 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Hilo Es Ketan Hitam 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Triple Pack: Tropicana Slim Korean Garlic Butter Cookies (5 Sch)", 73500, 52000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Avocado Coffee 4 Sch", 21200, 12100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Sambal Terasi 200 gr", 35600, 29700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Tropicana Slim Avocado Coffee 4 sch x 2", 42400, 24200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Bar Chocolate (12 Sch) + L-Men Protein Crunch BBQ Beef x 2", 159000, 107800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Buy 5 Get 5 L-Men Protein Crunch BBQ Beef (20gr)", 130000, 67500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Active Ketan Hitam 175gr', 48500, 20700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Es Ketan Hitam 10 sch', 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Sweetener Lemongrass Pandan 50 sch', 44200, 28900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 1 Get 1 FREE: Lokalate Kopi Berondong (10 sch)', 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['Buy 12 Get 12 - HiLo Chocolate Avocado Ready to Drink 200ml - Minuman Siap Minum Tinggi Kalsium', 152400, 84000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_order['product'] = data_order['product'].astype(str)
    data_SKU['product'] = data_SKU['product'].astype(str)

    data_order['product'] = data_order['product'].str.replace('L Men Gain Mass Chocolate 500 gr', 'L-Men Gain Mass Chocolate 500 gr')



    data_order = data_order.merge(data_SKU[['SKU', 'product', 'Harga Display', 'Harga Coret']].drop_duplicates('product'), how = 'left', on = 'product')

    data_order = data_order.reset_index(drop = True)

    indeks = data_order[data_order['product'] == "Lokalate Kopi Berondong 10's"].index.to_list()
    data_order['Harga Display'][indeks] = 15000
    data_order['Harga Coret'][indeks] = 15000
    indeks = data_order[data_order['SKU'].isnull()].index.to_list()
    for i in indeks:
        col = [x for x in data_SKU.columns if 'Alias Nama' in x]
        for j in col:
            if data_order['product'][i] in data_SKU[j].astype(str).values:
                SKU = data_SKU[data_SKU[j].astype(str) == data_order['product'][i]]['SKU'].values[0]
                data_order['SKU'][i] = SKU

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    data_SKU2 = pd.read_excel(r'SKU_File\data_SKU.xlsx')
    data_SKU2['Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

    for i in indeks:
        if str(data_order['product'][i]).lower() in data_SKU2['Nama Produk'].astype(str).str.lower().values:
            data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2['Nama Produk'].astype(str).str.lower()].values[0]

    list_alias_name = [x for x in data_SKU2.columns if 'Alias Nama' in x]

    for i in indeks:
        for j in list_alias_name:
            if str(data_order['product'][i]).lower() in data_SKU2[j].astype(str).str.lower().values:
                data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2[j].astype(str).str.lower()].values[0]

    
    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    if len(indeks) != 0:
        print('Alert SKU Missing')
        data_order['product'][indeks].drop_duplicates().to_excel('Alert SKU Missing.xlsx', index = False)
    else :
        data_order['phone'] = data_order['phone'].astype(str).str.replace('+628', '08', regex = False)

        data_order['zip'] = data_order['zip'].replace('.0', '', regex = False)

        data_order = data_order.rename(columns = {'order_id' : 'Sales Order ID', 'name' : 'Customer Name', 'product' : 'Item Name', 'price' : 'Price', 'shipping_cost' : 'Shipping Cost', 'address' : 'Shipping Address1', 'city' : 'Shipping City', 'zip' : 'Shipping Zip', 'province' : 'Shipping Province', 'phone' : 'Shipping Phone', 'courier' : 'Shipping Courier'})
        data_order['Channel Order ID'] = data_order['Sales Order ID']
        data_order['Invoice Number'] = data_order['Sales Order ID']
        data_order['Shipping Name'] = data_order['Customer Name']
        data_order['Shipping Address2'] = 0
        data_order['Shipping Country'] = 'Indonesia'
        data_order['AWB'] = 0
        data_order['Channel'] = 'Order Online Makassar'


        data_order['Order Date'] = pd.to_datetime(data_order['created_at'])


        data_order['SKU'] = data_order['SKU'].astype(str)
        data_order['Item Name'] = data_order['Item Name'].astype(str)
        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str).str.replace('(S)', '', regex = False)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        index = data_order[data_order['SKU'].astype(str) == '2306551174'].index.to_list()
        data_order['SKU'][index] = '2306592173'

        data_order = data_order.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('hd','', regex = False)
        temp['SKU'] = temp['SKU'].astype(str).str.replace('HD','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        data_order['Real SKU'] = data_order['Real SKU'].astype(str)
        data_order = data_order.merge(data_SKU2[['SKU', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Unbundling ====== 6/10")        
        # Forstok Unbundling    
        list_col = ['SKU'] + data_SKU2.columns[data_SKU2.columns.get_loc('Produk 1'):data_SKU2.columns.get_loc('Harga Organik 7')+1].to_list()
        data_order = data_order.merge(data_SKU2[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        list_pcs = [x for x in data_order.columns if 'PCS' in x]
        for i in list_pcs:
            data_order[i] = data_order[i] * data_order['Quantity']
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        indeks = data_order[data_order['Brand'] == 'Bundle'].index.to_list()
        data_order['Bundle Flag'] = np.nan
        data_order['Bundle Flag'][indeks] = 'Bundle'

        indeks = data_order[data_order['Brand'] == 'Bundle'][data_order[data_order['Brand'] == 'Bundle']['SKU'].astype(str).str.contains('(S)', regex = False)].index.to_list()
        data_order['SKU Produk 1'][indeks] = '(S)' + data_order['SKU Produk 1'][indeks].astype(str)
        data_order['SKU Produk 2'][indeks] = '(S)' + data_order['SKU Produk 2'][indeks].astype(str)
        data_order['SKU Produk 3'][indeks] = '(S)' + data_order['SKU Produk 3'][indeks].astype(str)
        data_order['SKU Produk 4'][indeks] = '(S)' + data_order['SKU Produk 4'][indeks].astype(str)
        data_order['SKU Produk 5'][indeks] = '(S)' + data_order['SKU Produk 5'][indeks].astype(str)
        data_order['SKU Produk 6'][indeks] = '(S)' + data_order['SKU Produk 6'][indeks].astype(str)
        data_order['SKU Produk 7'][indeks] = '(S)' + data_order['SKU Produk 7'][indeks].astype(str)

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Filling Date ====== 7/10")
        data_order['Date'] = np.nan
        data_order['Month'] = np.nan
        data_order['Year'] = np.nan

        for i in range(data_order.shape[0]):
            if int(data_order['Order Date'][i].strftime('%d')) <= 12:
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).year
            else :
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i]).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i]).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i]).year

        quarter = pd.DataFrame([['January', 1], ['February', 1], ['March', 1], ['April', 2], ['May', 2], ['June', 2], 
                ['July', 3], ['August', 3], ['September', 3],['October', 4], ['November', 4], ['December', 4]], columns = ['Bulan', 'Quarter'])
        data_order = data_order.merge(quarter, how = 'left', left_on = 'Month', right_on = 'Bulan')
        data_order = data_order.drop(['Bulan'], axis = 1)
        data_bulan = pd.DataFrame([{'Bulan' : 'December', 'Number' : 12} ,
                {'Bulan' : 'January' , 'Number': 1},
                {'Bulan' : 'February' , 'Number': 2},
                {'Bulan' : 'March' , 'Number': 3},
                {'Bulan' : 'April' , 'Number': 4},
                {'Bulan' : 'May' , 'Number': 5},
                {'Bulan' : 'June', 'Number': 6},
                {'Bulan' : 'July' , 'Number': 7},
                {'Bulan' : 'August', 'Number' : 8},
                {'Bulan' : 'September', 'Number' : 9},
                {'Bulan' : 'October' , 'Number': 10},
                {'Bulan' : 'November' , 'Number': 11}])
        temp = data_order.copy()
        temp['Day'] = temp['Date']
        temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
        temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
        data_order['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.week
        temp['Hour'] = pd.to_datetime(data_order['Order Date']).dt.hour
        temp['Minute'] = pd.to_datetime(data_order['Order Date']).dt.minute
        temp['Second'] = pd.to_datetime(data_order['Order Date']).dt.second
        data_order['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']])



        order_all = data_order.copy()
        order_all['Total'] = order_all['net_revenue']
        order_all['Price List NFI'] = np.nan
        order_all['Total Net'] = np.nan



        order_all = order_all.rename(columns={'Channel Order ID' : 'Order #',
                                                'Status' : 'Order Status',
                                                'Order Date' : 'Order date',
                                                'Item Name' :'Product Name',
                                                'Bundle Name' : 'Bundle',
                                                'Shipping Country' : 'Country',
                                                'Shipping Province' : 'Region',
                                                'Shipping City' : 'City',
                                                'Shipping Zip' : 'Zip Code',
                                                'Shipping Address1' : 'Address',
                                                'Shipping Phone' : 'Phone',
                                                'Quantity' : 'Qty. Invoiced',
                                                'Harga Display' : 'Regular Price',
                                                'net_revenue' : 'Subtotal'})

        

        order_all['Selling Price'] = order_all['Harga Coret'].astype(int)
        order_all['Kecamatan'] = np.nan
        order_all['Kelurahan'] = np.nan

        print("Filling Location")
        indeks = order_all[order_all['City'].astype(str).str.contains('/')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains('-')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[0]

        indeks = order_all[order_all['City'].astype(str).str.contains(',')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains(',')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[0]

        order_all['City'] = order_all['City'].astype(str).str.replace('Kab\.', 'Kabupaten' ,case = False)

        master_map = pd.read_csv(r'All Data/Province.csv', names = ['Kode Prov', 'Province'], header= 0)
        master_map2 = pd.read_csv(r'All Data/City.csv', names = ['Kode City', 'Kode Prov', 'City'], header = 0)
        master_map = master_map.merge(master_map2, how = 'right', on = 'Kode Prov')
        master_map['Kode Prov'][515] = 14
        master_map['Province'][515] = 'Riau'
        master_map['Kode Prov'] = master_map['Kode Prov'].astype(int)
        master_map['Province'] = master_map['Province'].str.title()
        master_map['City'] = master_map['City'].str.title()

        city = pd.read_excel(r'All Data/list_city.xlsx')
        temp = order_all.copy()
        temp['City'] = temp['City'].astype(str).str.lower()
        temp['City'] = temp['City'].astype(str).str.replace('kab. ', 'kabupaten ', regex = False, case = False)
        city['All City'] = city['All City'].astype(str).str.lower()
        temp = temp.merge(city.drop_duplicates('All City'), how = 'left', left_on = 'City', right_on = 'All City').set_index(temp.index)
        indeks = temp[temp['Real City'].notnull()].index.to_list()
        order_all['City'][indeks] = temp['Real City'][indeks]

        province = pd.read_excel(r'All Data/list_province.xlsx')
        temp = order_all.copy()
        temp['Region'] = temp['Region'].astype(str).str.lower()
        province['All Province'] = province['All Province'].astype(str).str.lower()
        temp = temp.merge(province.drop_duplicates('All Province'), how = 'left', left_on = 'Region', right_on = 'All Province').set_index(temp.index)
        indeks = temp[temp['Real Province'].notnull()].index.to_list()
        order_all['Region'][indeks] = temp['Real Province'][indeks]

        temp = order_all.copy()
        temp = temp[temp['Region'].isnull()]
        temp['Region'] = temp.merge(master_map, how = 'left', on = 'City').set_index(temp.index)['Province']
        order_all['Region'][temp.index] = temp['Region']  

        district = pd.read_excel(r'All Data/list_district.xlsx')
        temp = order_all.copy()
        temp['Kecamatan'] = temp['Kecamatan'].astype(str).str.lower()
        district['All District'] = district['All District'].astype(str).str.lower()
        temp = temp.merge(district.drop_duplicates('All District'), how = 'left', left_on = 'Kecamatan', right_on = 'All District').set_index(temp.index)
        indeks = temp[temp['Real District'].notnull()].index.to_list()
        order_all['Kecamatan'][indeks] = temp['Real District'][indeks]

        temp = order_all.copy()
        temp2 = temp[['Region', 'City', 'Kecamatan']].merge(master_map, how = 'left', on = 'City')
        indeks = temp2[temp2['Region'] != temp2['Province']][temp2[temp2['Region'] != temp2['Province']]['City'].notnull()].index.to_list()
        order_all['City'][indeks] = np.nan

        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        print("Unbundling")
        data_bundle1 = order_all[~order_all['Produk 1'].isnull()]
        data_bundle1['Bundle Name'] = data_bundle1['Product Name']
        data_bundle1['Product Name'] = data_bundle1['Produk 1']
        data_bundle1['SKU'] = data_bundle1['SKU Produk 1']
        data_bundle1['Qty. Invoiced'] = data_bundle1['PCS Produk 1']
        data_bundle1['Price List NFI'] = data_bundle1['Price List NFI 1']
        data_bundle1['Total Net'] = data_bundle1['Price List NFI 1']
        data_bundle1['Bundle Flag'] = np.nan

        data_bundle2 = order_all[~order_all['Produk 2'].isnull()]
        data_bundle2['Bundle Name'] = data_bundle2['Product Name']
        data_bundle2['Product Name'] = data_bundle2['Produk 2']
        data_bundle2['SKU'] = data_bundle2['SKU Produk 2']
        data_bundle2['Qty. Invoiced'] = data_bundle2['PCS Produk 2']
        data_bundle2['Price List NFI'] = data_bundle2['Price List NFI 2']
        data_bundle2['Total Net'] = data_bundle2['Price List NFI 2'] 
        data_bundle2['Bundle Flag'] = np.nan

        data_bundle3 = order_all[~order_all['Produk 3'].isnull()]
        data_bundle3['Bundle Name'] = data_bundle3['Product Name']
        data_bundle3['Product Name'] = data_bundle3['Produk 3']
        data_bundle3['SKU'] = data_bundle3['SKU Produk 3']
        data_bundle3['Qty. Invoiced'] = data_bundle3['PCS Produk 3']
        data_bundle3['Price List NFI'] = data_bundle3['Price List NFI 3']
        data_bundle3['Total Net'] = data_bundle3['Price List NFI 3'] 
        data_bundle3['Bundle Flag'] = np.nan

        data_bundle4 = order_all[~order_all['Produk 4'].isnull()]
        data_bundle4['Bundle Name'] = data_bundle4['Product Name']
        data_bundle4['Product Name'] = data_bundle4['Produk 4']
        data_bundle4['SKU'] = data_bundle4['SKU Produk 4']
        data_bundle4['Qty. Invoiced'] = data_bundle4['PCS Produk 4']
        data_bundle4['Price List NFI'] = data_bundle4['Price List NFI 4']
        data_bundle4['Total Net'] = data_bundle4['Price List NFI 4'] 
        data_bundle4['Bundle Flag'] = np.nan

        data_bundle5 = order_all[~order_all['Produk 5'].isnull()]
        data_bundle5['Bundle Name'] = data_bundle5['Product Name']
        data_bundle5['Product Name'] = data_bundle5['Produk 5']
        data_bundle5['SKU'] = data_bundle5['SKU Produk 5']
        data_bundle5['Qty. Invoiced'] = data_bundle5['PCS Produk 5']
        data_bundle5['Price List NFI'] = data_bundle5['Price List NFI 5']
        data_bundle5['Total Net'] = data_bundle5['Price List NFI 5']
        data_bundle5['Bundle Flag'] = np.nan

        data_bundle6 = order_all[~order_all['Produk 6'].isnull()]
        data_bundle6['Bundle Name'] = data_bundle6['Product Name']
        data_bundle6['Product Name'] = data_bundle6['Produk 6']
        data_bundle6['SKU'] = data_bundle6['SKU Produk 6']
        data_bundle6['Qty. Invoiced'] = data_bundle6['PCS Produk 6']
        data_bundle6['Price List NFI'] = data_bundle6['Price List NFI 6']
        data_bundle6['Total Net'] = data_bundle6['Price List NFI 6']
        data_bundle6['Bundle Flag'] = np.nan

        data_bundle7 = order_all[~order_all['Produk 7'].isnull()]
        data_bundle7['Bundle Name'] = data_bundle7['Product Name']
        data_bundle7['Product Name'] = data_bundle7['Produk 7']
        data_bundle7['SKU'] = data_bundle7['SKU Produk 7']
        data_bundle7['Qty. Invoiced'] = data_bundle7['PCS Produk 7']
        data_bundle7['Price List NFI'] = data_bundle7['Price List NFI 7']
        data_bundle7['Total Net'] = data_bundle7['Price List NFI 7']
        data_bundle7['Bundle Flag'] = np.nan

        data_bundle = data_bundle1.append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)
        data_bundle['SKU'] = data_bundle['SKU'].astype(str)
        data_bundle['SKU'] = data_bundle['SKU'].str.replace('\.0$', '', regex = True)
        data_bundle[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']] = data_bundle.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item_y', 'Parent SKU_y']]

        temp = data_bundle[data_bundle['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)

        indeks = data_bundle[data_bundle['Real SKU'].isnull()].index.to_list()
        data_bundle['Real SKU'][indeks] = temp['Real SKU_y'][indeks]
        data_bundle['Real Nama Produk'][indeks] = temp['Nama Produk'][indeks]
        data_bundle['Brand'][indeks] = temp['Brand_y'][indeks]
        data_bundle['Sub Brand'][indeks] = temp['Sub Brand_y'][indeks]
        data_bundle['Parent Item'][indeks] = temp['Parent Item_y'][indeks]
        data_bundle['Parent SKU'][indeks] = temp['Parent SKU_y'][indeks]

        print("Pricing")
        order_all = order_all.append(data_bundle, ignore_index = True, sort = False)

        colname = temp.columns[temp.columns.get_loc('Produk 1') : temp.columns.get_loc('Harga Cost 7') + 1]
        colname_str = [x for x in colname if 'Subtotal' not in x and 'Harga' not in x]
        colname_int = [x for x in colname if x not in colname_str]

        for i in colname_str:
            temp[i] = np.nan

        for i in colname_int:
            temp[i] = 0

        data_order = data_order.append(temp , ignore_index = True, sort = False)


        order_all = order_all.merge(data_SKU2[['SKU', 'Price List NFI', 'Harga Cost']].drop_duplicates('SKU'), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(order_all.index)
        order_all['Price List NFI_x'] = order_all['Price List NFI_x'].fillna(order_all['Price List NFI_y'])
        order_all =  order_all.drop(['Price List NFI_y', 'SKU_y'], axis = 1)
        order_all = order_all.rename(columns = {'SKU_x' : 'SKU', 'Price List NFI_x' : 'Price List NFI'})

        indeks = order_all[order_all['Product Name'] == 'HiLo Teen Chocolate 250gr'].index.to_list()
        order_all['Real Nama Produk'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Parent Item'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Brand'][indeks] = 'HiLo'
        order_all['Sub Brand'][indeks] = 'HILO TEEN'
        order_all['Price List NFI'][indeks] = 36850
        order_all['Harga Cost'][indeks] = 36850
        order_all['Real SKU'][indeks] = '2101651155'
        order_all['Parent SKU'][indeks] = '2101651155'


        order_all['Price List NFI'] = pd.to_numeric(order_all['Price List NFI']).astype(int)
        order_all['Harga Cost'] = pd.to_numeric(order_all['Harga Cost'], errors = 'coerce').fillna(0).astype(int)
        order_all['Qty. Invoiced'] = pd.to_numeric(order_all['Qty. Invoiced']).astype(int)

        order_all['Total Net'] = order_all['Price List NFI'] * order_all['Qty. Invoiced']
        order_all['Total Harga Cost'] = order_all['Harga Cost'] * order_all['Qty. Invoiced']
        order_all['Subtotal'] = order_all['Selling Price'] * order_all['Qty. Invoiced']
        order_all['Total'] = order_all['Selling Price'] * order_all['Qty. Invoiced']

        order_all = order_all.reset_index(drop = True)
        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        order_all['Seller Discount'] = order_all['discount']
        order_all['Shipping'] = order_all['Shipping Cost']

        temp = order_all[order_all['Brand'] != 'Bundle']
        temp['discount'] = temp['discount'] * temp['Total Harga Cost']/temp.groupby(['Order #'])['Total Harga Cost'].transform('sum')
        order_all['discount'][temp.index] = temp['discount']

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Subtotal', 'Total']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Total'][list_nobundle.index] = list_nobundle['Total_y']
        order_all['Subtotal'][list_nobundle.index] = list_nobundle['Subtotal_y']

        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Subtotal'] = temp['Subtotal'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        temp['Selling Price'] = temp['Subtotal']/temp['Qty. Invoiced']
        temp['Total'] = temp['Total'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Total'][temp.index] = temp['Total']
        order_all['Subtotal'][temp.index] = temp['Subtotal']
        order_all['Selling Price'][temp.index] = temp['Selling Price']


        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Seller Discount']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Seller Discount'][list_nobundle.index] = list_nobundle['Seller Discount_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Seller Discount'] = temp['Seller Discount'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Seller Discount'][temp.index] = temp['Seller Discount']


        temp = order_all[order_all['Bundle Name'].isnull()]
        temp_group = temp[['Order #','Shipping']].groupby(['Order #']).sum().reset_index()

        temp = order_all.merge(temp_group, how = 'left', on = 'Order #').set_index(order_all.index)
        temp['Shipping_x'] = temp['Shipping_y'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Shipping'][temp.index] = temp['Shipping_y']
        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Shipping']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Shipping'][list_nobundle.index] = list_nobundle['Shipping_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Shipping'] = temp['Shipping'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Shipping'][temp.index] = temp['Shipping']
        order_all['True datetime'] = pd.to_datetime(order_all['True datetime'])
        order_all['Promo'] = np.nan
        order_all['Discount MC'] = np.nan


        order_all['Warehouse Name'] = 'Primary Warehouse'
        order_all['Store'] = 'Order Online'

        order_all['Customer Email'] = order_all['email']
        order_all['Order Status'] = order_all['status']
        order_all['Payment Channel'] = order_all['payment_method']
        order_all['Coupon Code'] = order_all['coupon']
        order_all.columns.to_list()

        order_all_append = order_all[['Sales Order ID', 'Store',
            'Product Name',
            'Customer Name',
            'Phone',
            'Address',
            'Region',
            'City',
            'Zip Code',
            'payment_status',
            'Regular Price',
            'Shipping Courier',
            'Shipping Cost',
            'Subtotal',
            'Qty. Invoiced',
            'SKU',
            'Order #',
            'Invoice Number',
            'Shipping Name',
            'Shipping Address2',
            'Country',
            'AWB',
            'Channel',
            'Order date',
            'Real SKU',
            'Real Nama Produk',
            'Brand',
            'Sub Brand',
            'Parent Item',
            'Parent SKU',
            'Produk 1',
            'SKU Produk 1',
            'PCS Produk 1',
            'Price List NFI 1',
            'Subtotal Produk 1',
            'Harga Display 1',
            'Harga Cost 1',
            'Harga Organik 1',
            'Produk 2',
            'SKU Produk 2',
            'PCS Produk 2',
            'Price List NFI 2',
            'Subtotal Produk 2',
            'Harga Display 2',
            'Harga Cost 2',
            'Harga Organik 2',
            'Produk 3',
            'SKU Produk 3',
            'PCS Produk 3',
            'Price List NFI 3',
            'Subtotal Produk 3',
            'Harga Display 3',
            'Harga Cost 3',
            'Harga Organik 3',
            'Produk 4',
            'SKU Produk 4',
            'PCS Produk 4',
            'Price List NFI 4',
            'Subtotal Produk 4',
            'Harga Display 4',
            'Harga Cost 4',
            'Harga Organik 4',
            'Produk 5',
            'SKU Produk 5',
            'PCS Produk 5',
            'Price List NFI 5',
            'Subtotal Produk 5',
            'Harga Display 5',
            'Harga Cost 5',
            'Harga Organik 5',
            'Produk 6',
            'SKU Produk 6',
            'PCS Produk 6',
            'Price List NFI 6',
            'Subtotal Produk 6',
            'Harga Display 6',
            'Harga Cost 6',
            'Harga Organik 6',
            'Produk 7',
            'SKU Produk 7',
            'PCS Produk 7',
            'Price List NFI 7',
            'Subtotal Produk 7',
            'Harga Display 7',
            'Harga Cost 7',
            'Harga Organik 7',
            'Bundle Flag',
            'Date',
            'Month',
            'Year',
            'Quarter',
            'Week',
            'True datetime',
            'Total',
            'Price List NFI',
            'Total Net',
            'Selling Price',
            'Kecamatan',
            'Kelurahan',
            'Bundle Name',
            'Harga Cost',
            'Total Harga Cost',
            'Seller Discount',
            'Shipping',
            'Promo',
            'Discount MC',
            'Warehouse Name',
            'Customer Email',
            'Order Status',
            'Payment Channel',
            'Coupon Code']]

        data_all = data_all[~data_all['Order #'].astype(str).isin(order_all_append['Order #'].astype(str))]
        data_all = data_all.append(order_all_append, ignore_index = True, sort = False)

        order_online_makasar = True
        del file_name

if not order_online_medan:
                        
    import pandas as pd
    import numpy as np
    import requests
    import os
    
    print('order_online_medan')

    before = os.listdir(os.getcwd() + '/Input Data')

    options = Options()
    options.add_experimental_option("prefs", {
            "download.default_directory": os.path.abspath("Input Data"),
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            "safebrowsing.enabled": False
    })

    driver = webdriver.Chrome(options=options)
    driver.fullscreen_window()
    driver.get("https://app.orderonline.id")

    username = driver.find_element_by_name("email")
    username.clear()
    username.send_keys("customermedan@nutrimart.co.id")

    password = driver.find_element_by_name("password")
    password.send_keys("medanhomdel")
    password.click()

    driver.find_element_by_class_name("btn-submit").click()

    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="main-nav-dropdown"]/ul/li[3]/a'))).click()
    time.sleep(5)
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[1]/div/div[2]/div/div/div').click()
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="app"]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[1]/div[1]/ul/li[6]'))).click()
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[2]/button[2]').click()
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[3]/div/button').click()
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[3]/div/div/button[1]').click()

    time.sleep(10)

    after = os.listdir(os.getcwd() + '/Input Data')
    change = set(after) - set(before)
    if len(change) == 1:
        file_name = change.pop()
    elif len(change) == 0: 
        print("No file downloaded")
    else :
        print("More than one file downloaded")

    data_order = pd.read_csv(r'Input Data/' + str(file_name))
    driver.close()
#         data_order = pd.read_csv(r'Input Data/orderonline_orders_all_products_Jakarta.csv')
#             product_order = pd.read_csv(r'Input Data/orderonline_products_Jakarta.csv')

    data_order['order_id'] = data_order['order_id'].fillna(method='ffill')
    data_order = data_order.drop('quantity', axis = 1)
    temp = data_order.drop_duplicates('order_id').drop('product', axis = 1)
    data_order = data_order[['order_id', 'product']]
    data_order = data_order.merge(temp, how = 'left', on = 'order_id')
    data_order['order_id'] = data_order['order_id'].astype(int)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Shirataki Noodles 71gr (x2) ', 'Twin Pack: Tropicana Slim Shirataki Noodles 71gr ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Saus Tiram 200ml (x2) ', 'Twin Pack: Tropicana Slim Saus Tiram 200ml ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Sweetener Rose Vanilla 50 sch (x2) ', 'Twin Pack: Tropicana Slim Sweetener Rose Vanilla 50 sch ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Tropicana Slim Oat Drink 190 ml \(RTD\) x 4 pcs$', 'Tropicana Slim Oat Drink 190 ml (RTD) x 4 pcs (x1)')
    
    product = data_order['product'].str.split("\(x",1, expand = True)
    data_order['Quantity'] = product[1].astype(str).str.replace(')', '', regex = False).astype(int)
    data_order['product'] = product[0].str.strip().str.replace('  ', ' ').str.replace('6 SCH', '6sch').str.replace("W'Dank", "W'dank").str.replace('L-men', 'L-Men', case = False).str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("school", "School").str.replace("Rtd", "RTD").str.replace("tetra", "Tetra").str.replace("[6pcs]", "(6pcs)", regex = False)
    data_order['product'] = data_order['product'].str.replace("HiLo Thai Tea \(10sch\)$", 'HiLo Thai Tea (10 sch)', regex = True)
    data_order['product'] = data_order['product'].str.replace("NutriSari Jeruk Nipis 40'sachet (4 Renceng)", 'NutriSari Jeruk Nipis (40 sch)', regex = False)
    data_order['product'] = data_order['product'].str.replace("Tropicana Slim Sweetener Honey 50 sachet", 'Tropicana Slim Sweetener Honey (50sch)', regex = False)
    data_order['product'] = data_order['product'].str.replace("HiLo Teen Coklat 250gr", 'HiLo Teen Chocolate 250gr', regex = False)
    data_order.loc[data_order['product']=='HILO TEEN STRAWBERRY MILKSHAKE\xa0 12Dx500G','product']="HiLo Teen Strawberry Milkshake 500gr"
    


    data_SKU = pd.read_excel(r'Order Online\SKU buat andra updated maret 2021.xlsx')
    data_SKU = data_SKU.rename(columns = {'Product' : 'product', 'PL NFI' : 'Price List NFI'})
    data_SKU['SKU'] = data_SKU['SKU'].astype(str).str.replace('.0', '', regex = False)
    data_SKU = data_SKU[data_SKU['SKU'] != 'nan'][data_SKU[data_SKU['SKU'] != 'nan']['SKU'] != '0']
    data_SKU['product'] = data_SKU['product'].str.strip().str.replace('L-men', 'L-Men', case = False).str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("school", "School").str.replace("Rtd", "RTD").str.replace("tetra", "Tetra").str.replace("[6pcs]", "(6pcs)", regex = False)


    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Kecap Manis 200ml'].copy()
    price['product'] = 'Tropicana Slim Kecap Manis 200m'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Diabtx (50 sch)'].copy()
    price['product'] = 'Tropicana Slim Diabtx 50 sch'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Hokkaido Cheese 100gr'].copy()
    price['product'] = 'Tropicana Slim Hokaido Cheese 100gr'


    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['L-Men Bar Crunchy Chocolate 12sch', 5500, 5500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
 
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Mangga Gandaria', 45000, 45000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
 
    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Teen Vanilla Caramel 500gr', 71500, 71500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Paket Bundle HiLo Renceng (Hilo Chocolate Banana (10 sch) + Hilo Chocolate Taro (10 sch) + HiLo Thai Tea (10sch))', 35200, 35200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Berondong 10's", 15000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Kecap Asin 200 ml", 28500, 26000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim DIABTX (100 sch)", 87700, 75000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Teen Chocolate 750gr", 117800, 104000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Paket Ngopi Lokalate", 136000, 109200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Hi Protein 2 Go Chocolate (24 TETRAPAK)", 240000, 238000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 Tropicana Slim Goldenmil Vanilla (6sch)", 39160, 31000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Kawista", 17500, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["Paket Bundle HiLo (HiLo Active Chocolate 500gr + HiLo Teen Yoghurt Banana 250gr)", 122900, 101850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Strawberry Jam 375gr", 72600, 58500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Crunch BBQ Beef (20gr)", 13500, 10500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Cocopandan 40 sch", 52500, 52500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 - W'dank Empon Empon 10 Sachet Renceng", 35000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Semangka 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Nanas 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["W'Dank Empon-Empon 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Milk Skim Fiber Pro Plain 500gr", 135000, 106700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Es Teler 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: HiLo Es Teler 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Hilo Es Ketan Hitam 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Triple Pack: Tropicana Slim Korean Garlic Butter Cookies (5 Sch)", 73500, 52000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Avocado Coffee 4 Sch", 21200, 12100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Sambal Terasi 200 gr", 35600, 29700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Tropicana Slim Avocado Coffee 4 sch x 2", 42400, 24200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Bar Chocolate (12 Sch) + L-Men Protein Crunch BBQ Beef x 2", 159000, 107800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Buy 5 Get 5 L-Men Protein Crunch BBQ Beef (20gr)", 130000, 67500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Active Ketan Hitam 175gr', 48500, 20700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Es Ketan Hitam 10 sch', 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Sweetener Lemongrass Pandan 50 sch', 44200, 28900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 1 Get 1 FREE: Lokalate Kopi Berondong (10 sch)', 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Korean Garlic Butter Cookies 5 sch - Khusus Homdel', 11500, 11500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)HiLo Active Vanilla 500gr', 35450, 35450]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_order['product'] = data_order['product'].astype(str)
    data_SKU['product'] = data_SKU['product'].astype(str)

    data_order['product'] = data_order['product'].str.replace('L Men Gain Mass Chocolate 500 gr', 'L-Men Gain Mass Chocolate 500 gr')



    data_order = data_order.merge(data_SKU[['SKU', 'product', 'Harga Display', 'Harga Coret']].drop_duplicates('product'), how = 'left', on = 'product')

    data_order = data_order.reset_index(drop = True)

    indeks = data_order[data_order['product'] == "Lokalate Kopi Berondong 10's"].index.to_list()
    data_order['Harga Display'][indeks] = 15000
    data_order['Harga Coret'][indeks] = 15000
    indeks = data_order[data_order['SKU'].isnull()].index.to_list()
    for i in indeks:
        col = [x for x in data_SKU.columns if 'Alias Nama' in x]
        for j in col:
            if data_order['product'][i] in data_SKU[j].astype(str).values:
                SKU = data_SKU[data_SKU[j].astype(str) == data_order['product'][i]]['SKU'].values[0]
                data_order['SKU'][i] = SKU

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    data_SKU2 = pd.read_excel(r'SKU_File\data_SKU.xlsx')
    data_SKU2['Nama Produk'] = data_SKU2['Nama Produk'].astype(str)


    for i in indeks:
        if str(data_order['product'][i]).lower() in data_SKU2['Nama Produk'].astype(str).str.lower().values:
            data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2['Nama Produk'].astype(str).str.lower()].values[0]

    list_alias_name = [x for x in data_SKU2.columns if 'Alias Nama' in x]

    for i in indeks:
        for j in list_alias_name:
            if str(data_order['product'][i]).lower() in data_SKU2[j].astype(str).str.lower().values:
                data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2[j].astype(str).str.lower()].values[0]

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    if len(indeks) != 0:
        print('Alert SKU Missing')
        data_order['product'][indeks].drop_duplicates().to_excel('Alert SKU Missing.xlsx', index = False)
    else :
        data_order['phone'] = data_order['phone'].astype(str).str.replace('+628', '08', regex = False)

        data_order['zip'] = data_order['zip'].replace('.0', '', regex = False)

        data_order = data_order.rename(columns = {'order_id' : 'Sales Order ID', 'name' : 'Customer Name', 'product' : 'Item Name', 'price' : 'Price', 'shipping_cost' : 'Shipping Cost', 'address' : 'Shipping Address1', 'city' : 'Shipping City', 'zip' : 'Shipping Zip', 'province' : 'Shipping Province', 'phone' : 'Shipping Phone', 'courier' : 'Shipping Courier'})
        data_order['Channel Order ID'] = data_order['Sales Order ID']
        data_order['Invoice Number'] = data_order['Sales Order ID']
        data_order['Shipping Name'] = data_order['Customer Name']
        data_order['Shipping Address2'] = 0
        data_order['Shipping Country'] = 'Indonesia'
        data_order['AWB'] = 0
        data_order['Channel'] = 'Order Online Medan'


        data_order['Order Date'] = pd.to_datetime(data_order['created_at'])


        data_order['SKU'] = data_order['SKU'].astype(str)
        data_order['Item Name'] = data_order['Item Name'].astype(str)
        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str).str.replace('(S)', '', regex = False)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        index = data_order[data_order['SKU'].astype(str) == '2306551174'].index.to_list()
        data_order['SKU'][index] = '2306592173'

        data_order = data_order.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('hd','', regex = False)
        temp['SKU'] = temp['SKU'].astype(str).str.replace('HD','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        data_order['Real SKU'] = data_order['Real SKU'].astype(str)
        data_order = data_order.merge(data_SKU2[['SKU', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Unbundling ====== 6/10")        
        # Forstok Unbundling    
        list_col = ['SKU'] + data_SKU2.columns[data_SKU2.columns.get_loc('Produk 1'):data_SKU2.columns.get_loc('Harga Organik 7')+1].to_list()
        data_order = data_order.merge(data_SKU2[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        list_pcs = [x for x in data_order.columns if 'PCS' in x]
        for i in list_pcs:
            data_order[i] = data_order[i] * data_order['Quantity']
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        indeks = data_order[data_order['Brand'] == 'Bundle'].index.to_list()
        data_order['Bundle Flag'] = np.nan
        data_order['Bundle Flag'][indeks] = 'Bundle'

        indeks = data_order[data_order['Brand'] == 'Bundle'][data_order[data_order['Brand'] == 'Bundle']['SKU'].astype(str).str.contains('(S)', regex = False)].index.to_list()
        data_order['SKU Produk 1'][indeks] = '(S)' + data_order['SKU Produk 1'][indeks].astype(str)
        data_order['SKU Produk 2'][indeks] = '(S)' + data_order['SKU Produk 2'][indeks].astype(str)
        data_order['SKU Produk 3'][indeks] = '(S)' + data_order['SKU Produk 3'][indeks].astype(str)
        data_order['SKU Produk 4'][indeks] = '(S)' + data_order['SKU Produk 4'][indeks].astype(str)
        data_order['SKU Produk 5'][indeks] = '(S)' + data_order['SKU Produk 5'][indeks].astype(str)
        data_order['SKU Produk 6'][indeks] = '(S)' + data_order['SKU Produk 6'][indeks].astype(str)
        data_order['SKU Produk 7'][indeks] = '(S)' + data_order['SKU Produk 7'][indeks].astype(str)

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Filling Date ====== 7/10")
        data_order['Date'] = np.nan
        data_order['Month'] = np.nan
        data_order['Year'] = np.nan

        for i in range(data_order.shape[0]):
            if int(data_order['Order Date'][i].strftime('%d')) <= 12:
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).year
            else :
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i]).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i]).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i]).year

        quarter = pd.DataFrame([['January', 1], ['February', 1], ['March', 1], ['April', 2], ['May', 2], ['June', 2], 
                ['July', 3], ['August', 3], ['September', 3],['October', 4], ['November', 4], ['December', 4]], columns = ['Bulan', 'Quarter'])
        data_order = data_order.merge(quarter, how = 'left', left_on = 'Month', right_on = 'Bulan')
        data_order = data_order.drop(['Bulan'], axis = 1)
        data_bulan = pd.DataFrame([{'Bulan' : 'December', 'Number' : 12} ,
                {'Bulan' : 'January' , 'Number': 1},
                {'Bulan' : 'February' , 'Number': 2},
                {'Bulan' : 'March' , 'Number': 3},
                {'Bulan' : 'April' , 'Number': 4},
                {'Bulan' : 'May' , 'Number': 5},
                {'Bulan' : 'June', 'Number': 6},
                {'Bulan' : 'July' , 'Number': 7},
                {'Bulan' : 'August', 'Number' : 8},
                {'Bulan' : 'September', 'Number' : 9},
                {'Bulan' : 'October' , 'Number': 10},
                {'Bulan' : 'November' , 'Number': 11}])
        temp = data_order.copy()
        temp['Day'] = temp['Date']
        temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
        temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
        data_order['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.week
        temp['Hour'] = pd.to_datetime(data_order['Order Date']).dt.hour
        temp['Minute'] = pd.to_datetime(data_order['Order Date']).dt.minute
        temp['Second'] = pd.to_datetime(data_order['Order Date']).dt.second
        data_order['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']])



        order_all = data_order.copy()
        order_all['Total'] = order_all['net_revenue']
        order_all['Price List NFI'] = np.nan
        order_all['Total Net'] = np.nan



        order_all = order_all.rename(columns={'Channel Order ID' : 'Order #',
                                                'Status' : 'Order Status',
                                                'Order Date' : 'Order date',
                                                'Item Name' :'Product Name',
                                                'Bundle Name' : 'Bundle',
                                                'Shipping Country' : 'Country',
                                                'Shipping Province' : 'Region',
                                                'Shipping City' : 'City',
                                                'Shipping Zip' : 'Zip Code',
                                                'Shipping Address1' : 'Address',
                                                'Shipping Phone' : 'Phone',
                                                'Quantity' : 'Qty. Invoiced',
                                                'Harga Display' : 'Regular Price',
                                                'net_revenue' : 'Subtotal'})

        
        
        order_all['Selling Price'] = order_all['Harga Coret'].astype(int)
        order_all['Kecamatan'] = np.nan
        order_all['Kelurahan'] = np.nan

        print("Filling Location")
        indeks = order_all[order_all['City'].astype(str).str.contains('/')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains('-')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[0]

        indeks = order_all[order_all['City'].astype(str).str.contains(',')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains(',')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[0]

        order_all['City'] = order_all['City'].astype(str).str.replace('Kab\.', 'Kabupaten' ,case = False)

        master_map = pd.read_csv(r'All Data/Province.csv', names = ['Kode Prov', 'Province'], header= 0)
        master_map2 = pd.read_csv(r'All Data/City.csv', names = ['Kode City', 'Kode Prov', 'City'], header = 0)
        master_map = master_map.merge(master_map2, how = 'right', on = 'Kode Prov')
        master_map['Kode Prov'][515] = 14
        master_map['Province'][515] = 'Riau'
        master_map['Kode Prov'] = master_map['Kode Prov'].astype(int)
        master_map['Province'] = master_map['Province'].str.title()
        master_map['City'] = master_map['City'].str.title()

        city = pd.read_excel(r'All Data/list_city.xlsx')
        temp = order_all.copy()
        temp['City'] = temp['City'].astype(str).str.lower()
        temp['City'] = temp['City'].astype(str).str.replace('kab. ', 'kabupaten ', regex = False, case = False)
        city['All City'] = city['All City'].astype(str).str.lower()
        temp = temp.merge(city.drop_duplicates('All City'), how = 'left', left_on = 'City', right_on = 'All City').set_index(temp.index)
        indeks = temp[temp['Real City'].notnull()].index.to_list()
        order_all['City'][indeks] = temp['Real City'][indeks]

        province = pd.read_excel(r'All Data/list_province.xlsx')
        temp = order_all.copy()
        temp['Region'] = temp['Region'].astype(str).str.lower()
        province['All Province'] = province['All Province'].astype(str).str.lower()
        temp = temp.merge(province.drop_duplicates('All Province'), how = 'left', left_on = 'Region', right_on = 'All Province').set_index(temp.index)
        indeks = temp[temp['Real Province'].notnull()].index.to_list()
        order_all['Region'][indeks] = temp['Real Province'][indeks]

        temp = order_all.copy()
        temp = temp[temp['Region'].isnull()]
        temp['Region'] = temp.merge(master_map, how = 'left', on = 'City').set_index(temp.index)['Province']
        order_all['Region'][temp.index] = temp['Region']  

        district = pd.read_excel(r'All Data/list_district.xlsx')
        temp = order_all.copy()
        temp['Kecamatan'] = temp['Kecamatan'].astype(str).str.lower()
        district['All District'] = district['All District'].astype(str).str.lower()
        temp = temp.merge(district.drop_duplicates('All District'), how = 'left', left_on = 'Kecamatan', right_on = 'All District').set_index(temp.index)
        indeks = temp[temp['Real District'].notnull()].index.to_list()
        order_all['Kecamatan'][indeks] = temp['Real District'][indeks]

        temp = order_all.copy()
        temp2 = temp[['Region', 'City', 'Kecamatan']].merge(master_map, how = 'left', on = 'City')
        indeks = temp2[temp2['Region'] != temp2['Province']][temp2[temp2['Region'] != temp2['Province']]['City'].notnull()].index.to_list()
        order_all['City'][indeks] = np.nan

        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        print("Unbundling")
        data_bundle1 = order_all[~order_all['Produk 1'].isnull()]
        data_bundle1['Bundle Name'] = data_bundle1['Product Name']
        data_bundle1['Product Name'] = data_bundle1['Produk 1']
        data_bundle1['SKU'] = data_bundle1['SKU Produk 1']
        data_bundle1['Qty. Invoiced'] = data_bundle1['PCS Produk 1']
        data_bundle1['Price List NFI'] = data_bundle1['Price List NFI 1']
        data_bundle1['Total Net'] = data_bundle1['Price List NFI 1']
        data_bundle1['Bundle Flag'] = np.nan

        data_bundle2 = order_all[~order_all['Produk 2'].isnull()]
        data_bundle2['Bundle Name'] = data_bundle2['Product Name']
        data_bundle2['Product Name'] = data_bundle2['Produk 2']
        data_bundle2['SKU'] = data_bundle2['SKU Produk 2']
        data_bundle2['Qty. Invoiced'] = data_bundle2['PCS Produk 2']
        data_bundle2['Price List NFI'] = data_bundle2['Price List NFI 2']
        data_bundle2['Total Net'] = data_bundle2['Price List NFI 2'] 
        data_bundle2['Bundle Flag'] = np.nan

        data_bundle3 = order_all[~order_all['Produk 3'].isnull()]
        data_bundle3['Bundle Name'] = data_bundle3['Product Name']
        data_bundle3['Product Name'] = data_bundle3['Produk 3']
        data_bundle3['SKU'] = data_bundle3['SKU Produk 3']
        data_bundle3['Qty. Invoiced'] = data_bundle3['PCS Produk 3']
        data_bundle3['Price List NFI'] = data_bundle3['Price List NFI 3']
        data_bundle3['Total Net'] = data_bundle3['Price List NFI 3'] 
        data_bundle3['Bundle Flag'] = np.nan

        data_bundle4 = order_all[~order_all['Produk 4'].isnull()]
        data_bundle4['Bundle Name'] = data_bundle4['Product Name']
        data_bundle4['Product Name'] = data_bundle4['Produk 4']
        data_bundle4['SKU'] = data_bundle4['SKU Produk 4']
        data_bundle4['Qty. Invoiced'] = data_bundle4['PCS Produk 4']
        data_bundle4['Price List NFI'] = data_bundle4['Price List NFI 4']
        data_bundle4['Total Net'] = data_bundle4['Price List NFI 4'] 
        data_bundle4['Bundle Flag'] = np.nan

        data_bundle5 = order_all[~order_all['Produk 5'].isnull()]
        data_bundle5['Bundle Name'] = data_bundle5['Product Name']
        data_bundle5['Product Name'] = data_bundle5['Produk 5']
        data_bundle5['SKU'] = data_bundle5['SKU Produk 5']
        data_bundle5['Qty. Invoiced'] = data_bundle5['PCS Produk 5']
        data_bundle5['Price List NFI'] = data_bundle5['Price List NFI 5']
        data_bundle5['Total Net'] = data_bundle5['Price List NFI 5']
        data_bundle5['Bundle Flag'] = np.nan

        data_bundle6 = order_all[~order_all['Produk 6'].isnull()]
        data_bundle6['Bundle Name'] = data_bundle6['Product Name']
        data_bundle6['Product Name'] = data_bundle6['Produk 6']
        data_bundle6['SKU'] = data_bundle6['SKU Produk 6']
        data_bundle6['Qty. Invoiced'] = data_bundle6['PCS Produk 6']
        data_bundle6['Price List NFI'] = data_bundle6['Price List NFI 6']
        data_bundle6['Total Net'] = data_bundle6['Price List NFI 6']
        data_bundle6['Bundle Flag'] = np.nan

        data_bundle7 = order_all[~order_all['Produk 7'].isnull()]
        data_bundle7['Bundle Name'] = data_bundle7['Product Name']
        data_bundle7['Product Name'] = data_bundle7['Produk 7']
        data_bundle7['SKU'] = data_bundle7['SKU Produk 7']
        data_bundle7['Qty. Invoiced'] = data_bundle7['PCS Produk 7']
        data_bundle7['Price List NFI'] = data_bundle7['Price List NFI 7']
        data_bundle7['Total Net'] = data_bundle7['Price List NFI 7']
        data_bundle7['Bundle Flag'] = np.nan

        data_bundle = data_bundle1.append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)
        data_bundle['SKU'] = data_bundle['SKU'].astype(str)
        data_bundle['SKU'] = data_bundle['SKU'].str.replace('\.0$', '', regex = True)
        data_bundle[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']] = data_bundle.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item_y', 'Parent SKU_y']]

        temp = data_bundle[data_bundle['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)

        indeks = data_bundle[data_bundle['Real SKU'].isnull()].index.to_list()
        data_bundle['Real SKU'][indeks] = temp['Real SKU_y'][indeks]
        data_bundle['Real Nama Produk'][indeks] = temp['Nama Produk'][indeks]
        data_bundle['Brand'][indeks] = temp['Brand_y'][indeks]
        data_bundle['Sub Brand'][indeks] = temp['Sub Brand_y'][indeks]
        data_bundle['Parent Item'][indeks] = temp['Parent Item_y'][indeks]
        data_bundle['Parent SKU'][indeks] = temp['Parent SKU_y'][indeks]

        print("Pricing")
        order_all = order_all.append(data_bundle, ignore_index = True, sort = False)

        colname = temp.columns[temp.columns.get_loc('Produk 1') : temp.columns.get_loc('Harga Cost 7') + 1]
        colname_str = [x for x in colname if 'Subtotal' not in x and 'Harga' not in x]
        colname_int = [x for x in colname if x not in colname_str]

        for i in colname_str:
            temp[i] = np.nan

        for i in colname_int:
            temp[i] = 0

        data_order = data_order.append(temp , ignore_index = True, sort = False)


        order_all = order_all.merge(data_SKU2[['SKU', 'Price List NFI', 'Harga Cost']].drop_duplicates('SKU'), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(order_all.index)
        order_all['Price List NFI_x'] = order_all['Price List NFI_x'].fillna(order_all['Price List NFI_y'])
        order_all =  order_all.drop(['Price List NFI_y', 'SKU_y'], axis = 1)
        order_all = order_all.rename(columns = {'SKU_x' : 'SKU', 'Price List NFI_x' : 'Price List NFI'})

        indeks = order_all[order_all['Product Name'] == 'HiLo Teen Chocolate 250gr'].index.to_list()
        order_all['Real Nama Produk'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Parent Item'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Brand'][indeks] = 'HiLo'
        order_all['Sub Brand'][indeks] = 'HILO TEEN'
        order_all['Price List NFI'][indeks] = 36850
        order_all['Harga Cost'][indeks] = 36850
        order_all['Real SKU'][indeks] = '2101651155'
        order_all['Parent SKU'][indeks] = '2101651155'


        order_all['Price List NFI'] = pd.to_numeric(order_all['Price List NFI']).astype(int)
        #order_all['Harga Cost'] = pd.to_numeric(order_all['Harga Cost']).astype(int)
        order_all['Harga Cost'] = pd.to_numeric(order_all['Harga Cost'], errors = 'coerce').fillna(0).astype(int)
        order_all['Qty. Invoiced'] = pd.to_numeric(order_all['Qty. Invoiced']).astype(int)

        order_all['Total Net'] = order_all['Price List NFI'] * order_all['Qty. Invoiced']
        order_all['Total Harga Cost'] = order_all['Harga Cost'] * order_all['Qty. Invoiced']
        order_all['Subtotal'] = order_all['Selling Price'] * order_all['Qty. Invoiced']
        order_all['Total'] = order_all['Selling Price'] * order_all['Qty. Invoiced']

        order_all = order_all.reset_index(drop = True)
        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        order_all['Seller Discount'] = order_all['discount']
        order_all['Shipping'] = order_all['Shipping Cost']

        temp = order_all[order_all['Brand'] != 'Bundle']
        temp['discount'] = temp['discount'] * temp['Total Harga Cost']/temp.groupby(['Order #'])['Total Harga Cost'].transform('sum')
        order_all['discount'][temp.index] = temp['discount']

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Subtotal', 'Total']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Total'][list_nobundle.index] = list_nobundle['Total_y']
        order_all['Subtotal'][list_nobundle.index] = list_nobundle['Subtotal_y']

        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Subtotal'] = temp['Subtotal'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        temp['Selling Price'] = temp['Subtotal']/temp['Qty. Invoiced']
        temp['Total'] = temp['Total'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Total'][temp.index] = temp['Total']
        order_all['Subtotal'][temp.index] = temp['Subtotal']
        order_all['Selling Price'][temp.index] = temp['Selling Price']


        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Seller Discount']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Seller Discount'][list_nobundle.index] = list_nobundle['Seller Discount_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Seller Discount'] = temp['Seller Discount'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Seller Discount'][temp.index] = temp['Seller Discount']


        temp = order_all[order_all['Bundle Name'].isnull()]
        temp_group = temp[['Order #','Shipping']].groupby(['Order #']).sum().reset_index()

        temp = order_all.merge(temp_group, how = 'left', on = 'Order #').set_index(order_all.index)
        temp['Shipping_x'] = temp['Shipping_y'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Shipping'][temp.index] = temp['Shipping_y']
        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Shipping']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Shipping'][list_nobundle.index] = list_nobundle['Shipping_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Shipping'] = temp['Shipping'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Shipping'][temp.index] = temp['Shipping']
        order_all['True datetime'] = pd.to_datetime(order_all['True datetime'])
        order_all['Promo'] = np.nan
        order_all['Discount MC'] = np.nan


        order_all['Warehouse Name'] = 'Primary Warehouse'
        order_all['Store'] = 'Order Online'

        order_all['Customer Email'] = order_all['email']
        order_all['Order Status'] = order_all['status']
        order_all['Payment Channel'] = order_all['payment_method']
        order_all['Coupon Code'] = order_all['coupon']
        order_all.columns.to_list()

        order_all_append = order_all[['Sales Order ID', 'Store',
            'Product Name',
            'Customer Name',
            'Phone',
            'Address',
            'Region',
            'City',
            'Zip Code',
            'payment_status',
            'Regular Price',
            'Shipping Courier',
            'Shipping Cost',
            'Subtotal',
            'Qty. Invoiced',
            'SKU',
            'Order #',
            'Invoice Number',
            'Shipping Name',
            'Shipping Address2',
            'Country',
            'AWB',
            'Channel',
            'Order date',
            'Real SKU',
            'Real Nama Produk',
            'Brand',
            'Sub Brand',
            'Parent Item',
            'Parent SKU',
            'Produk 1',
            'SKU Produk 1',
            'PCS Produk 1',
            'Price List NFI 1',
            'Subtotal Produk 1',
            'Harga Display 1',
            'Harga Cost 1',
            'Harga Organik 1',
            'Produk 2',
            'SKU Produk 2',
            'PCS Produk 2',
            'Price List NFI 2',
            'Subtotal Produk 2',
            'Harga Display 2',
            'Harga Cost 2',
            'Harga Organik 2',
            'Produk 3',
            'SKU Produk 3',
            'PCS Produk 3',
            'Price List NFI 3',
            'Subtotal Produk 3',
            'Harga Display 3',
            'Harga Cost 3',
            'Harga Organik 3',
            'Produk 4',
            'SKU Produk 4',
            'PCS Produk 4',
            'Price List NFI 4',
            'Subtotal Produk 4',
            'Harga Display 4',
            'Harga Cost 4',
            'Harga Organik 4',
            'Produk 5',
            'SKU Produk 5',
            'PCS Produk 5',
            'Price List NFI 5',
            'Subtotal Produk 5',
            'Harga Display 5',
            'Harga Cost 5',
            'Harga Organik 5',
            'Produk 6',
            'SKU Produk 6',
            'PCS Produk 6',
            'Price List NFI 6',
            'Subtotal Produk 6',
            'Harga Display 6',
            'Harga Cost 6',
            'Harga Organik 6',
            'Produk 7',
            'SKU Produk 7',
            'PCS Produk 7',
            'Price List NFI 7',
            'Subtotal Produk 7',
            'Harga Display 7',
            'Harga Cost 7',
            'Harga Organik 7',
            'Bundle Flag',
            'Date',
            'Month',
            'Year',
            'Quarter',
            'Week',
            'True datetime',
            'Total',
            'Price List NFI',
            'Total Net',
            'Selling Price',
            'Kecamatan',
            'Kelurahan',
            'Bundle Name',
            'Harga Cost',
            'Total Harga Cost',
            'Seller Discount',
            'Shipping',
            'Promo',
            'Discount MC',
            'Warehouse Name',
            'Customer Email',
            'Order Status',
            'Payment Channel',
            'Coupon Code']]

        data_all = data_all[~data_all['Order #'].astype(str).isin(order_all_append['Order #'].astype(str))]
        data_all = data_all.append(order_all_append, ignore_index = True, sort = False)

        order_online_medan = True
        del file_name

if not order_online_samarinda:
                            
    import pandas as pd
    import numpy as np
    import requests
    import os

    print('order_online_samarinda')
    
    before = os.listdir(os.getcwd() + '/Input Data')

    options = Options()
    options.add_experimental_option("prefs", {
            "download.default_directory": os.path.abspath("Input Data"),
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            "safebrowsing.enabled": False
    })

    driver = webdriver.Chrome(options=options)
    driver.fullscreen_window()
    driver.get("https://app.orderonline.id")

    username = driver.find_element_by_name("email")
    username.clear()
    username.send_keys("customersamarinda@nutrimart.co.id")

    password = driver.find_element_by_name("password")
    password.send_keys("samarindahomdel")
    password.click()

    driver.find_element_by_class_name("btn-submit").click()

    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="main-nav-dropdown"]/ul/li[3]/a'))).click()
    time.sleep(5)
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[1]/div/div[2]/div/div/div').click()
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="app"]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[1]/div[1]/ul/li[6]'))).click()
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[2]/button[2]').click()
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[3]/div/button').click()
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[3]/div/div/button[1]').click()

    time.sleep(10)

    after = os.listdir(os.getcwd() + '/Input Data')
    change = set(after) - set(before)
    if len(change) == 1:
        file_name = change.pop()
    elif len(change) == 0: 
        print("No file downloaded")
    else :
        print("More than one file downloaded")

    data_order = pd.read_csv(r'Input Data/' + str(file_name))
    driver.close()

    data_order['order_id'] = data_order['order_id'].fillna(method='ffill')
    data_order = data_order.drop('quantity', axis = 1)
    temp = data_order.drop_duplicates('order_id').drop('product', axis = 1)
    data_order = data_order[['order_id', 'product']]
    data_order = data_order.merge(temp, how = 'left', on = 'order_id')
    data_order['order_id'] = data_order['order_id'].astype(int)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Shirataki Noodles 71gr (x2) ', 'Twin Pack: Tropicana Slim Shirataki Noodles 71gr ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Saus Tiram 200ml (x2) ', 'Twin Pack: Tropicana Slim Saus Tiram 200ml ', regex = False)

    product = data_order['product'].str.split("\(x",1, expand = True)
    data_order['Quantity'] = product[1].astype(str).str.replace(')', '', regex = False).astype(int)
    data_order['product'] = product[0].str.strip().str.replace('  ', ' ').str.replace('6 SCH', '6sch').str.replace("W'Dank", "W'dank").str.replace('L-men', 'L-Men').str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("school", "School").str.replace("Rtd", "RTD").str.replace('Honey 50 sachet', 'Honey (50sch)').str.replace('Teen Coklat', 'Teen Chocolate')

    data_order['product'] = data_order['product'].str.replace("HiLo Thai Tea \(10sch\)$", 'HiLo Thai Tea (10 sch)', regex = True)

    data_SKU = pd.read_excel(r'Order Online\SKU buat andra updated maret 2021.xlsx')
    data_SKU = data_SKU.rename(columns = {'Product' : 'product', 'PL NFI' : 'Price List NFI'})
    data_SKU['SKU'] = data_SKU['SKU'].astype(str).str.replace('.0', '', regex = False)
    data_SKU = data_SKU[data_SKU['SKU'] != 'nan'][data_SKU[data_SKU['SKU'] != 'nan']['SKU'] != '0']
    data_SKU['product'] = data_SKU['product'].str.strip().str.replace('L-men', 'L-Men').str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("school", "School").str.replace("Rtd", "RTD").str.replace('Honey 50 sachet', 'Honey (50sch)').str.replace('Teen Coklat', 'Teen Chocolate')


    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Kecap Manis 200ml'].copy()
    price['product'] = 'Tropicana Slim Kecap Manis 200m'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Diabtx (50 sch)'].copy()
    price['product'] = 'Tropicana Slim Diabtx 50 sch'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Hokkaido Cheese 100gr'].copy()
    price['product'] = 'Tropicana Slim Hokaido Cheese 100gr'


    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
 
    data_SKU = data_SKU.append(pd.DataFrame([['L-Men Bar Crunchy Chocolate 12sch', 5500, 5500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Mangga Gandaria', 45000, 45000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Teen Vanilla Caramel 500gr', 71500, 71500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Paket Bundle HiLo Renceng (Hilo Chocolate Banana (10 sch) + Hilo Chocolate Taro (10 sch) + HiLo Thai Tea (10sch))', 35200, 35200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Berondong 10's", 15000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Kecap Asin 200 ml", 28500, 26000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim DIABTX (100 sch)", 87700, 75000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Teen Chocolate 750gr", 117800, 104000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Paket Ngopi Lokalate", 136000, 109200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Hi Protein 2 Go Chocolate (24 TETRAPAK)", 240000, 238000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 Tropicana Slim Goldenmil Vanilla (6sch)", 39160, 31000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Kawista", 17500, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["Paket Bundle HiLo (HiLo Active Chocolate 500gr + HiLo Teen Yoghurt Banana 250gr)", 122900, 101850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Strawberry Jam 375gr", 72600, 58500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Crunch BBQ Beef (20gr)", 13500, 10500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Cocopandan 40 sch", 52500, 52500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 - W'dank Empon Empon 10 Sachet Renceng", 35000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Semangka 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Nanas 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["W'Dank Empon-Empon 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Milk Skim Fiber Pro Plain 500gr", 135000, 106700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Es Teler 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: HiLo Es Teler 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Hilo Es Ketan Hitam 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Triple Pack: Tropicana Slim Korean Garlic Butter Cookies (5 Sch)", 73500, 52000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Avocado Coffee 4 Sch", 21200, 12100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Sambal Terasi 200 gr", 35600, 29700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Tropicana Slim Avocado Coffee 4 sch x 2", 42400, 24200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Bar Chocolate (12 Sch) + L-Men Protein Crunch BBQ Beef x 2", 159000, 107800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Buy 5 Get 5 L-Men Protein Crunch BBQ Beef (20gr)", 130000, 67500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Active Ketan Hitam 175gr', 48500, 20700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Es Ketan Hitam 10 sch', 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Sweetener Lemongrass Pandan 50 sch', 44200, 28900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 1 Get 1 FREE: Lokalate Kopi Berondong (10 sch)', 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) L-Men Lose Weight Avocado Coffee 300g', 69250, 69250]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_order['product'] = data_order['product'].astype(str)
    data_SKU['product'] = data_SKU['product'].astype(str)

    data_order['product'] = data_order['product'].str.replace('L Men Gain Mass Chocolate 500 gr', 'L-Men Gain Mass Chocolate 500 gr')



    data_order = data_order.merge(data_SKU[['SKU', 'product', 'Harga Display', 'Harga Coret']].drop_duplicates('product'), how = 'left', on = 'product')

    data_order = data_order.reset_index(drop = True)

    indeks = data_order[data_order['product'] == "Lokalate Kopi Berondong 10's"].index.to_list()
    data_order['Harga Display'][indeks] = 15000
    data_order['Harga Coret'][indeks] = 15000
    indeks = data_order[data_order['SKU'].isnull()].index.to_list()
    for i in indeks:
        col = [x for x in data_SKU.columns if 'Alias Nama' in x]
        for j in col:
            if data_order['product'][i] in data_SKU[j].astype(str).values:
                SKU = data_SKU[data_SKU[j].astype(str) == data_order['product'][i]]['SKU'].values[0]
                data_order['SKU'][i] = SKU

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    data_SKU2 = pd.read_excel(r'SKU_File\data_SKU.xlsx')
    data_SKU2['Nama Produk'] = data_SKU2['Nama Produk'].astype(str)


    for i in indeks:
        if str(data_order['product'][i]).lower() in data_SKU2['Nama Produk'].astype(str).str.lower().values:
            data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2['Nama Produk'].astype(str).str.lower()].values[0]

    list_alias_name = [x for x in data_SKU2.columns if 'Alias Nama' in x]

    for i in indeks:
        for j in list_alias_name:
            if str(data_order['product'][i]).lower() in data_SKU2[j].astype(str).str.lower().values:
                data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2[j].astype(str).str.lower()].values[0]

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    if len(indeks) != 0:
        print('Alert SKU Missing')
        data_order['product'][indeks].drop_duplicates().to_excel('Alert SKU Missing.xlsx', index = False)
    else :
        data_order['phone'] = data_order['phone'].astype(str).str.replace('+628', '08', regex = False)

        data_order['zip'] = data_order['zip'].replace('.0', '', regex = False)

        data_order = data_order.rename(columns = {'order_id' : 'Sales Order ID', 'name' : 'Customer Name', 'product' : 'Item Name', 'price' : 'Price', 'shipping_cost' : 'Shipping Cost', 'address' : 'Shipping Address1', 'city' : 'Shipping City', 'zip' : 'Shipping Zip', 'province' : 'Shipping Province', 'phone' : 'Shipping Phone', 'courier' : 'Shipping Courier'})
        data_order['Channel Order ID'] = data_order['Sales Order ID']
        data_order['Invoice Number'] = data_order['Sales Order ID']
        data_order['Shipping Name'] = data_order['Customer Name']
        data_order['Shipping Address2'] = 0
        data_order['Shipping Country'] = 'Indonesia'
        data_order['AWB'] = 0
        data_order['Channel'] = 'Order Online Samarinda'


        data_order['Order Date'] = pd.to_datetime(data_order['created_at'])


        data_order['SKU'] = data_order['SKU'].astype(str)
        data_order['Item Name'] = data_order['Item Name'].astype(str)
        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str).str.replace('(S)', '', regex = False)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        index = data_order[data_order['SKU'].astype(str) == '2306551174'].index.to_list()
        data_order['SKU'][index] = '2306592173'

        data_order = data_order.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('hd','', regex = False)
        temp['SKU'] = temp['SKU'].astype(str).str.replace('HD','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        data_order['Real SKU'] = data_order['Real SKU'].astype(str)
        data_order = data_order.merge(data_SKU2[['SKU', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Unbundling ====== 6/10")        
        # Forstok Unbundling    
        list_col = ['SKU'] + data_SKU2.columns[data_SKU2.columns.get_loc('Produk 1'):data_SKU2.columns.get_loc('Harga Organik 7')+1].to_list()
        data_order = data_order.merge(data_SKU2[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        list_pcs = [x for x in data_order.columns if 'PCS' in x]
        for i in list_pcs:
            data_order[i] = data_order[i] * data_order['Quantity']
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        indeks = data_order[data_order['Brand'] == 'Bundle'].index.to_list()
        data_order['Bundle Flag'] = np.nan
        data_order['Bundle Flag'][indeks] = 'Bundle'

        indeks = data_order[data_order['Brand'] == 'Bundle'][data_order[data_order['Brand'] == 'Bundle']['SKU'].astype(str).str.contains('(S)', regex = False)].index.to_list()
        data_order['SKU Produk 1'][indeks] = '(S)' + data_order['SKU Produk 1'][indeks].astype(str)
        data_order['SKU Produk 2'][indeks] = '(S)' + data_order['SKU Produk 2'][indeks].astype(str)
        data_order['SKU Produk 3'][indeks] = '(S)' + data_order['SKU Produk 3'][indeks].astype(str)
        data_order['SKU Produk 4'][indeks] = '(S)' + data_order['SKU Produk 4'][indeks].astype(str)
        data_order['SKU Produk 5'][indeks] = '(S)' + data_order['SKU Produk 5'][indeks].astype(str)
        data_order['SKU Produk 6'][indeks] = '(S)' + data_order['SKU Produk 6'][indeks].astype(str)
        data_order['SKU Produk 7'][indeks] = '(S)' + data_order['SKU Produk 7'][indeks].astype(str)

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Filling Date ====== 7/10")
        data_order['Date'] = np.nan
        data_order['Month'] = np.nan
        data_order['Year'] = np.nan

        for i in range(data_order.shape[0]):
            if int(data_order['Order Date'][i].strftime('%d')) <= 12:
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).year
            else :
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i]).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i]).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i]).year

        quarter = pd.DataFrame([['January', 1], ['February', 1], ['March', 1], ['April', 2], ['May', 2], ['June', 2], 
                ['July', 3], ['August', 3], ['September', 3],['October', 4], ['November', 4], ['December', 4]], columns = ['Bulan', 'Quarter'])
        data_order = data_order.merge(quarter, how = 'left', left_on = 'Month', right_on = 'Bulan')
        data_order = data_order.drop(['Bulan'], axis = 1)
        data_bulan = pd.DataFrame([{'Bulan' : 'December', 'Number' : 12} ,
                {'Bulan' : 'January' , 'Number': 1},
                {'Bulan' : 'February' , 'Number': 2},
                {'Bulan' : 'March' , 'Number': 3},
                {'Bulan' : 'April' , 'Number': 4},
                {'Bulan' : 'May' , 'Number': 5},
                {'Bulan' : 'June', 'Number': 6},
                {'Bulan' : 'July' , 'Number': 7},
                {'Bulan' : 'August', 'Number' : 8},
                {'Bulan' : 'September', 'Number' : 9},
                {'Bulan' : 'October' , 'Number': 10},
                {'Bulan' : 'November' , 'Number': 11}])
        temp = data_order.copy()
        temp['Day'] = temp['Date']
        temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
        temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
        data_order['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.week
        temp['Hour'] = pd.to_datetime(data_order['Order Date']).dt.hour
        temp['Minute'] = pd.to_datetime(data_order['Order Date']).dt.minute
        temp['Second'] = pd.to_datetime(data_order['Order Date']).dt.second
        data_order['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']])



        order_all = data_order.copy()
        order_all['Total'] = order_all['net_revenue']
        order_all['Price List NFI'] = np.nan
        order_all['Total Net'] = np.nan



        order_all = order_all.rename(columns={'Channel Order ID' : 'Order #',
                                                'Status' : 'Order Status',
                                                'Order Date' : 'Order date',
                                                'Item Name' :'Product Name',
                                                'Bundle Name' : 'Bundle',
                                                'Shipping Country' : 'Country',
                                                'Shipping Province' : 'Region',
                                                'Shipping City' : 'City',
                                                'Shipping Zip' : 'Zip Code',
                                                'Shipping Address1' : 'Address',
                                                'Shipping Phone' : 'Phone',
                                                'Quantity' : 'Qty. Invoiced',
                                                'Harga Display' : 'Regular Price',
                                                'net_revenue' : 'Subtotal'})


        order_all['Selling Price'] = order_all['Harga Coret'].astype(int)
        order_all['Kecamatan'] = np.nan
        order_all['Kelurahan'] = np.nan

        print("Filling Location")
        indeks = order_all[order_all['City'].astype(str).str.contains('/')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains('-')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[0]

        indeks = order_all[order_all['City'].astype(str).str.contains(',')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains(',')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[0]

        order_all['City'] = order_all['City'].astype(str).str.replace('Kab\.', 'Kabupaten' ,case = False)

        master_map = pd.read_csv(r'All Data/Province.csv', names = ['Kode Prov', 'Province'], header= 0)
        master_map2 = pd.read_csv(r'All Data/City.csv', names = ['Kode City', 'Kode Prov', 'City'], header = 0)
        master_map = master_map.merge(master_map2, how = 'right', on = 'Kode Prov')
        master_map['Kode Prov'][515] = 14
        master_map['Province'][515] = 'Riau'
        master_map['Kode Prov'] = master_map['Kode Prov'].astype(int)
        master_map['Province'] = master_map['Province'].str.title()
        master_map['City'] = master_map['City'].str.title()

        city = pd.read_excel(r'All Data/list_city.xlsx')
        temp = order_all.copy()
        temp['City'] = temp['City'].astype(str).str.lower()
        temp['City'] = temp['City'].astype(str).str.replace('kab. ', 'kabupaten ', regex = False, case = False)
        city['All City'] = city['All City'].astype(str).str.lower()
        temp = temp.merge(city.drop_duplicates('All City'), how = 'left', left_on = 'City', right_on = 'All City').set_index(temp.index)
        indeks = temp[temp['Real City'].notnull()].index.to_list()
        order_all['City'][indeks] = temp['Real City'][indeks]

        province = pd.read_excel(r'All Data/list_province.xlsx')
        temp = order_all.copy()
        temp['Region'] = temp['Region'].astype(str).str.lower()
        province['All Province'] = province['All Province'].astype(str).str.lower()
        temp = temp.merge(province.drop_duplicates('All Province'), how = 'left', left_on = 'Region', right_on = 'All Province').set_index(temp.index)
        indeks = temp[temp['Real Province'].notnull()].index.to_list()
        order_all['Region'][indeks] = temp['Real Province'][indeks]

        temp = order_all.copy()
        temp = temp[temp['Region'].isnull()]
        temp['Region'] = temp.merge(master_map, how = 'left', on = 'City').set_index(temp.index)['Province']
        order_all['Region'][temp.index] = temp['Region']  

        district = pd.read_excel(r'All Data/list_district.xlsx')
        temp = order_all.copy()
        temp['Kecamatan'] = temp['Kecamatan'].astype(str).str.lower()
        district['All District'] = district['All District'].astype(str).str.lower()
        temp = temp.merge(district.drop_duplicates('All District'), how = 'left', left_on = 'Kecamatan', right_on = 'All District').set_index(temp.index)
        indeks = temp[temp['Real District'].notnull()].index.to_list()
        order_all['Kecamatan'][indeks] = temp['Real District'][indeks]

        temp = order_all.copy()
        temp2 = temp[['Region', 'City', 'Kecamatan']].merge(master_map, how = 'left', on = 'City')
        indeks = temp2[temp2['Region'] != temp2['Province']][temp2[temp2['Region'] != temp2['Province']]['City'].notnull()].index.to_list()
        order_all['City'][indeks] = np.nan

        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        print("Unbundling")
        data_bundle1 = order_all[~order_all['Produk 1'].isnull()]
        data_bundle1['Bundle Name'] = data_bundle1['Product Name']
        data_bundle1['Product Name'] = data_bundle1['Produk 1']
        data_bundle1['SKU'] = data_bundle1['SKU Produk 1']
        data_bundle1['Qty. Invoiced'] = data_bundle1['PCS Produk 1']
        data_bundle1['Price List NFI'] = data_bundle1['Price List NFI 1']
        data_bundle1['Total Net'] = data_bundle1['Price List NFI 1']
        data_bundle1['Bundle Flag'] = np.nan

        data_bundle2 = order_all[~order_all['Produk 2'].isnull()]
        data_bundle2['Bundle Name'] = data_bundle2['Product Name']
        data_bundle2['Product Name'] = data_bundle2['Produk 2']
        data_bundle2['SKU'] = data_bundle2['SKU Produk 2']
        data_bundle2['Qty. Invoiced'] = data_bundle2['PCS Produk 2']
        data_bundle2['Price List NFI'] = data_bundle2['Price List NFI 2']
        data_bundle2['Total Net'] = data_bundle2['Price List NFI 2'] 
        data_bundle2['Bundle Flag'] = np.nan

        data_bundle3 = order_all[~order_all['Produk 3'].isnull()]
        data_bundle3['Bundle Name'] = data_bundle3['Product Name']
        data_bundle3['Product Name'] = data_bundle3['Produk 3']
        data_bundle3['SKU'] = data_bundle3['SKU Produk 3']
        data_bundle3['Qty. Invoiced'] = data_bundle3['PCS Produk 3']
        data_bundle3['Price List NFI'] = data_bundle3['Price List NFI 3']
        data_bundle3['Total Net'] = data_bundle3['Price List NFI 3'] 
        data_bundle3['Bundle Flag'] = np.nan

        data_bundle4 = order_all[~order_all['Produk 4'].isnull()]
        data_bundle4['Bundle Name'] = data_bundle4['Product Name']
        data_bundle4['Product Name'] = data_bundle4['Produk 4']
        data_bundle4['SKU'] = data_bundle4['SKU Produk 4']
        data_bundle4['Qty. Invoiced'] = data_bundle4['PCS Produk 4']
        data_bundle4['Price List NFI'] = data_bundle4['Price List NFI 4']
        data_bundle4['Total Net'] = data_bundle4['Price List NFI 4'] 
        data_bundle4['Bundle Flag'] = np.nan

        data_bundle5 = order_all[~order_all['Produk 5'].isnull()]
        data_bundle5['Bundle Name'] = data_bundle5['Product Name']
        data_bundle5['Product Name'] = data_bundle5['Produk 5']
        data_bundle5['SKU'] = data_bundle5['SKU Produk 5']
        data_bundle5['Qty. Invoiced'] = data_bundle5['PCS Produk 5']
        data_bundle5['Price List NFI'] = data_bundle5['Price List NFI 5']
        data_bundle5['Total Net'] = data_bundle5['Price List NFI 5']
        data_bundle5['Bundle Flag'] = np.nan

        data_bundle6 = order_all[~order_all['Produk 6'].isnull()]
        data_bundle6['Bundle Name'] = data_bundle6['Product Name']
        data_bundle6['Product Name'] = data_bundle6['Produk 6']
        data_bundle6['SKU'] = data_bundle6['SKU Produk 6']
        data_bundle6['Qty. Invoiced'] = data_bundle6['PCS Produk 6']
        data_bundle6['Price List NFI'] = data_bundle6['Price List NFI 6']
        data_bundle6['Total Net'] = data_bundle6['Price List NFI 6']
        data_bundle6['Bundle Flag'] = np.nan

        data_bundle7 = order_all[~order_all['Produk 7'].isnull()]
        data_bundle7['Bundle Name'] = data_bundle7['Product Name']
        data_bundle7['Product Name'] = data_bundle7['Produk 7']
        data_bundle7['SKU'] = data_bundle7['SKU Produk 7']
        data_bundle7['Qty. Invoiced'] = data_bundle7['PCS Produk 7']
        data_bundle7['Price List NFI'] = data_bundle7['Price List NFI 7']
        data_bundle7['Total Net'] = data_bundle7['Price List NFI 7']
        data_bundle7['Bundle Flag'] = np.nan

        data_bundle = data_bundle1.append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)
        data_bundle['SKU'] = data_bundle['SKU'].astype(str)
        data_bundle['SKU'] = data_bundle['SKU'].str.replace('\.0$', '', regex = True)
        data_bundle[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']] = data_bundle.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item_y', 'Parent SKU_y']]

        temp = data_bundle[data_bundle['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)

        indeks = data_bundle[data_bundle['Real SKU'].isnull()].index.to_list()
        data_bundle['Real SKU'][indeks] = temp['Real SKU_y'][indeks]
        data_bundle['Real Nama Produk'][indeks] = temp['Nama Produk'][indeks]
        data_bundle['Brand'][indeks] = temp['Brand_y'][indeks]
        data_bundle['Sub Brand'][indeks] = temp['Sub Brand_y'][indeks]
        data_bundle['Parent Item'][indeks] = temp['Parent Item_y'][indeks]
        data_bundle['Parent SKU'][indeks] = temp['Parent SKU_y'][indeks]

        print("Pricing")
        order_all = order_all.append(data_bundle, ignore_index = True, sort = False)

        colname = temp.columns[temp.columns.get_loc('Produk 1') : temp.columns.get_loc('Harga Cost 7') + 1]
        colname_str = [x for x in colname if 'Subtotal' not in x and 'Harga' not in x]
        colname_int = [x for x in colname if x not in colname_str]

        for i in colname_str:
            temp[i] = np.nan

        for i in colname_int:
            temp[i] = 0

        data_order = data_order.append(temp , ignore_index = True, sort = False)


        order_all = order_all.merge(data_SKU2[['SKU', 'Price List NFI', 'Harga Cost']].drop_duplicates('SKU'), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(order_all.index)
        order_all['Price List NFI_x'] = order_all['Price List NFI_x'].fillna(order_all['Price List NFI_y'])
        order_all =  order_all.drop(['Price List NFI_y', 'SKU_y'], axis = 1)
        order_all = order_all.rename(columns = {'SKU_x' : 'SKU', 'Price List NFI_x' : 'Price List NFI'})

        indeks = order_all[order_all['Product Name'] == 'HiLo Teen Chocolate 250gr'].index.to_list()
        order_all['Real Nama Produk'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Parent Item'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Brand'][indeks] = 'HiLo'
        order_all['Sub Brand'][indeks] = 'HILO TEEN'
        order_all['Price List NFI'][indeks] = 36850
        order_all['Harga Cost'][indeks] = 36850
        order_all['Real SKU'][indeks] = '2101651155'
        order_all['Parent SKU'][indeks] = '2101651155'


        order_all['Price List NFI'] = pd.to_numeric(order_all['Price List NFI']).astype(int)
        order_all['Harga Cost'] = pd.to_numeric(order_all['Harga Cost']).astype(int)
        order_all['Qty. Invoiced'] = pd.to_numeric(order_all['Qty. Invoiced']).astype(int)

        order_all['Total Net'] = order_all['Price List NFI'] * order_all['Qty. Invoiced']
        order_all['Total Harga Cost'] = order_all['Harga Cost'] * order_all['Qty. Invoiced']
        order_all['Subtotal'] = order_all['Selling Price'] * order_all['Qty. Invoiced']
        order_all['Total'] = order_all['Selling Price'] * order_all['Qty. Invoiced']

        order_all = order_all.reset_index(drop = True)
        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        order_all['Seller Discount'] = order_all['discount']
        order_all['Shipping'] = order_all['Shipping Cost']

        temp = order_all[order_all['Brand'] != 'Bundle']
        temp['discount'] = temp['discount'] * temp['Total Harga Cost']/temp.groupby(['Order #'])['Total Harga Cost'].transform('sum')
        order_all['discount'][temp.index] = temp['discount']

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Subtotal', 'Total']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Total'][list_nobundle.index] = list_nobundle['Total_y']
        order_all['Subtotal'][list_nobundle.index] = list_nobundle['Subtotal_y']

        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Subtotal'] = temp['Subtotal'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        temp['Selling Price'] = temp['Subtotal']/temp['Qty. Invoiced']
        temp['Total'] = temp['Total'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Total'][temp.index] = temp['Total']
        order_all['Subtotal'][temp.index] = temp['Subtotal']
        order_all['Selling Price'][temp.index] = temp['Selling Price']


        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Seller Discount']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Seller Discount'][list_nobundle.index] = list_nobundle['Seller Discount_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Seller Discount'] = temp['Seller Discount'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Seller Discount'][temp.index] = temp['Seller Discount']


        temp = order_all[order_all['Bundle Name'].isnull()]
        temp_group = temp[['Order #','Shipping']].groupby(['Order #']).sum().reset_index()

        temp = order_all.merge(temp_group, how = 'left', on = 'Order #').set_index(order_all.index)
        temp['Shipping_x'] = temp['Shipping_y'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Shipping'][temp.index] = temp['Shipping_y']
        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Shipping']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Shipping'][list_nobundle.index] = list_nobundle['Shipping_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Shipping'] = temp['Shipping'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Shipping'][temp.index] = temp['Shipping']
        order_all['True datetime'] = pd.to_datetime(order_all['True datetime'])
        order_all['Promo'] = np.nan
        order_all['Discount MC'] = np.nan


        order_all['Warehouse Name'] = 'Primary Warehouse'
        order_all['Store'] = 'Order Online'

        order_all['Customer Email'] = order_all['email']
        order_all['Order Status'] = order_all['status']
        order_all['Payment Channel'] = order_all['payment_method']
        order_all['Coupon Code'] = order_all['coupon']
        order_all.columns.to_list()

        order_all_append = order_all[['Sales Order ID', 'Store',
            'Product Name',
            'Customer Name',
            'Phone',
            'Address',
            'Region',
            'City',
            'Zip Code',
            'payment_status',
            'Regular Price',
            'Shipping Courier',
            'Shipping Cost',
            'Subtotal',
            'Qty. Invoiced',
            'SKU',
            'Order #',
            'Invoice Number',
            'Shipping Name',
            'Shipping Address2',
            'Country',
            'AWB',
            'Channel',
            'Order date',
            'Real SKU',
            'Real Nama Produk',
            'Brand',
            'Sub Brand',
            'Parent Item',
            'Parent SKU',
            'Produk 1',
            'SKU Produk 1',
            'PCS Produk 1',
            'Price List NFI 1',
            'Subtotal Produk 1',
            'Harga Display 1',
            'Harga Cost 1',
            'Harga Organik 1',
            'Produk 2',
            'SKU Produk 2',
            'PCS Produk 2',
            'Price List NFI 2',
            'Subtotal Produk 2',
            'Harga Display 2',
            'Harga Cost 2',
            'Harga Organik 2',
            'Produk 3',
            'SKU Produk 3',
            'PCS Produk 3',
            'Price List NFI 3',
            'Subtotal Produk 3',
            'Harga Display 3',
            'Harga Cost 3',
            'Harga Organik 3',
            'Produk 4',
            'SKU Produk 4',
            'PCS Produk 4',
            'Price List NFI 4',
            'Subtotal Produk 4',
            'Harga Display 4',
            'Harga Cost 4',
            'Harga Organik 4',
            'Produk 5',
            'SKU Produk 5',
            'PCS Produk 5',
            'Price List NFI 5',
            'Subtotal Produk 5',
            'Harga Display 5',
            'Harga Cost 5',
            'Harga Organik 5',
            'Produk 6',
            'SKU Produk 6',
            'PCS Produk 6',
            'Price List NFI 6',
            'Subtotal Produk 6',
            'Harga Display 6',
            'Harga Cost 6',
            'Harga Organik 6',
            'Produk 7',
            'SKU Produk 7',
            'PCS Produk 7',
            'Price List NFI 7',
            'Subtotal Produk 7',
            'Harga Display 7',
            'Harga Cost 7',
            'Harga Organik 7',
            'Bundle Flag',
            'Date',
            'Month',
            'Year',
            'Quarter',
            'Week',
            'True datetime',
            'Total',
            'Price List NFI',
            'Total Net',
            'Selling Price',
            'Kecamatan',
            'Kelurahan',
            'Bundle Name',
            'Harga Cost',
            'Total Harga Cost',
            'Seller Discount',
            'Shipping',
            'Promo',
            'Discount MC',
            'Warehouse Name',
            'Customer Email',
            'Order Status',
            'Payment Channel',
            'Coupon Code']]

        data_all = data_all[~data_all['Order #'].astype(str).isin(order_all_append['Order #'].astype(str))]
        data_all = data_all.append(order_all_append, ignore_index = True, sort = False)
        data_all_aft_order = data_all.copy()
        del file_name
        order_online_samarinda = True
        
if not order_online_lampung:
                            
    import pandas as pd
    import numpy as np
    import requests
    import os

    print('order_online_lampung')
    
    before = os.listdir(os.getcwd() + '/Input Data')

    options = Options()
    options.add_experimental_option("prefs", {
            "download.default_directory": os.path.abspath("Input Data"),
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            "safebrowsing.enabled": False
    })

    driver = webdriver.Chrome(options=options)
    driver.fullscreen_window()
    driver.get("https://app.orderonline.id")

    username = driver.find_element_by_name("email")
    username.clear()
    username.send_keys("customerlampung@nutrimart.co.id")

    password = driver.find_element_by_name("password")
    password.send_keys("lampunghomdel")
    password.click()

    driver.find_element_by_class_name("btn-submit").click()
    time.sleep(15)
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="main-nav-dropdown"]/ul/li[3]/a'))).click()
    time.sleep(5)
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[1]/div/div[2]/div/div/div').click()
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="app"]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[1]/div[1]/ul/li[6]'))).click()
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[2]/button[2]').click()
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[3]/div/button').click()
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[3]/div/div/button[1]').click()

    time.sleep(10)

    after = os.listdir(os.getcwd() + '/Input Data')
    change = set(after) - set(before)
    if len(change) == 1:
        file_name = change.pop()
    elif len(change) == 0: 
        print("No file downloaded")
    else :
        print("More than one file downloaded")

    data_order = pd.read_csv(r'Input Data/' + str(file_name))
    driver.close()


    data_order['order_id'] = data_order['order_id'].fillna(method='ffill')
    data_order = data_order.drop('quantity', axis = 1)
    temp = data_order.drop_duplicates('order_id').drop('product', axis = 1)
    data_order = data_order[['order_id', 'product']]
    data_order = data_order.merge(temp, how = 'left', on = 'order_id')
    data_order['order_id'] = data_order['order_id'].astype(int)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Shirataki Noodles 71gr (x2) ', 'Twin Pack: Tropicana Slim Shirataki Noodles 71gr ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Saus Tiram 200ml (x2) ', 'Twin Pack: Tropicana Slim Saus Tiram 200ml ', regex = False)

    product = data_order['product'].str.split("\(x",1, expand = True)
    data_order['Quantity'] = product[1].astype(str).str.replace(')', '', regex = False).astype(int)
    data_order['product'] = product[0].str.strip().str.replace('  ', ' ').str.replace('6 SCH', '6sch').str.replace("W'Dank", "W'dank").str.replace('L-men', 'L-Men').str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("school", "School").str.replace("Rtd", "RTD").str.replace('Honey 50 sachet', 'Honey (50sch)').str.replace('Teen Coklat', 'Teen Chocolate').str.replace('40 sch', '40sch').str.replace('6 tetrapack', '6 Tetrapack').str.replace("Diabetx 100s", "Diabtx (100 sch)").str.replace("Alpukat 10's", "Alpukat (10sch)").str.replace("Kawista 10 sch", "Kawista (10sch)")




    data_order['product'] = data_order['product'].str.replace("HiLo Thai Tea \(10sch\)$", 'HiLo Thai Tea (10 sch)', regex = True)

    data_SKU = pd.read_excel(r'Order Online\SKU buat andra updated maret 2021.xlsx')
    data_SKU = data_SKU.rename(columns = {'Product' : 'product', 'PL NFI' : 'Price List NFI'})
    data_SKU['SKU'] = data_SKU['SKU'].astype(str).str.replace('.0', '', regex = False)
    data_SKU = data_SKU[data_SKU['SKU'] != 'nan'][data_SKU[data_SKU['SKU'] != 'nan']['SKU'] != '0']
    data_SKU['product'] = data_SKU['product'].str.strip().str.replace('L-men', 'L-Men').str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("school", "School").str.replace("Rtd", "RTD").str.replace('Honey 50 sachet', 'Honey (50sch)').str.replace('Teen Coklat', 'Teen Chocolate').str.replace('40 sch', '40sch').str.replace('6 tetrapack', '6 Tetrapack').str.replace("Diabetx 100s", "Diabtx (100 sch)").str.replace("Alpukat 10's", "Alpukat (10sch)").str.replace("Kawista 10 sch", "Kawista (10sch)")


    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Kecap Manis 200ml'].copy()
    price['product'] = 'Tropicana Slim Kecap Manis 200m'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Diabtx (50 sch)'].copy()
    price['product'] = 'Tropicana Slim Diabtx 50 sch'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Hokkaido Cheese 100gr'].copy()
    price['product'] = 'Tropicana Slim Hokaido Cheese 100gr'


    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
   
    data_SKU = data_SKU.append(pd.DataFrame([['L-Men Bar Crunchy Chocolate 12sch', 5500, 5500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
 
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Mangga Gandaria', 45000, 45000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
  
    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Teen Vanilla Caramel 500gr', 71500, 71500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Paket Bundle HiLo Renceng (Hilo Chocolate Banana (10 sch) + Hilo Chocolate Taro (10 sch) + HiLo Thai Tea (10sch))', 35200, 35200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Berondong 10's", 15000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Kecap Asin 200 ml", 28500, 26000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim DIABTX (100 sch)", 87700, 75000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Teen Chocolate 750gr", 117800, 104000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Paket Ngopi Lokalate", 136000, 109200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Hi Protein 2 Go Chocolate (24 TETRAPAK)", 240000, 238000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 Tropicana Slim Goldenmil Vanilla (6sch)", 39160, 31000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Kawista", 17500, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["Paket Bundle HiLo (HiLo Active Chocolate 500gr + HiLo Teen Yoghurt Banana 250gr)", 122900, 101850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Strawberry Jam 375gr", 72600, 58500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Crunch BBQ Beef (20gr)", 13500, 10500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Cocopandan 40 sch", 52500, 52500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 - W'dank Empon Empon 10 Sachet Renceng", 35000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Semangka 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Nanas 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["W'Dank Empon-Empon 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Milk Skim Fiber Pro Plain 500gr", 135000, 106700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Es Teler 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: HiLo Es Teler 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Hilo Es Ketan Hitam 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Triple Pack: Tropicana Slim Korean Garlic Butter Cookies (5 Sch)", 73500, 52000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Avocado Coffee 4 Sch", 21200, 12100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Sambal Terasi 200 gr", 35600, 29700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Tropicana Slim Avocado Coffee 4 sch x 2", 42400, 24200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Bar Chocolate (12 Sch) + L-Men Protein Crunch BBQ Beef x 2", 159000, 107800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Buy 5 Get 5 L-Men Protein Crunch BBQ Beef (20gr)", 130000, 67500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Active Ketan Hitam 175gr', 48500, 20700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Es Ketan Hitam 10 sch', 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Sweetener Lemongrass Pandan 50 sch', 44200, 28900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 1 Get 1 FREE: Lokalate Kopi Berondong (10 sch)', 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo School Susu Coklat (10 sch) Gusset - Khusus Homdel', 45800, 22900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)HiLo School Chocolate 500gr', 40050, 40050]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo School Susu Coklat (10 sch) Gusset - Khusus Homdel', 45800, 22900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)HiLo School Chocolate 500gr', 40050, 40050]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo School Susu Coklat (10 sch) Gusset - Khusus Homdel', 45800, 22900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED)HiLo School Chocolate 500gr', 40050, 40050]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) - NutriSari Lychee Tea 40 sch', 40050, 40050]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    
    data_order['product'] = data_order['product'].astype(str)
    data_SKU['product'] = data_SKU['product'].astype(str)

    data_order['product'] = data_order['product'].str.replace('L Men Gain Mass Chocolate 500 gr', 'L-Men Gain Mass Chocolate 500 gr')



    data_order = data_order.merge(data_SKU[['SKU', 'product', 'Harga Display', 'Harga Coret']].drop_duplicates('product'), how = 'left', on = 'product')
   
    data_order = data_order.reset_index(drop = True)

    indeks = data_order[data_order['product'] == "Lokalate Kopi Berondong 10's"].index.to_list()
    data_order['Harga Display'][indeks] = 15000
    data_order['Harga Coret'][indeks] = 15000
    indeks = data_order[data_order['SKU'].isnull()].index.to_list()
    for i in indeks:
        col = [x for x in data_SKU.columns if 'Alias Nama' in x]
        for j in col:
            if data_order['product'][i] in data_SKU[j].astype(str).values:
                SKU = data_SKU[data_SKU[j].astype(str) == data_order['product'][i]]['SKU'].values[0]
                data_order['SKU'][i] = SKU

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    data_SKU2 = pd.read_excel(r'SKU_File\data_SKU.xlsx')
    data_SKU2['Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

    s = requests.Session()
    s.get("http://tatanama.pythonanywhere.com")
    s.post("http://tatanama.pythonanywhere.com", data = {'username' : 'ecommerce', 'password' : 'ecommerce'})
    r = s.get("http://tatanama.pythonanywhere.com/download")

    with open(r'SKU_File\Master tatanama.xlsx', 'wb') as output:
        output.write(r.content)

    if os.path.isfile(r'SKU_File\Master tatanama.xlsx') :    
        SKU_append = pd.read_excel(r'SKU_File\Master tatanama.xlsx')
        SKU_append.columns = [x.replace('_', ' ') for x in SKU_append.columns]
        data_SKU2 = data_SKU2[~data_SKU2['SKU'].astype(str).isin(SKU_append['SKU'].astype(str))]
        data_SKU2 = data_SKU2.append(SKU_append, ignore_index = True, sort = False)

    to_excel = data_SKU2.to_excel(r'SKU_File\data_SKU.xlsx', index = False)

    for i in indeks:
        if str(data_order['product'][i]).lower() in data_SKU2['Nama Produk'].astype(str).str.lower().values:
            data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2['Nama Produk'].astype(str).str.lower()].values[0]

    list_alias_name = [x for x in data_SKU2.columns if 'Alias Nama' in x]

    for i in indeks:
        for j in list_alias_name:
            if str(data_order['product'][i]).lower() in data_SKU2[j].astype(str).str.lower().values:
                data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2[j].astype(str).str.lower()].values[0]

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    if len(indeks) != 0:
        print('Alert SKU Missing')
        data_order['product'][indeks].drop_duplicates().to_excel('Alert SKU Missing.xlsx', index = False)
    else :
        data_order['phone'] = data_order['phone'].astype(str).str.replace('+628', '08', regex = False)

        data_order['zip'] = data_order['zip'].replace('.0', '', regex = False)

        data_order = data_order.rename(columns = {'order_id' : 'Sales Order ID', 'name' : 'Customer Name', 'product' : 'Item Name', 'price' : 'Price', 'shipping_cost' : 'Shipping Cost', 'address' : 'Shipping Address1', 'city' : 'Shipping City', 'zip' : 'Shipping Zip', 'province' : 'Shipping Province', 'phone' : 'Shipping Phone', 'courier' : 'Shipping Courier'})
        data_order['Channel Order ID'] = data_order['Sales Order ID']
        data_order['Invoice Number'] = data_order['Sales Order ID']
        data_order['Shipping Name'] = data_order['Customer Name']
        data_order['Shipping Address2'] = 0
        data_order['Shipping Country'] = 'Indonesia'
        data_order['AWB'] = 0
        data_order['Channel'] = 'Order Online Lampung'


        data_order['Order Date'] = pd.to_datetime(data_order['created_at'])


        data_order['SKU'] = data_order['SKU'].astype(str)
        data_order['Item Name'] = data_order['Item Name'].astype(str)
        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str).str.replace('(S)', '', regex = False)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        index = data_order[data_order['SKU'].astype(str) == '2306551174'].index.to_list()
        data_order['SKU'][index] = '2306592173'

        data_order = data_order.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('hd','', regex = False)
        temp['SKU'] = temp['SKU'].astype(str).str.replace('HD','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        data_order['Real SKU'] = data_order['Real SKU'].astype(str)
        data_order = data_order.merge(data_SKU2[['SKU', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Unbundling ====== 6/10")        
        # Forstok Unbundling    
        list_col = ['SKU'] + data_SKU2.columns[data_SKU2.columns.get_loc('Produk 1'):data_SKU2.columns.get_loc('Harga Organik 7')+1].to_list()
        data_order = data_order.merge(data_SKU2[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        list_pcs = [x for x in data_order.columns if 'PCS' in x]
        for i in list_pcs:
            data_order[i] = data_order[i] * data_order['Quantity']
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        indeks = data_order[data_order['Brand'] == 'Bundle'].index.to_list()
        data_order['Bundle Flag'] = np.nan
        data_order['Bundle Flag'][indeks] = 'Bundle'

        indeks = data_order[data_order['Brand'] == 'Bundle'][data_order[data_order['Brand'] == 'Bundle']['SKU'].astype(str).str.contains('(S)', regex = False)].index.to_list()
        data_order['SKU Produk 1'][indeks] = '(S)' + data_order['SKU Produk 1'][indeks].astype(str)
        data_order['SKU Produk 2'][indeks] = '(S)' + data_order['SKU Produk 2'][indeks].astype(str)
        data_order['SKU Produk 3'][indeks] = '(S)' + data_order['SKU Produk 3'][indeks].astype(str)
        data_order['SKU Produk 4'][indeks] = '(S)' + data_order['SKU Produk 4'][indeks].astype(str)
        data_order['SKU Produk 5'][indeks] = '(S)' + data_order['SKU Produk 5'][indeks].astype(str)
        data_order['SKU Produk 6'][indeks] = '(S)' + data_order['SKU Produk 6'][indeks].astype(str)
        data_order['SKU Produk 7'][indeks] = '(S)' + data_order['SKU Produk 7'][indeks].astype(str)

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Filling Date ====== 7/10")
        data_order['Date'] = np.nan
        data_order['Month'] = np.nan
        data_order['Year'] = np.nan

        for i in range(data_order.shape[0]):
            if int(data_order['Order Date'][i].strftime('%d')) <= 12:
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).year
            else :
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i]).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i]).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i]).year

        quarter = pd.DataFrame([['January', 1], ['February', 1], ['March', 1], ['April', 2], ['May', 2], ['June', 2], 
                ['July', 3], ['August', 3], ['September', 3],['October', 4], ['November', 4], ['December', 4]], columns = ['Bulan', 'Quarter'])
        data_order = data_order.merge(quarter, how = 'left', left_on = 'Month', right_on = 'Bulan')
        data_order = data_order.drop(['Bulan'], axis = 1)
        data_bulan = pd.DataFrame([{'Bulan' : 'December', 'Number' : 12} ,
                {'Bulan' : 'January' , 'Number': 1},
                {'Bulan' : 'February' , 'Number': 2},
                {'Bulan' : 'March' , 'Number': 3},
                {'Bulan' : 'April' , 'Number': 4},
                {'Bulan' : 'May' , 'Number': 5},
                {'Bulan' : 'June', 'Number': 6},
                {'Bulan' : 'July' , 'Number': 7},
                {'Bulan' : 'August', 'Number' : 8},
                {'Bulan' : 'September', 'Number' : 9},
                {'Bulan' : 'October' , 'Number': 10},
                {'Bulan' : 'November' , 'Number': 11}])
        temp = data_order.copy()
        temp['Day'] = temp['Date']
        temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
        temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
        data_order['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.week
        temp['Hour'] = pd.to_datetime(data_order['Order Date']).dt.hour
        temp['Minute'] = pd.to_datetime(data_order['Order Date']).dt.minute
        temp['Second'] = pd.to_datetime(data_order['Order Date']).dt.second
        data_order['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']])



        order_all = data_order.copy()
        order_all['Total'] = order_all['net_revenue']
        order_all['Price List NFI'] = np.nan
        order_all['Total Net'] = np.nan



        order_all = order_all.rename(columns={'Channel Order ID' : 'Order #',
                                                'Status' : 'Order Status',
                                                'Order Date' : 'Order date',
                                                'Item Name' :'Product Name',
                                                'Bundle Name' : 'Bundle',
                                                'Shipping Country' : 'Country',
                                                'Shipping Province' : 'Region',
                                                'Shipping City' : 'City',
                                                'Shipping Zip' : 'Zip Code',
                                                'Shipping Address1' : 'Address',
                                                'Shipping Phone' : 'Phone',
                                                'Quantity' : 'Qty. Invoiced',
                                                'Harga Display' : 'Regular Price',
                                                'net_revenue' : 'Subtotal'})


        order_all['Selling Price'] = order_all['Harga Coret'].astype(int)
        order_all['Kecamatan'] = np.nan
        order_all['Kelurahan'] = np.nan

        print("Filling Location")
        indeks = order_all[order_all['City'].astype(str).str.contains('/')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains('-')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[0]

        indeks = order_all[order_all['City'].astype(str).str.contains(',')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains(',')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[0]

        order_all['City'] = order_all['City'].astype(str).str.replace('Kab\.', 'Kabupaten' ,case = False)

        master_map = pd.read_csv(r'All Data/Province.csv', names = ['Kode Prov', 'Province'], header= 0)
        master_map2 = pd.read_csv(r'All Data/City.csv', names = ['Kode City', 'Kode Prov', 'City'], header = 0)
        master_map = master_map.merge(master_map2, how = 'right', on = 'Kode Prov')
        master_map['Kode Prov'][515] = 14
        master_map['Province'][515] = 'Riau'
        master_map['Kode Prov'] = master_map['Kode Prov'].astype(int)
        master_map['Province'] = master_map['Province'].str.title()
        master_map['City'] = master_map['City'].str.title()

        city = pd.read_excel(r'All Data/list_city.xlsx')
        temp = order_all.copy()
        temp['City'] = temp['City'].astype(str).str.lower()
        temp['City'] = temp['City'].astype(str).str.replace('kab. ', 'kabupaten ', regex = False, case = False)
        city['All City'] = city['All City'].astype(str).str.lower()
        temp = temp.merge(city.drop_duplicates('All City'), how = 'left', left_on = 'City', right_on = 'All City').set_index(temp.index)
        indeks = temp[temp['Real City'].notnull()].index.to_list()
        order_all['City'][indeks] = temp['Real City'][indeks]

        province = pd.read_excel(r'All Data/list_province.xlsx')
        temp = order_all.copy()
        temp['Region'] = temp['Region'].astype(str).str.lower()
        province['All Province'] = province['All Province'].astype(str).str.lower()
        temp = temp.merge(province.drop_duplicates('All Province'), how = 'left', left_on = 'Region', right_on = 'All Province').set_index(temp.index)
        indeks = temp[temp['Real Province'].notnull()].index.to_list()
        order_all['Region'][indeks] = temp['Real Province'][indeks]

        temp = order_all.copy()
        temp = temp[temp['Region'].isnull()]
        temp['Region'] = temp.merge(master_map, how = 'left', on = 'City').set_index(temp.index)['Province']
        order_all['Region'][temp.index] = temp['Region']  

        district = pd.read_excel(r'All Data/list_district.xlsx')
        temp = order_all.copy()
        temp['Kecamatan'] = temp['Kecamatan'].astype(str).str.lower()
        district['All District'] = district['All District'].astype(str).str.lower()
        temp = temp.merge(district.drop_duplicates('All District'), how = 'left', left_on = 'Kecamatan', right_on = 'All District').set_index(temp.index)
        indeks = temp[temp['Real District'].notnull()].index.to_list()
        order_all['Kecamatan'][indeks] = temp['Real District'][indeks]

        temp = order_all.copy()
        temp2 = temp[['Region', 'City', 'Kecamatan']].merge(master_map, how = 'left', on = 'City')
        indeks = temp2[temp2['Region'] != temp2['Province']][temp2[temp2['Region'] != temp2['Province']]['City'].notnull()].index.to_list()
        order_all['City'][indeks] = np.nan

        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        print("Unbundling")
        data_bundle1 = order_all[~order_all['Produk 1'].isnull()]
        data_bundle1['Bundle Name'] = data_bundle1['Product Name']
        data_bundle1['Product Name'] = data_bundle1['Produk 1']
        data_bundle1['SKU'] = data_bundle1['SKU Produk 1']
        data_bundle1['Qty. Invoiced'] = data_bundle1['PCS Produk 1']
        data_bundle1['Price List NFI'] = data_bundle1['Price List NFI 1']
        data_bundle1['Total Net'] = data_bundle1['Price List NFI 1']
        data_bundle1['Bundle Flag'] = np.nan

        data_bundle2 = order_all[~order_all['Produk 2'].isnull()]
        data_bundle2['Bundle Name'] = data_bundle2['Product Name']
        data_bundle2['Product Name'] = data_bundle2['Produk 2']
        data_bundle2['SKU'] = data_bundle2['SKU Produk 2']
        data_bundle2['Qty. Invoiced'] = data_bundle2['PCS Produk 2']
        data_bundle2['Price List NFI'] = data_bundle2['Price List NFI 2']
        data_bundle2['Total Net'] = data_bundle2['Price List NFI 2'] 
        data_bundle2['Bundle Flag'] = np.nan

        data_bundle3 = order_all[~order_all['Produk 3'].isnull()]
        data_bundle3['Bundle Name'] = data_bundle3['Product Name']
        data_bundle3['Product Name'] = data_bundle3['Produk 3']
        data_bundle3['SKU'] = data_bundle3['SKU Produk 3']
        data_bundle3['Qty. Invoiced'] = data_bundle3['PCS Produk 3']
        data_bundle3['Price List NFI'] = data_bundle3['Price List NFI 3']
        data_bundle3['Total Net'] = data_bundle3['Price List NFI 3'] 
        data_bundle3['Bundle Flag'] = np.nan

        data_bundle4 = order_all[~order_all['Produk 4'].isnull()]
        data_bundle4['Bundle Name'] = data_bundle4['Product Name']
        data_bundle4['Product Name'] = data_bundle4['Produk 4']
        data_bundle4['SKU'] = data_bundle4['SKU Produk 4']
        data_bundle4['Qty. Invoiced'] = data_bundle4['PCS Produk 4']
        data_bundle4['Price List NFI'] = data_bundle4['Price List NFI 4']
        data_bundle4['Total Net'] = data_bundle4['Price List NFI 4'] 
        data_bundle4['Bundle Flag'] = np.nan

        data_bundle5 = order_all[~order_all['Produk 5'].isnull()]
        data_bundle5['Bundle Name'] = data_bundle5['Product Name']
        data_bundle5['Product Name'] = data_bundle5['Produk 5']
        data_bundle5['SKU'] = data_bundle5['SKU Produk 5']
        data_bundle5['Qty. Invoiced'] = data_bundle5['PCS Produk 5']
        data_bundle5['Price List NFI'] = data_bundle5['Price List NFI 5']
        data_bundle5['Total Net'] = data_bundle5['Price List NFI 5']
        data_bundle5['Bundle Flag'] = np.nan

        data_bundle6 = order_all[~order_all['Produk 6'].isnull()]
        data_bundle6['Bundle Name'] = data_bundle6['Product Name']
        data_bundle6['Product Name'] = data_bundle6['Produk 6']
        data_bundle6['SKU'] = data_bundle6['SKU Produk 6']
        data_bundle6['Qty. Invoiced'] = data_bundle6['PCS Produk 6']
        data_bundle6['Price List NFI'] = data_bundle6['Price List NFI 6']
        data_bundle6['Total Net'] = data_bundle6['Price List NFI 6']
        data_bundle6['Bundle Flag'] = np.nan

        data_bundle7 = order_all[~order_all['Produk 7'].isnull()]
        data_bundle7['Bundle Name'] = data_bundle7['Product Name']
        data_bundle7['Product Name'] = data_bundle7['Produk 7']
        data_bundle7['SKU'] = data_bundle7['SKU Produk 7']
        data_bundle7['Qty. Invoiced'] = data_bundle7['PCS Produk 7']
        data_bundle7['Price List NFI'] = data_bundle7['Price List NFI 7']
        data_bundle7['Total Net'] = data_bundle7['Price List NFI 7']
        data_bundle7['Bundle Flag'] = np.nan

        data_bundle = data_bundle1.append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)
        data_bundle['SKU'] = data_bundle['SKU'].astype(str)
        data_bundle['SKU'] = data_bundle['SKU'].str.replace('\.0$', '', regex = True)
        data_bundle[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']] = data_bundle.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item_y', 'Parent SKU_y']]

        temp = data_bundle[data_bundle['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)

        indeks = data_bundle[data_bundle['Real SKU'].isnull()].index.to_list()
        data_bundle['Real SKU'][indeks] = temp['Real SKU_y'][indeks]
        data_bundle['Real Nama Produk'][indeks] = temp['Nama Produk'][indeks]
        data_bundle['Brand'][indeks] = temp['Brand_y'][indeks]
        data_bundle['Sub Brand'][indeks] = temp['Sub Brand_y'][indeks]
        data_bundle['Parent Item'][indeks] = temp['Parent Item_y'][indeks]
        data_bundle['Parent SKU'][indeks] = temp['Parent SKU_y'][indeks]

        print("Pricing")
        order_all = order_all.append(data_bundle, ignore_index = True, sort = False)

        colname = temp.columns[temp.columns.get_loc('Produk 1') : temp.columns.get_loc('Harga Cost 7') + 1]
        colname_str = [x for x in colname if 'Subtotal' not in x and 'Harga' not in x]
        colname_int = [x for x in colname if x not in colname_str]

        for i in colname_str:
            temp[i] = np.nan

        for i in colname_int:
            temp[i] = 0

        data_order = data_order.append(temp , ignore_index = True, sort = False)


        order_all = order_all.merge(data_SKU2[['SKU', 'Price List NFI', 'Harga Cost']].drop_duplicates('SKU'), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(order_all.index)
        order_all['Price List NFI_x'] = order_all['Price List NFI_x'].fillna(order_all['Price List NFI_y'])
        order_all =  order_all.drop(['Price List NFI_y', 'SKU_y'], axis = 1)
        order_all = order_all.rename(columns = {'SKU_x' : 'SKU', 'Price List NFI_x' : 'Price List NFI'})

        indeks = order_all[order_all['Product Name'] == 'HiLo Teen Chocolate 250gr'].index.to_list()
        order_all['Real Nama Produk'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Parent Item'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Brand'][indeks] = 'HiLo'
        order_all['Sub Brand'][indeks] = 'HILO TEEN'
        order_all['Price List NFI'][indeks] = 36850
        order_all['Harga Cost'][indeks] = 36850
        order_all['Real SKU'][indeks] = '2101651155'
        order_all['Parent SKU'][indeks] = '2101651155'


        order_all['Price List NFI'] = pd.to_numeric(order_all['Price List NFI']).astype(int)
        order_all['Harga Cost'] = pd.to_numeric(order_all['Harga Cost']).astype(int)
        order_all['Qty. Invoiced'] = pd.to_numeric(order_all['Qty. Invoiced']).astype(int)

        order_all['Total Net'] = order_all['Price List NFI'] * order_all['Qty. Invoiced']
        order_all['Total Harga Cost'] = order_all['Harga Cost'] * order_all['Qty. Invoiced']
        order_all['Subtotal'] = order_all['Selling Price'] * order_all['Qty. Invoiced']
        order_all['Total'] = order_all['Selling Price'] * order_all['Qty. Invoiced']

        order_all = order_all.reset_index(drop = True)
        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        order_all['Seller Discount'] = order_all['discount']
        order_all['Shipping'] = order_all['Shipping Cost']

        temp = order_all[order_all['Brand'] != 'Bundle']
        temp['discount'] = temp['discount'] * temp['Total Harga Cost']/temp.groupby(['Order #'])['Total Harga Cost'].transform('sum')
        order_all['discount'][temp.index] = temp['discount']

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Subtotal', 'Total']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Total'][list_nobundle.index] = list_nobundle['Total_y']
        order_all['Subtotal'][list_nobundle.index] = list_nobundle['Subtotal_y']

        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Subtotal'] = temp['Subtotal'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        temp['Selling Price'] = temp['Subtotal']/temp['Qty. Invoiced']
        temp['Total'] = temp['Total'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Total'][temp.index] = temp['Total']
        order_all['Subtotal'][temp.index] = temp['Subtotal']
        order_all['Selling Price'][temp.index] = temp['Selling Price']


        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Seller Discount']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Seller Discount'][list_nobundle.index] = list_nobundle['Seller Discount_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Seller Discount'] = temp['Seller Discount'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Seller Discount'][temp.index] = temp['Seller Discount']


        temp = order_all[order_all['Bundle Name'].isnull()]
        temp_group = temp[['Order #','Shipping']].groupby(['Order #']).sum().reset_index()

        temp = order_all.merge(temp_group, how = 'left', on = 'Order #').set_index(order_all.index)
        temp['Shipping_x'] = temp['Shipping_y'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Shipping'][temp.index] = temp['Shipping_y']
        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Shipping']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Shipping'][list_nobundle.index] = list_nobundle['Shipping_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Shipping'] = temp['Shipping'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Shipping'][temp.index] = temp['Shipping']
        order_all['True datetime'] = pd.to_datetime(order_all['True datetime'])
        order_all['Promo'] = np.nan
        order_all['Discount MC'] = np.nan


        order_all['Warehouse Name'] = 'Primary Warehouse'
        order_all['Store'] = 'Order Online'

        order_all['Customer Email'] = order_all['email']
        order_all['Order Status'] = order_all['status']
        order_all['Payment Channel'] = order_all['payment_method']
        order_all['Coupon Code'] = order_all['coupon']
        order_all.columns.to_list()

        order_all_append = order_all[['Sales Order ID', 'Store',
            'Product Name',
            'Customer Name',
            'Phone',
            'Address',
            'Region',
            'City',
            'Zip Code',
            'payment_status',
            'Regular Price',
            'Shipping Courier',
            'Shipping Cost',
            'Subtotal',
            'Qty. Invoiced',
            'SKU',
            'Order #',
            'Invoice Number',
            'Shipping Name',
            'Shipping Address2',
            'Country',
            'AWB',
            'Channel',
            'Order date',
            'Real SKU',
            'Real Nama Produk',
            'Brand',
            'Sub Brand',
            'Parent Item',
            'Parent SKU',
            'Produk 1',
            'SKU Produk 1',
            'PCS Produk 1',
            'Price List NFI 1',
            'Subtotal Produk 1',
            'Harga Display 1',
            'Harga Cost 1',
            'Harga Organik 1',
            'Produk 2',
            'SKU Produk 2',
            'PCS Produk 2',
            'Price List NFI 2',
            'Subtotal Produk 2',
            'Harga Display 2',
            'Harga Cost 2',
            'Harga Organik 2',
            'Produk 3',
            'SKU Produk 3',
            'PCS Produk 3',
            'Price List NFI 3',
            'Subtotal Produk 3',
            'Harga Display 3',
            'Harga Cost 3',
            'Harga Organik 3',
            'Produk 4',
            'SKU Produk 4',
            'PCS Produk 4',
            'Price List NFI 4',
            'Subtotal Produk 4',
            'Harga Display 4',
            'Harga Cost 4',
            'Harga Organik 4',
            'Produk 5',
            'SKU Produk 5',
            'PCS Produk 5',
            'Price List NFI 5',
            'Subtotal Produk 5',
            'Harga Display 5',
            'Harga Cost 5',
            'Harga Organik 5',
            'Produk 6',
            'SKU Produk 6',
            'PCS Produk 6',
            'Price List NFI 6',
            'Subtotal Produk 6',
            'Harga Display 6',
            'Harga Cost 6',
            'Harga Organik 6',
            'Produk 7',
            'SKU Produk 7',
            'PCS Produk 7',
            'Price List NFI 7',
            'Subtotal Produk 7',
            'Harga Display 7',
            'Harga Cost 7',
            'Harga Organik 7',
            'Bundle Flag',
            'Date',
            'Month',
            'Year',
            'Quarter',
            'Week',
            'True datetime',
            'Total',
            'Price List NFI',
            'Total Net',
            'Selling Price',
            'Kecamatan',
            'Kelurahan',
            'Bundle Name',
            'Harga Cost',
            'Total Harga Cost',
            'Seller Discount',
            'Shipping',
            'Promo',
            'Discount MC',
            'Warehouse Name',
            'Customer Email',
            'Order Status',
            'Payment Channel',
            'Coupon Code']]

        data_all = data_all[~data_all['Order #'].astype(str).isin(order_all_append['Order #'].astype(str))]
        data_all = data_all.append(order_all_append, ignore_index = True, sort = False)
#         data_all_aft_order = data_all.copy()
        del file_name
        order_online_lampung = True
        

        
if not order_online_pekanbaru:
                            
    import pandas as pd
    import numpy as np
    import requests
    import os

    print('order_online_pekanbaru')
    
    before = os.listdir(os.getcwd() + '/Input Data')

    options = Options()
    options.add_experimental_option("prefs", {
            "download.default_directory": os.path.abspath("Input Data"),
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            "safebrowsing.enabled": False
    })

    driver = webdriver.Chrome(options=options)
    driver.fullscreen_window()
    driver.get("https://app.orderonline.id")

    username = driver.find_element_by_name("email")
    username.clear()
    username.send_keys("customerpekanbaru@nutrimart.co.id")

    password = driver.find_element_by_name("password")
    password.send_keys("pekanbaruhomdel1")
    password.click()

    driver.find_element_by_class_name("btn-submit").click()
    time.sleep(15)
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="main-nav-dropdown"]/ul/li[3]/a'))).click()
    time.sleep(5)
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[1]/div/div[2]/div/div/div').click()
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="app"]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[1]/div[1]/ul/li[6]'))).click()
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[2]/button[2]').click()
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[3]/div/button').click()
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[3]/div/div/button[1]').click()

    time.sleep(25)

    after = os.listdir(os.getcwd() + '/Input Data')
    change = set(after) - set(before)
    if len(change) == 1:
        file_name = change.pop()
    elif len(change) == 0: 
        print("No file downloaded")
    else :
        print("More than one file downloaded")

    data_order = pd.read_csv(r'Input Data/' + str(file_name))
    driver.close()


    data_order['order_id'] = data_order['order_id'].fillna(method='ffill')
    data_order = data_order.drop('quantity', axis = 1)
    temp = data_order.drop_duplicates('order_id').drop('product', axis = 1)
    data_order = data_order[['order_id', 'product']]
    data_order = data_order.merge(temp, how = 'left', on = 'order_id')
    data_order['order_id'] = data_order['order_id'].astype(int)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Shirataki Noodles 71gr (x2) ', 'Twin Pack: Tropicana Slim Shirataki Noodles 71gr ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Saus Tiram 200ml (x2) ', 'Twin Pack: Tropicana Slim Saus Tiram 200ml ', regex = False)
    data_order.loc[data_order['product']=='HILO TEEN STRAWBERRY MILKSHAKE\xa0 12Dx500G','product']="HiLo Teen Strawberry Milkshake 500gr"
    
    product = data_order['product'].str.split("\(x",1, expand = True)

    product.loc[product[0]=="HiLo Teen Taro 500gr",1]="1)"
    data_order['Quantity'] = product[1].astype(str).str.replace(')', '', regex = False).astype(int)
    data_order['product'] = product[0].str.strip().str.replace('  ', ' ').str.replace('6 SCH', '6sch').str.replace("W'Dank", "W'dank").str.replace('L-men', 'L-Men').str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("school", "School").str.replace("Rtd", "RTD").str.replace('Honey 50 sachet', 'Honey (50sch)').str.replace('Teen Coklat', 'Teen Chocolate').str.replace('40 sch', '40sch')

    data_order['product'] = data_order['product'].str.replace("HiLo Thai Tea \(10sch\)$", 'HiLo Thai Tea (10 sch)', regex = True)

    data_SKU = pd.read_excel(r'Order Online\SKU buat andra updated maret 2021.xlsx')
    data_SKU = data_SKU.rename(columns = {'Product' : 'product', 'PL NFI' : 'Price List NFI'})
    data_SKU['SKU'] = data_SKU['SKU'].astype(str).str.replace('.0', '', regex = False)
    data_SKU = data_SKU[data_SKU['SKU'] != 'nan'][data_SKU[data_SKU['SKU'] != 'nan']['SKU'] != '0']
    data_SKU['product'] = data_SKU['product'].str.strip().str.replace('L-men', 'L-Men').str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("school", "School").str.replace("Rtd", "RTD").str.replace('Honey 50 sachet', 'Honey (50sch)').str.replace('Teen Coklat', 'Teen Chocolate').str.replace('40 sch', '40sch')


    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Kecap Manis 200ml'].copy()
    price['product'] = 'Tropicana Slim Kecap Manis 200m'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Diabtx (50 sch)'].copy()
    price['product'] = 'Tropicana Slim Diabtx 50 sch'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Hokkaido Cheese 100gr'].copy()
    price['product'] = 'Tropicana Slim Hokaido Cheese 100gr'


    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
   
    data_SKU = data_SKU.append(pd.DataFrame([['L-Men Bar Crunchy Chocolate 12sch', 5500, 5500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
  
    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Mangga Gandaria', 45000, 45000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
  
    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Teen Vanilla Caramel 500gr', 71500, 71500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Paket Bundle HiLo Renceng (Hilo Chocolate Banana (10 sch) + Hilo Chocolate Taro (10 sch) + HiLo Thai Tea (10sch))', 35200, 35200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Berondong 10's", 15000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Kecap Asin 200 ml", 28500, 26000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim DIABTX (100 sch)", 87700, 75000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Teen Chocolate 750gr", 117800, 104000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Paket Ngopi Lokalate", 136000, 109200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Hi Protein 2 Go Chocolate (24 TETRAPAK)", 240000, 238000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 Tropicana Slim Goldenmil Vanilla (6sch)", 39160, 31000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Kawista", 17500, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["Paket Bundle HiLo (HiLo Active Chocolate 500gr + HiLo Teen Yoghurt Banana 250gr)", 122900, 101850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Strawberry Jam 375gr", 72600, 58500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Crunch BBQ Beef (20gr)", 13500, 10500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Cocopandan 40 sch", 52500, 52500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 - W'dank Empon Empon 10 Sachet Renceng", 35000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Semangka 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Nanas 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["W'Dank Empon-Empon 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Milk Skim Fiber Pro Plain 500gr", 135000, 106700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Es Teler 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: HiLo Es Teler 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Hilo Es Ketan Hitam 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Triple Pack: Tropicana Slim Korean Garlic Butter Cookies (5 Sch)", 73500, 52000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Avocado Coffee 4 Sch", 21200, 12100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Sambal Terasi 200 gr", 35600, 29700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Tropicana Slim Avocado Coffee 4 sch x 2", 42400, 24200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Bar Chocolate (12 Sch) + L-Men Protein Crunch BBQ Beef x 2", 159000, 107800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Buy 5 Get 5 L-Men Protein Crunch BBQ Beef (20gr)", 130000, 67500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Active Ketan Hitam 175gr', 48500, 20700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Es Ketan Hitam 10 sch', 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Sweetener Lemongrass Pandan 50 sch', 44200, 28900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 1 Get 1 FREE: Lokalate Kopi Berondong (10 sch)', 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) - HiLo School Cotton Candy 500g', 38850, 38850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) HiLo Active Chocolate 1000 gr', 61500, 61500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Sweetener Jahe 50 sch', 44600, 22300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 12 Get 12 - HiLo Chocolate Avocado Ready to Drink 200ml - Minuman Siap Minum Tinggi Kalsium', 152400, 84000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)   
    
    data_order['product'] = data_order['product'].astype(str)
    data_SKU['product'] = data_SKU['product'].astype(str)

    data_order['product'] = data_order['product'].str.replace('L Men Gain Mass Chocolate 500 gr', 'L-Men Gain Mass Chocolate 500 gr')
    data_order.loc[data_order['product']=='HILO TEEN STRAWBERRY MILKSHAKE\xa0 12Dx500G','product']="HiLo Teen Strawberry Milkshake 500gr"



    data_order = data_order.merge(data_SKU[['SKU', 'product', 'Harga Display', 'Harga Coret']].drop_duplicates('product'), how = 'left', on = 'product')

    data_order = data_order.reset_index(drop = True)

    indeks = data_order[data_order['product'] == "Lokalate Kopi Berondong 10's"].index.to_list()
    data_order['Harga Display'][indeks] = 15000
    data_order['Harga Coret'][indeks] = 15000
    indeks = data_order[data_order['SKU'].isnull()].index.to_list()
    for i in indeks:
        col = [x for x in data_SKU.columns if 'Alias Nama' in x]
        for j in col:
            if data_order['product'][i] in data_SKU[j].astype(str).values:
                SKU = data_SKU[data_SKU[j].astype(str) == data_order['product'][i]]['SKU'].values[0]
                data_order['SKU'][i] = SKU

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    data_SKU2 = pd.read_excel(r'SKU_File\data_SKU.xlsx')
    data_SKU2['Nama Produk'] = data_SKU2['Nama Produk'].astype(str)



    for i in indeks:
        if str(data_order['product'][i]).lower() in data_SKU2['Nama Produk'].astype(str).str.lower().values:
            data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2['Nama Produk'].astype(str).str.lower()].values[0]

    list_alias_name = [x for x in data_SKU2.columns if 'Alias Nama' in x]

    for i in indeks:
        for j in list_alias_name:
            if str(data_order['product'][i]).lower() in data_SKU2[j].astype(str).str.lower().values:
                data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2[j].astype(str).str.lower()].values[0]

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    if len(indeks) != 0:
        print('Alert SKU Missing')
        data_order['product'][indeks].drop_duplicates().to_excel('Alert SKU Missing.xlsx', index = False)
    else :
        data_order['phone'] = data_order['phone'].astype(str).str.replace('+628', '08', regex = False)

        data_order['zip'] = data_order['zip'].replace('.0', '', regex = False)

        data_order = data_order.rename(columns = {'order_id' : 'Sales Order ID', 'name' : 'Customer Name', 'product' : 'Item Name', 'price' : 'Price', 'shipping_cost' : 'Shipping Cost', 'address' : 'Shipping Address1', 'city' : 'Shipping City', 'zip' : 'Shipping Zip', 'province' : 'Shipping Province', 'phone' : 'Shipping Phone', 'courier' : 'Shipping Courier'})
        data_order['Channel Order ID'] = data_order['Sales Order ID']
        data_order['Invoice Number'] = data_order['Sales Order ID']
        data_order['Shipping Name'] = data_order['Customer Name']
        data_order['Shipping Address2'] = 0
        data_order['Shipping Country'] = 'Indonesia'
        data_order['AWB'] = 0
        data_order['Channel'] = 'Order Online Pekanbaru'

        data_order['Order Date'] = pd.to_datetime(data_order['created_at'])

        data_order['SKU'] = data_order['SKU'].astype(str)
        data_order['Item Name'] = data_order['Item Name'].astype(str)
        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str).str.replace('(S)', '', regex = False)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        index = data_order[data_order['SKU'].astype(str) == '2306551174'].index.to_list()
        data_order['SKU'][index] = '2306592173'

        data_order = data_order.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('hd','', regex = False)
        temp['SKU'] = temp['SKU'].astype(str).str.replace('HD','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        data_order['Real SKU'] = data_order['Real SKU'].astype(str)
        data_order = data_order.merge(data_SKU2[['SKU', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Unbundling ====== 6/10")        
        # Forstok Unbundling    
        list_col = ['SKU'] + data_SKU2.columns[data_SKU2.columns.get_loc('Produk 1'):data_SKU2.columns.get_loc('Harga Organik 7')+1].to_list()
        data_order = data_order.merge(data_SKU2[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        list_pcs = [x for x in data_order.columns if 'PCS' in x]
        for i in list_pcs:
            data_order[i] = data_order[i] * data_order['Quantity']
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        indeks = data_order[data_order['Brand'] == 'Bundle'].index.to_list()
        data_order['Bundle Flag'] = np.nan
        data_order['Bundle Flag'][indeks] = 'Bundle'

        indeks = data_order[data_order['Brand'] == 'Bundle'][data_order[data_order['Brand'] == 'Bundle']['SKU'].astype(str).str.contains('(S)', regex = False)].index.to_list()
        data_order['SKU Produk 1'][indeks] = '(S)' + data_order['SKU Produk 1'][indeks].astype(str)
        data_order['SKU Produk 2'][indeks] = '(S)' + data_order['SKU Produk 2'][indeks].astype(str)
        data_order['SKU Produk 3'][indeks] = '(S)' + data_order['SKU Produk 3'][indeks].astype(str)
        data_order['SKU Produk 4'][indeks] = '(S)' + data_order['SKU Produk 4'][indeks].astype(str)
        data_order['SKU Produk 5'][indeks] = '(S)' + data_order['SKU Produk 5'][indeks].astype(str)
        data_order['SKU Produk 6'][indeks] = '(S)' + data_order['SKU Produk 6'][indeks].astype(str)
        data_order['SKU Produk 7'][indeks] = '(S)' + data_order['SKU Produk 7'][indeks].astype(str)

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Filling Date ====== 7/10")
        data_order['Date'] = np.nan
        data_order['Month'] = np.nan
        data_order['Year'] = np.nan

        for i in range(data_order.shape[0]):
            if int(data_order['Order Date'][i].strftime('%d')) <= 12:
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).year
            else :
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i]).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i]).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i]).year

        quarter = pd.DataFrame([['January', 1], ['February', 1], ['March', 1], ['April', 2], ['May', 2], ['June', 2], 
                ['July', 3], ['August', 3], ['September', 3],['October', 4], ['November', 4], ['December', 4]], columns = ['Bulan', 'Quarter'])
        data_order = data_order.merge(quarter, how = 'left', left_on = 'Month', right_on = 'Bulan')
        data_order = data_order.drop(['Bulan'], axis = 1)
        data_bulan = pd.DataFrame([{'Bulan' : 'December', 'Number' : 12} ,
                {'Bulan' : 'January' , 'Number': 1},
                {'Bulan' : 'February' , 'Number': 2},
                {'Bulan' : 'March' , 'Number': 3},
                {'Bulan' : 'April' , 'Number': 4},
                {'Bulan' : 'May' , 'Number': 5},
                {'Bulan' : 'June', 'Number': 6},
                {'Bulan' : 'July' , 'Number': 7},
                {'Bulan' : 'August', 'Number' : 8},
                {'Bulan' : 'September', 'Number' : 9},
                {'Bulan' : 'October' , 'Number': 10},
                {'Bulan' : 'November' , 'Number': 11}])
        temp = data_order.copy()
        temp['Day'] = temp['Date']
        temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
        temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
        data_order['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.week
        temp['Hour'] = pd.to_datetime(data_order['Order Date']).dt.hour
        temp['Minute'] = pd.to_datetime(data_order['Order Date']).dt.minute
        temp['Second'] = pd.to_datetime(data_order['Order Date']).dt.second
        data_order['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']])



        order_all = data_order.copy()
        order_all['Total'] = order_all['net_revenue']
        order_all['Price List NFI'] = np.nan
        order_all['Total Net'] = np.nan



        order_all = order_all.rename(columns={'Channel Order ID' : 'Order #',
                                                'Status' : 'Order Status',
                                                'Order Date' : 'Order date',
                                                'Item Name' :'Product Name',
                                                'Bundle Name' : 'Bundle',
                                                'Shipping Country' : 'Country',
                                                'Shipping Province' : 'Region',
                                                'Shipping City' : 'City',
                                                'Shipping Zip' : 'Zip Code',
                                                'Shipping Address1' : 'Address',
                                                'Shipping Phone' : 'Phone',
                                                'Quantity' : 'Qty. Invoiced',
                                                'Harga Display' : 'Regular Price',
                                                'net_revenue' : 'Subtotal'})
        
        order_all = order_all.reset_index(drop = True)

        order_all['Selling Price'] = order_all['Harga Coret'].astype(int)
        order_all['Kecamatan'] = np.nan
        order_all['Kelurahan'] = np.nan

        print("Filling Location")
        indeks = order_all[order_all['City'].astype(str).str.contains('/')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains('-')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[0]

        indeks = order_all[order_all['City'].astype(str).str.contains(',')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains(',')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[0]

        order_all['City'] = order_all['City'].astype(str).str.replace('Kab\.', 'Kabupaten' ,case = False)

        master_map = pd.read_csv(r'All Data/Province.csv', names = ['Kode Prov', 'Province'], header= 0)
        master_map2 = pd.read_csv(r'All Data/City.csv', names = ['Kode City', 'Kode Prov', 'City'], header = 0)
        master_map = master_map.merge(master_map2, how = 'right', on = 'Kode Prov')
        master_map['Kode Prov'][515] = 14
        master_map['Province'][515] = 'Riau'
        master_map['Kode Prov'] = master_map['Kode Prov'].astype(int)
        master_map['Province'] = master_map['Province'].str.title()
        master_map['City'] = master_map['City'].str.title()

        city = pd.read_excel(r'All Data/list_city.xlsx')
        temp = order_all.copy()
        temp['City'] = temp['City'].astype(str).str.lower()
        temp['City'] = temp['City'].astype(str).str.replace('kab. ', 'kabupaten ', regex = False, case = False)
        city['All City'] = city['All City'].astype(str).str.lower()
        temp = temp.merge(city.drop_duplicates('All City'), how = 'left', left_on = 'City', right_on = 'All City').set_index(temp.index)
        indeks = temp[temp['Real City'].notnull()].index.to_list()
        order_all['City'][indeks] = temp['Real City'][indeks]

        province = pd.read_excel(r'All Data/list_province.xlsx')
        temp = order_all.copy()
        temp['Region'] = temp['Region'].astype(str).str.lower()
        province['All Province'] = province['All Province'].astype(str).str.lower()
        temp = temp.merge(province.drop_duplicates('All Province'), how = 'left', left_on = 'Region', right_on = 'All Province').set_index(temp.index)
        indeks = temp[temp['Real Province'].notnull()].index.to_list()
        order_all['Region'][indeks] = temp['Real Province'][indeks]

        temp = order_all.copy()
        temp = temp[temp['Region'].isnull()]
        temp['Region'] = temp.merge(master_map, how = 'left', on = 'City').set_index(temp.index)['Province']
        order_all['Region'][temp.index] = temp['Region']  

        district = pd.read_excel(r'All Data/list_district.xlsx')
        temp = order_all.copy()
        temp['Kecamatan'] = temp['Kecamatan'].astype(str).str.lower()
        district['All District'] = district['All District'].astype(str).str.lower()
        temp = temp.merge(district.drop_duplicates('All District'), how = 'left', left_on = 'Kecamatan', right_on = 'All District').set_index(temp.index)
        indeks = temp[temp['Real District'].notnull()].index.to_list()
        order_all['Kecamatan'][indeks] = temp['Real District'][indeks]

        temp = order_all.copy()
        temp2 = temp[['Region', 'City', 'Kecamatan']].merge(master_map, how = 'left', on = 'City')
        indeks = temp2[temp2['Region'] != temp2['Province']][temp2[temp2['Region'] != temp2['Province']]['City'].notnull()].index.to_list()
        order_all['City'][indeks] = np.nan

        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        print("Unbundling")
        data_bundle1 = order_all[~order_all['Produk 1'].isnull()]
        data_bundle1['Bundle Name'] = data_bundle1['Product Name']
        data_bundle1['Product Name'] = data_bundle1['Produk 1']
        data_bundle1['SKU'] = data_bundle1['SKU Produk 1']
        data_bundle1['Qty. Invoiced'] = data_bundle1['PCS Produk 1']
        data_bundle1['Price List NFI'] = data_bundle1['Price List NFI 1']
        data_bundle1['Total Net'] = data_bundle1['Price List NFI 1']
        data_bundle1['Bundle Flag'] = np.nan

        data_bundle2 = order_all[~order_all['Produk 2'].isnull()]
        data_bundle2['Bundle Name'] = data_bundle2['Product Name']
        data_bundle2['Product Name'] = data_bundle2['Produk 2']
        data_bundle2['SKU'] = data_bundle2['SKU Produk 2']
        data_bundle2['Qty. Invoiced'] = data_bundle2['PCS Produk 2']
        data_bundle2['Price List NFI'] = data_bundle2['Price List NFI 2']
        data_bundle2['Total Net'] = data_bundle2['Price List NFI 2'] 
        data_bundle2['Bundle Flag'] = np.nan

        data_bundle3 = order_all[~order_all['Produk 3'].isnull()]
        data_bundle3['Bundle Name'] = data_bundle3['Product Name']
        data_bundle3['Product Name'] = data_bundle3['Produk 3']
        data_bundle3['SKU'] = data_bundle3['SKU Produk 3']
        data_bundle3['Qty. Invoiced'] = data_bundle3['PCS Produk 3']
        data_bundle3['Price List NFI'] = data_bundle3['Price List NFI 3']
        data_bundle3['Total Net'] = data_bundle3['Price List NFI 3'] 
        data_bundle3['Bundle Flag'] = np.nan

        data_bundle4 = order_all[~order_all['Produk 4'].isnull()]
        data_bundle4['Bundle Name'] = data_bundle4['Product Name']
        data_bundle4['Product Name'] = data_bundle4['Produk 4']
        data_bundle4['SKU'] = data_bundle4['SKU Produk 4']
        data_bundle4['Qty. Invoiced'] = data_bundle4['PCS Produk 4']
        data_bundle4['Price List NFI'] = data_bundle4['Price List NFI 4']
        data_bundle4['Total Net'] = data_bundle4['Price List NFI 4'] 
        data_bundle4['Bundle Flag'] = np.nan

        data_bundle5 = order_all[~order_all['Produk 5'].isnull()]
        data_bundle5['Bundle Name'] = data_bundle5['Product Name']
        data_bundle5['Product Name'] = data_bundle5['Produk 5']
        data_bundle5['SKU'] = data_bundle5['SKU Produk 5']
        data_bundle5['Qty. Invoiced'] = data_bundle5['PCS Produk 5']
        data_bundle5['Price List NFI'] = data_bundle5['Price List NFI 5']
        data_bundle5['Total Net'] = data_bundle5['Price List NFI 5']
        data_bundle5['Bundle Flag'] = np.nan

        data_bundle6 = order_all[~order_all['Produk 6'].isnull()]
        data_bundle6['Bundle Name'] = data_bundle6['Product Name']
        data_bundle6['Product Name'] = data_bundle6['Produk 6']
        data_bundle6['SKU'] = data_bundle6['SKU Produk 6']
        data_bundle6['Qty. Invoiced'] = data_bundle6['PCS Produk 6']
        data_bundle6['Price List NFI'] = data_bundle6['Price List NFI 6']
        data_bundle6['Total Net'] = data_bundle6['Price List NFI 6']
        data_bundle6['Bundle Flag'] = np.nan

        data_bundle7 = order_all[~order_all['Produk 7'].isnull()]
        data_bundle7['Bundle Name'] = data_bundle7['Product Name']
        data_bundle7['Product Name'] = data_bundle7['Produk 7']
        data_bundle7['SKU'] = data_bundle7['SKU Produk 7']
        data_bundle7['Qty. Invoiced'] = data_bundle7['PCS Produk 7']
        data_bundle7['Price List NFI'] = data_bundle7['Price List NFI 7']
        data_bundle7['Total Net'] = data_bundle7['Price List NFI 7']
        data_bundle7['Bundle Flag'] = np.nan

        data_bundle = data_bundle1.append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)
        data_bundle['SKU'] = data_bundle['SKU'].astype(str)
        data_bundle['SKU'] = data_bundle['SKU'].str.replace('\.0$', '', regex = True)
        data_bundle[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']] = data_bundle.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item_y', 'Parent SKU_y']]

        temp = data_bundle[data_bundle['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)

        indeks = data_bundle[data_bundle['Real SKU'].isnull()].index.to_list()
        data_bundle['Real SKU'][indeks] = temp['Real SKU_y'][indeks]
        data_bundle['Real Nama Produk'][indeks] = temp['Nama Produk'][indeks]
        data_bundle['Brand'][indeks] = temp['Brand_y'][indeks]
        data_bundle['Sub Brand'][indeks] = temp['Sub Brand_y'][indeks]
        data_bundle['Parent Item'][indeks] = temp['Parent Item_y'][indeks]
        data_bundle['Parent SKU'][indeks] = temp['Parent SKU_y'][indeks]

        print("Pricing")
        order_all = order_all.append(data_bundle, ignore_index = True, sort = False)

        colname = temp.columns[temp.columns.get_loc('Produk 1') : temp.columns.get_loc('Harga Cost 7') + 1]
        colname_str = [x for x in colname if 'Subtotal' not in x and 'Harga' not in x]
        colname_int = [x for x in colname if x not in colname_str]

        for i in colname_str:
            temp[i] = np.nan

        for i in colname_int:
            temp[i] = 0

        data_order = data_order.append(temp , ignore_index = True, sort = False)


        order_all = order_all.merge(data_SKU2[['SKU', 'Price List NFI', 'Harga Cost']].drop_duplicates('SKU'), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(order_all.index)
        order_all['Price List NFI_x'] = order_all['Price List NFI_x'].fillna(order_all['Price List NFI_y'])
        order_all =  order_all.drop(['Price List NFI_y', 'SKU_y'], axis = 1)
        order_all = order_all.rename(columns = {'SKU_x' : 'SKU', 'Price List NFI_x' : 'Price List NFI'})

        indeks = order_all[order_all['Product Name'] == 'HiLo Teen Chocolate 250gr'].index.to_list()
        order_all['Real Nama Produk'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Parent Item'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Brand'][indeks] = 'HiLo'
        order_all['Sub Brand'][indeks] = 'HILO TEEN'
        order_all['Price List NFI'][indeks] = 36850
        order_all['Harga Cost'][indeks] = 36850
        order_all['Real SKU'][indeks] = '2101651155'
        order_all['Parent SKU'][indeks] = '2101651155'


        order_all['Price List NFI'] = pd.to_numeric(order_all['Price List NFI']).astype(int)
        order_all['Harga Cost'] = pd.to_numeric(order_all['Harga Cost']).astype(int)
        order_all['Qty. Invoiced'] = pd.to_numeric(order_all['Qty. Invoiced']).astype(int)

        order_all['Total Net'] = order_all['Price List NFI'] * order_all['Qty. Invoiced']
        order_all['Total Harga Cost'] = order_all['Harga Cost'] * order_all['Qty. Invoiced']
        order_all['Subtotal'] = order_all['Selling Price'] * order_all['Qty. Invoiced']
        order_all['Total'] = order_all['Selling Price'] * order_all['Qty. Invoiced']

        order_all = order_all.reset_index(drop = True)
        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        order_all['Seller Discount'] = order_all['discount']
        order_all['Shipping'] = order_all['Shipping Cost']

        temp = order_all[order_all['Brand'] != 'Bundle']
        temp['discount'] = temp['discount'] * temp['Total Harga Cost']/temp.groupby(['Order #'])['Total Harga Cost'].transform('sum')
        order_all['discount'][temp.index] = temp['discount']

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Subtotal', 'Total']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Total'][list_nobundle.index] = list_nobundle['Total_y']
        order_all['Subtotal'][list_nobundle.index] = list_nobundle['Subtotal_y']

        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Subtotal'] = temp['Subtotal'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        temp['Selling Price'] = temp['Subtotal']/temp['Qty. Invoiced']
        temp['Total'] = temp['Total'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Total'][temp.index] = temp['Total']
        order_all['Subtotal'][temp.index] = temp['Subtotal']
        order_all['Selling Price'][temp.index] = temp['Selling Price']


        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Seller Discount']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Seller Discount'][list_nobundle.index] = list_nobundle['Seller Discount_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Seller Discount'] = temp['Seller Discount'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Seller Discount'][temp.index] = temp['Seller Discount']


        temp = order_all[order_all['Bundle Name'].isnull()]
        temp_group = temp[['Order #','Shipping']].groupby(['Order #']).sum().reset_index()

        temp = order_all.merge(temp_group, how = 'left', on = 'Order #').set_index(order_all.index)
        temp['Shipping_x'] = temp['Shipping_y'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Shipping'][temp.index] = temp['Shipping_y']
        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Shipping']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Shipping'][list_nobundle.index] = list_nobundle['Shipping_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Shipping'] = temp['Shipping'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Shipping'][temp.index] = temp['Shipping']
        order_all['True datetime'] = pd.to_datetime(order_all['True datetime'])
        order_all['Promo'] = np.nan
        order_all['Discount MC'] = np.nan


        order_all['Warehouse Name'] = 'Primary Warehouse'
        order_all['Store'] = 'Order Online'

        order_all['Customer Email'] = order_all['email']
        order_all['Order Status'] = order_all['status']
        order_all['Payment Channel'] = order_all['payment_method']
        order_all['Coupon Code'] = order_all['coupon']
        order_all.columns.to_list()

        order_all_append = order_all[['Sales Order ID', 'Store',
            'Product Name',
            'Customer Name',
            'Phone',
            'Address',
            'Region',
            'City',
            'Zip Code',
            'payment_status',
            'Regular Price',
            'Shipping Courier',
            'Shipping Cost',
            'Subtotal',
            'Qty. Invoiced',
            'SKU',
            'Order #',
            'Invoice Number',
            'Shipping Name',
            'Shipping Address2',
            'Country',
            'AWB',
            'Channel',
            'Order date',
            'Real SKU',
            'Real Nama Produk',
            'Brand',
            'Sub Brand',
            'Parent Item',
            'Parent SKU',
            'Produk 1',
            'SKU Produk 1',
            'PCS Produk 1',
            'Price List NFI 1',
            'Subtotal Produk 1',
            'Harga Display 1',
            'Harga Cost 1',
            'Harga Organik 1',
            'Produk 2',
            'SKU Produk 2',
            'PCS Produk 2',
            'Price List NFI 2',
            'Subtotal Produk 2',
            'Harga Display 2',
            'Harga Cost 2',
            'Harga Organik 2',
            'Produk 3',
            'SKU Produk 3',
            'PCS Produk 3',
            'Price List NFI 3',
            'Subtotal Produk 3',
            'Harga Display 3',
            'Harga Cost 3',
            'Harga Organik 3',
            'Produk 4',
            'SKU Produk 4',
            'PCS Produk 4',
            'Price List NFI 4',
            'Subtotal Produk 4',
            'Harga Display 4',
            'Harga Cost 4',
            'Harga Organik 4',
            'Produk 5',
            'SKU Produk 5',
            'PCS Produk 5',
            'Price List NFI 5',
            'Subtotal Produk 5',
            'Harga Display 5',
            'Harga Cost 5',
            'Harga Organik 5',
            'Produk 6',
            'SKU Produk 6',
            'PCS Produk 6',
            'Price List NFI 6',
            'Subtotal Produk 6',
            'Harga Display 6',
            'Harga Cost 6',
            'Harga Organik 6',
            'Produk 7',
            'SKU Produk 7',
            'PCS Produk 7',
            'Price List NFI 7',
            'Subtotal Produk 7',
            'Harga Display 7',
            'Harga Cost 7',
            'Harga Organik 7',
            'Bundle Flag',
            'Date',
            'Month',
            'Year',
            'Quarter',
            'Week',
            'True datetime',
            'Total',
            'Price List NFI',
            'Total Net',
            'Selling Price',
            'Kecamatan',
            'Kelurahan',
            'Bundle Name',
            'Harga Cost',
            'Total Harga Cost',
            'Seller Discount',
            'Shipping',
            'Promo',
            'Discount MC',
            'Warehouse Name',
            'Customer Email',
            'Order Status',
            'Payment Channel',
            'Coupon Code']]

        data_all = data_all[~data_all['Order #'].astype(str).isin(order_all_append['Order #'].astype(str))]
        data_all = data_all.append(order_all_append, ignore_index = True, sort = False)

        del file_name
        order_online_pekanbaru = True

if not order_online_banjarmasin:

    import pandas as pd
    import numpy as np
    import requests
    import os

    print('order_online_banjarmasin')

    before = os.listdir(os.getcwd() + '/Input Data')

    options = Options()
    options.add_experimental_option("prefs", {
            "download.default_directory": os.path.abspath("Input Data"),
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            "safebrowsing.enabled": False
    })

    driver = webdriver.Chrome(options=options)
    driver.fullscreen_window()
    driver.get("https://app.orderonline.id")

    username = driver.find_element_by_name("email")
    username.clear()
    username.send_keys("banjarmasinpromosi.nhd2022@gmail.com")

    password = driver.find_element_by_name("password")
    password.send_keys("nhdcuan")
    password.click()

    driver.find_element_by_class_name("btn-submit").click()
    time.sleep(15)
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="main-nav-dropdown"]/ul/li[3]/a'))).click()
    time.sleep(5)
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[1]/div/div[2]/div/div/div').click()
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="app"]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[1]/div[1]/ul/li[6]'))).click()
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[2]/button[2]').click()
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[3]/div/button').click()
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[3]/div/div/button[1]').click()

    time.sleep(25)

    after = os.listdir(os.getcwd() + '/Input Data')
    change = set(after) - set(before)
    if len(change) == 1:
        file_name = change.pop()
    elif len(change) == 0: 
        print("No file downloaded")
    else :
        print("More than one file downloaded")

    data_order = pd.read_csv(r'Input Data/' + str(file_name))
    driver.close()

    data_order['order_id'] = data_order['order_id'].fillna(method='ffill')
    data_order = data_order.drop('quantity', axis = 1)
    temp = data_order.drop_duplicates('order_id').drop('product', axis = 1)
    data_order = data_order[['order_id', 'product']]
    data_order = data_order.merge(temp, how = 'left', on = 'order_id')
    data_order['order_id'] = data_order['order_id'].astype(int)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Shirataki Noodles 71gr (x2) ', 'Twin Pack: Tropicana Slim Shirataki Noodles 71gr ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Saus Tiram 200ml (x2) ', 'Twin Pack: Tropicana Slim Saus Tiram 200ml ', regex = False)

    product = data_order['product'].str.split("\(x",1, expand = True)

    product.loc[product[0]=="HiLo Teen Taro 500gr",1]="1)"
    data_order['Quantity'] = product[1].astype(str).str.replace(')', '', regex = False).astype(int)
    data_order['product'] = product[0].str.strip().str.replace('  ', ' ').str.replace('6 SCH', '6sch').str.replace("W'Dank", "W'dank").str.replace('L-men', 'L-Men').str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("school", "School").str.replace("Rtd", "RTD").str.replace('Honey 50 sachet', 'Honey (50sch)').str.replace('Teen Coklat', 'Teen Chocolate').str.replace('40 sch', '40sch')

    data_order['product'] = data_order['product'].str.replace("HiLo Thai Tea \(10sch\)$", 'HiLo Thai Tea (10 sch)', regex = True)

    data_SKU = pd.read_excel(r'Order Online\SKU buat andra updated maret 2021.xlsx')
    data_SKU = data_SKU.rename(columns = {'Product' : 'product', 'PL NFI' : 'Price List NFI'})
    data_SKU['SKU'] = data_SKU['SKU'].astype(str).str.replace('.0', '', regex = False)
    data_SKU = data_SKU[data_SKU['SKU'] != 'nan'][data_SKU[data_SKU['SKU'] != 'nan']['SKU'] != '0']
    data_SKU['product'] = data_SKU['product'].str.strip().str.replace('L-men', 'L-Men').str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("school", "School").str.replace("Rtd", "RTD").str.replace('Honey 50 sachet', 'Honey (50sch)').str.replace('Teen Coklat', 'Teen Chocolate').str.replace('40 sch', '40sch')


    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Kecap Manis 200ml'].copy()
    price['product'] = 'Tropicana Slim Kecap Manis 200m'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Diabtx (50 sch)'].copy()
    price['product'] = 'Tropicana Slim Diabtx 50 sch'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Hokkaido Cheese 100gr'].copy()
    price['product'] = 'Tropicana Slim Hokaido Cheese 100gr'


    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['L-Men Bar Crunchy Chocolate 12sch', 5500, 5500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Mangga Gandaria', 45000, 45000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Teen Vanilla Caramel 500gr', 71500, 71500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Paket Bundle HiLo Renceng (Hilo Chocolate Banana (10 sch) + Hilo Chocolate Taro (10 sch) + HiLo Thai Tea (10sch))', 35200, 35200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Berondong 10's", 15000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Kecap Asin 200 ml", 28500, 26000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim DIABTX (100 sch)", 87700, 75000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Teen Chocolate 750gr", 117800, 104000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Paket Ngopi Lokalate", 136000, 109200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Hi Protein 2 Go Chocolate (24 TETRAPAK)", 240000, 238000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 Tropicana Slim Goldenmil Vanilla (6sch)", 39160, 31000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Kawista", 17500, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["Paket Bundle HiLo (HiLo Active Chocolate 500gr + HiLo Teen Yoghurt Banana 250gr)", 122900, 101850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Strawberry Jam 375gr", 72600, 58500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Crunch BBQ Beef (20gr)", 13500, 10500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Cocopandan 40 sch", 52500, 52500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 - W'dank Empon Empon 10 Sachet Renceng", 35000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Semangka 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Nanas 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["W'Dank Empon-Empon 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Milk Skim Fiber Pro Plain 500gr", 135000, 106700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Es Teler 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: HiLo Es Teler 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Hilo Es Ketan Hitam 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Triple Pack: Tropicana Slim Korean Garlic Butter Cookies (5 Sch)", 73500, 52000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Avocado Coffee 4 Sch", 21200, 12100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Sambal Terasi 200 gr", 35600, 29700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Tropicana Slim Avocado Coffee 4 sch x 2", 42400, 24200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Bar Chocolate (12 Sch) + L-Men Protein Crunch BBQ Beef x 2", 159000, 107800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Buy 5 Get 5 L-Men Protein Crunch BBQ Beef (20gr)", 130000, 67500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Active Ketan Hitam 175gr', 48500, 20700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Es Ketan Hitam 10 sch', 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Sweetener Lemongrass Pandan 50 sch', 44200, 28900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 1 Get 1 FREE: Lokalate Kopi Berondong (10 sch)', 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['(Near ED) Tropicana Slim Sweetener Jahe 50 sch - Khusus Homdel', 44600, 22300]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 12 Get 12 - HiLo Chocolate Avocado Ready to Drink 200ml - Minuman Siap Minum Tinggi Kalsium', 152400, 84000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Multigrain Original 500g', 92400, 92400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Almond Milk Coconut 200gr', 51900, 51900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    
    data_order['product'] = data_order['product'].astype(str)
    data_SKU['product'] = data_SKU['product'].astype(str)

    data_order['product'] = data_order['product'].str.replace('L Men Gain Mass Chocolate 500 gr', 'L-Men Gain Mass Chocolate 500 gr')



    data_order = data_order.merge(data_SKU[['SKU', 'product', 'Harga Display', 'Harga Coret']].drop_duplicates('product'), how = 'left', on = 'product')
    #             data_order = data_order.merge(product_order[['title', 'price']].drop_duplicates('title'), how = 'left', left_on = 'product', right_on = 'title').drop('title', axis = 1)
    # data_order = data_order[data_order['SKU'].notnull()]
    data_order = data_order.reset_index(drop = True)

    indeks = data_order[data_order['product'] == "Lokalate Kopi Berondong 10's"].index.to_list()
    data_order['Harga Display'][indeks] = 15000
    data_order['Harga Coret'][indeks] = 15000
    indeks = data_order[data_order['SKU'].isnull()].index.to_list()
    for i in indeks:
        col = [x for x in data_SKU.columns if 'Alias Nama' in x]
        for j in col:
            if data_order['product'][i] in data_SKU[j].astype(str).values:
                SKU = data_SKU[data_SKU[j].astype(str) == data_order['product'][i]]['SKU'].values[0]
                data_order['SKU'][i] = SKU

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    data_SKU2 = pd.read_excel(r'SKU_File\data_SKU.xlsx')
    data_SKU2['Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

    #         s = requests.Session()
    #         s.get("http://tatanama.pythonanywhere.com")
    #         s.post("http://tatanama.pythonanywhere.com", data = {'username' : 'ecommerce', 'password' : 'ecommerce'})
    #         r = s.get("http://tatanama.pythonanywhere.com/download")

    #         with open(r'C:\Users\andra.miftah\Demo 9\SKU_File/Master tatanama.xlsx', 'wb') as output:
    #             output.write(r.content)

    #         if os.path.isfile(r'C:\Users\andra.miftah\Demo 9\SKU_File/Master tatanama.xlsx') :    
    #             SKU_append = pd.read_excel(r'C:\Users\andra.miftah\Demo 9\SKU_File/Master tatanama.xlsx')
    #             SKU_append.columns = [x.replace('_', ' ') for x in SKU_append.columns]
    #             data_SKU2 = data_SKU2[~data_SKU2['SKU'].astype(str).isin(SKU_append['SKU'].astype(str))]
    #             data_SKU2 = data_SKU2.append(SKU_append, ignore_index = True, sort = False)

    # to_excel = data_SKU.to_excel(r'C:\Users\andra.miftah\Demo 9\SKU_File/data_SKU.xlsx', index = False)

    for i in indeks:
        if str(data_order['product'][i]).lower() in data_SKU2['Nama Produk'].astype(str).str.lower().values:
            data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2['Nama Produk'].astype(str).str.lower()].values[0]

    list_alias_name = [x for x in data_SKU2.columns if 'Alias Nama' in x]

    for i in indeks:
        for j in list_alias_name:
            if str(data_order['product'][i]).lower() in data_SKU2[j].astype(str).str.lower().values:
                data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2[j].astype(str).str.lower()].values[0]

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    if len(indeks) != 0:
        print('Alert SKU Missing')
        data_order['product'][indeks].drop_duplicates().to_excel('Alert SKU Missing.xlsx', index = False)
    else :
        data_order['phone'] = data_order['phone'].astype(str).str.replace('+628', '08', regex = False)

        data_order['zip'] = data_order['zip'].replace('.0', '', regex = False)

        data_order = data_order.rename(columns = {'order_id' : 'Sales Order ID', 'name' : 'Customer Name', 'product' : 'Item Name', 'price' : 'Price', 'shipping_cost' : 'Shipping Cost', 'address' : 'Shipping Address1', 'city' : 'Shipping City', 'zip' : 'Shipping Zip', 'province' : 'Shipping Province', 'phone' : 'Shipping Phone', 'courier' : 'Shipping Courier'})
        data_order['Channel Order ID'] = data_order['Sales Order ID']
        data_order['Invoice Number'] = data_order['Sales Order ID']
        data_order['Shipping Name'] = data_order['Customer Name']
        data_order['Shipping Address2'] = 0
        data_order['Shipping Country'] = 'Indonesia'
        data_order['AWB'] = 0
        data_order['Channel'] = 'Order Online Banjarmasin'


        data_order['Order Date'] = pd.to_datetime(data_order['created_at'])


        data_order['SKU'] = data_order['SKU'].astype(str)
        data_order['Item Name'] = data_order['Item Name'].astype(str)
        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str).str.replace('(S)', '', regex = False)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        index = data_order[data_order['SKU'].astype(str) == '2306551174'].index.to_list()
        data_order['SKU'][index] = '2306592173'

        data_order = data_order.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('hd','', regex = False)
        temp['SKU'] = temp['SKU'].astype(str).str.replace('HD','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        data_order['Real SKU'] = data_order['Real SKU'].astype(str)
        data_order = data_order.merge(data_SKU2[['SKU', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Unbundling ====== 6/10")        
        # Forstok Unbundling    
        list_col = ['SKU'] + data_SKU2.columns[data_SKU2.columns.get_loc('Produk 1'):data_SKU2.columns.get_loc('Harga Organik 7')+1].to_list()
        data_order = data_order.merge(data_SKU2[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        list_pcs = [x for x in data_order.columns if 'PCS' in x]
        for i in list_pcs:
            data_order[i] = data_order[i] * data_order['Quantity']
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        indeks = data_order[data_order['Brand'] == 'Bundle'].index.to_list()
        data_order['Bundle Flag'] = np.nan
        data_order['Bundle Flag'][indeks] = 'Bundle'

        indeks = data_order[data_order['Brand'] == 'Bundle'][data_order[data_order['Brand'] == 'Bundle']['SKU'].astype(str).str.contains('(S)', regex = False)].index.to_list()
        data_order['SKU Produk 1'][indeks] = '(S)' + data_order['SKU Produk 1'][indeks].astype(str)
        data_order['SKU Produk 2'][indeks] = '(S)' + data_order['SKU Produk 2'][indeks].astype(str)
        data_order['SKU Produk 3'][indeks] = '(S)' + data_order['SKU Produk 3'][indeks].astype(str)
        data_order['SKU Produk 4'][indeks] = '(S)' + data_order['SKU Produk 4'][indeks].astype(str)
        data_order['SKU Produk 5'][indeks] = '(S)' + data_order['SKU Produk 5'][indeks].astype(str)
        data_order['SKU Produk 6'][indeks] = '(S)' + data_order['SKU Produk 6'][indeks].astype(str)
        data_order['SKU Produk 7'][indeks] = '(S)' + data_order['SKU Produk 7'][indeks].astype(str)

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Filling Date ====== 7/10")
        data_order['Date'] = np.nan
        data_order['Month'] = np.nan
        data_order['Year'] = np.nan

        for i in range(data_order.shape[0]):
            if int(data_order['Order Date'][i].strftime('%d')) <= 12:
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).year
            else :
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i]).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i]).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i]).year

        quarter = pd.DataFrame([['January', 1], ['February', 1], ['March', 1], ['April', 2], ['May', 2], ['June', 2], 
                ['July', 3], ['August', 3], ['September', 3],['October', 4], ['November', 4], ['December', 4]], columns = ['Bulan', 'Quarter'])
        data_order = data_order.merge(quarter, how = 'left', left_on = 'Month', right_on = 'Bulan')
        data_order = data_order.drop(['Bulan'], axis = 1)
        data_bulan = pd.DataFrame([{'Bulan' : 'December', 'Number' : 12} ,
                {'Bulan' : 'January' , 'Number': 1},
                {'Bulan' : 'February' , 'Number': 2},
                {'Bulan' : 'March' , 'Number': 3},
                {'Bulan' : 'April' , 'Number': 4},
                {'Bulan' : 'May' , 'Number': 5},
                {'Bulan' : 'June', 'Number': 6},
                {'Bulan' : 'July' , 'Number': 7},
                {'Bulan' : 'August', 'Number' : 8},
                {'Bulan' : 'September', 'Number' : 9},
                {'Bulan' : 'October' , 'Number': 10},
                {'Bulan' : 'November' , 'Number': 11}])
        temp = data_order.copy()
        temp['Day'] = temp['Date']
        temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
        temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
        data_order['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.week
        temp['Hour'] = pd.to_datetime(data_order['Order Date']).dt.hour
        temp['Minute'] = pd.to_datetime(data_order['Order Date']).dt.minute
        temp['Second'] = pd.to_datetime(data_order['Order Date']).dt.second
        data_order['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']])



        order_all = data_order.copy()
        order_all['Total'] = order_all['net_revenue']
        order_all['Price List NFI'] = np.nan
        order_all['Total Net'] = np.nan



        order_all = order_all.rename(columns={'Channel Order ID' : 'Order #',
                                                'Status' : 'Order Status',
                                                'Order Date' : 'Order date',
                                                'Item Name' :'Product Name',
                                                'Bundle Name' : 'Bundle',
                                                'Shipping Country' : 'Country',
                                                'Shipping Province' : 'Region',
                                                'Shipping City' : 'City',
                                                'Shipping Zip' : 'Zip Code',
                                                'Shipping Address1' : 'Address',
                                                'Shipping Phone' : 'Phone',
                                                'Quantity' : 'Qty. Invoiced',
                                                'Harga Display' : 'Regular Price',
                                                'net_revenue' : 'Subtotal'})

        order_all = order_all.reset_index(drop = True)

        order_all['Selling Price'] = order_all['Harga Coret'].astype(int)
        order_all['Kecamatan'] = np.nan
        order_all['Kelurahan'] = np.nan

        print("Filling Location")
        indeks = order_all[order_all['City'].astype(str).str.contains('/')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains('-')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[0]

        indeks = order_all[order_all['City'].astype(str).str.contains(',')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains(',')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[0]

        order_all['City'] = order_all['City'].astype(str).str.replace('Kab\.', 'Kabupaten' ,case = False)

        master_map = pd.read_csv(r'All Data/Province.csv', names = ['Kode Prov', 'Province'], header= 0)
        master_map2 = pd.read_csv(r'All Data/City.csv', names = ['Kode City', 'Kode Prov', 'City'], header = 0)
        master_map = master_map.merge(master_map2, how = 'right', on = 'Kode Prov')
        master_map['Kode Prov'][515] = 14
        master_map['Province'][515] = 'Riau'
        master_map['Kode Prov'] = master_map['Kode Prov'].astype(int)
        master_map['Province'] = master_map['Province'].str.title()
        master_map['City'] = master_map['City'].str.title()

        city = pd.read_excel(r'All Data/list_city.xlsx')
        temp = order_all.copy()
        temp['City'] = temp['City'].astype(str).str.lower()
        temp['City'] = temp['City'].astype(str).str.replace('kab. ', 'kabupaten ', regex = False, case = False)
        city['All City'] = city['All City'].astype(str).str.lower()
        temp = temp.merge(city.drop_duplicates('All City'), how = 'left', left_on = 'City', right_on = 'All City').set_index(temp.index)
        indeks = temp[temp['Real City'].notnull()].index.to_list()
        order_all['City'][indeks] = temp['Real City'][indeks]

        province = pd.read_excel(r'All Data/list_province.xlsx')
        temp = order_all.copy()
        temp['Region'] = temp['Region'].astype(str).str.lower()
        province['All Province'] = province['All Province'].astype(str).str.lower()
        temp = temp.merge(province.drop_duplicates('All Province'), how = 'left', left_on = 'Region', right_on = 'All Province').set_index(temp.index)
        indeks = temp[temp['Real Province'].notnull()].index.to_list()
        order_all['Region'][indeks] = temp['Real Province'][indeks]

        temp = order_all.copy()
        temp = temp[temp['Region'].isnull()]
        temp['Region'] = temp.merge(master_map, how = 'left', on = 'City').set_index(temp.index)['Province']
        order_all['Region'][temp.index] = temp['Region']  

        district = pd.read_excel(r'All Data/list_district.xlsx')
        temp = order_all.copy()
        temp['Kecamatan'] = temp['Kecamatan'].astype(str).str.lower()
        district['All District'] = district['All District'].astype(str).str.lower()
        temp = temp.merge(district.drop_duplicates('All District'), how = 'left', left_on = 'Kecamatan', right_on = 'All District').set_index(temp.index)
        indeks = temp[temp['Real District'].notnull()].index.to_list()
        order_all['Kecamatan'][indeks] = temp['Real District'][indeks]

        temp = order_all.copy()
        temp2 = temp[['Region', 'City', 'Kecamatan']].merge(master_map, how = 'left', on = 'City')
        indeks = temp2[temp2['Region'] != temp2['Province']][temp2[temp2['Region'] != temp2['Province']]['City'].notnull()].index.to_list()
        order_all['City'][indeks] = np.nan

        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        print("Unbundling")
        data_bundle1 = order_all[~order_all['Produk 1'].isnull()]
        data_bundle1['Bundle Name'] = data_bundle1['Product Name']
        data_bundle1['Product Name'] = data_bundle1['Produk 1']
        data_bundle1['SKU'] = data_bundle1['SKU Produk 1']
        data_bundle1['Qty. Invoiced'] = data_bundle1['PCS Produk 1']
        data_bundle1['Price List NFI'] = data_bundle1['Price List NFI 1']
        data_bundle1['Total Net'] = data_bundle1['Price List NFI 1']
        data_bundle1['Bundle Flag'] = np.nan

        data_bundle2 = order_all[~order_all['Produk 2'].isnull()]
        data_bundle2['Bundle Name'] = data_bundle2['Product Name']
        data_bundle2['Product Name'] = data_bundle2['Produk 2']
        data_bundle2['SKU'] = data_bundle2['SKU Produk 2']
        data_bundle2['Qty. Invoiced'] = data_bundle2['PCS Produk 2']
        data_bundle2['Price List NFI'] = data_bundle2['Price List NFI 2']
        data_bundle2['Total Net'] = data_bundle2['Price List NFI 2'] 
        data_bundle2['Bundle Flag'] = np.nan

        data_bundle3 = order_all[~order_all['Produk 3'].isnull()]
        data_bundle3['Bundle Name'] = data_bundle3['Product Name']
        data_bundle3['Product Name'] = data_bundle3['Produk 3']
        data_bundle3['SKU'] = data_bundle3['SKU Produk 3']
        data_bundle3['Qty. Invoiced'] = data_bundle3['PCS Produk 3']
        data_bundle3['Price List NFI'] = data_bundle3['Price List NFI 3']
        data_bundle3['Total Net'] = data_bundle3['Price List NFI 3'] 
        data_bundle3['Bundle Flag'] = np.nan

        data_bundle4 = order_all[~order_all['Produk 4'].isnull()]
        data_bundle4['Bundle Name'] = data_bundle4['Product Name']
        data_bundle4['Product Name'] = data_bundle4['Produk 4']
        data_bundle4['SKU'] = data_bundle4['SKU Produk 4']
        data_bundle4['Qty. Invoiced'] = data_bundle4['PCS Produk 4']
        data_bundle4['Price List NFI'] = data_bundle4['Price List NFI 4']
        data_bundle4['Total Net'] = data_bundle4['Price List NFI 4'] 
        data_bundle4['Bundle Flag'] = np.nan

        data_bundle5 = order_all[~order_all['Produk 5'].isnull()]
        data_bundle5['Bundle Name'] = data_bundle5['Product Name']
        data_bundle5['Product Name'] = data_bundle5['Produk 5']
        data_bundle5['SKU'] = data_bundle5['SKU Produk 5']
        data_bundle5['Qty. Invoiced'] = data_bundle5['PCS Produk 5']
        data_bundle5['Price List NFI'] = data_bundle5['Price List NFI 5']
        data_bundle5['Total Net'] = data_bundle5['Price List NFI 5']
        data_bundle5['Bundle Flag'] = np.nan

        data_bundle6 = order_all[~order_all['Produk 6'].isnull()]
        data_bundle6['Bundle Name'] = data_bundle6['Product Name']
        data_bundle6['Product Name'] = data_bundle6['Produk 6']
        data_bundle6['SKU'] = data_bundle6['SKU Produk 6']
        data_bundle6['Qty. Invoiced'] = data_bundle6['PCS Produk 6']
        data_bundle6['Price List NFI'] = data_bundle6['Price List NFI 6']
        data_bundle6['Total Net'] = data_bundle6['Price List NFI 6']
        data_bundle6['Bundle Flag'] = np.nan

        data_bundle7 = order_all[~order_all['Produk 7'].isnull()]
        data_bundle7['Bundle Name'] = data_bundle7['Product Name']
        data_bundle7['Product Name'] = data_bundle7['Produk 7']
        data_bundle7['SKU'] = data_bundle7['SKU Produk 7']
        data_bundle7['Qty. Invoiced'] = data_bundle7['PCS Produk 7']
        data_bundle7['Price List NFI'] = data_bundle7['Price List NFI 7']
        data_bundle7['Total Net'] = data_bundle7['Price List NFI 7']
        data_bundle7['Bundle Flag'] = np.nan

        data_bundle = data_bundle1.append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)
        data_bundle['SKU'] = data_bundle['SKU'].astype(str)
        data_bundle['SKU'] = data_bundle['SKU'].str.replace('\.0$', '', regex = True)
        data_bundle[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']] = data_bundle.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item_y', 'Parent SKU_y']]

        temp = data_bundle[data_bundle['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)

        indeks = data_bundle[data_bundle['Real SKU'].isnull()].index.to_list()
        data_bundle['Real SKU'][indeks] = temp['Real SKU_y'][indeks]
        data_bundle['Real Nama Produk'][indeks] = temp['Nama Produk'][indeks]
        data_bundle['Brand'][indeks] = temp['Brand_y'][indeks]
        data_bundle['Sub Brand'][indeks] = temp['Sub Brand_y'][indeks]
        data_bundle['Parent Item'][indeks] = temp['Parent Item_y'][indeks]
        data_bundle['Parent SKU'][indeks] = temp['Parent SKU_y'][indeks]

        print("Pricing")
        order_all = order_all.append(data_bundle, ignore_index = True, sort = False)

        colname = temp.columns[temp.columns.get_loc('Produk 1') : temp.columns.get_loc('Harga Cost 7') + 1]
        colname_str = [x for x in colname if 'Subtotal' not in x and 'Harga' not in x]
        colname_int = [x for x in colname if x not in colname_str]

        for i in colname_str:
            temp[i] = np.nan

        for i in colname_int:
            temp[i] = 0

        data_order = data_order.append(temp , ignore_index = True, sort = False)


        order_all = order_all.merge(data_SKU2[['SKU', 'Price List NFI', 'Harga Cost']].drop_duplicates('SKU'), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(order_all.index)
        order_all['Price List NFI_x'] = order_all['Price List NFI_x'].fillna(order_all['Price List NFI_y'])
        order_all =  order_all.drop(['Price List NFI_y', 'SKU_y'], axis = 1)
        order_all = order_all.rename(columns = {'SKU_x' : 'SKU', 'Price List NFI_x' : 'Price List NFI'})

        indeks = order_all[order_all['Product Name'] == 'HiLo Teen Chocolate 250gr'].index.to_list()
        order_all['Real Nama Produk'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Parent Item'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Brand'][indeks] = 'HiLo'
        order_all['Sub Brand'][indeks] = 'HILO TEEN'
        order_all['Price List NFI'][indeks] = 36850
        order_all['Harga Cost'][indeks] = 36850
        order_all['Real SKU'][indeks] = '2101651155'
        order_all['Parent SKU'][indeks] = '2101651155'


        order_all['Price List NFI'] = pd.to_numeric(order_all['Price List NFI']).astype(int)
        order_all['Harga Cost'] = pd.to_numeric(order_all['Harga Cost']).astype(int)
        order_all['Qty. Invoiced'] = pd.to_numeric(order_all['Qty. Invoiced']).astype(int)

        order_all['Total Net'] = order_all['Price List NFI'] * order_all['Qty. Invoiced']
        order_all['Total Harga Cost'] = order_all['Harga Cost'] * order_all['Qty. Invoiced']
        order_all['Subtotal'] = order_all['Selling Price'] * order_all['Qty. Invoiced']
        order_all['Total'] = order_all['Selling Price'] * order_all['Qty. Invoiced']

        order_all = order_all.reset_index(drop = True)
        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        order_all['Seller Discount'] = order_all['discount']
        order_all['Shipping'] = order_all['Shipping Cost']

        temp = order_all[order_all['Brand'] != 'Bundle']
        temp['discount'] = temp['discount'] * temp['Total Harga Cost']/temp.groupby(['Order #'])['Total Harga Cost'].transform('sum')
        order_all['discount'][temp.index] = temp['discount']

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Subtotal', 'Total']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Total'][list_nobundle.index] = list_nobundle['Total_y']
        order_all['Subtotal'][list_nobundle.index] = list_nobundle['Subtotal_y']

        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Subtotal'] = temp['Subtotal'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        temp['Selling Price'] = temp['Subtotal']/temp['Qty. Invoiced']
        temp['Total'] = temp['Total'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Total'][temp.index] = temp['Total']
        order_all['Subtotal'][temp.index] = temp['Subtotal']
        order_all['Selling Price'][temp.index] = temp['Selling Price']


        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Seller Discount']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Seller Discount'][list_nobundle.index] = list_nobundle['Seller Discount_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Seller Discount'] = temp['Seller Discount'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Seller Discount'][temp.index] = temp['Seller Discount']


        temp = order_all[order_all['Bundle Name'].isnull()]
        temp_group = temp[['Order #','Shipping']].groupby(['Order #']).sum().reset_index()

        temp = order_all.merge(temp_group, how = 'left', on = 'Order #').set_index(order_all.index)
        temp['Shipping_x'] = temp['Shipping_y'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Shipping'][temp.index] = temp['Shipping_y']
        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Shipping']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Shipping'][list_nobundle.index] = list_nobundle['Shipping_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Shipping'] = temp['Shipping'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Shipping'][temp.index] = temp['Shipping']
        order_all['True datetime'] = pd.to_datetime(order_all['True datetime'])
        order_all['Promo'] = np.nan
        order_all['Discount MC'] = np.nan


        order_all['Warehouse Name'] = 'Primary Warehouse'
        order_all['Store'] = 'Order Online'

        order_all['Customer Email'] = order_all['email']
        order_all['Order Status'] = order_all['status']
        order_all['Payment Channel'] = order_all['payment_method']
        order_all['Coupon Code'] = order_all['coupon']
        order_all.columns.to_list()

        order_all_append = order_all[['Sales Order ID', 'Store',
            'Product Name',
            'Customer Name',
            'Phone',
            'Address',
            'Region',
            'City',
            'Zip Code',
            'payment_status',
            'Regular Price',
            'Shipping Courier',
            'Shipping Cost',
            'Subtotal',
            'Qty. Invoiced',
            'SKU',
            'Order #',
            'Invoice Number',
            'Shipping Name',
            'Shipping Address2',
            'Country',
            'AWB',
            'Channel',
            'Order date',
            'Real SKU',
            'Real Nama Produk',
            'Brand',
            'Sub Brand',
            'Parent Item',
            'Parent SKU',
            'Produk 1',
            'SKU Produk 1',
            'PCS Produk 1',
            'Price List NFI 1',
            'Subtotal Produk 1',
            'Harga Display 1',
            'Harga Cost 1',
            'Harga Organik 1',
            'Produk 2',
            'SKU Produk 2',
            'PCS Produk 2',
            'Price List NFI 2',
            'Subtotal Produk 2',
            'Harga Display 2',
            'Harga Cost 2',
            'Harga Organik 2',
            'Produk 3',
            'SKU Produk 3',
            'PCS Produk 3',
            'Price List NFI 3',
            'Subtotal Produk 3',
            'Harga Display 3',
            'Harga Cost 3',
            'Harga Organik 3',
            'Produk 4',
            'SKU Produk 4',
            'PCS Produk 4',
            'Price List NFI 4',
            'Subtotal Produk 4',
            'Harga Display 4',
            'Harga Cost 4',
            'Harga Organik 4',
            'Produk 5',
            'SKU Produk 5',
            'PCS Produk 5',
            'Price List NFI 5',
            'Subtotal Produk 5',
            'Harga Display 5',
            'Harga Cost 5',
            'Harga Organik 5',
            'Produk 6',
            'SKU Produk 6',
            'PCS Produk 6',
            'Price List NFI 6',
            'Subtotal Produk 6',
            'Harga Display 6',
            'Harga Cost 6',
            'Harga Organik 6',
            'Produk 7',
            'SKU Produk 7',
            'PCS Produk 7',
            'Price List NFI 7',
            'Subtotal Produk 7',
            'Harga Display 7',
            'Harga Cost 7',
            'Harga Organik 7',
            'Bundle Flag',
            'Date',
            'Month',
            'Year',
            'Quarter',
            'Week',
            'True datetime',
            'Total',
            'Price List NFI',
            'Total Net',
            'Selling Price',
            'Kecamatan',
            'Kelurahan',
            'Bundle Name',
            'Harga Cost',
            'Total Harga Cost',
            'Seller Discount',
            'Shipping',
            'Promo',
            'Discount MC',
            'Warehouse Name',
            'Customer Email',
            'Order Status',
            'Payment Channel',
            'Coupon Code']]

        data_all = data_all[~data_all['Order #'].astype(str).isin(order_all_append['Order #'].astype(str))]
        data_all = data_all.append(order_all_append, ignore_index = True, sort = False)

        del file_name
        order_online_banjarmasin = True

if not order_online_jabar:

    import pandas as pd
    import numpy as np
    import requests
    import os

    print('order_online_jabar')

    before = os.listdir(os.getcwd() + '/Input Data')

    options = Options()
    options.add_experimental_option("prefs", {
            "download.default_directory": os.path.abspath(r"Input Data"),
            "download.directory_upgrade": True,
            "safebrowsing_for_trusted_sources_enabled": False,
            "safebrowsing.enabled": False
    })

    driver = webdriver.Chrome(options=options)
    driver.fullscreen_window()
    driver.get("https://app.orderonline.id")

    username = driver.find_element_by_name("email")
    username.clear()
    username.send_keys("nutrimartbandung1@gmail.com")

    password = driver.find_element_by_name("password")
    password.send_keys("nhdcuancuan")
    password.click()

    driver.find_element_by_class_name("btn-submit").click()
    time.sleep(15)
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="main-nav-dropdown"]/ul/li[3]/a'))).click()
    time.sleep(5)
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[1]/div/div[2]/div/div/div').click()
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="app"]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[1]/div[1]/ul/li[6]'))).click()
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[1]/div/div[2]/div/div/div[2]/div[2]/button[2]').click()
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[3]/div/button').click()
    driver.find_element_by_xpath('//*[@id="app"]/main/div/div[1]/div[3]/div/div/button[1]').click()

    time.sleep(25)

    after = os.listdir(os.getcwd() + '/Input Data')
    change = set(after) - set(before)
    if len(change) == 1:
        file_name = change.pop()
    elif len(change) == 0: 
        print("No file downloaded")
    else :
        print("More than one file downloaded")

    data_order = pd.read_csv(r'Input Data/' + str(file_name))
    driver.close()

    data_order['order_id'] = data_order['order_id'].fillna(method='ffill')
    data_order = data_order.drop('quantity', axis = 1)
    temp = data_order.drop_duplicates('order_id').drop('product', axis = 1)
    data_order = data_order[['order_id', 'product']]
    data_order = data_order.merge(temp, how = 'left', on = 'order_id')
    data_order['order_id'] = data_order['order_id'].astype(int)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Shirataki Noodles 71gr (x2) ', 'Twin Pack: Tropicana Slim Shirataki Noodles 71gr ', regex = False)
    data_order['product'] = data_order['product'].astype(str).str.replace('Twin Pack: Tropicana Slim Saus Tiram 200ml (x2) ', 'Twin Pack: Tropicana Slim Saus Tiram 200ml ', regex = False)

    product = data_order['product'].str.split("\(x",1, expand = True)

    product.loc[product[0]=="HiLo Teen Taro 500gr",1]="1)"
    data_order['Quantity'] = product[1].astype(str).str.replace(')', '', regex = False).astype(int)
    data_order['product'] = product[0].str.strip().str.replace('  ', ' ').str.replace('6 SCH', '6sch').str.replace("W'Dank", "W'dank").str.replace('L-men', 'L-Men').str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("school", "School").str.replace("Rtd", "RTD").str.replace('Honey 50 sachet', 'Honey (50sch)').str.replace('Teen Coklat', 'Teen Chocolate').str.replace('40 sch', '40sch')

    data_order['product'] = data_order['product'].str.replace("HiLo Thai Tea \(10sch\)$", 'HiLo Thai Tea (10 sch)', regex = True)

    data_SKU = pd.read_excel(r'Order Online\SKU buat andra updated maret 2021.xlsx')
    data_SKU = data_SKU.rename(columns = {'Product' : 'product', 'PL NFI' : 'Price List NFI'})
    data_SKU['SKU'] = data_SKU['SKU'].astype(str).str.replace('.0', '', regex = False)
    data_SKU = data_SKU[data_SKU['SKU'] != 'nan'][data_SKU[data_SKU['SKU'] != 'nan']['SKU'] != '0']
    data_SKU['product'] = data_SKU['product'].str.strip().str.replace('L-men', 'L-Men').str.replace("Hilo", "HiLo").str.replace("Sch", "sch").str.replace("Ml", "ml").str.replace("Gr", "gr").str.replace("DIABTX", "Diabtx").str.replace("Empon-empon", "Empon-Empon").str.replace("Nutrisari", "NutriSari").str.replace("X", "x").str.replace("original", "Original").str.replace("hilo", "HiLo").str.replace("Bbq", "BBQ").str.replace("school", "School").str.replace("Rtd", "RTD").str.replace('Honey 50 sachet', 'Honey (50sch)').str.replace('Teen Coklat', 'Teen Chocolate').str.replace('40 sch', '40sch')


    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Kecap Manis 200ml'].copy()
    price['product'] = 'Tropicana Slim Kecap Manis 200m'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Diabtx (50 sch)'].copy()
    price['product'] = 'Tropicana Slim Diabtx 50 sch'
    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)
    price = data_SKU[data_SKU['product'] == 'Tropicana Slim Hokkaido Cheese 100gr'].copy()
    price['product'] = 'Tropicana Slim Hokaido Cheese 100gr'


    data_SKU = data_SKU.append(price, ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['L-Men Bar Crunchy Chocolate 12sch', 5500, 5500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['NutriSari Mangga Gandaria', 45000, 45000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Teen Vanilla Caramel 500gr', 71500, 71500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Paket Bundle HiLo Renceng (Hilo Chocolate Banana (10 sch) + Hilo Chocolate Taro (10 sch) + HiLo Thai Tea (10sch))', 35200, 35200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Berondong 10's", 15000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Kecap Asin 200 ml", 28500, 26000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim DIABTX (100 sch)", 87700, 75000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Teen Chocolate 750gr", 117800, 104000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Paket Ngopi Lokalate", 136000, 109200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Hi Protein 2 Go Chocolate (24 TETRAPAK)", 240000, 238000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 Tropicana Slim Goldenmil Vanilla (6sch)", 39160, 31000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Lokalate Kopi Kawista", 17500, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["Paket Bundle HiLo (HiLo Active Chocolate 500gr + HiLo Teen Yoghurt Banana 250gr)", 122900, 101850]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Strawberry Jam 375gr", 72600, 58500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Crunch BBQ Beef (20gr)", 13500, 10500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Cocopandan 40 sch", 52500, 52500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)

    data_SKU = data_SKU.append(pd.DataFrame([["BUY 1 GET 1 - W'dank Empon Empon 10 Sachet Renceng", 35000, 15000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Semangka 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["NutriSari Nanas 40 sch", 62000, 42000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["W'Dank Empon-Empon 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Milk Skim Fiber Pro Plain 500gr", 135000, 106700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["HiLo Es Teler 10 sch", 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: HiLo Es Teler 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Hilo Es Ketan Hitam 10 sch x 2", 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Triple Pack: Tropicana Slim Korean Garlic Butter Cookies (5 Sch)", 73500, 52000]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Avocado Coffee 4 Sch", 21200, 12100]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Tropicana Slim Sambal Terasi 200 gr", 35600, 29700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Twin Pack: Tropicana Slim Avocado Coffee 4 sch x 2", 42400, 24200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["L-Men Protein Bar Chocolate (12 Sch) + L-Men Protein Crunch BBQ Beef x 2", 159000, 107800]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([["Buy 5 Get 5 L-Men Protein Crunch BBQ Beef (20gr)", 130000, 67500]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['HiLo Active Ketan Hitam 175gr', 48500, 20700]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Hilo Es Ketan Hitam 10 sch', 17500, 13200]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Tropicana Slim Sweetener Lemongrass Pandan 50 sch', 44200, 28900]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)
    data_SKU = data_SKU.append(pd.DataFrame([['Buy 1 Get 1 FREE: Lokalate Kopi Berondong (10 sch)', 35000, 26400]], columns = ['product', 'Harga Display', 'Harga Coret']), ignore_index = True, sort = False)


    data_order['product'] = data_order['product'].astype(str)
    data_SKU['product'] = data_SKU['product'].astype(str)

    data_order['product'] = data_order['product'].str.replace('L Men Gain Mass Chocolate 500 gr', 'L-Men Gain Mass Chocolate 500 gr')



    data_order = data_order.merge(data_SKU[['SKU', 'product', 'Harga Display', 'Harga Coret']].drop_duplicates('product'), how = 'left', on = 'product')
    data_order = data_order.reset_index(drop = True)

    indeks = data_order[data_order['product'] == "Lokalate Kopi Berondong 10's"].index.to_list()
    data_order['Harga Display'][indeks] = 15000
    data_order['Harga Coret'][indeks] = 15000
    indeks = data_order[data_order['SKU'].isnull()].index.to_list()
    for i in indeks:
        col = [x for x in data_SKU.columns if 'Alias Nama' in x]
        for j in col:
            if data_order['product'][i] in data_SKU[j].astype(str).values:
                SKU = data_SKU[data_SKU[j].astype(str) == data_order['product'][i]]['SKU'].values[0]
                data_order['SKU'][i] = SKU

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    data_SKU2 = pd.read_excel(r'SKU_File\data_SKU.xlsx')
    data_SKU2['Nama Produk'] = data_SKU2['Nama Produk'].astype(str)


    for i in indeks:
        if str(data_order['product'][i]).lower() in data_SKU2['Nama Produk'].astype(str).str.lower().values:
            data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2['Nama Produk'].astype(str).str.lower()].values[0]

    list_alias_name = [x for x in data_SKU2.columns if 'Alias Nama' in x]

    for i in indeks:
        for j in list_alias_name:
            if str(data_order['product'][i]).lower() in data_SKU2[j].astype(str).str.lower().values:
                data_order['SKU'][i] = data_SKU2['SKU'].loc[str(data_order['product'][i]).lower() == data_SKU2[j].astype(str).str.lower()].values[0]

    indeks = data_order[data_order['SKU'].isnull()].index.to_list()

    if len(indeks) != 0:
        print('Alert SKU Missing')
        data_order['product'][indeks].drop_duplicates().to_excel('Alert SKU Missing.xlsx', index = False)
    else :
        data_order['phone'] = data_order['phone'].astype(str).str.replace('+628', '08', regex = False)

        data_order['zip'] = data_order['zip'].replace('.0', '', regex = False)

        data_order = data_order.rename(columns = {'order_id' : 'Sales Order ID', 'name' : 'Customer Name', 'product' : 'Item Name', 'price' : 'Price', 'shipping_cost' : 'Shipping Cost', 'address' : 'Shipping Address1', 'city' : 'Shipping City', 'zip' : 'Shipping Zip', 'province' : 'Shipping Province', 'phone' : 'Shipping Phone', 'courier' : 'Shipping Courier'})
        data_order['Channel Order ID'] = data_order['Sales Order ID']
        data_order['Invoice Number'] = data_order['Sales Order ID']
        data_order['Shipping Name'] = data_order['Customer Name']
        data_order['Shipping Address2'] = 0
        data_order['Shipping Country'] = 'Indonesia'
        data_order['AWB'] = 0
        data_order['Channel'] = 'Order Online Jabar'


        data_order['Order Date'] = pd.to_datetime(data_order['created_at'])


        data_order['SKU'] = data_order['SKU'].astype(str)
        data_order['Item Name'] = data_order['Item Name'].astype(str)
        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str).str.replace('(S)', '', regex = False)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        index = data_order[data_order['SKU'].astype(str) == '2306551174'].index.to_list()
        data_order['SKU'][index] = '2306592173'

        data_order = data_order.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        temp = data_order[data_order['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('hd','', regex = False)
        temp['SKU'] = temp['SKU'].astype(str).str.replace('HD','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)
        temp['Real SKU_x'] = temp['Real SKU_x'].fillna(temp['Real SKU_y'])
        temp['Real Nama Produk_x'] = temp['Real Nama Produk_x'].fillna(temp['Real Nama Produk_y'])
        temp = temp.drop(['Real SKU_y', 'Real Nama Produk_y'], axis = 1)
        temp = temp.rename(columns = {'Real SKU_x' : 'Real SKU', 'Real Nama Produk_x' : 'Real Nama Produk'})

        indeks = data_order[data_order['Real SKU'].isnull()].index.to_list()
        data_order['Real SKU'][indeks] = temp['Real SKU'][indeks]
        data_order['Real Nama Produk'][indeks] = temp['Real Nama Produk'][indeks]

        data_order['Real SKU'] = data_order['Real SKU'].astype(str)
        data_order = data_order.merge(data_SKU2[['SKU', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Unbundling ====== 6/10")        
        # Forstok Unbundling    
        list_col = ['SKU'] + data_SKU2.columns[data_SKU2.columns.get_loc('Produk 1'):data_SKU2.columns.get_loc('Harga Organik 7')+1].to_list()
        data_order = data_order.merge(data_SKU2[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
        list_pcs = [x for x in data_order.columns if 'PCS' in x]
        for i in list_pcs:
            data_order[i] = data_order[i] * data_order['Quantity']
        data_order = data_order.drop(['SKU_y'], axis = 1)
        data_order = data_order.rename(columns = {'SKU_x':'SKU'})

        indeks = data_order[data_order['Brand'] == 'Bundle'].index.to_list()
        data_order['Bundle Flag'] = np.nan
        data_order['Bundle Flag'][indeks] = 'Bundle'

        indeks = data_order[data_order['Brand'] == 'Bundle'][data_order[data_order['Brand'] == 'Bundle']['SKU'].astype(str).str.contains('(S)', regex = False)].index.to_list()
        data_order['SKU Produk 1'][indeks] = '(S)' + data_order['SKU Produk 1'][indeks].astype(str)
        data_order['SKU Produk 2'][indeks] = '(S)' + data_order['SKU Produk 2'][indeks].astype(str)
        data_order['SKU Produk 3'][indeks] = '(S)' + data_order['SKU Produk 3'][indeks].astype(str)
        data_order['SKU Produk 4'][indeks] = '(S)' + data_order['SKU Produk 4'][indeks].astype(str)
        data_order['SKU Produk 5'][indeks] = '(S)' + data_order['SKU Produk 5'][indeks].astype(str)
        data_order['SKU Produk 6'][indeks] = '(S)' + data_order['SKU Produk 6'][indeks].astype(str)
        data_order['SKU Produk 7'][indeks] = '(S)' + data_order['SKU Produk 7'][indeks].astype(str)

        print("--- %s seconds ---" % (time.time() - start_time))
        print("Filling Date ====== 7/10")
        data_order['Date'] = np.nan
        data_order['Month'] = np.nan
        data_order['Year'] = np.nan

        for i in range(data_order.shape[0]):
            if int(data_order['Order Date'][i].strftime('%d')) <= 12:
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i].strftime('%Y-%d-%m %H:%M')).year
            else :
                data_order['Date'][i] = pd.to_datetime(data_order['Order Date'][i]).day
                data_order['Month'][i] = pd.to_datetime(data_order['Order Date'][i]).month_name()
                data_order['Year'][i] = pd.to_datetime(data_order['Order Date'][i]).year

        quarter = pd.DataFrame([['January', 1], ['February', 1], ['March', 1], ['April', 2], ['May', 2], ['June', 2], 
                ['July', 3], ['August', 3], ['September', 3],['October', 4], ['November', 4], ['December', 4]], columns = ['Bulan', 'Quarter'])
        data_order = data_order.merge(quarter, how = 'left', left_on = 'Month', right_on = 'Bulan')
        data_order = data_order.drop(['Bulan'], axis = 1)
        data_bulan = pd.DataFrame([{'Bulan' : 'December', 'Number' : 12} ,
                {'Bulan' : 'January' , 'Number': 1},
                {'Bulan' : 'February' , 'Number': 2},
                {'Bulan' : 'March' , 'Number': 3},
                {'Bulan' : 'April' , 'Number': 4},
                {'Bulan' : 'May' , 'Number': 5},
                {'Bulan' : 'June', 'Number': 6},
                {'Bulan' : 'July' , 'Number': 7},
                {'Bulan' : 'August', 'Number' : 8},
                {'Bulan' : 'September', 'Number' : 9},
                {'Bulan' : 'October' , 'Number': 10},
                {'Bulan' : 'November' , 'Number': 11}])
        temp = data_order.copy()
        temp['Day'] = temp['Date']
        temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
        temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
        data_order['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.week
        temp['Hour'] = pd.to_datetime(data_order['Order Date']).dt.hour
        temp['Minute'] = pd.to_datetime(data_order['Order Date']).dt.minute
        temp['Second'] = pd.to_datetime(data_order['Order Date']).dt.second
        data_order['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']])



        order_all = data_order.copy()
        order_all['Total'] = order_all['net_revenue']
        order_all['Price List NFI'] = np.nan
        order_all['Total Net'] = np.nan



        order_all = order_all.rename(columns={'Channel Order ID' : 'Order #',
                                                'Status' : 'Order Status',
                                                'Order Date' : 'Order date',
                                                'Item Name' :'Product Name',
                                                'Bundle Name' : 'Bundle',
                                                'Shipping Country' : 'Country',
                                                'Shipping Province' : 'Region',
                                                'Shipping City' : 'City',
                                                'Shipping Zip' : 'Zip Code',
                                                'Shipping Address1' : 'Address',
                                                'Shipping Phone' : 'Phone',
                                                'Quantity' : 'Qty. Invoiced',
                                                'Harga Display' : 'Regular Price',
                                                'net_revenue' : 'Subtotal'})

        order_all = order_all.reset_index(drop = True)

        order_all['Selling Price'] = order_all['Harga Coret'].astype(int)
        order_all['Kecamatan'] = np.nan
        order_all['Kelurahan'] = np.nan

        print("Filling Location")
        indeks = order_all[order_all['City'].astype(str).str.contains('/')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split('/', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains('-')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split('-', n = 1,expand = True)[0]

        indeks = order_all[order_all['City'].astype(str).str.contains(',')]['City'].index.to_list()
        if len(indeks)>0:
            order_all['Kecamatan'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['City'][indeks] = order_all['City'][indeks].str.split(',', n = 1,expand = True)[0]

        indeks = order_all[order_all['Kecamatan'].astype(str).str.contains(',')]['Kecamatan'].index.to_list()
        if len(indeks)>0:
            order_all['Kelurahan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[1]
            order_all['Kecamatan'][indeks] = order_all['Kecamatan'][indeks].str.split(',', n = 1,expand = True)[0]

        order_all['City'] = order_all['City'].astype(str).str.replace('Kab\.', 'Kabupaten' ,case = False)

        master_map = pd.read_csv(r'All Data/Province.csv', names = ['Kode Prov', 'Province'], header= 0)
        master_map2 = pd.read_csv(r'All Data/City.csv', names = ['Kode City', 'Kode Prov', 'City'], header = 0)
        master_map = master_map.merge(master_map2, how = 'right', on = 'Kode Prov')
        master_map['Kode Prov'][515] = 14
        master_map['Province'][515] = 'Riau'
        master_map['Kode Prov'] = master_map['Kode Prov'].astype(int)
        master_map['Province'] = master_map['Province'].str.title()
        master_map['City'] = master_map['City'].str.title()

        city = pd.read_excel(r'All Data/list_city.xlsx')
        temp = order_all.copy()
        temp['City'] = temp['City'].astype(str).str.lower()
        temp['City'] = temp['City'].astype(str).str.replace('kab. ', 'kabupaten ', regex = False, case = False)
        city['All City'] = city['All City'].astype(str).str.lower()
        temp = temp.merge(city.drop_duplicates('All City'), how = 'left', left_on = 'City', right_on = 'All City').set_index(temp.index)
        indeks = temp[temp['Real City'].notnull()].index.to_list()
        order_all['City'][indeks] = temp['Real City'][indeks]

        province = pd.read_excel(r'All Data/list_province.xlsx')
        temp = order_all.copy()
        temp['Region'] = temp['Region'].astype(str).str.lower()
        province['All Province'] = province['All Province'].astype(str).str.lower()
        temp = temp.merge(province.drop_duplicates('All Province'), how = 'left', left_on = 'Region', right_on = 'All Province').set_index(temp.index)
        indeks = temp[temp['Real Province'].notnull()].index.to_list()
        order_all['Region'][indeks] = temp['Real Province'][indeks]

        temp = order_all.copy()
        temp = temp[temp['Region'].isnull()]
        temp['Region'] = temp.merge(master_map, how = 'left', on = 'City').set_index(temp.index)['Province']
        order_all['Region'][temp.index] = temp['Region']  

        district = pd.read_excel(r'All Data/list_district.xlsx')
        temp = order_all.copy()
        temp['Kecamatan'] = temp['Kecamatan'].astype(str).str.lower()
        district['All District'] = district['All District'].astype(str).str.lower()
        temp = temp.merge(district.drop_duplicates('All District'), how = 'left', left_on = 'Kecamatan', right_on = 'All District').set_index(temp.index)
        indeks = temp[temp['Real District'].notnull()].index.to_list()
        order_all['Kecamatan'][indeks] = temp['Real District'][indeks]

        temp = order_all.copy()
        temp2 = temp[['Region', 'City', 'Kecamatan']].merge(master_map, how = 'left', on = 'City')
        indeks = temp2[temp2['Region'] != temp2['Province']][temp2[temp2['Region'] != temp2['Province']]['City'].notnull()].index.to_list()
        order_all['City'][indeks] = np.nan

        data_SKU2['Real SKU'] = data_SKU2['SKU'].astype(str)
        data_SKU2['Real Nama Produk'] = data_SKU2['Nama Produk'].astype(str)

        print("Unbundling")
        data_bundle1 = order_all[~order_all['Produk 1'].isnull()]
        data_bundle1['Bundle Name'] = data_bundle1['Product Name']
        data_bundle1['Product Name'] = data_bundle1['Produk 1']
        data_bundle1['SKU'] = data_bundle1['SKU Produk 1']
        data_bundle1['Qty. Invoiced'] = data_bundle1['PCS Produk 1']
        data_bundle1['Price List NFI'] = data_bundle1['Price List NFI 1']
        data_bundle1['Total Net'] = data_bundle1['Price List NFI 1']
        data_bundle1['Bundle Flag'] = np.nan

        data_bundle2 = order_all[~order_all['Produk 2'].isnull()]
        data_bundle2['Bundle Name'] = data_bundle2['Product Name']
        data_bundle2['Product Name'] = data_bundle2['Produk 2']
        data_bundle2['SKU'] = data_bundle2['SKU Produk 2']
        data_bundle2['Qty. Invoiced'] = data_bundle2['PCS Produk 2']
        data_bundle2['Price List NFI'] = data_bundle2['Price List NFI 2']
        data_bundle2['Total Net'] = data_bundle2['Price List NFI 2'] 
        data_bundle2['Bundle Flag'] = np.nan

        data_bundle3 = order_all[~order_all['Produk 3'].isnull()]
        data_bundle3['Bundle Name'] = data_bundle3['Product Name']
        data_bundle3['Product Name'] = data_bundle3['Produk 3']
        data_bundle3['SKU'] = data_bundle3['SKU Produk 3']
        data_bundle3['Qty. Invoiced'] = data_bundle3['PCS Produk 3']
        data_bundle3['Price List NFI'] = data_bundle3['Price List NFI 3']
        data_bundle3['Total Net'] = data_bundle3['Price List NFI 3'] 
        data_bundle3['Bundle Flag'] = np.nan

        data_bundle4 = order_all[~order_all['Produk 4'].isnull()]
        data_bundle4['Bundle Name'] = data_bundle4['Product Name']
        data_bundle4['Product Name'] = data_bundle4['Produk 4']
        data_bundle4['SKU'] = data_bundle4['SKU Produk 4']
        data_bundle4['Qty. Invoiced'] = data_bundle4['PCS Produk 4']
        data_bundle4['Price List NFI'] = data_bundle4['Price List NFI 4']
        data_bundle4['Total Net'] = data_bundle4['Price List NFI 4'] 
        data_bundle4['Bundle Flag'] = np.nan

        data_bundle5 = order_all[~order_all['Produk 5'].isnull()]
        data_bundle5['Bundle Name'] = data_bundle5['Product Name']
        data_bundle5['Product Name'] = data_bundle5['Produk 5']
        data_bundle5['SKU'] = data_bundle5['SKU Produk 5']
        data_bundle5['Qty. Invoiced'] = data_bundle5['PCS Produk 5']
        data_bundle5['Price List NFI'] = data_bundle5['Price List NFI 5']
        data_bundle5['Total Net'] = data_bundle5['Price List NFI 5']
        data_bundle5['Bundle Flag'] = np.nan

        data_bundle6 = order_all[~order_all['Produk 6'].isnull()]
        data_bundle6['Bundle Name'] = data_bundle6['Product Name']
        data_bundle6['Product Name'] = data_bundle6['Produk 6']
        data_bundle6['SKU'] = data_bundle6['SKU Produk 6']
        data_bundle6['Qty. Invoiced'] = data_bundle6['PCS Produk 6']
        data_bundle6['Price List NFI'] = data_bundle6['Price List NFI 6']
        data_bundle6['Total Net'] = data_bundle6['Price List NFI 6']
        data_bundle6['Bundle Flag'] = np.nan

        data_bundle7 = order_all[~order_all['Produk 7'].isnull()]
        data_bundle7['Bundle Name'] = data_bundle7['Product Name']
        data_bundle7['Product Name'] = data_bundle7['Produk 7']
        data_bundle7['SKU'] = data_bundle7['SKU Produk 7']
        data_bundle7['Qty. Invoiced'] = data_bundle7['PCS Produk 7']
        data_bundle7['Price List NFI'] = data_bundle7['Price List NFI 7']
        data_bundle7['Total Net'] = data_bundle7['Price List NFI 7']
        data_bundle7['Bundle Flag'] = np.nan

        data_bundle = data_bundle1.append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)
        data_bundle['SKU'] = data_bundle['SKU'].astype(str)
        data_bundle['SKU'] = data_bundle['SKU'].str.replace('\.0$', '', regex = True)
        data_bundle[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']] = data_bundle.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item_y', 'Parent SKU_y']]

        temp = data_bundle[data_bundle['Real SKU'].isnull()].copy()
        temp['SKU'] = temp['SKU'].astype(str).str.replace('(S)','', regex = False)
        temp = temp.merge(data_SKU2[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp.index)

        indeks = data_bundle[data_bundle['Real SKU'].isnull()].index.to_list()
        data_bundle['Real SKU'][indeks] = temp['Real SKU_y'][indeks]
        data_bundle['Real Nama Produk'][indeks] = temp['Nama Produk'][indeks]
        data_bundle['Brand'][indeks] = temp['Brand_y'][indeks]
        data_bundle['Sub Brand'][indeks] = temp['Sub Brand_y'][indeks]
        data_bundle['Parent Item'][indeks] = temp['Parent Item_y'][indeks]
        data_bundle['Parent SKU'][indeks] = temp['Parent SKU_y'][indeks]

        print("Pricing")
        order_all = order_all.append(data_bundle, ignore_index = True, sort = False)

        colname = temp.columns[temp.columns.get_loc('Produk 1') : temp.columns.get_loc('Harga Cost 7') + 1]
        colname_str = [x for x in colname if 'Subtotal' not in x and 'Harga' not in x]
        colname_int = [x for x in colname if x not in colname_str]

        for i in colname_str:
            temp[i] = np.nan

        for i in colname_int:
            temp[i] = 0

        data_order = data_order.append(temp , ignore_index = True, sort = False)


        order_all = order_all.merge(data_SKU2[['SKU', 'Price List NFI', 'Harga Cost']].drop_duplicates('SKU'), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(order_all.index)
        order_all['Price List NFI_x'] = order_all['Price List NFI_x'].fillna(order_all['Price List NFI_y'])
        order_all =  order_all.drop(['Price List NFI_y', 'SKU_y'], axis = 1)
        order_all = order_all.rename(columns = {'SKU_x' : 'SKU', 'Price List NFI_x' : 'Price List NFI'})

        indeks = order_all[order_all['Product Name'] == 'HiLo Teen Chocolate 250gr'].index.to_list()
        order_all['Real Nama Produk'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Parent Item'][indeks] = 'HiLo Teen Chocolate 250gr'
        order_all['Brand'][indeks] = 'HiLo'
        order_all['Sub Brand'][indeks] = 'HILO TEEN'
        order_all['Price List NFI'][indeks] = 36850
        order_all['Harga Cost'][indeks] = 36850
        order_all['Real SKU'][indeks] = '2101651155'
        order_all['Parent SKU'][indeks] = '2101651155'


        order_all['Price List NFI'] = pd.to_numeric(order_all['Price List NFI']).astype(int)
        order_all['Harga Cost'] = pd.to_numeric(order_all['Harga Cost']).astype(int)
        order_all['Qty. Invoiced'] = pd.to_numeric(order_all['Qty. Invoiced']).astype(int)

        order_all['Total Net'] = order_all['Price List NFI'] * order_all['Qty. Invoiced']
        order_all['Total Harga Cost'] = order_all['Harga Cost'] * order_all['Qty. Invoiced']
        order_all['Subtotal'] = order_all['Selling Price'] * order_all['Qty. Invoiced']
        order_all['Total'] = order_all['Selling Price'] * order_all['Qty. Invoiced']

        order_all = order_all.reset_index(drop = True)
        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        order_all['Seller Discount'] = order_all['discount']
        order_all['Shipping'] = order_all['Shipping Cost']

        temp = order_all[order_all['Brand'] != 'Bundle']
        temp['discount'] = temp['discount'] * temp['Total Harga Cost']/temp.groupby(['Order #'])['Total Harga Cost'].transform('sum')
        order_all['discount'][temp.index] = temp['discount']

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Subtotal', 'Total']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Total'][list_nobundle.index] = list_nobundle['Total_y']
        order_all['Subtotal'][list_nobundle.index] = list_nobundle['Subtotal_y']

        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Subtotal'] = temp['Subtotal'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        temp['Selling Price'] = temp['Subtotal']/temp['Qty. Invoiced']
        temp['Total'] = temp['Total'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Total'][temp.index] = temp['Total']
        order_all['Subtotal'][temp.index] = temp['Subtotal']
        order_all['Selling Price'][temp.index] = temp['Selling Price']


        order_all['Order #'] = order_all['Order #'].astype(str).str.replace('.0', '', regex = False)

        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Seller Discount']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Seller Discount'][list_nobundle.index] = list_nobundle['Seller Discount_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Seller Discount'] = temp['Seller Discount'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Seller Discount'][temp.index] = temp['Seller Discount']


        temp = order_all[order_all['Bundle Name'].isnull()]
        temp_group = temp[['Order #','Shipping']].groupby(['Order #']).sum().reset_index()

        temp = order_all.merge(temp_group, how = 'left', on = 'Order #').set_index(order_all.index)
        temp['Shipping_x'] = temp['Shipping_y'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')

        order_all['Shipping'][temp.index] = temp['Shipping_y']
        list_bundle = order_all[order_all['Bundle Flag'] == 'Bundle'][['Order #', 'Product Name', 'Shipping']].groupby(['Order #', 'Product Name']).sum().reset_index()
        list_nobundle = order_all[order_all['Bundle Name'].notnull()]
        list_nobundle = list_nobundle.merge(list_bundle, how = 'left', left_on = ['Order #', 'Bundle Name'], right_on = ['Order #', 'Product Name']).set_index(list_nobundle.index)
        list_nobundle

        order_all['Shipping'][list_nobundle.index] = list_nobundle['Shipping_y']
        temp = order_all[order_all['Bundle Name'].notnull()]
        temp['Shipping'] = temp['Shipping'] * temp['Total Harga Cost']/temp.groupby(['Order #', 'Bundle Name'])['Total Harga Cost'].transform('sum')
        order_all['Shipping'][temp.index] = temp['Shipping']
        order_all['True datetime'] = pd.to_datetime(order_all['True datetime'])
        order_all['Promo'] = np.nan
        order_all['Discount MC'] = np.nan


        order_all['Warehouse Name'] = 'Primary Warehouse'
        order_all['Store'] = 'Order Online'

        order_all['Customer Email'] = order_all['email']
        order_all['Order Status'] = order_all['status']
        order_all['Payment Channel'] = order_all['payment_method']
        order_all['Coupon Code'] = order_all['coupon']
        order_all.columns.to_list()

        order_all_append = order_all[['Sales Order ID', 'Store',
            'Product Name',
            'Customer Name',
            'Phone',
            'Address',
            'Region',
            'City',
            'Zip Code',
            'payment_status',
            'Regular Price',
            'Shipping Courier',
            'Shipping Cost',
            'Subtotal',
            'Qty. Invoiced',
            'SKU',
            'Order #',
            'Invoice Number',
            'Shipping Name',
            'Shipping Address2',
            'Country',
            'AWB',
            'Channel',
            'Order date',
            'Real SKU',
            'Real Nama Produk',
            'Brand',
            'Sub Brand',
            'Parent Item',
            'Parent SKU',
            'Produk 1',
            'SKU Produk 1',
            'PCS Produk 1',
            'Price List NFI 1',
            'Subtotal Produk 1',
            'Harga Display 1',
            'Harga Cost 1',
            'Harga Organik 1',
            'Produk 2',
            'SKU Produk 2',
            'PCS Produk 2',
            'Price List NFI 2',
            'Subtotal Produk 2',
            'Harga Display 2',
            'Harga Cost 2',
            'Harga Organik 2',
            'Produk 3',
            'SKU Produk 3',
            'PCS Produk 3',
            'Price List NFI 3',
            'Subtotal Produk 3',
            'Harga Display 3',
            'Harga Cost 3',
            'Harga Organik 3',
            'Produk 4',
            'SKU Produk 4',
            'PCS Produk 4',
            'Price List NFI 4',
            'Subtotal Produk 4',
            'Harga Display 4',
            'Harga Cost 4',
            'Harga Organik 4',
            'Produk 5',
            'SKU Produk 5',
            'PCS Produk 5',
            'Price List NFI 5',
            'Subtotal Produk 5',
            'Harga Display 5',
            'Harga Cost 5',
            'Harga Organik 5',
            'Produk 6',
            'SKU Produk 6',
            'PCS Produk 6',
            'Price List NFI 6',
            'Subtotal Produk 6',
            'Harga Display 6',
            'Harga Cost 6',
            'Harga Organik 6',
            'Produk 7',
            'SKU Produk 7',
            'PCS Produk 7',
            'Price List NFI 7',
            'Subtotal Produk 7',
            'Harga Display 7',
            'Harga Cost 7',
            'Harga Organik 7',
            'Bundle Flag',
            'Date',
            'Month',
            'Year',
            'Quarter',
            'Week',
            'True datetime',
            'Total',
            'Price List NFI',
            'Total Net',
            'Selling Price',
            'Kecamatan',
            'Kelurahan',
            'Bundle Name',
            'Harga Cost',
            'Total Harga Cost',
            'Seller Discount',
            'Shipping',
            'Promo',
            'Discount MC',
            'Warehouse Name',
            'Customer Email',
            'Order Status',
            'Payment Channel',
            'Coupon Code']]

        data_all = data_all[~data_all['Order #'].astype(str).isin(order_all_append['Order #'].astype(str))]
        data_all = data_all.append(order_all_append, ignore_index = True, sort = False)
        
        del file_name
        order_online_jabar = True
        order_online_cond = True
print('OK')

order_online_makasar
--- 53.00232291221619 seconds ---
Unbundling ====== 6/10
--- 53.03647422790527 seconds ---
Filling Date ====== 7/10
Filling Location
Unbundling
Pricing
order_online_medan
--- 141.66093111038208 seconds ---
Unbundling ====== 6/10
--- 141.76606607437134 seconds ---
Filling Date ====== 7/10
Filling Location
Unbundling
Pricing
order_online_samarinda
--- 210.44138717651367 seconds ---
Unbundling ====== 6/10
--- 210.4815104007721 seconds ---
Filling Date ====== 7/10
Filling Location
Unbundling
Pricing
order_online_lampung
--- 361.3245692253113 seconds ---
Unbundling ====== 6/10
--- 361.3543064594269 seconds ---
Filling Date ====== 7/10
Filling Location
Unbundling
Pricing
order_online_pekanbaru
--- 443.00952982902527 seconds ---
Unbundling ====== 6/10
--- 443.1067113876343 seconds ---
Filling Date ====== 7/10
Filling Location
Unbundling
Pricing
order_online_banjarmasin
--- 536.0420114994049 seconds ---
Unbundling ====== 6/10
--- 536.1066081523895 seconds ---
Filling Date 

In [50]:
order_all[order_all['Price List NFI'].isnull()]

46    nan
Name: Real SKU, dtype: object

In [17]:
print('done')

done


In [22]:
### optional cek item yang error
### to do: add to code jatim jkt jateng
### add to sheet sisanya
order_all[order_all['Regular Price'].isnull()][['SKU','Product Name']].drop_duplicates().values#.to_excel('nhd ed.xlsx',index=False)#.values

array([], shape=(0, 2), dtype=object)

In [23]:
### optional cman utk cek ada kota homdel yang belum ke run atau tidak
for n in [order_online_jatim,order_online_jkt,
          order_online_jateng,order_online_bali,
          order_online_makasar, order_online_medan,
          order_online_samarinda,order_online_lampung,
          order_online_pekanbaru,order_online_banjarmasin,order_online_jabar]:
    print (n)

True
True
True
True
True
True
True
True
True
True
True


In [24]:
### Run - 9
### download file tabi di Main_Sheet_(timo)_(2)
### replace file di var data_nubi
### idea: use sql

import pandas as pd
import numpy as np
import os
import glob

data_all = data_all[data_all['Store Type'] != 'Retail Online']
data_nubi = pd.read_csv(r"C:\Users\steven.nathanael\Downloads\Main_Sheet_(timo)_(2)_Full_Data_data (6).csv")

data_nubi.loc[data_nubi['Customer SO Desc']=='INDOPASIFIK FARMA MEDIKA INDONESIA, PT',['Customer SO Group Area','Customer SO Parent Desc']]='ECOM RETAIL - INDOPASIFIK FARMA MEDIKA INDONESIA'

data_nubi = data_nubi[data_nubi['Customer SO Group Area'].astype(str).str.contains('ECOM RETAIL')]

data_SKU = pd.read_excel(r'SKU_File\data_SKU.xlsx')

data_nubi['Sales Order ID'] = 'Nubi -' + data_nubi['Customer SO Parent Desc'].astype(str)
data_nubi['Channel Order ID'] = 'Nubi -' + data_nubi['Customer SO Parent Desc'].astype(str)
data_nubi['Customer Name'] = 'Nubi Retail Online'

print("Filling Date ====== 7/10")

data_nubi['Date'] = data_nubi['Day of Invoice Date SO']
data_nubi['Month'] = data_nubi['Month of Invoice Date SO']
data_nubi['Year'] = data_nubi['Year of Invoice Date SO']

quarter = pd.DataFrame([['January', 1], ['February', 1], ['March', 1], ['April', 2], ['May', 2], ['June', 2], 
        ['July', 3], ['August', 3], ['September', 3],['October', 4], ['November', 4], ['December', 4]], columns = ['Bulan', 'Quarter'])
data_nubi = data_nubi.merge(quarter, how = 'left', left_on = 'Month', right_on = 'Bulan')
data_nubi = data_nubi.drop(['Bulan'], axis = 1)
data_bulan = pd.DataFrame([{'Bulan' : 'December', 'Number' : 12} ,
        {'Bulan' : 'January' , 'Number': 1},
        {'Bulan' : 'February' , 'Number': 2},
        {'Bulan' : 'March' , 'Number': 3},
        {'Bulan' : 'April' , 'Number': 4},
        {'Bulan' : 'May' , 'Number': 5},
        {'Bulan' : 'June', 'Number': 6},
        {'Bulan' : 'July' , 'Number': 7},
        {'Bulan' : 'August', 'Number' : 8},
        {'Bulan' : 'September', 'Number' : 9},
        {'Bulan' : 'October' , 'Number': 10},
        {'Bulan' : 'November' , 'Number': 11}])
temp = data_nubi.copy()
temp['Day'] = temp['Date']
temp = temp.merge(data_bulan, how = 'left', left_on = 'Month', right_on='Bulan')
temp= temp.rename(columns = {'Month' : 'Bulan', 'Number' : 'Month'})
data_nubi['Week'] = pd.to_datetime(temp[['Year', 'Month', 'Day']]).dt.week
data_nubi['True datetime'] = pd.to_datetime(temp[['Year', 'Month', 'Day']])

data_nubi['Order Date'] = data_nubi['Day of Invoice Date SO']
data_nubi['Paid Date'] = data_nubi['Day of Invoice Date SO']
data_nubi['Status'] = 'Delivered'
data_nubi['Channel'] = data_nubi['Customer SO Parent Desc']
data_nubi['Store'] = data_nubi['Customer SO Group Area']
data_nubi['Currency Code'] = "IDR"
data_nubi['Warehouse Name'] = data_nubi['Customer SO Desc']
data_nubi['Regular Price'] = data_nubi['Sales Value Netto SO'] / data_nubi['Sales Qty SO']
data_nubi['Store Type'] = 'Retail Online'

indeks = data_nubi[data_nubi['Regular Price'].isnull()].index.to_list()
data_nubi['Regular Price'][indeks] = 0

data_nubi['Selling Price'] = data_nubi['Sales Value Netto SO'] / data_nubi['Sales Qty SO']
indeks = data_nubi[data_nubi['Selling Price'].isnull()].index.to_list()
data_nubi['Selling Price'][indeks] = 0

indeks = data_nubi[data_nubi['Sales Qty SO'] == 0].index.to_list()
data_nubi['Selling Price'][indeks] = 0
data_nubi['Regular Price'][indeks] = 0

indeks = data_nubi[data_nubi['Sales Value Netto SO'] == 0].index.to_list()
data_nubi['Selling Price'][indeks] = 0
data_nubi['Regular Price'][indeks] = 0

data_nubi['Sub Total'] = data_nubi['Sales Value Netto SO']
data_nubi['Gross Sales'] = data_nubi['Sales Value Netto SO']

data_nubi = data_nubi.rename(columns = {'Item Group Code' : 'SKU', 'Item Group' : 'Item Name', 'Sales Qty SO' : 'Quantity'})

data_nubi = data_nubi[['Sales Order ID', 'Channel Order ID', 'Customer Name', 'Order Date', 'Paid Date','Status','Channel', 'Store', 'Currency Code', 'Warehouse Name', 'Regular Price', 'Selling Price',
              'Sub Total', 'Gross Sales', 'SKU', 'Item Name', 'Quantity', 'Date', 'Month', 'Year', 'True datetime', "Week", 'Brand', 'Category']]

data_nubi['Order Date'] = data_nubi['True datetime']
data_nubi['Quarter'] = pd.to_datetime(data_nubi['True datetime']).dt.quarter

print("Filling Brand ====== 5/10")
data_nubi['SKU'] = data_nubi['SKU'].astype(str)
data_nubi['Item Name'] = data_nubi['Item Name'].astype(str)
data_SKU['Real SKU'] = data_SKU['SKU']
data_SKU['Real Nama Produk'] = data_SKU['Nama Produk'].astype(str)


data_nubi = data_nubi.merge(data_SKU[['Real SKU', 'Real Nama Produk']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')
data_nubi['Real SKU'] = data_nubi['Real SKU'].astype(str)
data_nubi = data_nubi.merge(data_SKU[['SKU', 'Sub Brand', 'Parent Item', 'Parent SKU','Price List NFI']].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU')
data_nubi['Parent Item'] = data_nubi['Parent Item'].fillna(data_nubi['Item Name'])
data_nubi = data_nubi.drop(['SKU_y'], axis = 1)
data_nubi = data_nubi.rename(columns = {'SKU_x':'SKU'})

indeks = data_nubi[data_nubi['Real SKU'].astype(str) == 'nan'].index.to_list()
data_nubi['Real SKU'][indeks] = data_nubi['SKU'][indeks]

indeks = data_nubi[data_nubi['Real Nama Produk'].astype(str) == 'nan'].index.to_list()
data_nubi['Real Nama Produk'][indeks] = data_nubi['Item Name'][indeks]

data_nubi['Sub Brand'] = data_nubi['Category']

data_nubi['Total'] = data_nubi['Sub Total']
data_nubi['Total Net'] = np.nan

data_nubi = data_nubi.rename(columns={'Channel Order ID' : 'Order #',
                                        'Status' : 'Order Status',
                                        'Order Date' : 'Order date',
                                        'Item Name' :'Product Name',
                                        'Bundle Name' : 'Bundle',
                                        'Shipping Country' : 'Country',
                                        'Shipping Province' : 'Region',
                                        'Shipping City' : 'City',
                                        'Shipping Zip' : 'Zip Code',
                                        'Shipping Address1' : 'Address',
                                        'Shipping Phone' : 'Phone',
                                        'Quantity' : 'Qty. Invoiced',
                                        'Item Price' : 'Regular Price',
                                        'Sub Total' : 'Subtotal'})


temp = data_nubi[data_nubi['Price List NFI'].isnull()]
skuoff=pd.read_excel('data_supp\item gk tau harga.xlsx')
skuoff=skuoff[['Real SKU','Price List NFI']]
skuoff['Real SKU']=skuoff['Real SKU'].astype(str)#.info()
display(temp[~temp['Real SKU'].isin(skuoff['Real SKU'].unique())])
temp=temp.merge(skuoff,how='left',on='Real SKU').rename(columns=({'Price List NFI_y':'Price List NFI'}))
data_nubi=data_nubi[data_nubi['Price List NFI'].notnull()].append(temp)


data_nubi['Price List NFI'] = pd.to_numeric(data_nubi['Price List NFI']).astype(int)
data_nubi['Qty. Invoiced'] = pd.to_numeric(data_nubi['Qty. Invoiced']).fillna(0).astype(int)

data_nubi['Total Net'] = data_nubi['Price List NFI'] * data_nubi['Qty. Invoiced']

data_nubi = data_nubi.reset_index(drop = True)

data_nubi['True datetime'] = pd.to_datetime(data_nubi['True datetime'])
data_nubi['Promo'] = np.nan
data_nubi['Discount MC'] = np.nan
data_nubi['Store Type'] = 'Retail Online'

for i in data_nubi['Brand'].unique():
    indeks = data_nubi[data_nubi['Brand'] == i].index.to_list()
    if i == 'NUTRISARI':
        data_nubi['Brand'][indeks] = 'NS'
    elif i == 'HI LO':
        data_nubi['Brand'][indeks] = 'HiLo'
        
temp = data_nubi[data_nubi['Store'] == 'WARUNG PINTAR DISTRIBUSI - BOGOR, PT'].copy()
data_nubi = data_nubi[data_nubi['Store'] != 'WARUNG PINTAR DISTRIBUSI - BOGOR, PT']

temp['Store'] = 'WARUNG PINTAR'
temp['Channel'] = 'WARUNG PINTAR DISTRIBUSI, PT'
temp['Warehouse Name'] = 'WARUNG PINTAR DISTRIBUSI - BOGOR, PT'

data_nubi = data_nubi.append(temp, ignore_index = True, sort = False)

indeks = data_nubi[data_nubi['Store'] == 'SHOPEE'].index.to_list()
data_nubi['Store'][indeks] = 'Shopee Sell-In'


C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_10148\4209893953.py:12: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  data_nubi = pd.read_csv(r"C:\Users\steven.nathanael\Downloads\Main_Sheet_(timo)_(2)_Full_Data_data (6).csv")


Filling Date ====== 7/10
Filling Brand ====== 5/10


,Sales Order ID,Order #,Customer Name,Order date,Paid Date,Order Status,Channel,Store,Currency Code,Warehouse Name,...,Category,Quarter,Real SKU,Real Nama Produk,Sub Brand,Parent Item,Parent SKU,Price List NFI,Total,Total Net


In [25]:
### ini run
from datetime import date
import glob
import os
cutoff=str((date.today().replace(day=1)-timedelta(1)).replace(day=1))
data_nubinew=data_nubi[data_nubi['True datetime']>=cutoff]

list_of_files = glob.glob('data_nubi\*.xlsx') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)
print(latest_file)

data_nubi=pd.read_excel(latest_file)
# data_nubi['Total Net Before ]
data_nubi=data_nubi[data_nubi['True datetime']<cutoff]
          
data_nubi=data_nubi.append(data_nubinew)

data_nubi.to_excel(f"data_nubi\data_nubi_{date.today()}.xlsx",index=False)

data_nubi\data_nubi_2022-11-28.xlsx


In [22]:
data_nubi[data_nubi['Year']==2022].groupby(['Month','Store'])[['Total Net']].sum().reset_index().pivot(columns='Month',index='Store')['Total Net'].reset_index().fillna(0)#.drop('Month')

Month,Store,April,August,February,January,July,June,March,May,November,October,September
0,"ASTRO TECHNOLOGIES - TANGERANG, PT",0.0,0.0,50292000.0,6629040.0,0.0,0.0,261485400.0,0.0,0.0,0.0,0.0
1,"CIBINONG CENTER INDUSTRIAL ESTATE , PT",0.0,0.0,0.0,0.0,0.0,0.0,250607064.0,0.0,0.0,0.0,0.0
2,ECOM RETAIL - ASTRO,5004324.0,87549696.0,0.0,0.0,75724200.0,31898736.0,0.0,13320000.0,16277040.0,34133832.0,114376176.0
3,ECOM RETAIL - BAYI NINJA JAKARTA,21581100.0,12182148.0,0.0,0.0,9896760.0,10478520.0,0.0,0.0,5850588.0,13598508.0,13274268.0
4,ECOM RETAIL - EMOS,668131200.0,273379680.0,0.0,0.0,272433960.0,471195000.0,0.0,216516600.0,420126120.0,338088240.0,290282760.0
5,ECOM RETAIL - INDOPASIFIK FARMA MEDIKA INDONESIA,0.0,58048560.0,0.0,0.0,8764560.0,0.0,0.0,0.0,11908080.0,14558760.0,27159480.0
6,ECOM RETAIL - JD ID,16596720.0,15158160.0,0.0,0.0,20095440.0,6726600.0,0.0,0.0,0.0,5069370.0,12227760.0
7,ECOM RETAIL - LIFEPACK,0.0,0.0,0.0,0.0,0.0,10389600.0,0.0,9597060.0,0.0,0.0,0.0
8,ECOM RETAIL - SAYUR BOX JAKARTA,81112140.0,19793520.0,0.0,0.0,4155840.0,207976260.0,0.0,29856780.0,14585400.0,10722600.0,11055600.0
9,ECOM RETAIL - SAYUR BOX SURABAYA,28919940.0,41836344.0,0.0,0.0,28147380.0,55053336.0,0.0,0.0,0.0,732600.0,0.0


In [26]:
## ini run
data_all['Store'] = data_all['Store'].astype(str).str.replace('JD Indonesia - Nutrimart', 'JD Indonesia', regex = False)

data_all = data_all[data_all['Store'].isin(['JD Indonesia', 'Lazada', 'Bukalapak', 'Blibli', 'Tokopedia',
       'Shopee', 'Elevenia', 'Nutrimart', 'Order Online','Aladin Mall','TikTok'])]
data_all = data_all.append(data_nubi, ignore_index = True, sort = False)

indeks = data_all[data_all['Store'].isin(['JD Indonesia', 'Lazada', 'Bukalapak', 'Blibli', 'Tokopedia',
       'Shopee', 'Elevenia','Aladin Mall','TikTok'])].index.to_list()
data_all['Store Type'][indeks] = 'Marketplace'

indeks = data_all[data_all['Store'].isin(['Nutrimart', 'Order Online'])].index.to_list()
data_all['Store Type'][indeks] = 'Organic'

indeks = data_all[data_all['Order #'].astype(str).str.contains('Nubi')].index.to_list()
data_all['Store Type'][indeks] = 'Retail Online'

In [27]:
## ini run juga
indeks = data_all[data_all['Product Name'] == 'L-Men Bar Crunchy Chocolate (12sch)'].index.to_list()
data_all['Real Nama Produk'][indeks] = 'L-Men Bar Crunchy Chocolate 12sch'
data_all['Parent Item'][indeks] = 'L-Men Bar Crunchy Chocolate (12 Sch)'
data_all['Brand'][indeks] = 'L-Men'
data_all['Sub Brand'][indeks] = 'L-MEN POWDER'
data_all['Real SKU'][indeks] = '2306592173'
data_all['Parent SKU'][indeks] = '2306592173'

indeks = data_all[data_all['Product Name'] == 'Heavenly Blush Yogurt Drink To Go Peach (1 pc)'].index.to_list()
data_all['Brand'][indeks] = 'Heavenly Blush'

indeks = data_all[data_all['Brand'] == 'HB'].index.to_list()
data_all['Brand'][indeks] = 'Heavenly Blush'

In [28]:
## ini run juga
data_all['Region Group'] = np.nan

index = data_all[data_all['Region'].isin(['Dki Jakarta', 'Banten'])].index.to_list()
data_all['Region Group'][index] = 'Jabodetabek'

list_city = ['Kota Bekasi', 'Kabupaten Bekasi', 'Kab. Bekasi', 'Kota Bogor', 'Kabupaten Bogor', 'Kab. Bogor', 'Kota Depok', 'Kota Tangerang',
             'Kota Tangerang Selatan','Kabupaten Tangerang', 'Kab. Tangerang']

index = data_all[data_all['City'].isin(list_city)].index.to_list()
data_all['Region Group'][index] = 'Jabodetabek'

data_all['Region Group'] = data_all['Region Group'].fillna(data_all['Region'])

In [29]:
print("LET'S TRY FOR EMAIL")

LET'S TRY FOR EMAIL


In [32]:
print("Export Master Data")

data_all['True datetime'] = pd.to_datetime(data_all['True datetime'])

data_all['PL Before PPN'] = data_all['Price List NFI']/1.1
data_all.loc[data_all['True datetime']>='2022-04-01','PL Before PPN']= data_all['Price List NFI']/1.11
data_all['Total Net Before PPN'] = data_all['PL Before PPN'] * data_all['Qty. Invoiced']

index = data_all[
    (pd.to_datetime(data_all['True datetime']) > '2020-09-11') &
    (data_all['Real SKU'].astype(str) == '1101909453') 
].index.to_list()

data_all['Real SKU'][index] = '1101909331'

index = data_all[
    (pd.to_datetime(data_all['True datetime']) > '2020-09-11') &
    (data_all['Real SKU'].astype(str) == '1101569453') 
].index.to_list()

data_all['Real SKU'][index] = '1101569326'

index = data_all[
    (pd.to_datetime(data_all['True datetime']) > '2020-09-11') &
    (data_all['Real SKU'].astype(str) == '1101907453') 
].index.to_list()

data_all['Real SKU'][index] = '1101907331'

index = data_all[data_all['Real SKU'] == '(B) 2101656'].index.to_list()
data_all['Real SKU'][index] = '(B)2101656'

index = data_all[data_all['Real SKU'] == '(B)1101989453'].index.to_list()
data_all['Brand'][index] = 'Bonus Produk'

data_all.loc[data_all['SKU'].isin(['(U)2104115163', '(U)2101809250', '(U)2309005305',
                                   '(U)2101845443',"(U)2101947250", '(B)71210295',
                                   '(B)2106323165']),'Brand']='Bonus Produk'


if 'Order Online' not in data_now:
    data_now = data_now + ' With Order Online'
SKU = pd.read_excel(r'SKU_File/data_SKU.xlsx')
SKU_baru = pd.read_excel(r"SKU_File/Subbrand Baru.xlsx")

SKU_final = SKU.copy()
SKU_baru['Item Group Code'] = SKU_baru['Item Group Code'].astype(str).str.replace('.0','', regex = False)
SKU_final['SKU Clean'] = SKU_final['SKU'].astype(str).str.replace('(S)', '', regex = False).str.replace('(R)', '', regex = False).str.replace('(B)', '', regex = False).str.replace('(50%)', '', regex = False).str.replace('(E)', '', regex = False)
SKU_final = SKU_final.merge(SKU_baru.drop_duplicates('Item Group Code'), how = 'left', left_on = 'SKU Clean', right_on = 'Item Group Code')

no_pair = SKU[SKU['SKU'].astype(str).isin(SKU_final[SKU_final['Category Baru'].isnull()]['SKU'].astype(str))]
SKU_final = SKU_final[~SKU_final['SKU'].astype(str).isin(no_pair['SKU'].astype(str))]

no_pair['SKU Clean'] = no_pair['SKU'].astype(str).str.replace('(S)', '', regex = False).str.replace('(R)', '', regex = False).str.replace('(B)', '', regex = False).str.replace('(50%)', '', regex = False).str.replace('(E)', '', regex = False).str.replace('E', '', regex = False)
SKU_baru['Item Group Code'] = SKU_baru['Product Code'].astype(str).str.replace('.0','', regex = False)

index = no_pair[no_pair['SKU'].astype(str) == '1101984451'].index[0]
no_pair['SKU Clean'][index] = '1101984453'

index = no_pair[no_pair['SKU'].astype(str) == '2104393210'].index[0]
no_pair['SKU Clean'][index] = '2104392210'

no_pair = no_pair.merge(SKU_baru.drop_duplicates('Product Code'), how = 'left', left_on = 'SKU Clean', right_on = 'Product Code')

SKU_final = SKU_final.append(no_pair, ignore_index = True, sort = False)

index = SKU_final[SKU_final['SKU'].astype(str) == '1101569360'].index[0]
SKU_final['Category Baru'][index] = 'NS MODERN'
SKU_final['Unnamed: 3'][index] = "NS JERUK PERAS REF 12DX500G"

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)1101569360'].index[0]
SKU_final['Category Baru'][index] = 'NS MODERN'
SKU_final['Unnamed: 3'][index] = "NS JERUK PERAS REF 12DX500G"

index = SKU_final[SKU_final['SKU'].astype(str) == '1101686036'].index[0]
SKU_final['Category Baru'][index] = 'WDANK TRADITIONAL'
SKU_final['Unnamed: 3'][index] = "W'DANK LKLT KOPI KAWISTA PLS 12RX10SX15G"

index = SKU_final[SKU_final['SKU'].astype(str).isin(["(E)2104407",'2104407'])].index.to_list()
SKU_final['Category Baru'][index] = 'TS MERAH'
SKU_final['Unnamed: 3'][index] = 'TS EXTRA VIRGIN OLIVE OIL 12BTLX500ML'

index = SKU_final[SKU_final['SKU'].astype(str) == '2305551106'].index[0]
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN PLATINUM CHOCO LATTE 6DX6SX33.5G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2305551106'].index[0]
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN PLATINUM CHOCO LATTE 6DX6SX33.5G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(B)2305551106'].index[0]
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN PLATINUM CHOCO LATTE 6DX6SX33.5G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2304034180'].index[0]
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN GM MANGGA 6DX500G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['(R)1101588453', '1101588453', '(B)(R)1101588453', '1101588016'])].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS MILKY ORANGE PLS 4PX40SX11G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(R)1101588453'].index[0]
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS MILKY ORANGE PLS 4PX40SX11G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101452195'].index[0]
SKU_final['Category Baru'][index] = 'HILO ACTIVE'
SKU_final['Unnamed: 3'][index] = 'HILO ACTIVE CHOCOLATE 6DX1000G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101500195'].index[0]
SKU_final['Category Baru'][index] = 'HILO GOLD'
SKU_final['Unnamed: 3'][index] = 'HILO GOLD PLAIN 6DX1000G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101524180'].index[0]
SKU_final['Category Baru'][index] = 'HILO GOLD'
SKU_final['Unnamed: 3'][index] = 'HILO GOLD BISCUIT CEREAL 12DX500G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101453195'].index[0]
SKU_final['Category Baru'][index] = 'HILO SCHOOL'
SKU_final['Unnamed: 3'][index] = 'HILO SCHOOL CHOCOLATE 6DX1000G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101651195'].index[0]
SKU_final['Category Baru'][index] = 'HILO TEEN'
SKU_final['Unnamed: 3'][index] = 'HILO TEEN CHOCOLATE 6DX1000G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2102546125'].index[0]
SKU_final['Category Baru'][index] = 'TS KUNING - SWT POWDER'
SKU_final['Unnamed: 3'][index] = 'TS SWT JAHE 24Dx50SX2.5G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(B)2102546125'].index[0]
SKU_final['Category Baru'][index] = 'TS KUNING - SWT POWDER'
SKU_final['Unnamed: 3'][index] = 'TS SWT JAHE 24Dx50SX2.5G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2102546125'].index[0]
SKU_final['Category Baru'][index] = 'TS KUNING - SWT POWDER'
SKU_final['Unnamed: 3'][index] = 'TS SWT JAHE 24Dx50SX2.5G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2304034180'].index[0]
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN GM MANGGA 6DX500G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['(R)1101989453', '1101989453', '(B)(R)1101989453'])].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS ES CINCAU PLS 4PX40SX13G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['(R)1101954453', '1101954453', '(B)(R)1101954453'])].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS SEMANGKA PLS 4PX40SX13G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['(R)1101927453', '1101927453', '(B)(R)1101927453','(E)1101927453'])].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS NANAS PLS 4PX40SX13G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['1101538453'])].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS ANGGUR HIJAU PLS 4PX40SX11G '

index = SKU_final[SKU_final['SKU'].astype(str) == '2104170104'].index[0]
SKU_final['Category Baru'][index] = 'TS KUNING OTHERS'
SKU_final['Unnamed: 3'][index] = 'TS WHITE COFFEE 12DX4SX15G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['1101656318', '(B)1101656318', '(B)(R)1101656318','(E)1101656318'])].index.to_list()
SKU_final['Category Baru'][index] = 'WDANK TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'LOKALATE KOPI BERONDONG PLS 12RX10SX15G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['(R)1101930453', '(B)(R)1101930453', '1101930453',"(E)1101930453"])].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS COCOPANDAN PLS 4PX40SX14G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['71210158', '(J)71110121'])].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET NUTRISARI'
SKU_final['Unnamed: 3'][index] = 'PAKET NUTRISARI 25 RASA'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['(J)71110126'])].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET NUTRISARI'
SKU_final['Unnamed: 3'][index] = 'PAKET NUTRISARI 25 RASA FRESHSTART'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['1102574110'])].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET NUTRISARI'
SKU_final['Unnamed: 3'][index] = 'NS MIX PAKET BALI 12Dx10S'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['71210165'])].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET WDANK'
SKU_final['Unnamed: 3'][index] = 'PAKET KEHANGATAN WDANK'

index = SKU_final[SKU_final['SKU'].astype(str) == '2104170164'].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING OTHERS'
SKU_final['Unnamed: 3'][index] = 'TS WHITE COFFEE 12DX4SX20G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2309005300', '2309005305','(E)2309005300'])].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN PROTEIN CRUNCH BBQ BEEF 20BX20G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['(E)1101688317','1101688317'])].index.to_list()
SKU_final['Category Baru'][index] = 'WDANK TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS W’DANK EMPON-EMPON PLS 12Rx10Sx12G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2306551173'].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN BAR CRUNCHY CHOCOLATE 6SBX12SX22G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2306551173'].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN BAR CRUNCHY CHOCOLATE 6SBX12SX22G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101813443'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'HILO ES TELER PLS 8RX10SX15G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2101813443'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'HILO ES TELER PLS 8RX10SX15G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2101813036'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'HILO ES TELER FC 36PX10SX15G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101864443'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'HILO ES PISANG IJO PLS 8RX10SX15G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101845360'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'HILO ES KETAN HITAM REF 12BAGX500G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101845443'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'HILO ES KETAN HITAM PLS 8RX10SX14G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2101845443'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'HILO ES KETAN HITAM PLS 8RX10SX14G'

index = SKU_final[SKU_final['SKU'].astype(str) == '71210166'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET WDANK'
SKU_final['Unnamed: 3'][index] = 'PAKET LOKALATE UNTUK SOBATKU'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2304008180'])].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN GAINMASS TARO 6DX500G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2305551161'])].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN PLATINUM CHOCO LATTE 6DX6SX38.5G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2305507288',"(E)2305507288"])].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN PLATINUM KACANG HIJAU 6KLRX800G'

index = SKU_final[SKU_final['SKU'].astype(str) == '71210163'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET NUTRISARI'
SKU_final['Unnamed: 3'][index] = 'PAKET HAMPERS IMLEK NUTRISARI'

index = SKU_final[SKU_final['SKU'].astype(str) == '2104508105'].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING OTHERS'
SKU_final['Unnamed: 3'][index] = 'TS KOREAN GARLIC BUTTER COOKIES 12DX5SX20G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2104148164'].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING OTHERS'
SKU_final['Unnamed: 3'][index] = 'TS MINT COCOA 12DX4SX15G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2T00803164'].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING OTHERS'
SKU_final['Unnamed: 3'][index] = 'TS ORANGE COCOA 12DX4SX15G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2104214210'].index.to_list()
SKU_final['Category Baru'][index] = 'TS MERAH'
SKU_final['Unnamed: 3'][index] = 'TS SAMBAL TERASI 24BTLX200G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2104214210'].index.to_list()
SKU_final['Category Baru'][index] = 'TS MERAH'
SKU_final['Unnamed: 3'][index] = 'TS SAMBAL TERASI 24BTLX200G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2104115163'].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING OTHERS'
SKU_final['Unnamed: 3'][index] = 'TS AVOCADO COFFEE 12DX4SX20G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2101609180','(E)2101609180'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TEEN'
SKU_final['Unnamed: 3'][index] = 'HILO TEEN TARO 12DX500G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2101445144','(E)2101445144'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO ACTIVE'
SKU_final['Unnamed: 3'][index] = 'HILO ACTIVE KETAN HITAM 12DX175G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2101485195'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO ACTIVE'
SKU_final['Unnamed: 3'][index] = 'HILO ACTIVE VANILLA 6DX1000G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2102526125'].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING - SWT POWDER'
SKU_final['Unnamed: 3'][index] = 'TS SWT LEMONGRASS PANDAN 24DX50SX2G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2104394210'].index.to_list()
SKU_final['Category Baru'][index] = 'TS MERAH'
SKU_final['Unnamed: 3'][index] = 'TS SAUS TIRAM 24BTLX200ML'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2305545288',"(E)2305545288"])].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN PLATINUM KETAN HITAM 6KLRX800G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2106023014', '(B)2106023014',"(E)2106023014"])].index.to_list()
SKU_final['Category Baru'][index] = 'TS MERAH'
SKU_final['Unnamed: 3'][index] = 'TS SHIRATAKI NOODLES 40PX71G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2T03201001'])].index.to_list()
SKU_final['Category Baru'][index] = 'TS MERAH'
SKU_final['Unnamed: 3'][index] = 'TS BUMBU KALDU AYAM JAMUR 24PX100G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2106323165'])].index.to_list()
SKU_final['Category Baru'][index] = 'TS MERAH'
SKU_final['Unnamed: 3'][index] = 'TS BUMBU SAUS TELUR ASIN 24Dx3Sx22G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110113'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET HILO'
SKU_final['Unnamed: 3'][index] = 'PAKET TAKJIL HILO DESSERT'

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110110'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET TS'
SKU_final['Unnamed: 3'][index] = 'HAMPER IDUL FITRI TS'

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110129'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET TS'
SKU_final['Unnamed: 3'][index] = 'PAKET TS BEAT HYPERTENSION 2022'

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110114'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET NUTRISARI'
SKU_final['Unnamed: 3'][index] = 'PAKET RAMADHAN NUTRISARI 2021'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101413144'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO ACTIVE'
SKU_final['Unnamed: 3'][index] = 'HILO ACTIVE ES TELER 12DX175G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2101413144'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO ACTIVE'
SKU_final['Unnamed: 3'][index] = 'HILO ACTIVE ES TELER 12DX175G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110111'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET WDANK'
SKU_final['Unnamed: 3'][index] = 'PAKET KEHANGATAN WDANK - RAMADHAN EDITION'

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110115'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET TS'
SKU_final['Unnamed: 3'][index] = 'GETPLUS HAMPERS IDUL FITRI TS'

index = SKU_final[SKU_final['SKU'].astype(str) == '2102584125'].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING - SWT POWDER'
SKU_final['Unnamed: 3'][index] = 'TS SWT ROSE VANILLA 24DX50SX2G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101642180'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TEEN'
SKU_final['Unnamed: 3'][index] = 'HILO TEEN POPCORN CARAMEL 12DX500G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2101642180'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TEEN'
SKU_final['Unnamed: 3'][index] = 'HILO TEEN POPCORN CARAMEL 12DX500G'

index = SKU_final[SKU_final['SKU'].astype(str) == '1101543453'].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS APEL JERUK PLS 4PX40SX14G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(B) 2101656'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO RTD'
SKU_final['Unnamed: 3'][index] = 'HILO TEEN RTD COFFEE TIRAMISU 24PX200ML'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101816250'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO RTD'
SKU_final['Unnamed: 3'][index] = 'HILO RTD VANILA COOKIES 24PX200ML'

index = SKU_final[SKU_final['SKU'].astype(str) == '2106386249'].index.to_list()
SKU_final['Category Baru'][index] = 'TS BIRU'
SKU_final['Unnamed: 3'][index] = 'TS RTD OAT DRINK VANILLICIOUS 24PX190ML'

index = SKU_final[SKU_final['SKU'].astype(str) == '2105028180'].index.to_list()
SKU_final['Category Baru'][index] = 'TS BIRU'
SKU_final['Unnamed: 3'][index] = 'TS LOW FAT MILK KOREAN STRAWBERRY 12DX500G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2104241210'].index.to_list()
SKU_final['Category Baru'][index] = 'TS MERAH'
SKU_final['Unnamed: 3'][index] = 'TS MAYONNAISE ROASTED SESAME 24BTLX200G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2104241210'].index.to_list()
SKU_final['Category Baru'][index] = 'TS MERAH'
SKU_final['Unnamed: 3'][index] = 'TS MAYONNAISE ROASTED SESAME 24BTLX200G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['1101996453',"(E)1101996453"])].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS ES RUJAK JERUK BALI PLS 4PX40SX14G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2300542155'].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN DAILY POPCORN CARAMEL 12DX250G'

index = SKU_final[SKU_final['SKU'].astype(str) == '1102110453'].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS NUTRI C1000 JERUK PLS 4PX40SX6G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110117'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET NUTRISARI'
SKU_final['Unnamed: 3'][index] = 'NUTRILOGI SERI MBA NANA SI PENUH PESONA'

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110116'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET NUTRISARI'
SKU_final['Unnamed: 3'][index] = 'NUTRILOGI SERI MAS KAKA YANG BANYAK AKAL'

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110119'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET NUTRISARI'
SKU_final['Unnamed: 3'][index] = 'NUTRILOGI SERI JEJE SI JELI'

index = SKU_final[SKU_final['SKU'].astype(str) == '1102560453'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET NUTRISARI'
SKU_final['Unnamed: 3'][index] = 'NUTRISARI 4 RASA SUMMER PACKAGE 40 SACHET'

index = SKU_final[SKU_final['SKU'].astype(str) == '1102561453'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET NUTRISARI'
SKU_final['Unnamed: 3'][index] = 'NUTRISARI 4 RASA LOCAL PACKAGE 40 SACHET'

index = SKU_final[SKU_final['SKU'].astype(str) == '2104543105'].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING OTHERS'
SKU_final['Unnamed: 3'][index] = 'TS KOREAN GOGUMA COOKIES 12DX5SX20G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101651155'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TEEN'
SKU_final['Unnamed: 3'][index] = 'HILO TEEN CHOCOLATE 12DX250G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110118'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET NUTRISARI'
SKU_final['Unnamed: 3'][index] = 'PAKET ORANGE ADVENTURE'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101578180'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO GOLD'
SKU_final['Unnamed: 3'][index] = 'HILO GOLD SWEET POTATO 12DX500G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101551195'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO GOLD'
SKU_final['Unnamed: 3'][index] = 'HILO GOLD CHOCOLATE 6DX1000G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101700166'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO GOLD'
SKU_final['Unnamed: 3'][index] = 'HILO PLATINUM ORIGINAL 12DX12SX30G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101710110'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO GOLD'
SKU_final['Unnamed: 3'][index] = 'HILO JOINT PLUS 12DX10SX14G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2101710110'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO GOLD'
SKU_final['Unnamed: 3'][index] = 'HILO JOINT PLUS 12DX10SX14G'

index = SKU_final[SKU_final['SKU'].astype(str) == '1101693318'].index.to_list()
SKU_final['Category Baru'][index] = 'WDANK TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'LOKALATE KOPI TAPE KETAN PLS 12RX10SX15G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)1101693318'].index.to_list()
SKU_final['Category Baru'][index] = 'WDANK TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'LOKALATE KOPI TAPE KETAN PLS 12RX10SX15G'

index = SKU_final[SKU_final['SKU'].astype(str) == '1101694318'].index.to_list()
SKU_final['Category Baru'][index] = 'WDANK TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'LOKALATE KOPI ANDALIMAN PLS 12RX10SX15G'

index = SKU_final[SKU_final['SKU'].astype(str) == '1101653318'].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = "NS W'DANK SARABBA EXTRA PLS 12RX10SX15G"

index = SKU_final[SKU_final['SKU'].astype(str) == '1101618318'].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = "NS W'DANK MADU TEMULAWAK PLS 12RX10SX11G"

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110120'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET WDANK'
SKU_final['Unnamed: 3'][index] = 'PAKET LOKALATE #RASALOKAL'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101444180'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO SCHOOL'
SKU_final['Unnamed: 3'][index] = 'HILO SCHOOL STRAW CHEESECAKE 12DX500G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101486195'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO SCHOOL'
SKU_final['Unnamed: 3'][index] = 'HILO SCHOOL VANILLA 6DX1000G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2101487148', '(B)2101487148','(E)2101487148'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO ACTIVE'
SKU_final['Unnamed: 3'][index] = 'HILO ALMOND MILK COCONUT 12DX200G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2101469180'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO ACTIVE'
SKU_final['Unnamed: 3'][index] = 'HILO MULTIGRAIN ORIGINAL 12DX500G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2101683180', '(B)2101683180'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TEEN'
SKU_final['Unnamed: 3'][index] = 'HILO TEEN STRAW MILKSHAKE 12DX500G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2304097159'])].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN PLANTPROTEIN OGURA 6Dx216G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['(E)2304097159'])].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN PLANTPROTEIN OGURA 6DX216G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2106395308','(E)2106395308'])].index.to_list()
SKU_final['Category Baru'][index] = 'TS MERAH'
SKU_final['Unnamed: 3'][index] = 'TS SHIRATAKI RICE RASA NASI UDUK 24PX72G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['1102070350'])].index.to_list()
SKU_final['Category Baru'][index] = 'NS MODERN'
SKU_final['Unnamed: 3'][index] = "NS TEA LYCHEE TEA REF 12BAGX500G"

index = SKU_final[SKU_final['SKU'].astype(str).isin(['1101505453'])].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = "NS JERUK NIPIS JAHE PLS 4PX40SX11G"

index = SKU_final[SKU_final['SKU'].astype(str).isin(['1N01402250'])].index.to_list()
SKU_final['Category Baru'][index] = 'NS RTD'
SKU_final['Unnamed: 3'][index] = "NS RTD SQUEEZED ORANGE 24TPKX200ML"

index = SKU_final[SKU_final['SKU'].astype(str).isin(['1101595453'])].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = "NS ES KUWUD NIPIS PLS 4PX40SX11G"

index = SKU_final[SKU_final['SKU'].astype(str).isin(['1101591453'])].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = "NS GULA ASEM PLS 4PX40SX11G"

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2104589105','(B)2104589105'])].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING OTHERS'
SKU_final['Unnamed: 3'][index] = "TS COOKIES KLEPON 12DX5SX20G"

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2304515112'])].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = "L-MEN LOSE WEIGHT AVOCADO COFFEE 6DX12SX25G"

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2LD0905181'])].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = "L-MEN GAINMASS KLEPON LATTE 6DX500G"

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2101453607'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO SCHOOL'
SKU_final['Unnamed: 3'][index] = "HILO SCHOOL CHOCOLATE 12DX250G"

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2101459180'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO SCHOOL'
SKU_final['Unnamed: 3'][index] = "HILO SCHOOL BUBBLE GUM 12DX500G"

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2101478180'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO ACTIVE'
SKU_final['Unnamed: 3'][index] = "HILO ACTIVE CARAMEL LATTE 12DX500G"

index = SKU_final[SKU_final['SKU'].astype(str).isin(['(E)2101478180'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO ACTIVE'
SKU_final['Unnamed: 3'][index] = "HILO ACTIVE CARAMEL LATTE 12DX500G"

index = SKU_final[SKU_final['SKU'].astype(str) == '(J)71110124'].index.to_list()
SKU_final['Category Baru'][index] = 'PAKET TS'
SKU_final['Unnamed: 3'][index] = 'PARSEL NATAL TAHUN BARU'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101479180'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO SCHOOL'
SKU_final['Unnamed: 3'][index] = 'HILO SCHOOL COTTON CANDY 12DX500G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2HF1403250'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO RTD'
SKU_final['Unnamed: 3'][index] = 'HILO SCHOOL RTD COTTON CANDY 24TPKX200ML'

index = SKU_final[SKU_final['SKU'].astype(str) == '1101692318'].index.to_list()
SKU_final['Category Baru'][index] = 'WDANK TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'LOKALATE KOPI PISANG BAKAR EPE PLS 12RX10SX15G'

index = SKU_final[SKU_final['SKU'].astype(str) == '1101656360'].index.to_list()
SKU_final['Category Baru'][index] = 'WDANK TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'LOKALATE KOPI BERONDONG 12BAGX500G'

index = SKU_final[SKU_final['SKU'].astype(str) == '1101685360'].index.to_list()
SKU_final['Category Baru'][index] = 'WDANK TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'LOKALATE KOPI ALPUKAT 12BAGX500G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)1101686036'].index[0]
SKU_final['Category Baru'][index] = 'WDANK TRADITIONAL'
SKU_final['Unnamed: 3'][index] = "W'DANK LKLT KOPI KAWISTA PLS 12RX10SX15G"

index = SKU_final[SKU_final['SKU'].astype(str) == '2101888360'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'HILO KLEPON LATTE REF 12BAGX500G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101847360'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'HILO THAI TEA REF 12BAGX500G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2101819360'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'HILO TARO LATTE REF 12BAGX500G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2101481118'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'HILO SCHOOL CHOCOLATE CANDY 12SBX20SX8G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2106317152'])].index.to_list()
SKU_final['Category Baru'][index] = 'TS MERAH'
SKU_final['Unnamed: 3'][index] = 'TS INSTANT CAKE MIX BROWNIES 12Dx230G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2305559288'].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN PLATINUM BUBBLE GUM 6KLRX800G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2305584288'].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN PLATINUM VANILLA CARAMEL 6KLRX800G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2304576112'].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN LW MANGO STICKY RICE 6DX12SX25G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2105055180'])].index.to_list()
SKU_final['Category Baru'][index] = 'TS BIRU'
SKU_final['Unnamed: 3'][index] = 'TS LOW FAT MILK MACCHIATO COFFEE 12DX500G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2101618180'])].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TEEN'
SKU_final['Unnamed: 3'][index] = 'HILO TEEN BISCUIT CARAMEL 12DX500G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2154084141'])].index.to_list()
SKU_final['Category Baru'][index] = 'DIABETAMIL'
SKU_final['Unnamed: 3'][index] = 'DIABETAMIL MILK VANILLA 12Dx150G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['2104119110'])].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING OTHERS'
SKU_final['Unnamed: 3'][index] = 'TS SOY LATTE 12Dx10Sx15G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2T01812126'].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING - SWT POWDER'
SKU_final['Unnamed: 3'][index] = 'TS SWT GULA AREN 24DX50SX2G'

index = SKU_final[SKU_final['SKU'].astype(str).isin(['1102611360'])].index.to_list()
SKU_final['Category Baru'][index] = 'NS MODERN'
SKU_final['Unnamed: 3'][index] = "NS TEA LEMON TEA REF 12BAGX500G"

index = SKU_final[SKU_final['SKU'].astype(str) == '2HH1407250'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO RTD'
SKU_final['Unnamed: 3'][index] = 'HILO DRINK RTD CHOC AVOCADO 24TPKX200ML'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)1101694318'].index.to_list()
SKU_final['Category Baru'][index] = 'WDANK TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'LOKALATE KOPI ANDALIMAN PLS 12RX10SX15G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2101888360'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'HILO KLEPON LATTE REF 12BAGX500G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2102526125'].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING - SWT POWDER'
SKU_final['Unnamed: 3'][index] = 'TS SWT LEMONGRASS PANDAN 24DX50SX2G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)1101543453'].index.to_list()
SKU_final['Category Baru'][index] = 'NS TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'NS APEL JERUK PLS 4PX40SX14G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2101500195'].index[0]
SKU_final['Category Baru'][index] = 'HILO GOLD'
SKU_final['Unnamed: 3'][index] = 'HILO GOLD PLAIN 6DX1000G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2101444180'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO SCHOOL'
SKU_final['Unnamed: 3'][index] = 'HILO SCHOOL STRAW CHEESECAKE 12DX500G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2104394210'].index.to_list()
SKU_final['Category Baru'][index] = 'TS MERAH'
SKU_final['Unnamed: 3'][index] = 'TS SAUS TIRAM 24BTLX200ML'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2101651195'].index[0]
SKU_final['Category Baru'][index] = 'HILO TEEN'
SKU_final['Unnamed: 3'][index] = 'HILO TEEN CHOCOLATE 6DX1000G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)1101694108'].index.to_list()
SKU_final['Category Baru'][index] = 'WDANK TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'LOKALATE KOPI ANDALIMAN 12DX8SX15G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2HF0111003'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO SCHOOL'
SKU_final['Unnamed: 3'][index] = 'HI LO SCHOOL STRAWBERRY PLS 12RX10SX27G'

index = SKU_final[SKU_final['SKU'].astype(str) == '1101686318'].index.to_list()
SKU_final['Category Baru'][index] = 'WDANK TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'LOKALATE KOPI KAWISTA PLS 12RX10SX15G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2101453195'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO SCHOOL'
SKU_final['Unnamed: 3'][index] = 'HILO SCHOOL CHOCOLATE 6DX1000G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2HF0112021'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO SCHOOL'
SKU_final['Unnamed: 3'][index] = 'HI LO SCHOOL 3+ SOYA VANILLA MALT 12DX400G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2300542155'].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN DAILY POPCORN CARAMEL 12DX250G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2101578180'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO GOLD'
SKU_final['Unnamed: 3'][index] = 'HILO GOLD SWEET POTATO 12DX500G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2104508105'].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING OTHERS'
SKU_final['Unnamed: 3'][index] = 'TS KOREAN GARLIC BUTTER COOKIES 12DX5SX20G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)1101656108'].index.to_list()
SKU_final['Category Baru'][index] = 'WDANK TRADITIONAL'
SKU_final['Unnamed: 3'][index] = 'LOKALATE KOPI BERONDONG 12Dx8Sx15G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2104115110'].index.to_list()
SKU_final['Category Baru'][index] = 'TS KUNING OTHERS'
SKU_final['Unnamed: 3'][index] = 'TS AVOCADO COFFEE 12DX10SX14G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2101479180'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO SCHOOL'
SKU_final['Unnamed: 3'][index] = 'HILO SCHOOL COTTON CANDY 12DX500G'

index = SKU_final[SKU_final['SKU'].astype(str) == '2HF0113021'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO SCHOOL'
SKU_final['Unnamed: 3'][index] = 'HI LO SCHOOL 3+ STRAWBERRY POP 12DX400G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2101452195'].index.to_list()
SKU_final['Category Baru'][index] = 'HILO ACTIVE'
SKU_final['Unnamed: 3'][index] = 'HILO ACTIVE CHOCOLATE 6DX1000G'

index = SKU_final[SKU_final['SKU'].astype(str) == '(E)2304515112'].index.to_list()
SKU_final['Category Baru'][index] = 'L-MEN POWDER'
SKU_final['Unnamed: 3'][index] = 'L-MEN LOSE WEIGHT AVOCADO COFFEE 6DX12SX25G'

SKU = SKU.merge(SKU_final[['SKU', 'Category Baru', 'Unnamed: 3']].drop_duplicates('SKU'), how = 'left', on = 'SKU')
SKU = SKU.rename(columns = {'Unnamed: 3' : 'Exported Parent Item'})

index = data_all[data_all['Real SKU'] == '2101492P24'].index.to_list()
data_all['Brand'][index] = 'Bundle'
index = data_all[data_all['SKU'].isin(['71210111', '71210112'])].index.to_list()
data_all['Brand'][index] = 'Bundle'
index = data_all[data_all['Real SKU'] == '2306592173'].index.to_list()
data_all['Brand'][index] = 'L-Men'
indeks = data_all[data_all['Brand'] == 'TS'][data_all[data_all['Brand'] == 'TS']['Real SKU'].astype(str) == '2101453180'].index.to_list()
data_all['Real SKU'][indeks] = data_all['SKU'][indeks]
data_all['Parent Item'][indeks] = 'Tropicana Slim Milk Low Fat Vanilla 500gr'
index = data_all[data_all['Brand'] == "WDANK"].index.to_list()
data_all['Brand'][index] = 'NS'
index = data_all[data_all['Brand'] == "W'dank"].index.to_list()
data_all['Brand'][index] = 'NS'

data_all['Real SKU'] = data_all['Real SKU'].astype(str).str.replace('.0', '', regex = False)

no_name = data_all[data_all['Exported Parent Item'].isnull()]
no_name = no_name.merge(SKU[['SKU', 'Category Baru', 'Exported Parent Item']].drop_duplicates('SKU'), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(no_name.index)

data_all['Category Baru'][no_name.index] = no_name['Category Baru_y']
data_all['Exported Parent Item'][no_name.index] = no_name['Exported Parent Item_y']

data_SKU = SKU.copy()
data_SKU['Category Baru'] = data_SKU['Category Baru'].fillna(data_SKU['Sub Brand'])
data_SKU['Category Baru'] = data_SKU['Category Baru'].fillna(data_SKU['Brand'])
data_SKU['Sub Brand'] = data_SKU['Category Baru']
data_all['Sub Brand'] = data_all['Category Baru']

index = data_all[data_all['Real SKU'].astype(str) == '2102000155'].index.to_list()
data_all['Category Baru'][index] = 'TS KUNING - SWT POWDER'

index = data_all[data_all['Real SKU'].astype(str) == '2153000314'].index.to_list()
data_all['Category Baru'][index] = 'TS KUNING - SWT POWDER'

index = data_all[data_all['Real SKU'].astype(str) == '2304097159'].index.to_list()
data_all['Category Baru'][index] = 'L-MEN POWDER'
data_all['Sub Brand'][index] = 'L-MEN POWDER'

index = data_all[data_all['Real SKU'].astype(str) == '2101651155'].index.to_list()
data_all['Category Baru'][index] = 'HILO TEEN'
data_all['Exported Parent Item'][index] = 'HILO TEEN CHOCOLATE 12DX250G'



temp = data_all.copy()
no_name['Real SKU'] = no_name['Real SKU'].astype(str).str.replace('(S)','',regex = False).str.replace('.0','',regex = False)
data_SKU['SKU'] = data_SKU['SKU'].astype(str).str.replace('(S)','',regex = False).str.replace('.0','',regex = False)
data_all['Sub Brand'][no_name.index] = no_name.merge(data_SKU[['SKU', 'Sub Brand']].drop_duplicates('SKU'), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(no_name.index)['Sub Brand_y']

print('Filter Status')
index = data_all[data_all['Customer Name'] == 'Shopee Brand Portal'].index.to_list()
data_all['Order Status'][index] = 'Delivered'

data_success =data_all[data_all['Order Status'].isin([
        '"Delivered"', '"Ready to Ship"', '"Open"', '"Printed"',
       '"Shipped"', '"Delivered, Shipped"', '"Shipped, Ready to Ship"',
       '"Shipped, Delivered"', '"Ready to Ship, Delivered"',
       '"Ready to Ship, Shipped"', '"Delivered, Ready to Ship"',
       '"Delivered, Open"', '"Open, Delivered"', 'Delivered',
       '"Ready to Ship, Open"', '"Shipped, Open"','complete', 'payment_confirmed',
       'completed_to_wms', 'processing', 
       'ready_to_ship', 
       '"Open, Ready to Ship"', '"Open, Shipped, Ready to Ship"',
       'Sudah Settlement', '"Delivered, Shipped, Ready to Ship"',
       '"Ready to Ship, Shipped, Delivered"',
       '"Ready to Ship, Delivered, Shipped"', '"Open, Shipped"',
       '"Printed, Open"', '"Shipped, Printed"', '"Delivered, Printed"',
       '"Open, Printed"', '"Printed, Shipped"', '"Printed, Delivered"',
       '"Open, Delivered, Printed"', '"Open, Shipped, Printed"',
       '"Shipped, Open, Printed"', '"Ready to Ship, Printed"',
       '"Printed, Delivered, Open"', '"Delivered, Printed, Shipped"',
       '"Printed, Shipped, Open"', '"Delivered, Shipped, Open"',
       '"Shipped, Open, Ready to Ship"',
       '"Delivered, Open, Ready to Ship"', 
       '"Ready to Ship, Open, Shipped"', 
       'Sudah Isi Pickup Time', '"Shipped, Ready to Ship, Open"',
       '"Payout"',  '"Printed, Ready to Ship"',
       'Ready to Ship', 'Open',
       'Shipped', 'Delivered, Shipped', 
       'Delivered, Open', 'Printed', 'cod_sent',
       'entri_verified', 'incoming', 'completed', 'Open, Ready to Ship',
       'Open, Shipped', 'Ready to Ship, Shipped', 'completed_with_awb','Packed','Shipped','Delivered','Ready to Ship','Completed'
    ])]


all_single = data_success[data_success['Brand'].isin(['NS', 'TS', 'L-Men', 'HiLo'])]
all_single = all_single[all_single['Exported Parent Item'] != "W'DANK LKLT KOPI DURIAN PLS 12RX10SX15G"]


all_single = all_single[all_single['Store']!='MITRA TOKOPEDIA']
indeks = all_single[all_single['Store'].isin(['CARI SAYUR',  'EMOS', 'INOVASI DIGITAL NIAGA - KALIDERES, PT',
                                              'RITEL BERSAMA NASIONAL; PT', 'SAYUR BOX', 'TANIHUB'])].index.to_list()
all_single['Exported Parent Item'][indeks] = all_single['Product Name'][indeks]

indeks = all_single[(all_single['Store'].isin(['CARI SAYUR',  'EMOS', 'INOVASI DIGITAL NIAGA - KALIDERES, PT',
       'RITEL BERSAMA NASIONAL; PT', 'SAYUR BOX', 'TANIHUB']))&
                   (all_single['Sub Brand'].isnull())].index.to_list()
all_single['Sub Brand'][indeks] = all_single['Category'][indeks]

all_single['Total Net'] = all_single['Total Net'].astype(float).astype('int64')

bundle_name = data_all[data_all['Real SKU'] == 'PN50N86N87N89G162']['Product Name'].unique()
bundle_name = list(bundle_name) + ['NutriSari Orange Adventure (5 x 10 Sch)','NutriSari Bundle - Orange Adventure']
indeks = all_single[all_single['Bundle Name'].astype(str).isin(bundle_name)].index.to_list()
all_single['Brand'][indeks] = 'NS'
all_single['Sub Brand'][indeks] = 'PAKET NUTRISARI'
all_single['Exported Parent Item'][indeks] = 'PAKET ORANGE ADVENTURE'

all_single['Exported Parent Item'] = all_single['Exported Parent Item'].astype(str).str.replace('TS SHIRATAKI NOODLES 40Px71G', 'TS SHIRATAKI NOODLES 40PX71G')
all_single['Exported Parent Item'] = all_single['Exported Parent Item'].astype(str).str.replace('TS KOREAN GARLIC BUTTERCOOKIES12DX5SX20G', 'TS KOREAN GARLIC BUTTER COOKIES 12DX5SX20G')
all_single['Exported Parent Item'] = all_single['Exported Parent Item'].astype(str).str.replace('TS WHITE COFFEE 12DX4SX20G', 'TS WHITE COFFEE 12DX4SX15G')
all_single['Exported Parent Item'] = all_single['Exported Parent Item'].astype(str).str.replace('TS AVOCADO COFFEE 12DX4SX20G', 'TS AVOCADO COFFEE 12DX4SX14G')

all_single.loc[all_single['Real SKU'] == '2152051','Exported Parent Item']='TS DM COOKIES CHOCO 12DX10SX20G'#[['Year','Month','Brand','Store','Sub Brand','Real SKU','Real Nama Produk','Bundle Name']]#.drop_duplicates()
all_single.loc[all_single['Real SKU'] == '2152051','Sub Brand']='TS KUNING OTHERS'
all_single=all_single[all_single['Real SKU']!="(U)1102101155"]
all_single=all_single[all_single['Real SKU']!="(U)2307061250"]
all_single.loc[all_single['Order #']=='1000103864','Region Group']='Kab. Bogor'

zipnotmapped=all_single[(all_single['Region Group'].isnull())&(all_single['Store Type']=='Marketplace')&(all_single['Year']==2022)&(all_single['Zip Code'].notnull())]['Zip Code'].unique()#.to_excel('Gk ke mapping.xlsx',index=False)
for i in zipnotmapped:
    regionvalue=all_single[(all_single['Zip Code']==i)&(all_single['Region Group'].notnull())]['Region Group'].unique()[0]
    all_single.loc[(all_single['Region Group'].isnull())&(all_single['Region Group'].isnull())&(all_single['Store Type']=='Marketplace')&(all_single['Year']==2022),'Region Group']=regionvalue


indeks = all_single[all_single['Exported Parent Item'].isnull()].index.to_list()
indeks = indeks + all_single[all_single['Exported Parent Item'] == 'nan'].index.to_list()

if len(indeks) == 0:
    print("Prepare E-mailing")
    
    ########################################################
    
    table_brand = all_single[all_single['Store']=='Order Online'][['Brand','Sub Brand']].drop_duplicates().reset_index(drop = True)
    all_single['True datetime'] = pd.to_datetime(all_single['True datetime'])
    from datetime import datetime, timedelta
    today = datetime.today()
    yesterday = datetime.today() - timedelta(days=1)
    today = pd.to_datetime(today.strftime('%Y-%m-%d'))
    yesterday = pd.to_datetime(yesterday.strftime('%Y-%m-%d'))
    yesterday_sales = all_single[all_single['Store']=='Order Online'][all_single['True datetime'] >= yesterday][all_single[all_single['True datetime'] >= yesterday]['True datetime']<today]
    yesterday_sales = yesterday_sales.groupby(['Brand', 'Sub Brand'])['Total Net'].sum().reset_index()
    yesterday_sales['Total Net'] = yesterday_sales['Total Net']/1.1
    yesterday_sales = yesterday_sales.rename(columns = {'Total Net' : 'Yesterday Sales'})
    table_brand = table_brand.merge(yesterday_sales, how = 'left', on = ['Brand', 'Sub Brand'])

    if int(today.strftime('%d')) > 1 :
        mtd_date = datetime.today()-timedelta(days=int(today.strftime('%d'))-1)
        mtd_date = pd.to_datetime(mtd_date.strftime('%Y-%m-%d'))
    else :
        mtd_date = datetime.today()-timedelta(days=int(yesterday.strftime('%d')))
        mtd_date = pd.to_datetime(mtd_date.strftime('%Y-%m-%d'))

    mtd_sales = all_single[all_single['Store']=='Order Online'][all_single['True datetime'] >= mtd_date][all_single[all_single['True datetime'] >= mtd_date]['True datetime']<today]
    mtd_sales = mtd_sales.groupby(['Brand', 'Sub Brand'])['Total Net'].sum().reset_index()
    mtd_sales['Total Net'] = mtd_sales['Total Net']/1.1
    mtd_sales = mtd_sales.rename(columns = {'Total Net' : 'Local Sales'})
    table_brand = table_brand.merge(mtd_sales, how = 'left', on = ['Brand', 'Sub Brand'])

    from calendar import monthrange


    if int(today.strftime('%d')) > 1 :
        table_brand['Average This Month'] = table_brand['Local Sales']/(int(today.strftime('%d'))-1)
        number_of_days = monthrange(today.year, today.month)[1]
    else :
        table_brand['Average This Month'] = table_brand['Local Sales']/(int(yesterday.strftime('%d')))
        number_of_days = monthrange(yesterday.year, yesterday.month)[1]

    table_brand['Projected'] = table_brand['Average This Month'] * number_of_days

    from dateutil import rrule
    from dateutil.relativedelta import relativedelta

    start_date = datetime.today()-timedelta(days=int(today.strftime('%d'))-1)-relativedelta(months=3)
    start_date = pd.to_datetime(start_date.strftime('%Y-%m-%d'))
    end_date = datetime.today()-timedelta(days=int(today.strftime('%d')))
    for dt in rrule.rrule(rrule.MONTHLY, dtstart=start_date, until=end_date):
        first_date = pd.to_datetime(dt)
        print("First Date " + str(first_date))
        last_date = first_date + relativedelta(months=1)
        print("Last Date " + str(last_date))
        temp_sales = all_single[all_single['Store']=='Order Online'][all_single['True datetime'] >= first_date][all_single[all_single['True datetime'] >= first_date]['True datetime']<last_date]
        temp_sales = temp_sales.groupby(['Brand', 'Sub Brand'])['Total Net'].sum().reset_index()
        temp_sales['Total Net'] = temp_sales['Total Net']/1.1
        colname = str(first_date.month_name()) + ' ' + str(first_date.year)
        temp_sales = temp_sales.rename(columns = {'Total Net' : colname})
        table_brand = table_brand.merge(temp_sales, how = 'left', on = ['Brand', 'Sub Brand'])

    cols = list(table_brand)
    cols[len(cols)-1], cols[len(cols)-3] = cols[len(cols)-3], cols[len(cols)-1]
    table_brand = table_brand.loc[:,cols]
    table_brand = table_brand.reset_index(drop = True)
    table_brand = table_brand.drop('Average This Month', axis = 1)

    def highlight_max(x):
        return ['font-weight: bold' if v == x.loc[x.index.max()] else ''
                    for v in x]

    #     table_brand = table_brand[table_brand['Local Sales'].notnull()]
    table_brand['Yesterday Sales'] = table_brand['Yesterday Sales'].fillna(0).astype('int64')
    table_brand['Local Sales'] = table_brand['Local Sales'].fillna(0).astype('int64')
    table_brand = table_brand.sort_values(['Brand','Local Sales'], ascending = [True, False])

    table_ecom = table_brand.copy()

    for i in table_ecom.columns[2:]:
        table_ecom[i] = table_ecom[i].fillna(0).astype('int64')

    table_ecom = table_ecom.append(table_ecom.sum(numeric_only=True), ignore_index=True)
    # for i in table_ecom.columns[2:]:
    #     table_ecom[i] = table_ecom[i].apply(lambda x: "Rp {:,}".format(int(x))).str.replace(',','.', regex = False)
    #     table_ecom[i][table_ecom.index.max()] = "<b> {} </b>".format(table_ecom[i][table_ecom.index.max()])

    table_ecom = table_ecom.drop('Brand', axis = 1)
    table_ecom['Sub Brand'][table_ecom.index.max()] = '<b>Total E-Commerce (NHD) Sales<b>'
    table_ecom = table_ecom.rename(columns = {'Sub Brand' : ''})

    table_ecom2 = table_ecom.iloc[[-1]].reset_index(drop = True)

    ###############################################################

    table_brand = all_single[all_single['Store']!='Order Online'][['Brand','Sub Brand']].drop_duplicates().reset_index(drop = True)
    all_single['True datetime'] = pd.to_datetime(all_single['True datetime'])

    from datetime import datetime, timedelta

    today = datetime.today()
    yesterday = datetime.today() - timedelta(days=1)

    today = pd.to_datetime(today.strftime('%Y-%m-%d'))
    yesterday = pd.to_datetime(yesterday.strftime('%Y-%m-%d'))

    yesterday_sales = all_single[all_single['Store']!='Order Online'][all_single['True datetime'] >= yesterday][all_single[all_single['True datetime'] >= yesterday]['True datetime']<today]
    yesterday_sales = yesterday_sales.groupby(['Brand', 'Sub Brand'])['Total Net'].sum().reset_index()
    yesterday_sales['Total Net'] = yesterday_sales['Total Net']/1.1
    yesterday_sales = yesterday_sales.rename(columns = {'Total Net' : 'Yesterday Sales'})
    table_brand = table_brand.merge(yesterday_sales, how = 'left', on = ['Brand', 'Sub Brand'])

    if int(today.strftime('%d')) > 1 :
        mtd_date = datetime.today()-timedelta(days=int(today.strftime('%d'))-1)
        mtd_date = pd.to_datetime(mtd_date.strftime('%Y-%m-%d'))
    else :
        mtd_date = datetime.today()-timedelta(days=int(yesterday.strftime('%d')))
        mtd_date = pd.to_datetime(mtd_date.strftime('%Y-%m-%d'))

    mtd_sales = all_single[all_single['Store']!='Order Online'][all_single['True datetime'] >= mtd_date][all_single[all_single['True datetime'] >= mtd_date]['True datetime']<today]
    mtd_sales = mtd_sales.groupby(['Brand', 'Sub Brand'])['Total Net'].sum().reset_index()
    mtd_sales['Total Net'] = mtd_sales['Total Net']/1.1
    mtd_sales = mtd_sales.rename(columns = {'Total Net' : 'Local Sales'})
    table_brand = table_brand.merge(mtd_sales, how = 'left', on = ['Brand', 'Sub Brand'])

    from calendar import monthrange


    if int(today.strftime('%d')) > 1 :
        table_brand['Average This Month'] = table_brand['Local Sales']/(int(today.strftime('%d'))-1)
        number_of_days = monthrange(today.year, today.month)[1]
    else :
        table_brand['Average This Month'] = table_brand['Local Sales']/(int(yesterday.strftime('%d')))
        number_of_days = monthrange(yesterday.year, yesterday.month)[1]

    table_brand['Projected'] = table_brand['Average This Month'] * number_of_days

    from dateutil import rrule
    from dateutil.relativedelta import relativedelta

    start_date = datetime.today()-timedelta(days=int(today.strftime('%d'))-1)-relativedelta(months=3)
    start_date = pd.to_datetime(start_date.strftime('%Y-%m-%d'))
    end_date = datetime.today()-timedelta(days=int(today.strftime('%d')))
    for dt in rrule.rrule(rrule.MONTHLY, dtstart=start_date, until=end_date):
        first_date = pd.to_datetime(dt)
        print("First Date " + str(first_date))
        last_date = first_date + relativedelta(months=1)
        print("Last Date " + str(last_date))
        temp_sales = all_single[all_single['Store']!='Order Online'][all_single['True datetime'] >= first_date][all_single[all_single['True datetime'] >= first_date]['True datetime']<last_date]
        temp_sales = temp_sales.groupby(['Brand', 'Sub Brand'])['Total Net'].sum().reset_index()
        temp_sales['Total Net'] = temp_sales['Total Net']/1.1
        colname = str(first_date.month_name()) + ' ' + str(first_date.year)
        temp_sales = temp_sales.rename(columns = {'Total Net' : colname})
        table_brand = table_brand.merge(temp_sales, how = 'left', on = ['Brand', 'Sub Brand'])

    cols = list(table_brand)
    cols[len(cols)-1], cols[len(cols)-3] = cols[len(cols)-3], cols[len(cols)-1]
    table_brand = table_brand.loc[:,cols]
    table_brand = table_brand.reset_index(drop = True)
    table_brand = table_brand.drop('Average This Month', axis = 1)

    def highlight_max(x):
        return ['font-weight: bold' if v == x.loc[x.index.max()] else ''
                    for v in x]

    #     table_brand = table_brand[table_brand['Local Sales'].notnull()]
    table_brand['Yesterday Sales'] = table_brand['Yesterday Sales'].fillna(0).astype('int64')
    table_brand['Local Sales'] = table_brand['Local Sales'].fillna(0).astype('int64')
    table_brand = table_brand.sort_values(['Brand','Local Sales'], ascending = [True, False])

    table_ecom = table_brand.copy()

    for i in table_ecom.columns[2:]:
        table_ecom[i] = table_ecom[i].fillna(0).astype('int64')

    table_ecom = table_ecom.append(table_ecom.sum(numeric_only=True), ignore_index=True)
    # for i in table_ecom.columns[2:]:
    #     table_ecom[i] = table_ecom[i].apply(lambda x: "Rp {:,}".format(int(x))).str.replace(',','.', regex = False)
    #     table_ecom[i][table_ecom.index.max()] = "<b> {} </b>".format(table_ecom[i][table_ecom.index.max()])

    table_ecom = table_ecom.drop('Brand', axis = 1)
    table_ecom['Sub Brand'][table_ecom.index.max()] = '<b>Total E-Commerce (Non NHD) Sales</b>'
    table_ecom = table_ecom.rename(columns = {'Sub Brand' : ''})

    table_ecom1 = table_ecom.iloc[[-1]].reset_index(drop = True)
    # table_ecom=table_ecom1.append(table_ecom2,ignore_index=True,sort=False)
    # table_ecom=table_ecom.append(table_ecom.sum(numeric_only=True), ignore_index=True,sort=False)

    table_ecom=table_ecom1.append(table_ecom2,ignore_index=True,sort=False)
    table_ecom=table_ecom.append(table_ecom.sum(numeric_only=True), ignore_index=True)
    table_ecom.loc[(table_ecom[""].isnull()),""]='<b>Total E-Commerce Sales</b>'
    table_ecom

    for i in table_ecom.columns[1:]:
        table_ecom[i] = table_ecom[i].apply(lambda x: "Rp {:,}".format(int(x))).str.replace(',','.', regex = False)
        table_ecom[i][table_ecom.index.max()] = "<b> {} </b>".format(table_ecom[i][table_ecom.index.max()])

    ###############################################################
    from datetime import date
    sales_total=all_single.groupby(['Year','Month','True datetime','Store'],dropna=False)[['Total Net']].sum().reset_index()
    sales_total.loc[sales_total['Store']=='Order Online','Store Group']='<b>Total Ecommerce (NHD) Sales</b>'
    sales_total.loc[sales_total['Store']!='Order Online','Store Group']='<b>Total Ecommerce (Non NHD) Sales</b>'

    sales_total['Total Net Before PPN']=sales_total['Total Net']/1.1
    sales_total.loc[sales_total['True datetime']>='2022-04-01','Total Net Before PPN']=sales_total['Total Net']/1.11

    sales_total['Date']=pd.to_datetime(sales_total['True datetime']).dt.date

    col1=sales_total[(sales_total['Date']>=(date.today()-timedelta(1)))&
                        (sales_total['Date']<(date.today()))].groupby(['Store Group'])[['Total Net Before PPN']].sum().reset_index().rename(columns=({'Total Net Before PPN':'Yesterday Sales'}))

    if '<b>Total Ecommerce (NHD) Sales</b>' not in col1['Store Group'].unique():
        col1=col1.append(pd.DataFrame({'Store Group': ["<b>Total Ecommerce (NHD) Sales</b>"], 'Yesterday Sales': [0]}))

    col2=sales_total[(sales_total['Year']==(datetime.now()-timedelta(1)).year)&
                         (sales_total['Month']==(datetime.now()-timedelta(1)).strftime("%B"))&
                         (sales_total['Date']<(date.today()))].groupby(['Store Group'])[['Total Net Before PPN']].sum().reset_index().rename(columns=({'Total Net Before PPN':'Local Sales'}))
    lastmonth1=(date.today().replace(day=1)-timedelta(1))#.strftime("%B")
    lastmonth2=(lastmonth1.replace(day=1)-timedelta(1))#.strftime("%B")
    lastmonth3=(lastmonth2.replace(day=1)-timedelta(1))
    col3=col2.copy()
    maxday=monthrange((datetime.now()-timedelta(1)).year, (datetime.now()-timedelta(1)).month)[1]
    col3['Projected']=col3['Local Sales']*maxday/((datetime.now()-timedelta(1)).day)

    col4=sales_total[(sales_total['Year']==lastmonth1.year)&
                 (sales_total['Month']==lastmonth1.strftime("%B"))].groupby(['Store Group'])[['Total Net Before PPN']].sum().reset_index().rename(columns=({'Total Net Before PPN':f'{lastmonth1.strftime("%B")} {lastmonth1.year}'}))
    col5=sales_total[(sales_total['Year']==lastmonth2.year)&
                 (sales_total['Month']==lastmonth2.strftime("%B"))].groupby(['Store Group'])[['Total Net Before PPN']].sum().reset_index().rename(columns=({'Total Net Before PPN':f'{lastmonth2.strftime("%B")} {lastmonth2.year}'}))
    col6=sales_total[(sales_total['Year']==lastmonth3.year)&
                 (sales_total['Month']==lastmonth3.strftime("%B"))].groupby(['Store Group'])[['Total Net Before PPN']].sum().reset_index().rename(columns=({'Total Net Before PPN':f'{lastmonth3.strftime("%B")} {lastmonth3.year}'}))
    tblall=col1.merge(col2,how='left').merge(col3,how='left').merge(col4,how='left').merge(col5,how='left').merge(col6,how='left')
    tblall=tblall.fillna(int(0))
    tblall=tblall.sort_values('Store Group',ascending=False).reset_index(drop=True)

    total_row=tblall.sum(numeric_only=True)

    tblall=tblall.append(total_row,ignore_index=True)
    tblall.loc[tblall['Store Group'].isnull(),'Store Group']='<b>Total Ecommerce Sales</b>'
    tblall.rename(columns={'Store Group':""})
    for i in tblall.columns[1:]:
        tblall[i] = tblall[i].apply(lambda x: "Rp {:,}".format(int(x))).str.replace(',','.', regex = False)
        tblall[i][tblall.index.max()] = "<b> {} </b>".format(tblall[i][tblall.index.max()])

    # tblall   
    
    
    ###############################################################
    sales_region=all_single.groupby(['Year','Month','True datetime','Region Group'],dropna=False)[['Total Net']].sum().reset_index()
    mappingarea=pd.read_excel("data_supp\Database Area untuk Report.xlsx")
    mappingarea=mappingarea.rename(columns=({'Region':'Real Region',"City":'Region Group'}))
    sales_region=sales_region.merge(mappingarea,how='left')
    sales_region.loc[sales_region['Region Group'].isnull(),'Real Region']='Retail Online'
    
    
    sales_region['Total Net Before PPN']=sales_region['Total Net']/1.1
    sales_region.loc[sales_region['True datetime']>='2022-04-01','Total Net Before PPN']=sales_region['Total Net']/1.11
    sales_region['Date']=pd.to_datetime(sales_region['True datetime']).dt.date

    tblreg=sales_region[(sales_region['Date']>=(date.today()-timedelta(1)))&
                        (sales_region['Date']<(date.today()))].groupby(['Real Region'])[['Total Net Before PPN']].sum().reset_index().rename(columns=({'Total Net Before PPN':'Yesterday Sales'}))
    no_ytd_sales=mappingarea[~mappingarea['Real Region'].isin(tblreg['Real Region'].unique())]['Real Region'].unique()
    for i in no_ytd_sales:
        tblreg=tblreg.append(pd.DataFrame({'Real Region': [f"{i}"], 'Yesterday Sales': [0]}))
        tblreg=tblreg.sort_values('Real Region')    
    if 'Retail Online' not in tblreg['Real Region'].unique():
        tblreg=tblreg.append(pd.DataFrame({'Real Region': ["Retail Online"], 'Yesterday Sales': [0]}))

    tblreg2=sales_region[(sales_region['Year']==(datetime.now()-timedelta(1)).year)&
                         (sales_region['Month']==(datetime.now()-timedelta(1)).strftime("%B"))&
                         (sales_region['Date']<(date.today()))].groupby(['Real Region'])[['Total Net Before PPN']].sum().reset_index().rename(columns=({'Total Net Before PPN':'Local Sales'}))
    lastmonth1=(date.today().replace(day=1)-timedelta(1))#.strftime("%B")
    lastmonth2=(lastmonth1.replace(day=1)-timedelta(1))#.strftime("%B")
    lastmonth3=(lastmonth2.replace(day=1)-timedelta(1))
    tblreg3=tblreg2.copy()
    maxday=monthrange((datetime.now()-timedelta(1)).year, (datetime.now()-timedelta(1)).month)[1]
    tblreg3['Projected']=tblreg3['Local Sales']*maxday/((datetime.now()-timedelta(1)).day)

    tblregshopee=tblreg3[tblreg3["Real Region"]=='Retail Online']
    tblreg3=tblreg3[tblreg3["Real Region"]!='Retail Online']#.copy()

    totprj=tblreg3[tblreg3["Real Region"]!='Retail Online']['Projected'].sum()
    tblreg3['Projected %']=tblreg3['Projected']/totprj
    tblreg3=tblreg3.append(tblregshopee)

    tblreg3=tblreg3[['Real Region','Projected','Projected %']]
    tblreg4=sales_region[(sales_region['Year']==lastmonth1.year)&
                 (sales_region['Month']==lastmonth1.strftime("%B"))].groupby(['Real Region'])[['Total Net Before PPN']].sum().reset_index().rename(columns=({'Total Net Before PPN':f'{lastmonth1.strftime("%B")} {lastmonth1.year}'}))
    tblreg5=sales_region[(sales_region['Year']==lastmonth2.year)&
                 (sales_region['Month']==lastmonth2.strftime("%B"))].groupby(['Real Region'])[['Total Net Before PPN']].sum().reset_index().rename(columns=({'Total Net Before PPN':f'{lastmonth2.strftime("%B")} {lastmonth2.year}'}))
    tblreg6=sales_region[(sales_region['Year']==lastmonth3.year)&
                 (sales_region['Month']==lastmonth3.strftime("%B"))].groupby(['Real Region'])[['Total Net Before PPN']].sum().reset_index().rename(columns=({'Total Net Before PPN':f'{lastmonth3.strftime("%B")} {lastmonth3.year}'}))
    tblreg=tblreg.merge(tblreg2,how='left').merge(tblreg3,how='left').merge(tblreg4,how='left').merge(tblreg5,how='left').merge(tblreg6,how='left')
    tblreg=tblreg.rename(columns={'Real Region':'Region'})

    total_row=tblreg.sum(numeric_only=True)
    sub_total_row=tblreg[tblreg["Region"]!='Retail Online'].sum(numeric_only=True)
    # tblreg.loc[(tblreg["Region"].isnull()),"Region"]='<b>Total Sales</b>'
    res=pd.DataFrame()
    res=res.append(tblreg[tblreg["Region"]!='Retail Online']).append(sub_total_row,ignore_index=True).append(tblreg[tblreg["Region"]=='Retail Online'])
    res.loc[res["Region"].isnull(),'Region']='<b>Sub Total Sales</b>'
    res=res.append(total_row,ignore_index=True)
    res.loc[res["Region"].isnull(),'Region']='<b>Total Sales</b>'

    res.loc[(res['Region']=='Retail Online')&
               (res['Local Sales'].isnull()),'Local Sales']=0
    res.loc[(res['Region']=='Retail Online')&
               (res['Projected'].isnull()),'Projected']=0

    tblreg=res.copy()
    tblreg=tblreg.fillna(0)
    for i in tblreg.columns[1:4]:
        tblreg[i] = tblreg[i].apply(lambda x: "Rp {:,}".format(int(x))).str.replace(',','.', regex = False)
        tblreg[i][tblreg.index.max()] = "<b> {} </b>".format(tblreg[i][tblreg.index.max()])

    for i in tblreg.columns[4:5]:
        tblreg[i] = tblreg[i].apply(lambda x: "{:.2f} %".format(x*100))#.str.replace(',','.', regex = False)
        tblreg[i][tblreg.index.max()] = "<b> {} </b>".format(tblreg[i][tblreg.index.max()])


    for i in tblreg.columns[5:]:
        tblreg[i] = tblreg[i].apply(lambda x: "Rp {:,}".format(int(x))).str.replace(',','.', regex = False)
        tblreg[i][tblreg.index.max()] = "<b> {} </b>".format(tblreg[i][tblreg.index.max()])

    for i in tblreg.columns[1:]:
        tblreg[i][tblreg.index.max()-2] = "<b> {} </b>".format(tblreg[i][tblreg.index.max()-2])

    tblreg.loc[(tblreg["Projected %"]=="nan %"),"Projected %"]=''


    tblreg

    ###################################################################################

    for i in table_brand.columns[2:]:
        table_brand[i] = table_brand[i].fillna(0).astype('int64')

    table_brand_hilo = table_brand[table_brand['Brand'] == 'HiLo'].reset_index(drop = True)
    table_brand_hilo = table_brand_hilo.append(table_brand_hilo.sum(numeric_only=True), ignore_index=True)
    table_brand_hilo['Sub Brand'][table_brand_hilo.index.max()] = '<b>TOTAL<b>'
    table_brand_hilo['Brand'][table_brand_hilo.index.max()] = ''


    table_brand_ns= table_brand[table_brand['Brand'] == 'NS'].reset_index(drop = True)
    table_brand_ns = table_brand_ns.append(table_brand_ns.sum(numeric_only=True), ignore_index=True)
    table_brand_ns['Sub Brand'][table_brand_ns.index.max()] = '<b>TOTAL<b>'
    table_brand_ns['Brand'][table_brand_ns.index.max()] = ''


    table_brand_lmen = table_brand[table_brand['Brand'] == 'L-Men'].reset_index(drop = True)
    table_brand_lmen = table_brand_lmen.append(table_brand_lmen.sum(numeric_only=True), ignore_index=True)
    table_brand_lmen['Sub Brand'][table_brand_lmen.index.max()] = '<b>TOTAL<b>'
    table_brand_lmen['Brand'][table_brand_lmen.index.max()] = ''


    table_brand_ts = table_brand[table_brand['Brand'] == 'TS'].reset_index(drop = True)
    table_brand_ts = table_brand_ts.append(table_brand_ts.sum(numeric_only=True), ignore_index=True)
    table_brand_ts['Sub Brand'][table_brand_ts.index.max()] = '<b>TOTAL<b>'
    table_brand_ts['Brand'][table_brand_ts.index.max()] = ''


    for i in table_brand_hilo.columns[2:]:
        table_brand_hilo[i] = table_brand_hilo[i].apply(lambda x: "Rp {:,}".format(int(x))).str.replace(',','.', regex = False)
        table_brand_hilo[i][table_brand_hilo.index.max()] = "<b> {} </b>".format(table_brand_hilo[i][table_brand_hilo.index.max()])

    for i in table_brand_lmen.columns[2:]:
        table_brand_lmen[i] = table_brand_lmen[i].apply(lambda x: "Rp {:,}".format(int(x))).str.replace(',','.', regex = False)
        table_brand_lmen[i][table_brand_lmen.index.max()] = "<b> {} </b>".format(table_brand_lmen[i][table_brand_lmen.index.max()])

    for i in table_brand_ns.columns[2:]:
        table_brand_ns[i] = table_brand_ns[i].apply(lambda x: "Rp {:,}".format(int(x))).str.replace(',','.', regex = False)
        table_brand_ns[i][table_brand_ns.index.max()] = "<b> {} </b>".format(table_brand_ns[i][table_brand_ns.index.max()])

    for i in table_brand_ts.columns[2:]:
        table_brand_ts[i] = table_brand_ts[i].apply(lambda x: "Rp {:,}".format(int(x))).str.replace(',','.', regex = False)
        table_brand_ts[i][table_brand_ts.index.max()] = "<b> {} </b>".format(table_brand_ts[i][table_brand_ts.index.max()])

    table_brand = table_brand_hilo.append(pd.Series(), ignore_index = True, sort = False)
    table_brand = table_brand.append(table_brand_lmen, ignore_index = True, sort = False)
    table_brand = table_brand.append(pd.Series(), ignore_index = True, sort = False)
    table_brand = table_brand.append(table_brand_ns, ignore_index = True, sort = False)
    table_brand = table_brand.append(pd.Series(), ignore_index = True, sort = False)
    table_brand = table_brand.append(table_brand_ts, ignore_index = True, sort = False)
    for i in table_brand.columns:
        table_brand[i] = table_brand[i].fillna('')

    table_brand = table_brand[(table_brand == 'Rp 0').sum(1) < 5]
    table_brand = table_brand.reset_index(drop = True)


    table_item = all_single[['Brand', 'Sub Brand', 'Exported Parent Item']].drop_duplicates().reset_index(drop = True)
    all_single['True datetime'] = pd.to_datetime(all_single['True datetime'])
    from datetime import datetime, timedelta

    today = datetime.today()
    yesterday = datetime.today() - timedelta(days=1)

    today = pd.to_datetime(today.strftime('%Y-%m-%d'))
    yesterday = pd.to_datetime(yesterday.strftime('%Y-%m-%d'))

    yesterday_sales = all_single[all_single['True datetime'] >= yesterday][all_single[all_single['True datetime'] >= yesterday]['True datetime']<today]
    yesterday_sales = yesterday_sales.groupby(['Brand', 'Sub Brand', 'Exported Parent Item'])['Total Net'].sum().reset_index()
    yesterday_sales['Total Net'] = yesterday_sales['Total Net']/1.1
    yesterday_sales = yesterday_sales.rename(columns = {'Total Net' : 'Yesterday Sales'})
    table_item = table_item.merge(yesterday_sales, how = 'left', on = ['Brand', 'Sub Brand', 'Exported Parent Item'])

    if int(today.strftime('%d')) > 1 :
        mtd_date = datetime.today()-timedelta(days=int(today.strftime('%d'))-1)
        mtd_date = pd.to_datetime(mtd_date.strftime('%Y-%m-%d'))
    else :
        mtd_date = datetime.today()-timedelta(days=int(yesterday.strftime('%d')))
        mtd_date = pd.to_datetime(mtd_date.strftime('%Y-%m-%d'))

    mtd_sales = all_single[all_single['True datetime'] >= mtd_date][all_single[all_single['True datetime'] >= mtd_date]['True datetime']<today]
    mtd_sales = mtd_sales.groupby(['Brand', 'Sub Brand', 'Exported Parent Item'])['Total Net'].sum().reset_index()
    mtd_sales['Total Net'] = mtd_sales['Total Net']/1.1
    mtd_sales = mtd_sales.rename(columns = {'Total Net' : 'Local Sales'})
    table_item = table_item.merge(mtd_sales, how = 'left', on = ['Brand', 'Sub Brand', 'Exported Parent Item'])

    if int(today.strftime('%d')) > 1 :
        table_item['Average This Month'] = table_item['Local Sales']/(int(today.strftime('%d'))-1)
        number_of_days = monthrange(today.year, today.month)[1]
    else :
        table_item['Average This Month'] = table_item['Local Sales']/(int(yesterday.strftime('%d')))
        number_of_days = monthrange(yesterday.year, yesterday.month)[1]

    table_item['Projected'] = table_item['Average This Month'] * number_of_days

    from dateutil import rrule
    from dateutil.relativedelta import relativedelta

    start_date = datetime.today()-timedelta(days=int(today.strftime('%d'))-1)-relativedelta(months=3)
    start_date = pd.to_datetime(start_date.strftime('%Y-%m-%d'))
    end_date = datetime.today()-timedelta(days=int(today.strftime('%d')))
    for dt in rrule.rrule(rrule.MONTHLY, dtstart=start_date, until=end_date):
        first_date = pd.to_datetime(dt)
        last_date = first_date + relativedelta(months=1)
        temp_sales = all_single[all_single['True datetime'] >= first_date][all_single[all_single['True datetime'] >= first_date]['True datetime']<last_date]
        temp_sales = temp_sales.groupby(['Brand', 'Sub Brand', 'Exported Parent Item'])['Total Net'].sum().reset_index()
        temp_sales['Total Net'] = temp_sales['Total Net']/1.11
        colname = str(first_date.month_name()) + ' ' + str(first_date.year)
        temp_sales = temp_sales.rename(columns = {'Total Net' : colname})
        table_item = table_item.merge(temp_sales, how = 'left', on = ['Brand', 'Sub Brand', 'Exported Parent Item'])

    cols = list(table_item)
    cols[len(cols)-1], cols[len(cols)-3] = cols[len(cols)-3], cols[len(cols)-1]
    table_item = table_item.loc[:,cols]

    #     table_item = table_item[table_item['Local Sales'].notnull()]
    table_item['Yesterday Sales'] = pd.to_numeric(table_item['Yesterday Sales'], errors = 'coerce').fillna(0).astype('int64')
    table_item['Local Sales'] = pd.to_numeric(table_item['Local Sales'], errors = 'coerce').fillna(0).astype('int64')
    table_item = table_item.sort_values(['Brand','Local Sales'], ascending = [True, False])

    for i in table_item.columns[3:]:
        table_item[i] = table_item[i].fillna(0).apply(lambda x: "Rp {:,}".format(int(x))).str.replace(',','.', regex = False)

    table_item = table_item[(table_item == 'Rp 0').sum(1) < 5]
    table_item = table_item.reset_index(drop = True)
    table_item = table_item.drop('Average This Month', axis = 1)

    all_single = all_single.sort_values('True datetime')

    first_product = all_single.groupby('Exported Parent Item')['True datetime'].first().reset_index()

    product_launch = first_product[first_product['True datetime'] > pd.to_datetime(datetime.today() - relativedelta(months=3))]['Exported Parent Item'].unique()
    non_launch =  ["PARSEL NATAL TAHUN BARU",'HAMPER IDUL FITRI TS', 'GETPLUS HAMPERS IDUL FITRI TS', 'PAKET TAKJIL HILO DESSERT',
                   'HILO CHOCOLATE PLS 15RX10SX14G', 'HILO TEEN TARO 12DX500G', 'TS SAMBAL TERASI 24BTLX200G',
                   'TS AVOCADO COFFEE 12DX4SX20G', 'L-MEN PLATINUM KETAN HITAM 6KLRX800G', 'HILO AVOCADO CHOCOLATE PLS 15RX10SX14G', 'TS SHIRATAKI NOODLES 40PX71G',
                   'TS KOREAN GARLIC BUTTERCOOKIES12DX5SX20G','TS AVOCADO COFFEE 12DX4SX14G', 'TS SWT CLASSIC 24DX100G', 'HILO WHITE CHOCOLATE PLS 15RX10SX14G', 
                   'HILO CHOCO HAZELNUT PLS 15RX10SX14G', 'NS ANGGUR PLS 4PX40SX11G',"NS JERUK MANADO PLS 4PX40SX11G",
                   "NS TEA LYCHEE TEA REF 12BAGX500G","NUTRISARI 4 RASA SUMMER PACKAGE 40 SACHET","NUTRISARI 4 RASA LOCAL PACKAGE 40 SACHET",
                   'PAKET NUTRISARI 25 RASA FRESHSTART','PAKET TS BEAT HYPERTENSION 2022','HILO SCHOOL CHOCOLATE CANDY 12SBX20SX8G','HILO ES TELER FC 36PX10SX15G','NS BLEWAH FC 72PX10SX11G','NS JERUK MAROKO 72PX10SX14G']
    product_launch = [x for x in product_launch if x not in non_launch]

    table_launch = table_item[table_item['Exported Parent Item'].isin(product_launch)]
    table_item = table_item[~table_item['Exported Parent Item'].isin(product_launch)]

    table_hilo = table_item[table_item['Brand'] == 'HiLo'].reset_index(drop = True)
    table_ns= table_item[table_item['Brand'] == 'NS'].reset_index(drop = True)
    table_ns = table_ns[table_ns['Exported Parent Item'] != "W'DANK LKLT KOPI DURIAN PLS 12RX10SX15G"]
    table_lmen = table_item[table_item['Brand'] == 'L-Men'].reset_index(drop = True)
    table_ts = table_item[table_item['Brand'] == 'TS'].reset_index(drop = True)

    new_table_launch = pd.DataFrame()
    for i in table_launch['Brand'].unique():
        if i != table_launch['Brand'].unique()[-1]:
            new_table_launch = new_table_launch.append(table_launch[table_launch['Brand'] == i], ignore_index = True, sort = False).append(pd.Series(), ignore_index = True, sort = False)
        else :
            new_table_launch = new_table_launch.append(table_launch[table_launch['Brand'] == i], ignore_index = True, sort = False)

    for i in new_table_launch.columns:
        new_table_launch[i] = new_table_launch[i].fillna('')

    index = data_all[data_all['Sub Brand'] == 'WDANK TRADITIONAL'][data_all[data_all['Sub Brand'] == 'WDANK TRADITIONAL']['Bundle Flag'].isnull()].index.to_list()
    data_all['Brand'][index] = 'WDANK'


    print("Clean Data Finish")

    print("Clean Data Non Nubi Finish")


    print("Export Data 2020 Finish")


    print("Prepare Emailing")
  
    import win32com.client
    o = win32com.client.Dispatch("Outlook.Application")
    for oacc in o.Session.Accounts:
        if oacc.SmtpAddress == "customer@nutrifood.co.id":
            oacctouse = oacc
            break
    Msg = o.CreateItem(0)
    if oacctouse:
        Msg._oleobj_.Invoke(*(64209, 0, 8, 0, oacctouse))  # Msg.SendUsingAccount = oacctouseif to:

        to = ['steven.nathanael@nutrifood.co.id']

        list_to = ';'.join(to)

        Msg.To = list_to
        Msg.Subject = "Report E-commerce " + \
        str(datetime.today().date().strftime('%B %d, %Y'))


    html = '''
    <html>
    <head>
    <style>

        h2 {
            text-align: center;
            font-family: Helvetica, Arial, sans-serif;
        }
        table { 
            margin-left: auto;
            margin-right: auto;
        }
        table, th, td {
            border: 1px solid black;
            border-collapse: collapse;
        }
        th, td {
            padding: 5px;
            text-align: center;
            font-family: Helvetica, Arial, sans-serif;
            font-size: 90%;
        }
        table tbody tr:hover {
            background-color: #dddddd;
        }
        .wide {
            width: 90%; 
        }
        p

    </style>
    </head>
    <body>
        '''
    html = html + """
        <p> Dear all,</p>
        <p> Berikut data penjualan E-Com <b> (Before PPN) tanggal {tanggal} </b> dengan rincian channel sebagai berikut : </p>
        <p> Channel Marketplace : Blibli, Bukalapak, Elevenia, JD Indonesia, Lazada, Nutrimart, Order Online, Shopee, Tokopedia, Aladin Mall, TikTok </p>
        <p> Channel Retail : Cari Sayur, Emos, Farmaku, Inovasi Digital Niaga, Ritel Bersama Nasional, Sayurbox, Tanihub, Astro, Lifepack, Toko Now   </p>
        <p> Notes: Mohon maaf ada keterlambatan data untuk retail online </p>

    """.format(tanggal = str(datetime.today().date().strftime('%B %d, %Y')))

    html = html + tblall.to_html(classes='wide', escape=False).replace('&lt;b&gt;', '<b>').replace('&lt;/b&gt;', '</b>')

    html = html + """
        <p> Dan berikut data penjualan E-Com (Before PPN) per Area :</p>
        """

    html = html + tblreg.to_html(classes='wide', escape=False)

    html = html + """
        <p> Dan Berikut data penjualan E-Com <b> (Before PPN) tanggal {tanggal} </b> per Brand :</p>
    """.format(tanggal = str(datetime.today().date().strftime('%B %d, %Y'))) 


    html = html + table_brand.to_html(classes='wide', escape=False).replace('&lt;b&gt;', '<b>').replace('&lt;/b&gt;', '</b>')



    html = html + """
        <p> Dan berikut data penjualan E-Com (Before PPN) per Item :</p>
        <p> Item Launching : </p>"""

    html = html + new_table_launch.to_html(classes='wide', escape=False)

    html = html + "<p> HiLo : </p>"    

    html = html + table_hilo.to_html(classes='wide', escape=False)

    html = html + "<p> L-Men : </p>"    

    html = html + table_lmen.to_html(classes='wide', escape=False)

    html = html + "<p> NutriSari : </p>"    

    html = html + table_ns.to_html(classes='wide', escape=False)

    html = html + "<p> Tropicana Slim : </p>"    

    html = html + table_ts.to_html(classes='wide', escape=False)

    html = html + """
        <p></p>
        <p> Best regards, </p>
        <p> E-Commerce Nutrifood </p>
    <body>
    </html>
    """

    Msg.HTMLBody = html
    Msg.Send()
    print('Email Sent')
else :
    print('Missing Parent Item')

Export Master Data
Filter Status
Prepare E-mailing


C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_10148\425632950.py:874: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  yesterday_sales = all_single[all_single['Store']=='Order Online'][all_single['True datetime'] >= yesterday][all_single[all_single['True datetime'] >= yesterday]['True datetime']<today]
C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_10148\425632950.py:887: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  mtd_sales = all_single[all_single['Store']=='Order Online'][all_single['True datetime'] >= mtd_date][all_single[all_single['True datetime'] >= mtd_date]['True datetime']<today]


First Date 2022-08-01 00:00:00
Last Date 2022-09-01 00:00:00


C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_10148\425632950.py:916: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temp_sales = all_single[all_single['Store']=='Order Online'][all_single['True datetime'] >= first_date][all_single[all_single['True datetime'] >= first_date]['True datetime']<last_date]


First Date 2022-09-01 00:00:00
Last Date 2022-10-01 00:00:00


...

First Date 2022-09-01 00:00:00
Last Date 2022-10-01 00:00:00


C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_10148\425632950.py:1009: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temp_sales = all_single[all_single['Store']!='Order Online'][all_single['True datetime'] >= first_date][all_single[all_single['True datetime'] >= first_date]['True datetime']<last_date]


First Date 2022-10-01 00:00:00
Last Date 2022-11-01 00:00:00


C:\Users\steven.nathanael\AppData\Local\Temp\ipykernel_10148\425632950.py:1009: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temp_sales = all_single[all_single['Store']!='Order Online'][all_single['True datetime'] >= first_date][all_single[all_single['True datetime'] >= first_date]['True datetime']<last_date]


Clean Data Finish
Clean Data Non Nubi Finish
Export Data 2020 Finish
Prepare Emailing
Email Sent


In [65]:
print('Done')

Done


In [33]:
## ini cek kalau pembagian region sama total (tbl 1 & 2 Beda) update di file area utk report
sales_region[(sales_region['Region Group'].notnull())&(~sales_region['Region Group'].isin(mappingarea['Region Group'].unique()))][['Region Group']].values

array([], shape=(0, 1), dtype=object)

In [34]:
# if error missing parent item
# listing di kode atas, exported parent item liat tatanama akun admin login user&pass: nutrifood
all_single[all_single['Exported Parent Item'] == 'nan'][['Real SKU','Real Nama Produk','Category Baru']].drop_duplicates()#.values

,Real SKU,Real Nama Produk,Category Baru


In [32]:
all_single.columns

Index(['Order #', 'Sales Order ID', 'AWB', 'Paid Date', 'Order Status',
       'Order date', 'Week', 'Date', 'Month', 'Quarter',
       ...
       'Invoice Amount', 'Payment Received ID', 'Payment Received Ref. number',
       'Category Baru', 'Exported Parent Item', 'Unnamed: 0',
       'Price List NFI_x', 'Customer Type', 'Real Region', 'True datetime1'],
      dtype='object', length=198)

In [37]:
#CHECKING ORDER ONLINE
date2 = 6/11/2022
data_all['True datetime'] = pd.to_datetime(data_all['True datetime'])

In [49]:
data_all1 = data_all[(data_all['Year'] == 2022) & (data_all['Month'] == 'November') & (data_all['Date'] == 6)]

In [56]:
data_all1[data_all1['Store'] == "Order Online"][['Sales Order ID','Channel','Order Status']]

,Sales Order ID,Channel,Order Status
1590413,90217420,Order Online Medan,pending
1590414,90217420,Order Online Medan,pending
1590415,90217420,Order Online Medan,pending
1590416,90217420,Order Online Medan,pending
1590417,90217420,Order Online Medan,pending
1590418,90217420,Order Online Medan,pending
1590419,90217420,Order Online Medan,pending
1590420,90217420,Order Online Medan,pending
1590965,90217420,Order Online Medan,pending
1590966,90217420,Order Online Medan,pending


In [35]:
### run 12
### take out item yang gk masuk email automation
buang = ['PAKET HILO']
table_brand = table_brand[~table_brand['Sub Brand'].isin(buang)]
table_hilo = table_hilo[~table_hilo['Sub Brand'].isin(buang)]
table_hilo

buang = ['PAKET TS']
table_brand = table_brand[~table_brand['Sub Brand'].isin(buang)]
table_ts = table_ts[~table_ts['Sub Brand'].isin(buang)]
# table_ts

In [36]:
### run 13
### send email automation final ke user

import win32com.client

o = win32com.client.Dispatch("Outlook.Application")
for oacc in o.Session.Accounts:
    if oacc.SmtpAddress == "customer@nutrifood.co.id":
        oacctouse = oacc
        break

Msg = o.CreateItem(0)
if oacctouse:
    Msg._oleobj_.Invoke(*(64209, 0, 8, 0, oacctouse))  # Msg.SendUsingAccount = oacctouseif to:
    
    to=["mardi@nutrifood.co.id", "Elvina@nutrifood.co.id", "susana@nutrifood.co.id",
        "ignasius.dwi@nutrifood.co.id", "may@nutrifood.co.id", "Meirza@nutrifood.co.id", "christian.jesaya@nutrifood.co.id",
        "didit@nutrifood.co.id", "noviana@nutrifood.co.id","reynald.tjandra@nutrifood.co.id", "elvira@nutrifood.co.id",
        "manthovani.annice@nutrifood.co.id","geny.pitaloka@nutrifood.co.id", "joshua.suhendro@nutrifood.co.id","stephanie.wonoadi@nutrifood.co.id",
        "lisa.arianti@nutrifood.co.id",'charissa.lungkat@nutrifood.co.id','rosariani@nutrifood.co.id',"zakaria@nutrifood.co.id","geovano.satria@nutrifood.co.id","agus_s@nutrifood.co.id","romi.anggara@nutrifood.co.id",
        "edward@nutrifood.co.id", "elisa.putri@nutrifood.co.id","teddy.andreas@nutrifood.co.id", "muroby.rocky@nutrifood.co.id",
        "evelyn@nutrifood.co.id","sheila.odilia@nutrifood.co.id","timotius.giovandi@nutrifood.co.id",
        'ronaldo.yolanda@nutrifood.co.id','bagus.sindhu@nutrifood.co.id',"hendi.herdiansyah@nutrifood.co.id",'christovertand.wim@nutrifood.co.id',
        'steven.nathanael@nutrifood.co.id','lintang.purnomo@nutrifood.co.id','mikhael.justin@nutrifood.co.id']
    #to = ['steven.nathanael@nutrifood.co.id']

    list_to = ';'.join(to)

    Msg.To = list_to
    Msg.Subject = "Report E-commerce " + \
    str(datetime.today().date().strftime('%B %d, %Y'))
    




html = '''
<html>
<head>
<style>

    h2 {
        text-align: center;
        font-family: Helvetica, Arial, sans-serif;
    }
    table { 
        margin-left: auto;
        margin-right: auto;
    }
    table, th, td {
        border: 1px solid black;
        border-collapse: collapse;
    }
    th, td {
        padding: 5px;
        text-align: center;
        font-family: Helvetica, Arial, sans-serif;
        font-size: 90%;
    }
    table tbody tr:hover {
        background-color: #dddddd;
    }
    .wide {
        width: 90%; 
    }
    p

</style>
</head>
<body>
    '''
html = html + """
    <p> Dear all,</p>
    <p> Berikut data penjualan E-Com <b> (Before PPN) tanggal {tanggal} </b> dengan rincian channel sebagai berikut : </p>
    <p> Channel Marketplace : Blibli, Bukalapak, JD Indonesia, Lazada, Nutrimart, Order Online, Shopee, Tokopedia, Aladin Mall, TikTok </p>
    <p> Channel Retail : Cari Sayur, Emos, Farmaku, Inovasi Digital Niaga, Ritel Bersama Nasional, Sayurbox, Tanihub, Astro, Lifepack, Toko Now  </p>
""".format(tanggal = str(datetime.today().date().strftime('%B %d, %Y')))

html = html + tblall.to_html(classes='wide', escape=False).replace('&lt;b&gt;', '<b>').replace('&lt;/b&gt;', '</b>')

html = html + """
    <p> Dan berikut data penjualan E-Com (Before PPN) per Area :</p>
    """

html = html + tblreg.to_html(classes='wide', escape=False)

html = html + """
    <p> Dan Berikut data penjualan E-Com <b> (Before PPN) tanggal {tanggal} </b> per Brand :</p>
""".format(tanggal = str(datetime.today().date().strftime('%B %d, %Y'))) 


html = html + table_brand.to_html(classes='wide', escape=False).replace('&lt;b&gt;', '<b>').replace('&lt;/b&gt;', '</b>')



html = html + """
    <p> Dan berikut data penjualan E-Com (Before PPN) per Item :</p>
    <p> Item Launching : </p>"""

html = html + new_table_launch.to_html(classes='wide', escape=False)

html = html + "<p> HiLo : </p>"    

html = html + table_hilo.to_html(classes='wide', escape=False)

html = html + "<p> L-Men : </p>"    

html = html + table_lmen.to_html(classes='wide', escape=False)

html = html + "<p> NutriSari : </p>"    

html = html + table_ns.to_html(classes='wide', escape=False)

html = html + "<p> Tropicana Slim : </p>"    

html = html + table_ts.to_html(classes='wide', escape=False)

html = html + """
    <p></p>
    <p> Best regards, </p>
    <p> E-Commerce Nutrifood </p>
<body>
</html>
"""

Msg.HTMLBody = html
Msg.Send()

In [37]:
### run 14
### export masterdata

data_all.to_csv(r'D:\Masterdata\Clean Data\data_all_' + data_now + '.csv', sep = ';', index = False)

Export Lite Version

In [53]:
mappingarea=pd.read_excel("data_supp\Database Area untuk Report.xlsx")
mappingarea=mappingarea.rename(columns=({'Region':'Real Region',"City":'Region Group'}))
# temp=data_all.merge(mappingarea,how='left')
# temp.loc[temp['Region Group'].isnull(),'Real Region']='Retail Online'
for i in mappingarea['Real Region'].unique():
    print(i)
    reg=mappingarea[mappingarea['Real Region']==i]['Region Group'].unique()
    data_all.loc[data_all['Region Group'].isin(reg),'Real Region']=i
data_all.loc[data_all['Region Group'].isnull(),'Real Region']='Retail Online'

BARA
Jabodetabek
Jawa Barat
Jawa Tengah
Jawa Timur
Kalimantan
PUMA
Sulawesi
Sumatera 1
Sumatera 2


In [139]:
##NOT USED
from datetime import date
osf=pd.read_excel(r"data_supp\order filter.xlsx")
data_all[(data_all['Year'].isin([2021,2022]))&
         (data_all['Order Status'].isin(osf['order filter'].unique()))].groupby(['Year','Month','Store Type','Store','Channel','Real Region','Warehouse Name','Order Status','Region Group','Coupon Code','Brand','Sub Brand','Real SKU','Real Nama Produk','Bundle Name'],dropna=False).agg({'Order #':'nunique','Phone':'nunique','Qty. Invoiced':'sum','Total Net Before PPN':'sum'}).reset_index().to_excel(f'masterdata_{date.today()}_lite.xlsx',index=False)
# data_all.groupby(['Year','Month','Date','Store Type','Store','Channel','Order Status'])[['Order #']].nunique().reset_index().to_excel(f'Cek order gerak {date.today()}.xlsx',index=False)

In [39]:
from datetime import date
osf=pd.read_excel(r"data_supp\order filter.xlsx")
data_all[(data_all['Year'].isin([2022]))&
         (data_all['Order Status'].isin(osf['order filter'].unique()))].groupby(['Year','Month','Date','Store Type','Store','Channel','Warehouse Name','Order Status','Region Group','Coupon Code','Brand','Sub Brand','Real SKU','Real Nama Produk','Bundle Name'],dropna=False).agg({'Order #':'nunique','Phone':'nunique','Qty. Invoiced':'sum','Total Net Before PPN':'sum'}).reset_index().to_excel(f'D:\Masterdata\Masterdata Lite\masterdata_{date.today()}_lite new.xlsx',index=False)
# data_all.groupby(['Year','Month','Date','Store Type','Store','Channel','Order Status'])[['Order #']].nunique().reset_index().to_excel(f'Cek order gerak {date.today()}.xlsx',index=False)

In [56]:
##NOT USED
data_all[data_all['True datetime']>'2022-04-01'][['Order #','Order Status','Year','Month','Week','Date','Store Type','Store',
                                                  'Channel','Brand','Real SKU','Real Nama Produk','Bundle Name','Qty. Invoiced',
                                                  'Total Net Before PPN','Total','Coupon Code','Customer Name','Customer Email',
                                                  'Phone','Address','Warehouse Name','City','Region','Region Group','Real Region']].to_excel(f'Masterdata Lite Order {date.today()}.xlsx',index=False)

ValueError: This sheet is too large! Your sheet size is: 1050572, 26 Max sheet size is: 1048576, 16384

In [43]:
from datetime import date
osf=pd.read_excel(r"data_supp\order filter.xlsx")
resb=data_all[(data_all['Year'].isin([2021,2022]))&(data_all['Order Status'].isin(osf['order filter'].unique()))].groupby(['Year','Month','Date','Store Type','Store','Channel','Coupon Code','Brand','Sub Brand','Real SKU','Real Nama Produk','Bundle Name'],dropna=False).agg({'Order #':'nunique','Phone':'nunique','Qty. Invoiced':'sum','Total Net Before PPN':'sum'}).reset_index()
resb=resb[resb['Store Type']!='Organic']#['Store'].unique()
resa=data_all[(data_all['Year'].isin([2022,2021]))&(data_all['Order Status'].isin(osf['order filter'].unique()))&
                           (data_all['Store Type']=='Organic')][['Order #','Year','Month','Date','Store Type','Store','Channel','Coupon Code','Brand','Sub Brand','Real SKU','Real Nama Produk','Bundle Name','Qty. Invoiced','Total','Total Net','Total Net Before PPN','Customer Name','Customer Email','Phone','Payment Channel','City','Region','Real Region']]
print('tinggal append export')
resa.append(resb).to_excel(f'D:\Masterdata\Masterdata Lite\masterdata_{date.today()}_custlite v2.xlsx',index=False)

tinggal append export


In [72]:
data_all.loc[data_all['Store']=='Blibli','Warehouse Name']='Blibli Warehouse'
data_all[data_all['True datetime']>'2022-08-01'][['Order #','Order Status','Year','Month','Week','Date','Store Type','Store',
                                                  'Channel','Brand','Real SKU','Real Nama Produk','Bundle Name','Qty. Invoiced',
                                                  'Total Net Before PPN','Total','Coupon Code','Customer Name','Customer Email',
                                                  'Phone','Address','Warehouse Name','City','Region','Region Group','Real Region']].to_excel(f'D:\Masterdata\Masterdata Lite\Masterdata Lite Order {date.today()}.xlsx',index=False)

In [68]:
data_all.loc[data_all['Store']=='Blibli','Warehouse Name']='Blibli Warehouse'

In [71]:
data_all.loc[data_all['Store']=='Blibli']['Warehouse Name'].unique()

array(['Blibli Warehouse'], dtype=object)

In [67]:
for i in data_all.columns :
    print(i)

Order #
Sales Order ID
AWB
Paid Date
Order Status
Order date
Week
Date
Month
Quarter
Year
Channel
SKU
Brand
Product Name
Bundle Name
Price List NFI
Qty. Invoiced
Total Net
Sub Brand
Real SKU
Real Nama Produk
Parent Item
Parent SKU
Bundle Flag
Customer Email
Customer Name
Customer Group
Phone
Country
Region
City
Kecamatan
Kelurahan
Address
Zip Code
Shipping Name
Shipping Courier
Total
Coupon Code
Qty. Ordered
Qty. Shipped
Qty. Refunded
Item Price
Subtotal
Discounts
Tax
Total incl. Tax
Invoiced
Tax Invoiced
Invoiced incl. Tax
Refunded
Refunded incl. Tax
Cart Name Rule
Payment Channel
Voucher Amount
Discount Poin Reward
Ship Date
Discount Product
Produk 1
SKU Produk 1
PCS Produk 1
Price List NFI 1
Subtotal Produk 1
Harga Display 1
Harga Cost 1
Produk 2
SKU Produk 2
PCS Produk 2
Price List NFI 2
Subtotal Produk 2
Harga Display 2
Harga Cost 2
Produk 3
SKU Produk 3
PCS Produk 3
Price List NFI 3
Subtotal Produk 3
Harga Display 3
Harga Cost 3
Produk 4
SKU Produk 4
PCS Produk 4
Price List NFI 4